# Energy Production & Consumption Analysis

In [470]:
pip install country_converter

Note: you may need to restart the kernel to use updated packages.


In [471]:
# For Data Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 500)

# For Visualization
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
import plotly.offline as pyo

from plotly.subplots import make_subplots
from plotly.offline import plot

# For Additional Use
import datetime 
import calendar
import country_converter as cc

# Template
pio.templates.default = "plotly_dark"

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Import Statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose



### Overview

In [472]:
energy_data = pd.read_csv('World Energy Overview.csv')
energy_data = pd.DataFrame(energy_data)
energy_data['Date']= pd.to_datetime(energy_data['Date'])
energy_data.head()

,Date,Total Fossil Fuels Production,Nuclear Electric Power Production,Total Renewable Energy Production,Total Primary Energy Production,Primary Energy Imports,Primary Energy Exports,Primary Energy Net Imports,Primary Energy Stock Change and Other,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption
0,1973-01-31,4.932632,0.068103,0.403981,5.404715,1.173080,0.125781,1.047299,0.771858,6.747651,0.068103,0.403981,7.223873
1,1973-02-28,4.729582,0.064634,0.360900,5.155115,1.168005,0.120883,1.047122,0.390129,6.163095,0.064634,0.360900,6.592366
2,1973-03-31,4.946902,0.072494,0.400161,5.419556,1.309473,0.139950,1.169523,-0.067640,6.044647,0.072494,0.400161,6.521439
3,1973-04-30,4.716271,0.064070,0.380470,5.160812,1.085169,0.194185,0.890984,-0.110067,5.493184,0.064070,0.380470,5.941729
4,1973-05-31,4.956995,0.062111,0.392141,5.411246,1.162804,0.196775,0.966029,-0.305335,5.613551,0.062111,0.392141,6.071940


In [473]:
con_type_trend = energy_data[['Date', 'Total Fossil Fuels Consumption', 'Nuclear Electric Power Consumption','Total Renewable Energy Consumption','Total Primary Energy Consumption']]
con_type_trend.head()

,Date,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption
0,1973-01-31,6.747651,0.068103,0.403981,7.223873
1,1973-02-28,6.163095,0.064634,0.360900,6.592366
2,1973-03-31,6.044647,0.072494,0.400161,6.521439
3,1973-04-30,5.493184,0.064070,0.380470,5.941729
4,1973-05-31,5.613551,0.062111,0.392141,6.071940


In [474]:
con_type_trend_year = con_type_trend
con_type_trend_year['Date'] = con_type_trend['Date'].dt.year
con_type_trend_year.head()

,Date,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption
0,1973,6.747651,0.068103,0.403981,7.223873
1,1973,6.163095,0.064634,0.360900,6.592366
2,1973,6.044647,0.072494,0.400161,6.521439
3,1973,5.493184,0.064070,0.380470,5.941729
4,1973,5.613551,0.062111,0.392141,6.071940


In [475]:
con_type_trend_year =con_type_trend_year.groupby('Date').sum(numeric_only=True)
con_type_trend_year.head()

,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption
Date,,,,
1973,70.282079,0.910178,4.410937,75.651910
1974,67.871940,1.272084,4.741852,73.929185
1975,65.323405,1.899797,4.687121,71.931428
1976,69.070873,2.111119,4.727150,75.938523
1977,70.950267,2.701763,4.208967,77.920418


In [476]:
con_type_trend_year.reset_index(inplace=True)
con_type_trend_year.rename(columns={'Date': 'Year'}, inplace=True)
con_type_trend_year.head()

,Year,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption
0,1973,70.282079,0.910178,4.410937,75.651910
1,1974,67.871940,1.272084,4.741852,73.929185
2,1975,65.323405,1.899797,4.687121,71.931428
3,1976,69.070873,2.111119,4.727150,75.938523
4,1977,70.950267,2.701763,4.208967,77.920418


In [477]:
def consumption_changes(energy_names, energy_consumption_types):
    for energy_name, energy_consumption_type in zip(energy_names, energy_consumption_types):
        con_type_trend_year[energy_name + ' Changes '] = con_type_trend_year[energy_consumption_type].pct_change()*100

In [478]:
energy_con_names = ['Total Fossil Fuels Consumption','Nuclear Electric Power Consumption','Total Renewable Energy Consumption','Total Primary Energy Consumption']
energy_con_vol = ['Total Fossil Fuels Consumption','Nuclear Electric Power Consumption','Total Renewable Energy Consumption','Total Primary Energy Consumption']

In [479]:
consumption_changes(energy_con_names, energy_con_vol)
con_type_trend_year.head()

,Year,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption,Total Fossil Fuels Consumption Changes,Nuclear Electric Power Consumption Changes,Total Renewable Energy Consumption Changes,Total Primary Energy Consumption Changes
0,1973,70.282079,0.910178,4.410937,75.651910,NaN,NaN,NaN,NaN
1,1974,67.871940,1.272084,4.741852,73.929185,-3.429237,39.762112,7.502148,-2.277173
2,1975,65.323405,1.899797,4.687121,71.931428,-3.754917,49.345248,-1.154211,-2.702258
3,1976,69.070873,2.111119,4.727150,75.938523,5.736792,11.123399,0.854021,5.570715
4,1977,70.950267,2.701763,4.208967,77.920418,2.720965,27.977769,-10.961848,2.609868


In [480]:
con_type_trend_year = con_type_trend_year.set_index('Year')

In [481]:
con_type_trend_year.head()

,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption,Total Fossil Fuels Consumption Changes,Nuclear Electric Power Consumption Changes,Total Renewable Energy Consumption Changes,Total Primary Energy Consumption Changes
Year,,,,,,,,
1973,70.282079,0.910178,4.410937,75.651910,NaN,NaN,NaN,NaN
1974,67.871940,1.272084,4.741852,73.929185,-3.429237,39.762112,7.502148,-2.277173
1975,65.323405,1.899797,4.687121,71.931428,-3.754917,49.345248,-1.154211,-2.702258
1976,69.070873,2.111119,4.727150,75.938523,5.736792,11.123399,0.854021,5.570715
1977,70.950267,2.701763,4.208967,77.920418,2.720965,27.977769,-10.961848,2.609868


#### Production

In [482]:
prod_type_trend = energy_data[['Date', 'Total Fossil Fuels Production', 'Nuclear Electric Power Production', 'Total Renewable Energy Production', 'Total Primary Energy Production']]
prod_type_trend.head()

,Date,Total Fossil Fuels Production,Nuclear Electric Power Production,Total Renewable Energy Production,Total Primary Energy Production
0,1973-01-31,4.932632,0.068103,0.403981,5.404715
1,1973-02-28,4.729582,0.064634,0.360900,5.155115
2,1973-03-31,4.946902,0.072494,0.400161,5.419556
3,1973-04-30,4.716271,0.064070,0.380470,5.160812
4,1973-05-31,4.956995,0.062111,0.392141,5.411246


In [483]:
prod_type_trend_year = prod_type_trend
prod_type_trend_year['Date'] = prod_type_trend['Date'].dt.year
prod_type_trend_year.head()

,Date,Total Fossil Fuels Production,Nuclear Electric Power Production,Total Renewable Energy Production,Total Primary Energy Production
0,1973,4.932632,0.068103,0.403981,5.404715
1,1973,4.729582,0.064634,0.360900,5.155115
2,1973,4.946902,0.072494,0.400161,5.419556
3,1973,4.716271,0.064070,0.380470,5.160812
4,1973,4.956995,0.062111,0.392141,5.411246


In [484]:
prod_type_trend_year =prod_type_trend_year.groupby('Date').sum(numeric_only=True)
prod_type_trend_year.head()

,Total Fossil Fuels Production,Nuclear Electric Power Production,Total Renewable Energy Production,Total Primary Energy Production
Date,,,,
1973,58.209277,0.910178,4.410937,63.530389
1974,56.294375,1.272084,4.741852,62.308310
1975,54.696533,1.899797,4.687121,61.283451
1976,54.684224,2.111119,4.727150,61.522497
1977,55.062617,2.701763,4.208967,61.973344


In [485]:
prod_type_trend_year.reset_index(inplace=True)
prod_type_trend_year.rename(columns={'Date': 'Year'}, inplace=True)
prod_type_trend_year.head()

,Year,Total Fossil Fuels Production,Nuclear Electric Power Production,Total Renewable Energy Production,Total Primary Energy Production
0,1973,58.209277,0.910178,4.410937,63.530389
1,1974,56.294375,1.272084,4.741852,62.308310
2,1975,54.696533,1.899797,4.687121,61.283451
3,1976,54.684224,2.111119,4.727150,61.522497
4,1977,55.062617,2.701763,4.208967,61.973344


In [486]:
def production_changes(energy_names, energy_production_types):
    for energy_name, energy_production_type in zip(energy_names, energy_production_types):
        prod_type_trend_year[energy_name + ' Changes '] = prod_type_trend_year[energy_production_type].pct_change()*100

In [487]:
energy_prod_names = ['Total Fossil Fuels Production','Nuclear Electric Power Production','Total Renewable Energy Production','Total Primary Energy Production']
energy_prod_vol = ['Total Fossil Fuels Production','Nuclear Electric Power Production','Total Renewable Energy Production','Total Primary Energy Production']

In [488]:
production_changes(energy_prod_names, energy_prod_vol)

In [489]:
prod_type_trend_year = prod_type_trend_year.set_index('Year')
prod_type_trend_year.head()

,Total Fossil Fuels Production,Nuclear Electric Power Production,Total Renewable Energy Production,Total Primary Energy Production,Total Fossil Fuels Production Changes,Nuclear Electric Power Production Changes,Total Renewable Energy Production Changes,Total Primary Energy Production Changes
Year,,,,,,,,
1973,58.209277,0.910178,4.410937,63.530389,NaN,NaN,NaN,NaN
1974,56.294375,1.272084,4.741852,62.308310,-3.289685,39.762112,7.502148,-1.923613
1975,54.696533,1.899797,4.687121,61.283451,-2.838369,49.345248,-1.154211,-1.644819
1976,54.684224,2.111119,4.727150,61.522497,-0.022504,11.123399,0.854021,0.390066
1977,55.062617,2.701763,4.208967,61.973344,0.691960,27.977769,-10.961848,0.732816


In [490]:
fig_primary_con_prod_trend = go.Figure()
fig_primary_con_prod_trend = fig_primary_con_prod_trend.add_trace(go.Scatter(x=con_type_trend_year.index, y=con_type_trend_year['Total Primary Energy Consumption'], name='Total Primary Energy Consumption'))
fig_primary_con_prod_trend = fig_primary_con_prod_trend.add_trace(go.Scatter(x=prod_type_trend_year.index, y=prod_type_trend_year['Total Primary Energy Production'], name='Total Primary Energy Production'))
fig_primary_con_prod_trend = fig_primary_con_prod_trend.update_layout(title=' Figure 1: Total Primary Energy Consumption and Production', xaxis_title='Year', yaxis_title='Volume (qBTU)', title_x=0.5, title_y=0.9)
fig_primary_con_prod_trend.update_xaxes(rangeslider_visible=True)
fig_primary_con_prod_trend.show()

In [491]:
fig_primary_con_prod_changes = make_subplots(rows=1, cols=2, subplot_titles=('Total Primary Energy Consumption Changes', 'Total Primary Energy Production Changes'))
fig_primary_con_prod_changes.add_trace(go.Scatter(x=con_type_trend_year.index, y=con_type_trend_year['Total Primary Energy Consumption Changes '], name='Total Primary Energy Consumption Changes'), row=1, col=1)
fig_primary_con_prod_changes.add_trace(go.Scatter(x=prod_type_trend_year.index, y=prod_type_trend_year['Total Primary Energy Production Changes '], name='Total Primary Energy Production Changes'), row=1, col=2)
fig_primary_con_prod_changes = fig_primary_con_prod_changes.update_layout(title=' Figure 2.1: Total Primary Energy Consumption and Production Changes')
fig_primary_con_prod_changes.show()

In [492]:
fig_con_prod_lap = go.Figure()
fig_con_prod_lap = fig_con_prod_lap.add_trace(go.Scatter(x=con_type_trend_year.index, y=con_type_trend_year['Total Primary Energy Consumption Changes '], name='Total Primary Energy Consumption Changes',mode='lines + markers'))
fig_con_prod_lap = fig_con_prod_lap.add_trace(go.Scatter(x=prod_type_trend_year.index, y=prod_type_trend_year['Total Primary Energy Production Changes '], name='Total Primary Energy Production Changes',mode='lines + markers'))
fig_con_prod_lap = fig_con_prod_lap.update_layout(title=' Figure 2.2: Total Primary Energy Consumption and Production Changes')
fig_con_prod_lap.show()

In [493]:
con_type_trend_year.head()

,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption,Total Fossil Fuels Consumption Changes,Nuclear Electric Power Consumption Changes,Total Renewable Energy Consumption Changes,Total Primary Energy Consumption Changes
Year,,,,,,,,
1973,70.282079,0.910178,4.410937,75.651910,NaN,NaN,NaN,NaN
1974,67.871940,1.272084,4.741852,73.929185,-3.429237,39.762112,7.502148,-2.277173
1975,65.323405,1.899797,4.687121,71.931428,-3.754917,49.345248,-1.154211,-2.702258
1976,69.070873,2.111119,4.727150,75.938523,5.736792,11.123399,0.854021,5.570715
1977,70.950267,2.701763,4.208967,77.920418,2.720965,27.977769,-10.961848,2.609868


In [494]:
con_type_portion_year = con_type_trend_year[['Total Fossil Fuels Consumption','Nuclear Electric Power Consumption','Total Renewable Energy Consumption','Total Primary Energy Consumption']]

In [495]:
def con_type_portion(energy_con_names, energy_con_types):
    for energy_con_name, energy_con_type in zip(energy_con_names, energy_con_types):
        con_type_portion_year[energy_con_name + ' Portion '] = (con_type_portion_year[energy_con_type]/con_type_portion_year['Total Primary Energy Consumption'])*100

In [496]:
energy_con_names = ['Total Fossil Fuels Consumption','Nuclear Electric Consumption','Total Renewable Energy Consumption']
energy_con_types = ['Total Fossil Fuels Consumption','Nuclear Electric Power Consumption','Total Renewable Energy Consumption']
con_type_portion(energy_con_names, energy_con_types)

In [497]:
con_type_portion_year.head()

,Total Fossil Fuels Consumption,Nuclear Electric Power Consumption,Total Renewable Energy Consumption,Total Primary Energy Consumption,Total Fossil Fuels Consumption Portion,Nuclear Electric Consumption Portion,Total Renewable Energy Consumption Portion
Year,,,,,,,
1973,70.282079,0.910178,4.410937,75.651910,92.901923,1.203113,5.830569
1974,67.871940,1.272084,4.741852,73.929185,91.806693,1.720679,6.414046
1975,65.323405,1.899797,4.687121,71.931428,90.813441,2.641122,6.516096
1976,69.070873,2.111119,4.727150,75.938523,90.956303,2.780037,6.224970
1977,70.950267,2.701763,4.208967,77.920418,91.054782,3.467336,5.401623


In [498]:
prod_type_portion_year = prod_type_trend_year[['Total Fossil Fuels Production','Nuclear Electric Power Production','Total Renewable Energy Production','Total Primary Energy Production']]

In [499]:
def prod_type_portion(energy_prod_names, energy_prod_types):
    for energy_prod_name, energy_prod_type in zip(energy_prod_names, energy_prod_types):
        prod_type_portion_year[energy_prod_name + ' Portion '] = (prod_type_portion_year[energy_prod_type]/prod_type_portion_year['Total Primary Energy Production'])*100

In [500]:
prod_type_portion_year.columns

Index(['Total Fossil Fuels Production', 'Nuclear Electric Power Production',
       'Total Renewable Energy Production', 'Total Primary Energy Production'],
      dtype='object')

In [501]:
energy_prod_names = ['Total Fossil Fuels Production','Nuclear Electric Production','Total Renewable Energy Production','Total Primary Energy Production']
energy_prod_types = ['Total Fossil Fuels Production','Nuclear Electric Power Production','Total Renewable Energy Production','Total Primary Energy Production']
prod_type_portion(energy_prod_names, energy_prod_types)

In [502]:
prod_type_portion_year.head()

,Total Fossil Fuels Production,Nuclear Electric Power Production,Total Renewable Energy Production,Total Primary Energy Production,Total Fossil Fuels Production Portion,Nuclear Electric Production Portion,Total Renewable Energy Production Portion,Total Primary Energy Production Portion
Year,,,,,,,,
1973,58.209277,0.910178,4.410937,63.530389,91.624304,1.432666,6.943035,100.0
1974,56.294375,1.272084,4.741852,62.308310,90.348101,2.041596,7.610304,100.0
1975,54.696533,1.899797,4.687121,61.283451,89.251718,3.100016,7.648265,100.0
1976,54.684224,2.111119,4.727150,61.522497,88.884923,3.431459,7.683612,100.0
1977,55.062617,2.701763,4.208967,61.973344,88.848872,4.359557,6.791576,100.0


In [503]:
con_type_portion_year.columns

Index(['Total Fossil Fuels Consumption', 'Nuclear Electric Power Consumption',
       'Total Renewable Energy Consumption',
       'Total Primary Energy Consumption',
       'Total Fossil Fuels Consumption Portion ',
       'Nuclear Electric Consumption Portion ',
       'Total Renewable Energy Consumption Portion '],
      dtype='object')

In [504]:
fig_con_portion = go.Figure()
fig_con_portion.add_trace(go.Bar(x=con_type_portion_year.index, y=con_type_portion_year['Total Fossil Fuels Consumption Portion '], name='Total Fossil Fuels Consumption Portion'))
fig_con_portion.add_trace(go.Bar(x=con_type_portion_year.index, y=con_type_portion_year['Nuclear Electric Consumption Portion '], name='Nuclear Electric Consumption Portion'))
fig_con_portion.add_trace(go.Bar(x=con_type_portion_year.index, y=con_type_portion_year['Total Renewable Energy Consumption Portion '], name='Total Renewable Energy Consumption Portion'))
fig_con_portion.update_layout(title='Figure 3: Energy Consumption Shares by Type',barmode='stack')
fig_con_portion.show()

In [505]:
fig_prod_portion = go.Figure()
fig_prod_portion = fig_prod_portion.add_trace(go.Bar(x=prod_type_portion_year.index, y=prod_type_portion_year['Total Fossil Fuels Production Portion '], name='Total Fossil Fuels Production Portion'))
fig_prod_portion = fig_prod_portion.add_trace(go.Bar(x=prod_type_portion_year.index, y=prod_type_portion_year['Nuclear Electric Production Portion '], name='Nuclear Electric Production Portion'))
fig_prod_portion = fig_prod_portion.add_trace(go.Bar(x=prod_type_portion_year.index, y=prod_type_portion_year['Total Renewable Energy Production Portion '], name='Total Renewable Energy Production Portion'))
fig_prod_portion = fig_prod_portion.update_layout(title='Figure 4: Energy Production Shares by Type', barmode='stack')
fig_prod_portion.show()

### Analysis on Energy Consumption and Production

#### Analysis on Gloabl Energy Consumption & Production 

Energy consumption worldwide is on the increasing trend throughout 1973 to 2022, reaching its peak at 101.24 qBTU in 2018 (*Figure 1**), with the strongest growth between 1991 to 1995. During this period, total energy consumption increased by 11.34%, from 84.38 qBTU in 1991 to 93.94 qBTU, driven by strong economic growth arising from peace dividend following the end of Cold War. 

 
 

Despite the increasing trend in total energy consumption, several significant declines could be seen throughout this period. Following restoration of energy consumption in the aftermath of two oil shocks in 1970s (*1973-1974: Oil Embargo* & *Iranian Revolution 1979*), energy consumption declined to reach its lowest point in 7 years, 72.92 qBTU in 1983, driven by economic recession in major economies. In 2009, energy consumption declined to its lowest point in 13 years,93.94 qBTU, in the aftermath of The Great Recession. 

2022 recorded the worst slump in energy consumption, when total energy consumption declined to 91.34, the lowest in 27 years, driven down by Covid-19 outbreak. 

 
 

In general, energy production also shows a similar increasing trend throughout 1973 to 2022. However, contrary to the total energy consumption, it shows less volatility. Energy production reached its peaks in 2019, recording 101.49 qBTU production before falling to 7 years low at 95.75 qBTU  following the same trajectory recorded by energy consumption driven down by Covid-19 outbreak. After breaking up the continuous production growth from 2009, energy production recorded its highest growth, growing from 84.34 qBTU to rally to its peak in 2019. Another significant decline in energy production was in 1983, when it declined to 64.07 qBTU, the lowest in 4 years, driven down by slowing economic growth. 

 
 

Year by Year growth trends as captured in **Figure 2. 1** and **Figure 2.2** showed devastating impact of Covid-19 outbreak. Energy consumption fell drastically by 7.43% in 2020, as compared to 2019 while energy production fell by 5.65% within the same period. Highest Year on Year growth in energy consumption was recorded in 1976, as the energy consumption grew by 5.57%, driven by economic growth in major economies (*United States*, *West Germany* & *France*) and then newly emerging major economic power, Japan, as the economy recovered from recession in 1975. Attributed by economic expansion, yield from energy infrastructure and technological advancement, energy production recorded strongest growth in 2018, 8.71%. 

 
 

Throughout the period of 1973 to 2022, both energy consumption and production (**Figure 3** & **Figure 4**) made a gradual shift from fossil fuels to renewable energy. From meagre 5.83% of shares for total energy consumption in 1973, renewable energy consumption reached its peak at 13.19% in 2021. Renewable energy consumption grew throughout this period at the expense of nuclear and fossil fuel energy consumption. Nuclear energy consumption grew from 1.23% of energy consumption in 1973 to 8.89% in 2009, before gradually declining to 8.01%, the lowest in 20 years. Despite the growth of renewable and nuclear energy, fossil fuels stay the main source for energy consumption. It declined from 92.90% in 1973 to 78.65 in 2022. 

 
 

Energy production also displays a similar trend. Fossil fuel energy production declined from 91.62% in 1973 to 79.19 in 2022 while renewable energy shares increased from meagre 6.94% in 1973 to 13.03% in 2022. Nuclear energy production also displays a similar upward trend and reached its peak in 2007, producing 11.87% of global energy production. However, its shares gradually declined to 7.78% in 2022. Despite this trend, fossil fuels are still the main source for energy production with its contribution remaining 3/4 of the total energy production. 

 
 

The gradual shift from fossil fuels could be attributed to few factors: 

<ol> 

<li> Increasing investment in renewable energy, worldwide, especially in Europe and North America</li> 

<li>Growing concern on the impact of fossil fuels consumption</li> 

<li>Pivot towards sustainability policies and incentives both in public and private sector</li> 

<li>Technological advancement in renewable energy sector</li> 

<li>Energy security and diversification</li> 

</ol> 

 
 

However, similar trends on total global energy and consumption and their mix are not uniform throughout the regions. 

### Regional Energy Consumption & Production Analysis 

#### Africa

Throughout the period 1980 to 2022, energy consumption in Africa was derived from fossil fuels, being almost 90% of energy mix consumption. The main driver behind this is the growth of natural gas consumption, growing from 12.56% in 1973 to 29.46% in 2022. Total natural gas consumption displays an almost uninterrupted increasing trend through this period, increasing from 0.83 qBTU to its peak, 5.97 qBTU. A significant increase in natural gas consumption was recorded between 2010 to 2011, when it grew by 19.2%, from 3.75 qBTU to 4.47 qBTU. Despite the increase in natural gas consumption, Africa remains able to export its gas due to the net surplus between consumption and production. In 1980, its production is 0.78 qBTU and keep increasing to reach its peak in 2021, producing 9.69 qBTU of natural gas.   

 
 

The growth of natural gas consumption in Africa could be attributed to the following factors: 

<ol> 

<li> Economic growth and industrialization of many countries in Africa</li> 

<li> Discoveries of new gas reserves in several African countries</li> 

<li> Continuous investment in the required infrastructure for the distribution of natural gas</li> 

</ol> 

  

However, both natural gas consumption and production throughout the period concentrated in certain parts of the region.  Throughout the period, five countries in the region produced more than 90% of natural gas. Similarly, five countries in the region consumed more than 90% of total natural gas in Africa. Largest natural gas consumer and producer are: 

<ol> 

<li>Algeria</li> 

<li>Egypt</li> 

<li>Libya</li> 

<li>Nigeria</li> 

</ol> 

 
 

Besides natural gas, Africa is also powered by both coal and petroleum. Coal consumption displayed an increasing trend throughout the period from 2.11 qBTU in 1980 to its peak, 4.66 qBTU in 2019, before declining to 4.41 qBTU in 2021. The sharpest increase in coal consumption was recorded between 2018 and 2019, when total coal consumption increased from 4.26qBTU to 4.66 qBTU, an increase of 9.39%. A ten-year rally was recorded between 1980 to 1990, an impressive increase of 52.61%, from 2.11 qBTU to 3.22 qBTU. Similarly, coal production increased from 2.64 qBTU in 1980 to its peak, 6.17 qBTU in 2017 before declining to its 17-year low. The sharpest increase was recorded between 2002 and 2023 as production increased by 8.42%. However, the longest uptrend rally was recorded between 2009 to 2014, when coal production increased from 5.56 qBTU to 6.10 qBTU, an increase of 9.71%. 

 
 

Coal production in the region is much more concentrated compared with natural gas. It is heavily dominated by South Africa, producing at its peak, 98.25%, equivalent to 5.58 qBTU of total coal production in Africa. Similarly, coal consumption in Africa is dominated by South Africa, consuming at its peak, 93.04% of total coal consumption or 4.10 qBTU. 

 
 

Despite the increase of natural gas consumption throughout the region, petroleum is the major source of energy that is powering up Africa. Between 1980 to 20221, petroleum consumption contributed between 40% to 48% of energy consumption with its consumption showing an increasing trend from 3.07 qBTU in 1980 to 8.06 qBTU in 2021, a decrease of 8.57% from its peak in 2019. However, its production showed a downtrend after its peak in 2010. In 2010, it produced 21.95 qBTU. However, despite some recovery in its production in 2012, it continues it downtrend, reaching 26 years low in 2020.  

 
 

Similar concentrated consumption and production can also be seen for petroleum. In 1980, the top five producers contributed 93.43% of production from the region. While it is less concentrated in the next decades, it stays above 80% of the total production of the region. Petroleum consumption is less concentrated but stays high. The top five petroleum consumers consume more than 50% of total consumption in the region. Top five petroleum producer in the region are: 

<ol> 

<li>Nigeria</li> 

<li>Egypt</li> 

<li>Algeria</li> 

<li>Libya</li> 

</ol> 

While South Africa is not one of the top producers, it is one of top consumers for the region, consuming 16% to 22% of the total petroleum consumption in the region. 

 


The unequal distribution of energy consumption and production throughout the region can be attributed to the size of the economies. Africa's energy landscape is heavily influenced by its major economies, which include South Africa, Nigeria, Egypt, Algeria, Tunisia, and Angola. These economies play a pivotal role in shaping the region's energy consumption and production trends.

### Asia

Rapid economic development throughout Asia (excluding Middle East) saw the rapid increase in its energy consumption. In 1980, Asia consumed 51.02 qBTU of energy and experienced continuous growth throughout the period. By 2021, it consumed 274.24 qBTU, a five-fold increase in its consumption. Rapid expansion between 2004 and 2008 saw the region increased its energy consumption from 143.06 qBTU to 178.47 qBTU, an impressive increase of 24.75%. The impressive energy consumption growth in Asia saw it overtook Europe energy consumption in 1991. By 2002, it overtook North America energy consumption and appeared as the most power-hungry region. 

  

The surge in energy consumption within the region has spurred a parallel increase in energy production. Starting from 1980, Asia's energy production stood at 37.33 qBTU, and since then, it has steadily and consistently grown, reaching its peak in 2021 at an impressive 211.46 qBTU – a remarkable five-fold rise. Energy production in Asia overtook North America in 2003 and became the largest energy producer. 

Despite the increasing production throughout the period, the gap between energy production and consumption in Asia widening, a trend set since 1980, with the largest gap in 2016. While the gap decreased afterwards, it remains throughout the period. 

Asia was energized by fossil fuels, coal, petroleum, and natural gas with coal as the main source of energy. Coal consumption in Asia saw an uninterrupted rally since 1980, reaching its peak in 2021, consuming 132.74 qBTU in 2021. Coal consumption increased drastically between 2003 and 2007, increasing by 52.11%. With the exception in 2016 when coal production fell to 114.06 qBTU, lowest in 4 years, coal production is on uptrend. A significant production gain was recorded between 2003 and 2007 as production increased by 45.85, from 59 qBTU to 86.05 qBTU. Coal consumption deficit started to widen since 2004 with the widest deficit between 2015 and 2019. By 2019, coal consumption trend was reversed. 

Coal consumption contributed between 41.54% to 54.58% of the total energy consumption in Asia. While coal consumption in terms of volume keeps increasing, coal consumption contribution toward total energy consumption in Asia started to decline after reaching its peak in 2012, contributing only 48.4% of total energy consumption. The trend shows the emergence of another energy source to fulfill Asia's rising demand for energy. 

Nuclear and renewable energy is the emerging new source energy for consumption in Asia. In 1980, its consumption is meagre 3.71 qBTU. By 2021, its consumption stands at 37.43 qBTU. Nuclear and renewable energy experienced rapid growth between 2015 and 2020. Its consumption increased from 23.32 qBTU to 34.61 qBTU, an increase of 48.41%. It contributed 7.28% of total energy consumption in 1989 and by 2021, its contribution is 13.65%, making it the fastest source of energy in Asia. Unlike other sources of energy, nuclear and renewable energy can fulfill its demand, hence making it attractive. 

Delving further into nuclear and renewable energy consumption revealed that 80% of its consumption concentrated in Japan, China, India, South Korea, Taiwan, and Vietnam. Japan and China consumed up to 75% of total nuclear and renewable energy consumption. By 2020, China alone consumed 65% of nuclear and renewable energy consumption in Asia. The same trend could also be seen in its production trend. Except for 1980, the largest nuclear and renewable energy producers were similar largest consumers and have nuclear capability. Only in 2010, Vietnam, the first non-nuclear country become one of 5 largest nuclear and renewable energy in Asia. 

Asia emerged as the largest consumer for petroleum since 2004, when it overtook North America. Its petroleum consumption increased from 23.34 qBTU in 1980 to 70.57 qBTU in 2021, a three-fold increase. In contract, petroleum production in Asia is declining since 2015 to its 26-year lowest in 2021, producing at 15.22 qBTU. This led to ever growing petroleum consumption deficit for Asia. The highest consumption deficit was recorded in 2021 as the gap between petroleum consumption grew to 55.34 qBTU.

Petroleum consumption in Asia recorded uninterrupted increasing trend, apart from 2020, when consumption declined from 69.91 qBTU to 67.23 qBTU, a decline of 3.83%. Rather than indicating a retreat from its trend, 2020 is rather an exception as the region was heavily affected by Covid-19 outbreak. Post outbreak, the consumption rallying to its peak in 2021. Meanwhile, the region able to increase its production from 1980 to 2010, with the largest increase between 1982 and 1985. During the period, production increase from 10.18 qBTU to 12.41 qBTU, an increase of 21.9%. Its production plateauing between 1997 and 2009. Temporary hike in 2010 began the decline in petroleum production in Asia.

Natural gas is another source of energy for Asia. It emerged from contributing only 5.43% of the total energy consumption in 1980 to 12.23% of total consumption in 2021. Unlike petroleum, natural gas consumption throughout the period is continuously increasing and not even being dampened by Covid-19 outbreak in 2020. Rapid increase in natural gas consumption was recorded between 2015 and 2018. During this period, natural gas consumption increased by 19.46%, from 25.86 qBTU to 30.89 qBTU. Strong demand for natural gas was supported by increasing natural gas production with the strongest growth recorded between 2004 to 2008. During this period, production increased from 11.3 qBTU to 15.19 qBTU, an increase of 34.42%. Despite the increase, natural gas consumption deficit continued to grow, with the largest deficit recorded in 2021. In 2021, the deficit is 9 qBTU.
 
 The rise of energy consumption in Asia is driven by the rise of China as the new economic house. In 1980, total energy consumption in China is 15.94 qBTU and recorded uninterrupted increasing trend through out the period. By 2021, its total energy consumption increased by ten-fold, to 165.16 qBTU. China's total energy consumption increased at faster rate from 2002 to 2021 compared with the previous period. From 2002 to 2021, total energy consumption increased by 3-fold, from 47.71 qBTU to 165.16. In the previous decades, energy consumption increased from 19.10 qBTU to 44.56, a lower yet still impressive two-fold increase.

 Coal remains the main energy that powered China. Despite its decline trend as energy consumption mix, it contributed to 58.25% of total energy mix in China by the end of the period. Throughout the period, it nuclear and renewable energy mix in China's total energy mix keep increasing. It increased from a meagre of 3.10% to 14.97% of total energy consumption. Similar trend could also be observed for natural gas. Natural gas contribution towards China energy consumption mixes increased 3.04% in 1980 to 8.11% by the end of the period. The rise in the consumption of both nuclear & renewable energy and natural gas also led to the decline of petroleum consumption as part of China energy consumption mix. It declined from 21.58% in 1980 to 18.68% in 2021. Despite the increase in the consumption of nuclear & renewable energy, fossil fuels remained as the main source of energy consumption in China. Its consumption remains above 80% of total energy consumption mix.

 
 Its ever-rising energy consumption is further supported by increase in its energy production. In 1980, China produced 19.54% qBTU of energy and it continued to increase, with the exception in 2016. In 2016, it fell to 108.15 qBTU. By 2021, China produced 134.96 qBTu of energy. However, due to the higher energy consumption rate, increase in China's energy production could not fulfill energy demand for China. From 2002 onwards, consumption deficit widening, reaching its peak in 2016. 

### Middle East

Middle East is one of the regions that recorded positive net energy, producing more energy that required for the region. Its energy production steadily increased throughout the year with the exception two major points. In the first decade, its production decrease from 42.26 qBTU in 1980 to reach five years low, 25.78 qBTU, a decline of 39% of total production energy. Its production increased throughout the years and reached its peak in 2018, producing 89.69 qBTU. However, in 2020, its production fell by 5.62%, to 4 years low at 83.19 qBTU.  
  
Along with Asia and North America, Middle East is a crucial energy producer. It contributed between 8.14% to 15.61% towards global energy production. Until 2018, it was the only major energy producing region with net positive energy surplus.  
  
Petroleum and natural gas are two major energy producers. Throughout 1980 to 2021, both petroleum and natural gas produced more than 60% of total energy. In 1980, 63.4% of total energy produced by the combination of petroleum and natural gas, with 44.92% was produced by petroleum while 18.48% was produced by natural gas. Despite declining in the shares of energy produced by both petroleum and natural gas, it remains as the main source of energy production. 
 
In this context, Middle East is a crucial region. Petroleum and natural gas are two main energies produced by the region. In 1980, 96.17% of energy production mix is petroleum. However, throughout the period, natural gas share of energy production progressively increased. In 1980, it contributed a meagre 3.53% of energy production in the region. By the end of the period, 31.32% of energy produced by the region is in the form of natural gas. In contrast, both nuclear & renewable energy and coal made up less than 1% of the energy production in the Middle East.  
  
Middle East is the largest petroleum producer throughout 1980 to 2021, with exception of 1985. In 1980, Middle East contributed to 30.54% of global total petroleum production before declining to 19.1% in 1985, producing 23.14 qBTU of energy. In 1985, North America is the largest petroleum producer, producing 31.18 qBTU or 25.74% of total petroleum production. Afterwards, Middle East become the largest petroleum producer, reaching its peak by producing 64.79 qBTU in 2018, or 34.36% of total petroleum production. Some of the largest petroleum producers in the Middle East are Saudi Arabia, Iraq, Kuwait, Iran, and the United Arab Emirates, all of which also rank among the top ten petroleum producers in the world. 
 
Production of natural gas increased throughout 1980 to 2021. In 1980, Middle East produced 1.49 qBTU of energy. By the end of decade, its production increased to 3.77 qBTU, a two-fold increase in its production. In contract with petroleum production, natural gas production experienced a continuous and uninterrupted growth. By 2000, its production improved to 7.97 qBTU and by 2010, emerged as third largest producer, behind North America and Eurasia. The region reached its peak in 2021, producing 26.57 qBTU. 
 
Energy consumption in Middle East was dominated by petroleum and natural gas. 90% of the energy consumption in Middle East consist of petroleum and natural gas throughout the period. However, petroleum as part of energy mix in Middle East declined from its peak, 77.25% in 1982, to its lowest in 2021, 38.87% of energy consumption mix. In terms of consumption volume, petroleum consumption increased steadily from 1980 and reached its peak, 18.41 qBTU in 2018 before declined to 14 years low, 14.04 qBTU in 2021. 
  
Natural gas consumption increased at higher rate compared to petroleum. In 1980, natural gas consumption in Middle East stood at 1.37 qBTU. In 2014, its consumption overtook petroleum consumption at 16.85 qBTU. Its consumption reached its peak,21.34 qBTU in 2021. In terms of energy consumption mix, natural gas represents 23.37% of total energy consumption in 1980. By 1990, natural gas consumption increased to 32.56% of total energy consumption. By 2007, over half of the energy consumption mix in Middle East was in the form of natural gas. By 2021, natural gas represents almost 60% of the energy consumption in Middle East. 
 
Despite the increase in energy consumption. Middle East maintain its net energy surplus. With higher rate of increase in the energy production, both petroleum and natural gas, Middle East net energy surplus widening throughout the period, reaching its peak in 2018. 
 


#### Central & South America

Energy consumption in the region  recorded continous growth throughout the period with the exception of 2020. During Covid-19 outbreak in 2020, energy consumption decreased by 10% ffrom 28.44 qBTU to 25.82 qBTU. This was the lowest in 11 year.  Energy consumption in the region reached its peak in 2014 and 2015 as it consumed 29.2 qBTU.  

The region is powered up by combination of fossil fuel and nuclear & renewable energy. More than 90% of its energy consumption contributed by these source of energies. Petroleum is the main source of its energy consumption, followed by nuclear & renewable energy and natural gas.Coal consumption as energy consumption mix in the region is increasing. In 1980, it represents 3.44% of energy consumption and by 1985, it increased to 4.78%. It's contribution towards energy mix consumption however decline afterwards, reaching its lowest point in 2003, with 3.19% of energy consumption mix. The trend was reversed after that, and coal consumption in 2021 contributed to 4.71% of energy mix consumption.

Central & South America region recorded net energy surplus since 1980 and the surplus started to rapidly widening in 1991 until 2017. Afterwards, the energy surplus was tightening due to the fall in the energy production and increase in the energy consumption. Increase in the energy consumption in the region could be attributed to the increase in the consumption of nuclear & renewable. In 2017, the region consumed 7.86 qBTU of nuclear and renewable energy. By 2018, its consumption reached its peak at 8.08 qBTU, an increase of 2.8% before moderating to be between 7.91 qBTU and 7.93 qBTU for the remaining period.

Meanwhile,natural gas and petroleum consumption showed a declining trend. After reaching its peak in 2015, consuming 6.24 qBTU of natural gas, its consumption is on the declining trend. It fell by 4.81% to 5.94 qBTU and remain flat until Covid-19 outbreak in 2020. While natural gas consumption restored after the outbreak, it remained below pre-Covid consumption volume. The decline in the petroleum consumption started a year earlier, in 2014. In 2014, its consumption reached its peak, at 14.44 qBTU. Aftrewards, it progressively declined and reached its lowest in 2020, recorded 11.37 qBTU, lowest in 15 years.  Despite restored volume consumption in 2021, it was at 15 year low, 11.96 qBTU. After reached its peak in 2017, consumed at the volume of 1.32 qBTU, coal consumption remain relatively flat, with the excpetion in 2020.

### Total Consumption

In [506]:
total_all_con = pd.read_csv('Consumption_Data/Consumption_Total.csv')
total_all_con.tail()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
225,Eurasia,Russia,--,--,--,--,--,--,--,--,...,31.09814022,31.36027357,30.9650754,31.02390028,31.26933107,32.39329874,32.597945,32.81496851,34.21033961,894.408884
226,Eurasia,Tajikistan,--,--,--,--,--,--,--,--,...,0.210975397,0.215523806,0.215805158,0.222154268,0.226612228,0.241753209,0.256198715,0.254621103,0.255333992,6.796811
227,Eurasia,Turkmenistan,--,--,--,--,--,--,--,--,...,1.407979321,1.440514209,1.758353822,1.754720561,1.737786661,1.740037773,1.771733624,1.80034877,1.89508569,28.140879
228,Eurasia,Ukraine,--,--,--,--,--,--,--,--,...,4.904226787,4.383048096,3.845564092,3.771233197,3.684040518,3.747075892,3.543399757,3.454790016,3.544386604,167.558852
229,Eurasia,Uzbekistan,--,--,--,--,--,--,--,--,...,1.913352428,2.087359302,2.00379243,1.763372064,1.848783053,1.861009507,1.859892727,1.805319218,1.925119154,58.902105


#### Africa

In [507]:
total_all_con_africa = total_all_con[total_all_con['Continent']=='Africa']
total_all_con_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
0,Africa,Algeria,0.780695167,0.663391323,0.952188116,1.070561843,1.130786713,1.046418247,1.066300962,1.138318654,...,2.037584643,2.299888436,2.35214754,2.33598238,2.365986905,2.522119152,2.624873096,2.480810774,2.581123314,64.746181
1,Africa,Angola,0.058366148,0.057863688,0.062007899,0.066297007,0.059894701,0.073766696,0.072853256,0.075135479,...,0.347761278,0.382763726,0.39921871,0.364251517,0.347458949,0.359226946,0.404865374,0.402317536,0.328628418,7.403765
2,Africa,Benin,0.006526525,0.006574612,0.004640112,0.006731564,0.006673241,0.007698814,0.007501055,0.005942365,...,0.066471109,0.070275151,0.077723341,0.094785008,0.10051639,0.111506331,0.101325936,0.108653617,0.110194887,1.660207
3,Africa,Botswana,0.014748762,0.015462338,0.014427219,0.014460824,0.014444631,0.018865991,0.020060806,0.021144246,...,0.065375734,0.065661654,0.080008274,0.088008537,0.080969008,0.080276228,0.080755726,0.076835026,0.082892488,2.021542
4,Africa,Burkina Faso,0.0060363,0.006019807,0.006019807,0.006019807,0.0060363,0.005819147,0.006886218,0.007191413,...,0.0451141,0.042012755,0.053396002,0.051306252,0.05946458,0.066130673,0.067697934,0.064550875,0.068226015,0.988877


In [508]:
total_all_con_africa_sum = total_all_con_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_con_africa_sum = total_all_con_africa_sum.sum(axis=0) 
total_all_con_africa_sum = total_all_con_africa_sum.transpose()
total_all_con_africa_sum = pd.DataFrame(total_all_con_africa_sum)
total_all_con_africa_sum = total_all_con_africa_sum.rename(columns={0:'Total_Consumption_Africa'})
total_all_con_africa_sum.tail()

,Total_Consumption_Africa
2017,19.974224
2018,20.305719
2019,20.881698
2020,19.852882
2021,20.250196


#### Asia

In [509]:
total_all_con_asia = total_all_con[total_all_con['Continent']== 'Asia & Oceania']
total_all_con_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
57,Asia & Oceania,Afghanistan,0.026583217,0.029949458,0.031897815,0.039021803,0.03920224,0.038274961,0.039111199,0.063065158,...,0.11416577,0.096961201,0.111725826,0.101967815,0.100861238,0.115698197,0.103436586,0.09346919,0.092588907,2.531506
58,Asia & Oceania,American Samoa,0.005893112,0.006926477,0.00965509,0.006086904,0.006945454,0.006506691,0.006716584,0.006973009,...,0.004930591,0.004930529,0.004930145,0.004952661,0.004957574,0.004957029,0.004956034,0.004722483,0.00544641,0.296715
59,Asia & Oceania,Australia,2.98543583,3.049885599,3.231536108,3.106748752,3.149405326,3.289445776,3.339457591,3.510145689,...,5.81706753,5.752163477,5.778272494,6.018588429,6.199735644,6.167648593,6.20613677,5.832997906,5.922681343,198.066837
60,Asia & Oceania,Bangladesh,0.122373731,0.13263193,0.141023313,0.136119292,0.166146742,0.170785,0.185244035,0.200774298,...,1.127902426,1.204768817,1.333116046,1.434278744,1.478925297,1.486120069,1.586626464,1.550571791,1.711825006,28.280312
61,Asia & Oceania,Bhutan,0.000135023,0.000494827,0.000509713,0.000500903,0.000517838,0.000559007,0.002725977,0.009396006,...,0.061220231,0.058614448,0.061385491,0.063170956,0.062765098,0.060169277,0.071363938,0.070097187,0.073712887,1.264962


In [510]:
total_all_con_asia_sum = total_all_con_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_con_asia_sum = total_all_con_asia_sum.sum(axis=0)
total_all_con_asia_sum = total_all_con_asia_sum.transpose()
total_all_con_asia_sum = pd.DataFrame(total_all_con_asia_sum)
total_all_con_asia_sum = total_all_con_asia_sum.rename(columns={0:'Total_Consumption_Asia'})
total_all_con_asia_sum.head()

,Total_Consumption_Asia
1980,51.020781
1981,51.736257
1982,52.856163
1983,54.651351
1984,58.524602


#### Middle East

In [511]:
total_all_con_me = total_all_con[total_all_con['Continent']== 'Middle East']
total_all_con_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
106,Middle East,Bahrain,0.137079245,0.157930777,0.168400777,0.176852604,0.171630245,0.203998777,0.220987772,0.265819139,...,0.694685179,0.688650527,0.694730523,0.693186174,0.713164716,0.722134346,0.79228429,0.786935755,0.804407371,18.201202
107,Middle East,Iran,1.602263257,1.494842399,1.728156748,2.103023854,2.229336205,2.408062407,2.422930466,2.531707034,...,9.817213928,10.41672059,10.65539398,10.87725087,11.38359294,11.88447052,11.91806202,11.70262522,12.05034758,258.423821
108,Middle East,Iraq,0.529373261,0.481029294,0.462333916,0.55057257,0.53308641,0.572683953,0.620907014,0.637388756,...,1.728490974,1.663506701,1.434806985,1.768347218,2.364614779,2.630779714,2.53181196,2.163563152,2.088296564,49.042921
109,Middle East,Israel,0.355489502,0.360903568,0.37631468,0.369340376,0.33717559,0.356481988,0.384921918,0.42284982,...,0.920587767,0.972153424,1.013129901,1.0331426,1.070955104,1.050515185,1.015515642,0.970166147,1.030758639,30.201866
110,Middle East,Jordan,0.077824544,0.092295242,0.104880957,0.111173815,0.117788499,0.121586846,0.129563319,0.139507294,...,0.335986002,0.344741552,0.372458098,0.39698988,0.410723506,0.400927078,0.410986818,0.366864089,0.349322747,9.972381


In [512]:
total_all_con_me_sum = total_all_con_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_con_me_sum = total_all_con_me_sum.sum(axis=0)
total_all_con_me_sum = total_all_con_me_sum.transpose()
total_all_con_me_sum = pd.DataFrame(total_all_con_me_sum)
total_all_con_me_sum = total_all_con_me_sum.rename(columns={0:'Total_Consumption_Middle_East'})
total_all_con_me_sum.head()

,Total_Consumption_Middle_East
1980,5.876938
1981,6.150960
1982,6.422753
1983,7.196534
1984,7.810526


#### Central & South America

In [513]:
total_all_con_csa = total_all_con[total_all_con['Continent']== 'Central & South America']
total_all_con_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
120,Central & South America,Antarctica,0.00213195,0.002126125,0.002126125,0.002126125,0.002771535,0.002976575,0.002763963,0.002763963,...,0.00019569,0.000195674,0.000198729,0.000198478,0.000186509,0.000186509,0.000186509,0.000157013,0.000167087,0.085370
121,Central & South America,Antigua and Barbuda,0.005804855,0.005168746,0.007029494,0.004134997,0.004560958,0.004548496,0.004624394,0.005755917,...,0.009922379,0.010145155,0.010384333,0.010535492,0.010023144,0.010327377,0.010382211,0.008785966,0.009372914,0.322670
122,Central & South America,Argentina,1.713088126,1.669763391,1.749519261,1.773452331,1.899825996,1.821675672,1.951940041,1.957846267,...,3.764130595,3.777299811,3.832386308,3.836759294,3.667912266,3.642795329,3.704336987,3.247238793,3.5448976,115.538608
123,Central & South America,Aruba,--,--,--,--,--,--,0.001453868,0.001453868,...,0.017646844,0.017898388,0.018421825,0.017901295,0.018534464,0.017905592,0.018564912,0.015820885,0.016755668,0.513160
124,Central & South America,Barbados,0.014607404,0.015202213,0.015413404,0.016258171,0.015666255,0.017314129,0.018158895,0.017289876,...,0.025202074,0.021917332,0.023314344,0.02378657,0.022550358,0.02395075,0.024065923,0.020667281,0.022093626,0.890778


In [514]:
total_all_con_csa_sum = total_all_con_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_con_csa_sum = total_all_con_csa_sum.sum(axis=0)
total_all_con_csa_sum = total_all_con_csa_sum.transpose()
total_all_con_csa_sum = pd.DataFrame(total_all_con_csa_sum)
total_all_con_csa_sum = total_all_con_csa_sum.rename(columns={0:'Total_Consumption_Central_South_America'})
total_all_con_csa_sum.head()

,Total_Consumption_Central_South_America
1980,11.036419
1981,11.018623
1982,11.207669
1983,11.152724
1984,11.741093


#### North America

In [515]:
total_all_con_na = total_all_con[total_all_con['Continent']== 'North America']
total_all_con_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
165,North America,Bermuda,0.007977284,0.006908714,0.007327424,0.007746134,0.007557427,0.007327424,0.006898958,0.009947155,...,0.007802241,0.010104616,0.008497225,0.01081503,0.010668011,0.011036412,0.011029603,0.010088665,0.010186187,0.375344
166,North America,Canada,9.587301021,9.478642977,9.084953571,9.003453612,9.568578286,9.926910677,10.0339905,10.33093102,...,14.68606267,14.8042203,14.71640601,14.55476557,14.96439028,15.16095755,14.7497767,13.84903852,14.17114494,521.849654
167,North America,Greenland,0.007817076,0.007374328,0.007163633,0.005056682,0.008028349,0.007163633,0.004761882,0.003294473,...,0.012358677,0.011550802,0.01230661,0.012147638,0.012084296,0.012322115,0.012017163,0.012120341,0.012195101,0.397070
168,North America,Mexico,3.801224373,4.203000197,4.442704101,4.182654406,4.213411815,4.453974866,4.401642446,4.513598038,...,7.783456258,7.678793386,7.798257376,7.966803773,7.95239238,7.806440646,7.914050215,6.996789927,7.04856658,261.329234
169,North America,Saint Pierre and Miquelon,0.002129031,0.002123214,0.001910892,0.002123214,0.002129031,0.001910892,0.00235233,0.003024068,...,0.001306404,0.001361465,0.001361465,0.001386152,0.001361465,0.001481302,0.001273758,0.001165176,0.001176441,0.069092


In [516]:
total_all_con_na_sum = total_all_con_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_con_na_sum = total_all_con_na_sum.sum(axis=0)
total_all_con_na_sum = total_all_con_na_sum.transpose()
total_all_con_na_sum = pd.DataFrame(total_all_con_na_sum)
total_all_con_na_sum = total_all_con_na_sum.rename(columns={0:'Total_Consumption_North_America'})
total_all_con_na_sum.head()

,Total_Consumption_North_America
1980,91.427562
1981,89.755236
1982,86.590248
1983,86.116428
1984,90.370402


#### Europe

In [517]:
total_all_con_europe = total_all_con[total_all_con['Continent']== 'Europe']
total_all_con_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
172,Europe,Albania,0.162981822,0.162044493,0.162846117,0.169335851,0.170405722,0.170056255,0.166075378,0.166640528,...,0.131931118,0.116216393,0.116297302,0.135895705,0.125780475,0.14342046,0.113609057,0.10914868,0.137347715,5.040066
173,Europe,Austria,1.09738269,1.086021611,1.033423966,1.040713209,1.068711353,1.091841456,1.071648656,1.021749791,...,1.472655105,1.424002015,1.426288308,1.45911133,1.490301668,1.457465029,1.516470409,1.409519255,1.441018898,55.602679
174,Europe,Belgium,2.122077199,1.99587973,1.866576717,1.834552174,1.875209775,1.948695088,2.02757189,2.091754985,...,2.636916102,2.462887377,2.495016376,2.646344945,2.660533504,2.682506913,2.696035571,2.447575884,2.714421214,103.408869
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0.249862444,0.24194132,0.242312366,0.247680045,0.245790237,0.249928591,0.23730706,0.22039809,0.256852205,6.237125
176,Europe,Bulgaria,1.33896612,1.324224181,1.37165549,1.402375485,1.404325086,1.438387267,1.422900059,1.413156848,...,0.706069842,0.748545851,0.785803103,0.752018557,0.75854264,0.748856049,0.717490052,0.672789903,0.744132231,39.834917


In [518]:
total_all_con_europe_sum = total_all_con_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_con_europe_sum = total_all_con_europe_sum.sum(axis=0)
total_all_con_europe_sum = total_all_con_europe_sum.transpose()
total_all_con_europe_sum = pd.DataFrame(total_all_con_europe_sum)
total_all_con_europe_sum = total_all_con_europe_sum.rename(columns={0:'Total_Consumption_Europe'})
total_all_con_europe_sum.head()

,Total_Consumption_Europe
1980,75.367302
1981,73.029128
1982,72.632915
1983,73.018118
1984,74.125327


#### Eurasia

In [519]:
total_all_con_eurasia = total_all_con[total_all_con['Continent']== 'Eurasia']
total_all_con_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0.146905025,0.14973865,0.138737831,0.13972766,0.148910885,0.146478106,0.155712475,0.163567101,0.168413259,4.099553
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,0.567334314,0.58507,0.618316557,0.608957262,0.606640486,0.608430022,0.647238563,0.628515134,0.663350689,18.640861
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,1.118389227,1.120295341,0.953054341,0.944644515,0.940714123,0.991866151,0.961810248,0.923425044,1.016464377,31.835688
220,Eurasia,Former U.S.S.R.,51.56998421,50.95051081,52.64340399,53.80084789,56.52287146,58.53154281,59.80096869,61.83636662,...,--,--,--,--,--,--,--,--,--,697.882262
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.210155934,0.219500711,0.232753768,0.246018684,0.244609545,0.249497346,0.252947991,0.240251253,0.267159494,5.709198


In [520]:
total_all_con_eurasia_sum = total_all_con_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_con_eurasia_sum = total_all_con_eurasia_sum.sum(axis=0)
total_all_con_eurasia_sum = total_all_con_eurasia_sum.transpose()
total_all_con_eurasia_sum = pd.DataFrame(total_all_con_eurasia_sum)
total_all_con_eurasia_sum = total_all_con_eurasia_sum.rename(columns={0:'Total_Consumption_Eurasia'})
total_all_con_eurasia_sum.head()

,Total_Consumption_Eurasia
1980,51.569984
1981,50.950511
1982,52.643404
1983,53.800848
1984,56.522871


#### Total All by Year

In [521]:
total_all_con_sum = total_all_con_africa_sum.join(total_all_con_asia_sum).join(total_all_con_me_sum).join(total_all_con_csa_sum).join(total_all_con_na_sum).join(total_all_con_europe_sum).join(total_all_con_eurasia_sum)
total_all_con_sum['Total_All'] = total_all_con_sum.sum(axis=1)
total_all_con_sum['Africa_Portion'] = total_all_con_sum['Total_Consumption_Africa']/total_all_con_sum['Total_All']*100
total_all_con_sum['Asia_Portion'] = total_all_con_sum['Total_Consumption_Asia']/total_all_con_sum['Total_All']*100
total_all_con_sum['Middle_East_Portion'] = total_all_con_sum['Total_Consumption_Middle_East']/total_all_con_sum['Total_All']*100
total_all_con_sum['Central_South_America_Portion'] = total_all_con_sum['Total_Consumption_Central_South_America']/total_all_con_sum['Total_All']*100
total_all_con_sum['North_America_Portion'] = total_all_con_sum['Total_Consumption_North_America']/total_all_con_sum['Total_All']*100
total_all_con_sum['Europe_Portion'] = total_all_con_sum['Total_Consumption_Europe']/total_all_con_sum['Total_All']*100
total_all_con_sum['Eurasia_Portion'] = total_all_con_sum['Total_Consumption_Eurasia']/total_all_con_sum['Total_All']*100
total_all_con_sum.head()

,Total_Consumption_Africa,Total_Consumption_Asia,Total_Consumption_Middle_East,Total_Consumption_Central_South_America,Total_Consumption_North_America,Total_Consumption_Europe,Total_Consumption_Eurasia,Total_All,Africa_Portion,Asia_Portion,Middle_East_Portion,Central_South_America_Portion,North_America_Portion,Europe_Portion,Eurasia_Portion
1980,6.640928,51.020781,5.876938,11.036419,91.427562,75.367302,51.569984,292.939915,2.266993,17.416808,2.006192,3.767469,31.210346,25.727905,17.604287
1981,6.820238,51.736257,6.150960,11.018623,89.755236,73.029128,50.950511,289.460953,2.356186,17.873311,2.124971,3.806601,31.007718,25.229354,17.601860
1982,7.395432,52.856163,6.422753,11.207669,86.590248,72.632915,52.643404,289.748584,2.552362,18.242078,2.216664,3.868067,29.884615,25.067565,18.168649
1983,7.807166,54.651351,7.196534,11.152724,86.116428,73.018118,53.800848,293.743169,2.657820,18.605148,2.449941,3.796760,29.316913,24.857810,18.315608
1984,8.198115,58.524602,7.810526,11.741093,90.370402,74.125327,56.522871,307.292935,2.667850,19.045215,2.541720,3.820814,29.408552,24.122041,18.393808


In [522]:
fig_total_all_con_sum = go.Figure()
fig_total_all_con_sum = fig_total_all_con_sum.add_trace(go.Scatter(x=total_all_con_sum.index, y=total_all_con_sum['Total_Consumption_Africa'],name='Africa', mode='lines + markers'))
fig_total_all_con_sum = fig_total_all_con_sum.add_trace(go.Scatter(x=total_all_con_sum.index, y=total_all_con_sum['Total_Consumption_Asia'], name='Asia',mode='lines + markers'))
fig_total_all_con_sum = fig_total_all_con_sum.add_trace(go.Scatter(x=total_all_con_sum.index, y=total_all_con_sum['Total_Consumption_Middle_East'], name='Middle East',mode='lines + markers'))
fig_total_all_con_sum = fig_total_all_con_sum.add_trace(go.Scatter(x=total_all_con_sum.index, y=total_all_con_sum['Total_Consumption_Central_South_America'], name='Central South America',mode='lines + markers'))
fig_total_all_con_sum = fig_total_all_con_sum.add_trace(go.Scatter(x=total_all_con_sum.index, y=total_all_con_sum['Total_Consumption_North_America'], name='North America',mode='lines + markers'))
fig_total_all_con_sum = fig_total_all_con_sum.add_trace(go.Scatter(x=total_all_con_sum.index, y=total_all_con_sum['Total_Consumption_Europe'], name='Europe',mode='lines + markers'))
fig_total_all_con_sum = fig_total_all_con_sum.add_trace(go.Scatter(x=total_all_con_sum.index, y=total_all_con_sum['Total_Consumption_Eurasia'], name='Eurasia',mode='lines + markers'))
fig_total_all_con_sum = fig_total_all_con_sum.update_layout(title='Total Energy Consumption by Region 1980 - 2021')
fig_total_all_con_sum.show()

In [523]:
fig_total_con_portion_all = go.Figure()
fig_total_con_portion_all = fig_total_con_portion_all.add_trace(go.Bar(x=total_all_con_sum.index, y=total_all_con_sum['Africa_Portion'],name='Africa'))
fig_total_all_portion_all = fig_total_con_portion_all.add_trace(go.Bar(x=total_all_con_sum.index, y=total_all_con_sum['Asia_Portion'], name='Asia'))
fig_total_all_portion_all = fig_total_con_portion_all.add_trace(go.Bar(x=total_all_con_sum.index, y=total_all_con_sum['Middle_East_Portion'], name='Middle East'))
fig_total_all_portion_all = fig_total_con_portion_all.add_trace(go.Bar(x=total_all_con_sum.index, y=total_all_con_sum['Central_South_America_Portion'], name='Central South America'))
fig_total_all_portion_all = fig_total_con_portion_all.add_trace(go.Bar(x=total_all_con_sum.index, y=total_all_con_sum['North_America_Portion'], name='North America'))
fig_total_all_portion_all = fig_total_con_portion_all.add_trace(go.Bar(x=total_all_con_sum.index, y=total_all_con_sum['Europe_Portion'], name='Europe'))
fig_total_all_portion_all = fig_total_con_portion_all.add_trace(go.Bar(x=total_all_con_sum.index, y=total_all_con_sum['Eurasia_Portion'], name='Eurasia'))
fig_total_all_portion_all.update_layout(title='Figure 6: Energy Mix Consumption by Continent', template='plotly_dark',barmode='stack')
fig_total_all_portion_all.update_xaxes(rangeslider_visible=True)
fig_total_all_portion_all.show()

### Total Production

In [524]:
total_all_prod = pd.read_csv('Production_Data/Production_Total.csv')
total_all_prod.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
0,Africa,Algeria,2.803017355,3.03753686,3.224933779,3.606400483,3.859176003,3.907465538,3.968324779,4.218086334,...,6.510417691,6.561749872,6.696192523,6.910709728,6.830371007,6.755066467,6.474600013,5.972769931,6.656881917,238.118858
1,Africa,Angola,0.335098042,0.291293691,0.276262403,0.395407667,0.462550452,0.511269369,0.621432995,0.789428936,...,3.932204112,3.802528446,3.949946917,3.93529724,3.899020984,3.814151292,3.526638469,3.159336901,2.824017641,90.852722
2,Africa,Benin,in,0,0,0.00896,0.01573,0.01793,0.01793,0.01569,...,0.000019082,0.00001902,0.000055914,0.00009232,0.000055278,4.64E-05,7.18E-05,6.19E-05,7.07E-05,0.178363
3,Africa,Botswana,0.008262057,0.008484744,0.009241913,0.008796519,0.00875198,0.009731845,0.010912138,0.010845329,...,0.033334507,0.038240847,0.046562169,0.0419666,0.049654188,0.048447799,0.047231633,0.041991759,0.044392707,0.952582
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0.001106756,0.00096051,0.000987814,0.00143096,0.001455654,0.002066608,0.00190567,0.001859876,0.002054882,0.033589


In [525]:
total_all_prod['Continent'].unique()

array(['    Africa', '    Eurasia', '    Europe', '    Asia & Oceania',
       '    Middle East', '    North America',
       '    Central & South America'], dtype=object)

#### Africa

In [526]:
total_all_prod_africa = total_all_prod[total_all_prod['Continent']=='    Africa']
total_all_prod_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
0,Africa,Algeria,2.803017355,3.03753686,3.224933779,3.606400483,3.859176003,3.907465538,3.968324779,4.218086334,...,6.510417691,6.561749872,6.696192523,6.910709728,6.830371007,6.755066467,6.474600013,5.972769931,6.656881917,238.118858
1,Africa,Angola,0.335098042,0.291293691,0.276262403,0.395407667,0.462550452,0.511269369,0.621432995,0.789428936,...,3.932204112,3.802528446,3.949946917,3.93529724,3.899020984,3.814151292,3.526638469,3.159336901,2.824017641,90.852722
2,Africa,Benin,in,0,0,0.00896,0.01573,0.01793,0.01793,0.01569,...,0.000019082,0.00001902,0.000055914,0.00009232,0.000055278,4.64E-05,7.18E-05,6.19E-05,7.07E-05,0.178363
3,Africa,Botswana,0.008262057,0.008484744,0.009241913,0.008796519,0.00875198,0.009731845,0.010912138,0.010845329,...,0.033334507,0.038240847,0.046562169,0.0419666,0.049654188,0.048447799,0.047231633,0.041991759,0.044392707,0.952582
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0.001106756,0.00096051,0.000987814,0.00143096,0.001455654,0.002066608,0.00190567,0.001859876,0.002054882,0.033589


In [527]:
total_all_prod_africa_sum = total_all_prod_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_prod_africa_sum = total_all_prod_africa_sum.sum(axis=0)
total_all_prod_africa_sum = total_all_prod_africa_sum.transpose()
total_all_prod_africa_sum = pd.DataFrame(total_all_prod_africa_sum)
total_all_prod_africa_sum = total_all_prod_africa_sum.rename(columns={0:'Total_Production_Africa'})
total_all_prod_africa_sum.head()

,Total_Production_Africa
1980,17.186180
1981,14.911990
1982,15.328102
1983,16.003652
1984,17.430642


#### Asia

In [528]:
total_all_prod_asia = total_all_prod[total_all_prod['Continent']== '    Asia & Oceania']
total_all_prod_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
115,Asia & Oceania,Afghanistan,0.072561156,0.092723655,0.095067867,0.09971282,0.099827343,0.12096783,0.120549769,0.114835081,...,0.040816563,0.044789507,0.042057997,0.049505118,0.059500543,0.063175856,0.05530024,0.010260242,0.041213085,1.918961
116,Asia & Oceania,American Samoa,0,0,0,0,0,0,0,0,...,0.000019082,0.00001902,0.000018638,0.000027696,0.000046065,0.00004552,0.000044525,0.000043865,0.000043865,0.000317
117,Asia & Oceania,Australia,3.722297333,4.142083016,4.149179345,4.544673514,4.927573328,5.411039308,5.614002178,6.161929291,...,14.18255887,15.08070006,15.65180832,16.09826409,16.86993783,17.35916749,18.92253191,18.67941226,18.01231093,423.582598
118,Asia & Oceania,Bangladesh,0.053542502,0.06192692,0.068085551,0.072315474,0.099606438,0.103319364,0.112541005,0.130109003,...,0.876649729,0.916327572,1.01973903,1.125437842,1.078727254,1.070334519,1.060012798,0.982117452,0.936612494,20.191419
119,Asia & Oceania,Bhutan,6.32E-05,7.47E-05,8.58E-05,8.70E-05,9.02E-05,9.02E-05,0.00314707,0.012199748,...,0.073142871,0.06935489,0.073467189,0.074701868,0.073175883,0.065828474,0.082777394,0.08113648,0.0820511,1.504563


In [529]:
total_all_prod_asia_sum = total_all_prod_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_prod_asia_sum = total_all_prod_asia_sum.sum(axis=0)   
total_all_prod_asia_sum = total_all_prod_asia_sum.transpose()
total_all_prod_asia_sum = pd.DataFrame(total_all_prod_asia_sum)
total_all_prod_asia_sum = total_all_prod_asia_sum.rename(columns={0:'Total_Production_Asia'})
total_all_prod_asia_sum.head()

,Total_Production_Asia
1980,37.327228
1981,38.286615
1982,39.484112
1983,42.398905
1984,46.334497


#### Middle East

In [530]:
total_all_prod_me = total_all_prod[total_all_prod['Continent']=='    Middle East']
total_all_prod_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
164,Middle East,Bahrain,0.220752774,0.237047333,0.242845188,0.25078951,0.241762868,0.272163532,0.295236342,0.332325684,...,0.984466418,1.024558776,1.024245061,1.017875964,1.032321573,1.034465512,1.097055586,1.048226221,1.080748922,23.671534
165,Middle East,Iran,3.942762037,3.291377705,5.131137508,5.676622743,5.298678009,5.582412904,5.062637932,5.664076881,...,13.30027517,14.05689544,14.57354802,17.12264372,18.32903937,18.36550926,15.93315296,15.61895784,16.70374936,452.576902
166,Middle East,Iraq,5.450484369,2.163251456,2.189388134,2.168100966,2.61056118,3.090701288,3.659376524,4.576350776,...,6.642995706,7.299934739,8.753998827,9.64983899,9.969897213,10.32782472,10.59009376,9.184025239,9.240203455,207.726903
167,Middle East,Israel,0.081115,0.083075,0.005277,0.004238,0.004248,0.004238,0.0036146,0.002937814,...,0.171730118,0.297283039,0.321311165,0.362823896,0.380059717,0.390675367,0.400013275,0.559858651,0.688321258,4.217068
168,Middle East,Jordan,0,0,0,0,0,0,0.002211338,0.001287961,...,0.005542429,0.004938085,0.006659888,0.011821674,0.016576881,0.023131051,0.029803339,0.042973916,0.050146104,0.412696


In [531]:
total_all_prod_me_sum = total_all_prod_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_prod_me_sum = total_all_prod_me_sum.sum(axis=0)
total_all_prod_me_sum = total_all_prod_me_sum.transpose()
total_all_prod_me_sum = pd.DataFrame(total_all_prod_me_sum)
total_all_prod_me_sum = total_all_prod_me_sum.rename(columns={0:'Total_Production_Middle_East'})
total_all_prod_me_sum.head()

,Total_Production_Middle_East
1980,42.264127
1981,36.485050
1982,29.694303
1983,27.561035
1984,27.675984


In [532]:
fig_total_con_prod_energy_me = go.Figure()
fig_total_con_prod_energy_me = fig_total_con_prod_energy_me.add_trace(go.Scatter(x=total_all_con_me_sum.index, y=total_all_con_me_sum['Total_Consumption_Middle_East'], name='Consumption', mode='lines+markers'))
fig_total_con_prod_energy_me = fig_total_con_prod_energy_me.add_trace(go.Scatter(x=total_all_prod_me_sum.index, y=total_all_prod_me_sum['Total_Production_Middle_East'], name='Production', mode='lines+markers'))
fig_total_con_prod_energy_me = fig_total_con_prod_energy_me.update_layout(title='Middle East Consumption & Production', template='plotly_dark')
fig_total_con_prod_energy_me.show()

#### Central & South America

In [533]:
total_all_prod_csa = total_all_prod[total_all_prod['Continent']=='    Central & South America']
total_all_prod_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
184,Central & South America,Antarctica,0,0,0,0,0,0,0,0,...,4.77E-06,0.000004755,0.000009319,0.000009232,0,0,0,0,0,0.000229
185,Central & South America,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,5.72E-06,0.000005706,4.99E-05,0.00004616,5.66E-05,0.00009104,0.000115765,0.000140368,0.000171249,0.000694
186,Central & South America,Argentina,1.597945664,1.671287961,1.729804472,1.785477213,1.852960338,1.834828933,1.843831912,1.834331805,...,3.12684005,3.120516508,3.139683465,3.230273032,3.233848532,3.318160756,3.43406163,3.159079525,3.23367069,121.580775
187,Central & South America,Aruba,--,--,--,--,--,--,0,0,...,0.001459773,0.00151209,0.001630825,0.001380295,0.001324019,0.001335666,0.00130647,0.001287104,0.001287104,0.016248
188,Central & South America,Barbados,0.0004188,0.0004188,0.0025588,0.0025588,0.0047188,0.0046988,0.0046988,0.003309688,...,0.002998483,0.002761601,0.002927764,0.0029946,0.002987859,0.002993475,0.003084863,0.003277928,0.003399445,0.143106


In [534]:
total_all_prod_csa_sum = total_all_prod_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_prod_csa_sum = total_all_prod_csa_sum.sum(axis=0)
total_all_prod_csa_sum = total_all_prod_csa_sum.transpose()
total_all_prod_csa_sum = pd.DataFrame(total_all_prod_csa_sum)
total_all_prod_csa_sum = total_all_prod_csa_sum.rename(columns={0:'Total_Production_Central_South_America'})
total_all_prod_csa_sum.head()

,Total_Production_Central_South_America
1980,12.133860
1981,12.334737
1982,12.214567
1983,12.490824
1984,13.523598


In [535]:
fig_total_con_prod_energy_csa = go.Figure()
fig_total_con_prod_energy_csa = fig_total_con_prod_energy_csa.add_trace(go.Scatter(x=total_all_con_csa_sum.index, y=total_all_con_csa_sum['Total_Consumption_Central_South_America'], name='Consumption', mode='lines+markers'))
fig_total_con_prod_energy_csa = fig_total_con_prod_energy_csa.add_trace(go.Scatter(x=total_all_prod_csa_sum.index, y=total_all_prod_csa_sum['Total_Production_Central_South_America'], name='Production', mode='lines+markers'))
fig_total_con_prod_energy_csa = fig_total_con_prod_energy_csa.update_layout(title='Central South America Energy Consumption & Production', template='plotly_dark')
fig_total_con_prod_energy_csa.update_xaxes(rangeslider_visible=True)
fig_total_con_prod_energy_csa.show()


#### North America

In [536]:
total_all_prod_na = total_all_prod[total_all_prod['Continent']=='    North America']
total_all_prod_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
178,North America,Bermuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
179,North America,Canada,10.15605745,9.962604945,9.955898659,10.20044612,11.16288673,11.82409685,11.72122593,12.35934295,...,19.93668902,20.75752642,20.91416582,21.10709605,22.69970072,23.60884128,23.45886067,22.63761484,23.40553976,709.974346
180,North America,Greenland,0,0,0,0,0,0,0,0,...,0.003568334,0.00372792,0.003904661,0.003665104,0.003537792,0.003767417,0.003562,0.0043865,0.0043865,0.065164
181,North America,Mexico,5.713648968,6.742066139,7.79269697,7.658492701,7.886286362,7.82733977,7.098676353,7.347981678,...,8.610921048,8.380745447,7.693843621,7.340392727,6.672281536,6.249387956,5.785129564,5.825629906,5.867028136,344.105944
182,North America,Saint Pierre and Miquelon,0,0,0,0,0,0,0,0,...,0.000019082,0,0,0,0,0,0,0,0,0.000077


In [537]:
total_all_prod_na_sum = total_all_prod_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_prod_na_sum = total_all_prod_na_sum.sum(axis=0)
total_all_prod_na_sum = total_all_prod_na_sum.transpose()
total_all_prod_na_sum = pd.DataFrame(total_all_prod_na_sum)
total_all_prod_na_sum = total_all_prod_na_sum.rename(columns={0:'Total_Production_North_America'})
total_all_prod_na_sum.head()

,Total_Production_North_America
1980,83.016301
1981,83.614179
1982,84.275328
1983,81.925271
1984,87.848759


In [538]:
fig_con_prod_sum_na = go.Figure()
fig_con_prod_sum_na = fig_con_prod_sum_na.add_trace(go.Scatter(x=total_all_con_na_sum.index, y=total_all_con_na_sum['Total_Consumption_North_America'], name='Consumption', mode='lines+markers'))
fig_con_prod_sum_na = fig_con_prod_sum_na.add_trace(go.Scatter(x=total_all_prod_na_sum.index, y=total_all_prod_na_sum['Total_Production_North_America'], name='Production', mode='lines+markers'))
fig_con_prod_sum_na = fig_con_prod_sum_na.update_layout(title='North America Consumption & Production', template='plotly_dark')
fig_con_prod_sum_na.show()

#### Europe

In [539]:
total_all_prod_europe = total_all_prod[total_all_prod['Continent']=='    Europe']
total_all_prod_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
70,Europe,Albania,0.15556162,0.158691383,0.205944156,0.234693222,0.191025062,0.191729938,0.196595523,0.202388064,...,0.105636486,0.092961941,0.102254704,0.125356216,0.08259616,0.116337106,0.091399758,0.089594086,0.119993451,4.479150
71,Europe,Austria,0.477056917,0.470869747,0.472787608,0.46138511,0.445309168,0.462166206,0.461008282,0.403179699,...,0.567091102,0.563655235,0.530440303,0.550642005,0.549275289,0.522594805,0.56410935,0.542696228,0.505869402,21.607216
72,Europe,Belgium,0.299878697,0.299836636,0.335462088,0.407233784,0.450642822,0.512263482,0.545206953,0.538484471,...,0.560835952,0.483215695,0.41890767,0.588415272,0.591801415,0.465422264,0.629181031,0.552545461,0.695511494,20.516149
73,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0.158810535,0.146538416,0.140223655,0.145890683,0.139109216,0.163121808,0.153094878,0.135655114,0.161487699,4.051968
74,Europe,Bulgaria,0.323726039,0.346534535,0.377162949,0.39851169,0.412146624,0.391548685,0.412726871,0.42920315,...,0.425557186,0.459587042,0.492856445,0.449672131,0.463893071,0.463624626,0.43443392,0.389710765,0.45537389,17.501682


In [540]:
total_all_prod_europe_sum = total_all_prod_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_prod_europe_sum = total_all_prod_europe_sum.sum(axis=0)
total_all_prod_europe_sum = total_all_prod_europe_sum.transpose()
total_all_prod_europe_sum = pd.DataFrame(total_all_prod_europe_sum)
total_all_prod_europe_sum = total_all_prod_europe_sum.rename(columns={0:'Total_Production_Europe'})
total_all_prod_europe_sum.head()

,Total_Production_Europe
1980,42.908097
1981,43.827254
1982,45.268815
1983,47.266558
1984,47.881392


#### Eurasia

In [541]:
total_all_prod_eurasia = total_all_prod[total_all_prod['Continent']=='    Eurasia']
total_all_prod_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
57,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0.045980655,0.04536061,0.050525628,0.047246935,0.049007438,0.043337275,0.045262562,0.04674026,0.041807898,1.260966
58,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,2.502823304,2.479827844,2.50494981,2.461803915,2.339899183,2.378237826,2.518582437,2.417830699,2.701185932,50.734602
59,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0.069204931,0.068842132,0.0604334,0.075410608,0.078347109,0.082044079,0.085107583,0.091693233,0.147194883,2.528852
60,Eurasia,Former U.S.S.R.,61.50664227,61.83394113,63.86297682,65.40147273,68.25201144,70.31066975,72.98506044,75.03424507,...,--,--,--,--,--,--,--,--,--,840.491586
61,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.087483994,0.0858219,0.085257319,0.091450831,0.090499971,0.093905824,0.080794531,0.075729584,0.093804537,2.272900


In [542]:
total_all_prod_eurasia_sum = total_all_prod_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
total_all_prod_eurasia_sum = total_all_prod_eurasia_sum.sum(axis=0)
total_all_prod_eurasia_sum = total_all_prod_eurasia_sum.transpose()
total_all_prod_eurasia_sum = pd.DataFrame(total_all_prod_eurasia_sum)
total_all_prod_eurasia_sum = total_all_prod_eurasia_sum.rename(columns={0:'Total_Production_Eurasia'})
total_all_prod_eurasia_sum.head()

,Total_Production_Eurasia
1980,61.506642
1981,61.833941
1982,63.862977
1983,65.401473
1984,68.252011


#### Total All Production by Year

In [543]:
total_all_prod_sum = total_all_prod_africa_sum.join(total_all_prod_asia_sum).join(total_all_prod_me_sum).join(total_all_prod_csa_sum).join(total_all_prod_na_sum).join(total_all_prod_europe_sum).join(total_all_prod_eurasia_sum)
total_all_prod_sum['Total All'] = total_all_prod_sum.sum(axis=1)
total_all_prod_sum['Africa_Portion'] = total_all_prod_sum['Total_Production_Africa']/total_all_prod_sum['Total All']*100
total_all_prod_sum['Asia_Portion'] = total_all_prod_sum['Total_Production_Asia']/total_all_prod_sum['Total All']*100
total_all_prod_sum['Middle_East_Portion'] = total_all_prod_sum['Total_Production_Middle_East']/total_all_prod_sum['Total All']*100
total_all_prod_sum['Central_South_America_Portion'] = total_all_prod_sum['Total_Production_Central_South_America']/total_all_prod_sum['Total All']*100
total_all_prod_sum['North_America_Portion'] = total_all_prod_sum['Total_Production_North_America']/total_all_prod_sum['Total All']*100
total_all_prod_sum['Europe_Portion'] = total_all_prod_sum['Total_Production_Europe']/total_all_prod_sum['Total All']*100
total_all_prod_sum['Eurasia_Portion'] = total_all_prod_sum['Total_Production_Eurasia']/total_all_prod_sum['Total All']*100
total_all_prod_sum.head()

,Total_Production_Africa,Total_Production_Asia,Total_Production_Middle_East,Total_Production_Central_South_America,Total_Production_North_America,Total_Production_Europe,Total_Production_Eurasia,Total All,Africa_Portion,Asia_Portion,Middle_East_Portion,Central_South_America_Portion,North_America_Portion,Europe_Portion,Eurasia_Portion
1980,17.186180,37.327228,42.264127,12.133860,83.016301,42.908097,61.506642,296.342436,5.799433,12.595978,14.261922,4.094540,28.013639,14.479228,20.755260
1981,14.911990,38.286615,36.485050,12.334737,83.614179,43.827254,61.833941,291.293766,5.119227,13.143644,12.525174,4.234466,28.704418,15.045723,21.227348
1982,15.328102,39.484112,29.694303,12.214567,84.275328,45.268815,63.862977,290.128203,5.283217,13.609195,10.234890,4.210058,29.047617,15.603038,22.011985
1983,16.003652,42.398905,27.561035,12.490824,81.925271,47.266558,65.401473,293.047718,5.461108,14.468260,9.404965,4.262386,27.956291,16.129304,22.317687
1984,17.430642,46.334497,27.675984,13.523598,87.848759,47.881392,68.252011,308.946883,5.641954,14.997561,8.958169,4.377321,28.434907,15.498260,22.091827


In [544]:
fig_total_all_prod_sum = go.Figure()
fig_total_all_prod_sum = fig_total_all_prod_sum.add_trace(go.Scatter(x=total_all_prod_sum.index, y=total_all_prod_sum['Total_Production_Africa'], name='Africa', mode='lines + markers'))
fig_total_all_prod_sum = fig_total_all_prod_sum.add_trace(go.Scatter(x=total_all_prod_sum.index, y=total_all_prod_sum['Total_Production_Asia'], name='Asia',mode='lines + markers'))
fig_total_all_prod_sum = fig_total_all_prod_sum.add_trace(go.Scatter(x=total_all_prod_sum.index, y=total_all_prod_sum['Total_Production_Middle_East'], name='Middle East',mode='lines + markers'))
fig_total_all_prod_sum = fig_total_all_prod_sum.add_trace(go.Scatter(x=total_all_prod_sum.index, y=total_all_prod_sum['Total_Production_Central_South_America'], name='Central South America',mode='lines + markers'))
fig_total_all_prod_sum = fig_total_all_prod_sum.add_trace(go.Scatter(x=total_all_prod_sum.index, y=total_all_prod_sum['Total_Production_North_America'], name='North America',mode='lines + markers'))
fig_total_all_prod_sum = fig_total_all_prod_sum.add_trace(go.Scatter(x=total_all_prod_sum.index, y=total_all_prod_sum['Total_Production_Europe'], name='Europe',mode='lines + markers'))
fig_total_all_prod_sum = fig_total_all_prod_sum.add_trace(go.Scatter(x=total_all_prod_sum.index, y=total_all_prod_sum['Total_Production_Eurasia'], name='Eurasia',mode='lines + markers'))
fig_total_all_prod_sum.update_layout(title='Total Energy Production by Continent', template='plotly_dark',barmode='stack')
fig_total_all_prod_sum.update_xaxes(rangeslider_visible=True)
fig_total_all_prod_sum.show()

In [545]:
fig_total_prod_portion_all = go.Figure()
fig_total_prod_portion_all = fig_total_prod_portion_all.add_trace(go.Bar(x=total_all_prod_sum.index, y=total_all_prod_sum['Africa_Portion'],name='Africa'))
fig_total_prod_portion_all = fig_total_prod_portion_all.add_trace(go.Bar(x=total_all_prod_sum.index, y=total_all_prod_sum['Asia_Portion'], name='Asia'))
fig_total_prod_portion_all = fig_total_prod_portion_all.add_trace(go.Bar(x=total_all_prod_sum.index, y=total_all_prod_sum['Middle_East_Portion'], name='Middle East'))
fig_total_prod_portion_all = fig_total_prod_portion_all.add_trace(go.Bar(x=total_all_prod_sum.index, y=total_all_prod_sum['Central_South_America_Portion'], name='Central South America'))
fig_total_prod_portion_all = fig_total_prod_portion_all.add_trace(go.Bar(x=total_all_prod_sum.index, y=total_all_prod_sum['North_America_Portion'], name='North America'))
fig_total_prod_portion_all = fig_total_prod_portion_all.add_trace(go.Bar(x=total_all_prod_sum.index, y=total_all_prod_sum['Europe_Portion'], name='Europe'))
fig_total_prod_portion_all = fig_total_prod_portion_all.add_trace(go.Bar(x=total_all_prod_sum.index, y=total_all_prod_sum['Eurasia_Portion'], name='Eurasia'))
fig_total_prod_portion_all.update_layout(title='Total Production by Continent', template='plotly_dark',barmode='stack')
fig_total_prod_portion_all.update_xaxes(rangeslider_visible=True)
fig_total_prod_portion_all.show()

In [546]:
prod_con_gap_asia = go.Figure()
prod_con_gap_asia = prod_con_gap_asia.add_trace(go.Scatter(x=total_all_prod_asia_sum.index, y=total_all_prod_asia_sum['Total_Production_Asia'], name='Energy Production Asia', mode='lines+markers'))
prod_con_gap_asia = prod_con_gap_asia.add_trace(go.Scatter(x=total_all_prod_asia_sum.index, y=total_all_con_asia_sum['Total_Consumption_Asia'], name='Energy Consumption Asia', mode='lines+markers'))
prod_con_gap_asia.update_layout(title='Energy Production vs Energy Consumption Asia', template='plotly_dark')
prod_con_gap_asia.update_xaxes(rangeslider_visible=True)
prod_con_gap_asia.show()

In [547]:
total_all_prod_sum.head()

,Total_Production_Africa,Total_Production_Asia,Total_Production_Middle_East,Total_Production_Central_South_America,Total_Production_North_America,Total_Production_Europe,Total_Production_Eurasia,Total All,Africa_Portion,Asia_Portion,Middle_East_Portion,Central_South_America_Portion,North_America_Portion,Europe_Portion,Eurasia_Portion
1980,17.186180,37.327228,42.264127,12.133860,83.016301,42.908097,61.506642,296.342436,5.799433,12.595978,14.261922,4.094540,28.013639,14.479228,20.755260
1981,14.911990,38.286615,36.485050,12.334737,83.614179,43.827254,61.833941,291.293766,5.119227,13.143644,12.525174,4.234466,28.704418,15.045723,21.227348
1982,15.328102,39.484112,29.694303,12.214567,84.275328,45.268815,63.862977,290.128203,5.283217,13.609195,10.234890,4.210058,29.047617,15.603038,22.011985
1983,16.003652,42.398905,27.561035,12.490824,81.925271,47.266558,65.401473,293.047718,5.461108,14.468260,9.404965,4.262386,27.956291,16.129304,22.317687
1984,17.430642,46.334497,27.675984,13.523598,87.848759,47.881392,68.252011,308.946883,5.641954,14.997561,8.958169,4.377321,28.434907,15.498260,22.091827


### Regional Energy Analysis

#### Coal

In [548]:
con_coal = pd.read_csv('Consumption_Data/Consumption_Coal.csv')
con_coal.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
0,Africa,Algeria,0.002547398,0.007655064,0.020475801,0.033213824,0.034235723,0.034242132,0.034610211,0.035079318,...,0.001259621,0.000835328,0.000576922,0.000575219,0.000924225,0.000410418,0.000405436,0.000658437,0.00094109,0.594818
1,Africa,Angola,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,Africa,Benin,0,0,0,0,0,0,0,0,...,3.17E-05,0.000961001,0.001037911,0.001985751,0.002992505,0.002987991,0.002938642,0.001951128,0.003932631,0.019046
3,Africa,Botswana,0.008660526,0.009356617,0.010099701,0.00986717,0.009751175,0.010819193,0.01211937,0.012351123,...,0.014232074,0.016837001,0.027546501,0.038035577,0.035819574,0.033554487,0.031321952,0.029050749,0.032820821,0.808603
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [549]:
prod_coal = pd.read_csv('Production_Data/Production_Coal.csv')
prod_coal.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
0,Africa,Algeria,7.59E-05,0.000240414,0.000404907,0.0005694,0.000575727,0.000582054,0.000278374,0.000240414,...,0,0,0,0,0,0,0,0,0,0.000000
1,Africa,Angola,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Africa,Botswana,0.008262057,0.008484744,0.009241913,0.008796519,0.00875198,0.009731845,0.010912138,0.010845329,...,0.033315425,0.038212317,0.046534212,0.04192044,0.049608123,0.048393175,0.047178203,0.041939121,0.044340069,0.952191
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


#### Coal Production & Consumption by Regions

Africa

In [550]:
con_coal_africa = con_coal[con_coal['Continent']=='Africa']
con_coal_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
0,Africa,Algeria,0.002547398,0.007655064,0.020475801,0.033213824,0.034235723,0.034242132,0.034610211,0.035079318,...,0.001259621,0.000835328,0.000576922,0.000575219,0.000924225,0.000410418,0.000405436,0.000658437,0.00094109,0.594818
1,Africa,Angola,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,Africa,Benin,0,0,0,0,0,0,0,0,...,3.17E-05,0.000961001,0.001037911,0.001985751,0.002992505,0.002987991,0.002938642,0.001951128,0.003932631,0.019046
3,Africa,Botswana,0.008660526,0.009356617,0.010099701,0.00986717,0.009751175,0.010819193,0.01211937,0.012351123,...,0.014232074,0.016837001,0.027546501,0.038035577,0.035819574,0.033554487,0.031321952,0.029050749,0.032820821,0.808603
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [551]:

con_coal_africa_sum = con_coal_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_coal_africa_sum = con_coal_africa_sum.sum(axis=0)
con_coal_africa_sum = con_coal_africa_sum.transpose()
con_coal_africa_sum = pd.DataFrame(con_coal_africa_sum)
con_coal_africa_sum = con_coal_africa_sum.rename(columns={0:'Total_Coal_Consumption_Africa'})
con_coal_africa_sum.head()



,Total_Coal_Consumption_Africa
1980,2.106965
1981,2.296101
1982,2.464263
1983,2.598829
1984,2.763172


In [552]:
con_coal_africa_sum.columns

Index(['Total_Coal_Consumption_Africa'], dtype='object')

In [553]:
prod_coal_africa = prod_coal[prod_coal['Continent']=='Africa']
prod_coal_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
0,Africa,Algeria,7.59E-05,0.000240414,0.000404907,0.0005694,0.000575727,0.000582054,0.000278374,0.000240414,...,0,0,0,0,0,0,0,0,0,0.000000
1,Africa,Angola,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Africa,Botswana,0.008262057,0.008484744,0.009241913,0.008796519,0.00875198,0.009731845,0.010912138,0.010845329,...,0.033315425,0.038212317,0.046534212,0.04192044,0.049608123,0.048393175,0.047178203,0.041939121,0.044340069,0.952191
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [554]:
prod_coal_africa_sum = prod_coal_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_coal_africa_sum = prod_coal_africa_sum.sum(axis=0)
prod_coal_africa_sum = prod_coal_africa_sum.transpose()
prod_coal_africa_sum = pd.DataFrame(prod_coal_africa_sum)
prod_coal_africa_sum = prod_coal_africa_sum.rename(columns={0:'Total_Production_Africa'})
prod_coal_africa_sum.head()

,Total_Production_Africa
1980,2.644093
1981,2.914258
1982,3.154886
1983,3.308514
1984,3.695633


In [555]:
fig_coal_africa = go.Figure()
fig_coal_africa = fig_coal_africa.add_trace(go.Scatter(x=con_coal_africa_sum.index, y=con_coal_africa_sum['Total_Coal_Consumption_Africa'], name='Total Coal Consumption Africa'))
fig_coal_africa = fig_coal_africa.add_trace(go.Scatter(x=prod_coal_africa_sum.index, y=prod_coal_africa_sum['Total_Production_Africa'], name='Total Coal Production Africa'))
fig_coal_africa.update_layout(title='Total Coal Consumption Africa vs Total Coal Production Africa 1980 - 2021')
fig_coal_africa.show()

In [556]:
con_coal_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
0,Africa,Algeria,0.002547398,0.007655064,0.020475801,0.033213824,0.034235723,0.034242132,0.034610211,0.035079318,...,0.001259621,0.000835328,0.000576922,0.000575219,0.000924225,0.000410418,0.000405436,0.000658437,0.00094109,0.594818
1,Africa,Angola,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,Africa,Benin,0,0,0,0,0,0,0,0,...,3.17E-05,0.000961001,0.001037911,0.001985751,0.002992505,0.002987991,0.002938642,0.001951128,0.003932631,0.019046
3,Africa,Botswana,0.008660526,0.009356617,0.010099701,0.00986717,0.009751175,0.010819193,0.01211937,0.012351123,...,0.014232074,0.016837001,0.027546501,0.038035577,0.035819574,0.033554487,0.031321952,0.029050749,0.032820821,0.808603
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [557]:
con_coal_africa_sum.head()

,Total_Coal_Consumption_Africa
1980,2.106965
1981,2.296101
1982,2.464263
1983,2.598829
1984,2.763172


In [558]:
top_coal_consumption_africa_1980 = con_coal_africa[['Country','1980']]
top_coal_consumption_africa_1980['1980'] = pd.to_numeric(top_coal_consumption_africa_1980['1980'], errors='coerce')
top_coal_consumption_africa_1980 = top_coal_consumption_africa_1980.reset_index(drop=True).sort_values(by='1980', ascending=False, na_position='last')
top_coal_consumption_africa_1980['portion_africa'] = top_coal_consumption_africa_1980['1980']/con_coal_africa_sum.loc['1980'][0]*100
top_coal_consumption_africa_1980['cumulative'] = top_coal_consumption_africa_1980['portion_africa'].cumsum()
top_coal_consumption_africa_1980.head()

,Country,1980,portion_africa,cumulative
47,South Africa,1.965977,93.308472,93.308472
56,Zimbabwe,0.066210,3.142439,96.450910
15,Egypt,0.022549,1.070200,97.521111
55,Zambia,0.015137,0.718422,98.239532
34,Morocco,0.011864,0.563098,98.802630


In [559]:
top_coal_consumption_africa_1990 = con_coal_africa[['Country','1990']]
top_coal_consumption_africa_1990['1990'] = pd.to_numeric(top_coal_consumption_africa_1990['1990'], errors='coerce')
top_coal_consumption_africa_1990 = top_coal_consumption_africa_1990.reset_index(drop=True).sort_values(by='1990', ascending=False, na_position='last')
top_coal_consumption_africa_1990['portion_africa'] = top_coal_consumption_africa_1990['1990']/con_coal_africa_sum.loc['1990'][0]*100
top_coal_consumption_africa_1990['cumulative'] = top_coal_consumption_africa_1990['portion_africa'].cumsum()
top_coal_consumption_africa_1990.head()

,Country,1990,portion_africa,cumulative
47,South Africa,2.940747,91.461518,91.461518
56,Zimbabwe,0.121120,3.767012,95.228530
34,Morocco,0.042162,1.311311,96.539841
0,Algeria,0.033572,1.044140,97.583981
15,Egypt,0.031442,0.977878,98.561859


In [560]:
top_coal_consumption_africa_2000 = con_coal_africa[['Country','2000']]
top_coal_consumption_africa_2000['2000'] = pd.to_numeric(top_coal_consumption_africa_2000['2000'], errors='coerce')
top_coal_consumption_africa_2000 = top_coal_consumption_africa_2000.reset_index(drop=True).sort_values(by='2000', ascending=False, na_position='last')
top_coal_consumption_africa_2000['portion_africa'] = top_coal_consumption_africa_2000['2000']/con_coal_africa_sum.loc['2000'][0]*100
top_coal_consumption_africa_2000['cumulative'] = top_coal_consumption_africa_2000['portion_africa'].cumsum()
top_coal_consumption_africa_2000.head()

,Country,2000,portion_africa,cumulative
47,South Africa,3.447035,91.774944,91.774944
56,Zimbabwe,0.109078,2.904130,94.679074
34,Morocco,0.095294,2.537135,97.216208
15,Egypt,0.042705,1.137000,98.353208
3,Botswana,0.020854,0.555222,98.908430


In [561]:
top_coal_consumption_africa_2010 = con_coal_africa[['Country','2010']]
top_coal_consumption_africa_2010['2010'] = pd.to_numeric(top_coal_consumption_africa_2010['2010'], errors='coerce')
top_coal_consumption_africa_2010 = top_coal_consumption_africa_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_coal_consumption_africa_2010['portion_africa'] = top_coal_consumption_africa_2010['2010']/con_coal_africa_sum.loc['2010'][0]*100
top_coal_consumption_africa_2010['cumulative'] = top_coal_consumption_africa_2010['portion_africa'].cumsum()
top_coal_consumption_africa_2010.head()

,Country,2010,portion_africa,cumulative
47,South Africa,4.101369,93.044677,93.044677
34,Morocco,0.126385,2.867204,95.911881
56,Zimbabwe,0.075364,1.709726,97.621606
15,Egypt,0.040996,0.930046,98.551652
33,Mauritius,0.016553,0.375515,98.927167


In [562]:
top_coal_consumption_africa_2020 = con_coal_africa[['Country','2020']]
top_coal_consumption_africa_2020['2020'] = pd.to_numeric(top_coal_consumption_africa_2020['2020'], errors='coerce')
top_coal_consumption_africa_2020 = top_coal_consumption_africa_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_coal_consumption_africa_2020['portion_africa'] = top_coal_consumption_africa_2020['2020']/con_coal_africa_sum.loc['2020'][0]*100
top_coal_consumption_africa_2020['cumulative'] = top_coal_consumption_africa_2020['portion_africa'].cumsum()
top_coal_consumption_africa_2020.head()

,Country,2020,portion_africa,cumulative
47,South Africa,4.091200,89.150725,89.150725
34,Morocco,0.226368,4.932755,94.083480
56,Zimbabwe,0.052825,1.151108,95.234588
15,Egypt,0.048482,1.056467,96.291055
3,Botswana,0.029051,0.633041,96.924095


In [563]:
top_coal_production_africa_1980 = prod_coal_africa[['Country','1980']]
top_coal_production_africa_1980['1980'] = pd.to_numeric(top_coal_production_africa_1980['1980'], errors='coerce')
top_coal_production_africa_1980 = top_coal_production_africa_1980.reset_index(drop=True).sort_values(by='1980', ascending=False, na_position='last')
top_coal_production_africa_1980['portion_africa'] = top_coal_production_africa_1980['1980']/prod_coal_africa_sum.loc['1980'][0]*100
top_coal_production_africa_1980['cumulative'] = top_coal_production_africa_1980['portion_africa'].cumsum()
top_coal_production_africa_1980.head()

,Country,1980,portion_africa,cumulative
47,South Africa,2.525071,95.498583,95.498583
56,Zimbabwe,0.068388,2.586447,98.085030
34,Morocco,0.017209,0.650830,98.735860
55,Zambia,0.013941,0.527247,99.263106
3,Botswana,0.008262,0.312472,99.575579


In [564]:
top_coal_production_africa_1990 = prod_coal_africa[['Country','1990']]
top_coal_production_africa_1990['1990'] = pd.to_numeric(top_coal_production_africa_1990['1990'], errors='coerce')
top_coal_production_africa_1990 = top_coal_production_africa_1990.reset_index(drop=True).sort_values(by='1990', ascending=False, na_position='last')
top_coal_production_africa_1990['portion_africa'] = top_coal_production_africa_1990['1990']/prod_coal_africa_sum.loc['1990'][0]*100
top_coal_production_africa_1990['cumulative'] = top_coal_production_africa_1990['portion_africa'].cumsum()
top_coal_production_africa_1990.head()

,Country,1990,portion_africa,cumulative
47,South Africa,3.770739,95.148211,95.148211
56,Zimbabwe,0.135985,3.431359,98.579571
3,Botswana,0.017682,0.446178,99.025749
34,Morocco,0.013311,0.335888,99.361637
55,Zambia,0.011157,0.281522,99.643160


In [565]:
top_coal_production_africa_2000 = prod_coal_africa[['Country','2000']]
top_coal_production_africa_2000['2000'] = pd.to_numeric(top_coal_production_africa_2000['2000'], errors='coerce')
top_coal_production_africa_2000 = top_coal_production_africa_2000.reset_index(drop=True).sort_values(by='2000', ascending=False, na_position='last')
top_coal_production_africa_2000['portion_africa'] = top_coal_production_africa_2000['2000']/prod_coal_africa_sum.loc['2000'][0]*100
top_coal_production_africa_2000['cumulative'] = top_coal_production_africa_2000['portion_africa'].cumsum()
top_coal_production_africa_2000.head()

,Country,2000,portion_africa,cumulative
47,South Africa,4.929281,96.968519,96.968519
56,Zimbabwe,0.108709,2.138521,99.107040
3,Botswana,0.020933,0.411802,99.518842
18,Eswatini,0.008946,0.175990,99.694832
55,Zambia,0.005554,0.109253,99.804085


In [566]:
top_coal_production_africa_2010 = prod_coal_africa[['Country','2010']]
top_coal_production_africa_2010['2010'] = pd.to_numeric(top_coal_production_africa_2010['2010'], errors='coerce')
top_coal_production_africa_2010 = top_coal_production_africa_2010.reset_index(drop=True).sort_values(by='2010', ascending=False, na_position='last')
top_coal_production_africa_2010['portion_africa'] = top_coal_production_africa_2010['2010']/prod_coal_africa_sum.loc['2010'][0]*100
top_coal_production_africa_2010['cumulative'] = top_coal_production_africa_2010['portion_africa'].cumsum()
top_coal_production_africa_2010.head()

,Country,2010,portion_africa,cumulative
47,South Africa,5.579088,98.247691,98.247691
56,Zimbabwe,0.061772,1.087796,99.335488
3,Botswana,0.022002,0.387463,99.722950
37,Niger,0.004810,0.084700,99.807650
18,Eswatini,0.003379,0.059509,99.867159


In [567]:
top_coal_production_africa_2020 = prod_coal_africa[['Country','2020']]
top_coal_production_africa_2020['2020'] = pd.to_numeric(top_coal_production_africa_2020['2020'], errors='coerce')
top_coal_production_africa_2020 = top_coal_production_africa_2020.reset_index(drop=True).sort_values(by='2020', ascending=False, na_position='last')
top_coal_production_africa_2020['portion_africa'] = top_coal_production_africa_2020['2020']/prod_coal_africa_sum.loc['2020'][0]*100
top_coal_production_africa_2020['cumulative'] = top_coal_production_africa_2020['portion_africa'].cumsum()
top_coal_production_africa_2020.head()

,Country,2020,portion_africa,cumulative
47,South Africa,5.394765,93.881606,93.881606
35,Mozambique,0.187149,3.256835,97.138441
56,Zimbabwe,0.066801,1.162493,98.300934
3,Botswana,0.041939,0.729839,99.030773
55,Zambia,0.027466,0.477972,99.508746


In [568]:
prod_coal_africa_sum.head()

,Total_Production_Africa
1980,2.644093
1981,2.914258
1982,3.154886
1983,3.308514
1984,3.695633


#### Trial Start

#### Trial End

Asia & Ocenia

In [569]:
con_coal_asia = con_coal[con_coal['Continent']=='Asia & Oceania']
con_coal_asia.head()



,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
57,Asia & Oceania,Afghanistan,0.002479248,0.002604252,0.003020933,0.003020933,0.003083435,0.003145937,0.003333443,0.003479281,...,0.017724885,0.014260951,0.013874091,0.019768281,0.021566236,0.023364191,0.025162146,0.020746989,0.017505365,0.263517
58,Asia & Oceania,American Samoa,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
59,Asia & Oceania,Australia,1.287101904,1.317761034,1.394638904,1.325104889,1.267020237,1.362507004,1.354289254,1.488758228,...,1.818969993,1.726155494,1.777427343,1.851000109,1.834907654,1.750054627,1.69693128,1.602947598,1.670752446,73.907873
60,Asia & Oceania,Bangladesh,0.00048534,0.000508058,0.000619583,0.00033664,0.001280471,0.001085397,0.001240119,0.001373042,...,0.054883391,0.07062486,0.084577166,0.085734503,0.101429138,0.110142178,0.124351279,0.144656846,0.166928096,1.487205
61,Asia & Oceania,Bhutan,1.72E-05,1.81E-05,1.91E-05,2.00E-05,1.94E-05,2.07E-05,2.66E-05,0.000314929,...,0.001558439,0.001651907,0.001728535,0.002172304,0.003089036,0.003347667,0.00337493,0.003243704,0.003777277,0.051638


In [570]:
con_coal_asia_sum = con_coal_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_coal_asia_sum = con_coal_asia_sum.sum(axis=0)
con_coal_asia_sum = con_coal_asia_sum.transpose()
con_coal_asia_sum = pd.DataFrame(con_coal_asia_sum)
con_coal_asia_sum = con_coal_asia_sum.rename(columns={0:'Total_Coal_Consumption_Asia'})
con_coal_asia_sum.head()

,Total_Coal_Consumption_Asia
1980,21.191995
1981,21.812177
1982,23.192326
1983,24.446940
1984,26.824249


In [571]:
prod_coal_asia = prod_coal[prod_coal['Continent']=='Asia & Oceania']
prod_coal_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
57,Asia & Oceania,Afghanistan,0.002355286,0.00247404,0.002869886,0.002869886,0.002929263,0.00298864,0.003166771,0.003305317,...,0.026660219,0.030032826,0.027016481,0.033607314,0.043364915,0.048392158,0.042335716,0.001128161,0.03195941,0.410733
58,Asia & Oceania,American Samoa,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
59,Asia & Oceania,Australia,2.374452858,2.699360224,2.733467786,3.041359368,3.244336768,3.497189795,3.765758866,4.247154444,...,10.7738122,11.58072908,12.09463795,11.81982682,11.80927495,11.59034777,12.16039616,11.88807402,11.06245749,299.184514
60,Asia & Oceania,Bangladesh,0,0,0,0,0,0,0,0,...,0.017850598,0.019771364,0.014113455,0.021337206,0.024239233,0.019270294,0.019548666,0.021211938,0.021951944,0.276751
61,Asia & Oceania,Bhutan,8.29E-07,1.49E-06,2.16E-06,2.82E-06,6.64E-06,6.67E-06,1.33E-05,3.04E-05,...,0.0012896,0.00135839,0.001412681,0.001953708,0.00268722,0.003101914,0.003068739,0.002609357,0.003085327,0.045687


In [572]:
prod_coal_asia_sum = prod_coal_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_coal_asia_sum = prod_coal_asia_sum.sum(axis=0)
prod_coal_asia_sum = prod_coal_asia_sum.transpose()
prod_coal_asia_sum = pd.DataFrame(prod_coal_asia_sum)
prod_coal_asia_sum = prod_coal_asia_sum.rename(columns={0:'Total_Production_Asia'})
prod_coal_asia_sum.head()

,Total_Production_Asia
1980,20.250933
1981,20.834413
1982,22.087943
1983,23.581426
1984,25.692378


In [573]:
fig_coal_asia = go.Figure()
fig_coal_asia = fig_coal_asia.add_trace(go.Scatter(x=con_coal_asia_sum.index, y=con_coal_asia_sum['Total_Coal_Consumption_Asia'], name='Total Coal Consumption Asia'))  
fig_coal_asia = fig_coal_asia.add_trace(go.Scatter(x=prod_coal_asia_sum.index, y=prod_coal_asia_sum['Total_Production_Asia'], name='Total Coal Production Asia'))
fig_coal_asia.update_layout(title='Total Coal Consumption Asia vs Total Coal Production Asia 1980 - 2021')
fig_coal_asia.show()

In [574]:
top_coal_consumption_asia_1980 = con_coal_asia[['Country','1980']]
top_coal_consumption_asia_1980['1980'] = pd.to_numeric(top_coal_consumption_asia_1980['1980'], errors='coerce')
top_coal_consumption_asia_1980 = top_coal_consumption_asia_1980.reset_index(drop=True).sort_values(by='1980', ascending=False, na_position='last')
top_coal_consumption_asia_1980['portion_asia'] = top_coal_consumption_asia_1980['1980']/prod_coal_asia_sum.loc['1980'][0]*100
top_coal_consumption_asia_1980['cumulative'] = top_coal_consumption_asia_1980['portion_asia'].cumsum()
top_coal_consumption_asia_1980.head()

,Country,1980,portion_asia,cumulative
8,China,13.809654,68.192680,68.192680
17,Japan,2.396834,11.835673,80.028354
15,India,1.824435,9.009140,89.037493
2,Australia,1.287102,6.355766,95.393259
30,North Korea,0.801533,3.958006,99.351265


In [575]:
top_coal_consumption_asia_1990 = con_coal_asia[['Country','1990']]
top_coal_consumption_asia_1990['1990'] = pd.to_numeric(top_coal_consumption_asia_1990['1990'], errors='coerce')
top_coal_consumption_asia_1990 = top_coal_consumption_asia_1990.reset_index(drop=True).sort_values(by='1990', ascending=False, na_position='last')
top_coal_consumption_asia_1990['portion_asia'] = top_coal_consumption_asia_1990['1990']/prod_coal_asia_sum.loc['1990'][0]*100
top_coal_consumption_asia_1990['cumulative'] = top_coal_consumption_asia_1990['portion_asia'].cumsum()
top_coal_consumption_asia_1990.head()

,Country,1990,portion_asia,cumulative
8,China,21.620772,62.719828,62.719828
15,India,3.715616,10.778653,73.498481
17,Japan,3.252765,9.435967,82.934447
2,Australia,1.582636,4.591077,87.525525
38,South Korea,1.181529,3.427506,90.953030


In [576]:
top_coal_consumption_asia_2000 = con_coal_asia[['Country','2000']]
top_coal_consumption_asia_2000['2000'] = pd.to_numeric(top_coal_consumption_asia_2000['2000'], errors='coerce')
top_coal_consumption_asia_2000 = top_coal_consumption_asia_2000.reset_index(drop=True).sort_values(by='2000', ascending=False, na_position='last')
top_coal_consumption_asia_2000['portion_asia'] = top_coal_consumption_asia_2000['2000']/prod_coal_asia_sum.loc['2000'][0]*100
top_coal_consumption_asia_2000['cumulative'] = top_coal_consumption_asia_2000['portion_asia'].cumsum()
top_coal_consumption_asia_2000.head()


,Country,2000,portion_asia,cumulative
8,China,29.868460,64.630851,64.630851
15,India,5.726695,12.391706,77.022557
17,Japan,3.941798,8.529457,85.552014
2,Australia,2.024106,4.379861,89.931875
38,South Korea,1.821616,3.941703,93.873578


In [577]:
top_coal_consumption_asia_2010 = con_coal_asia[['Country','2010']]
top_coal_consumption_asia_2010['2010'] = pd.to_numeric(top_coal_consumption_asia_2010['2010'], errors='coerce')
top_coal_consumption_asia_2010 = top_coal_consumption_asia_2010.reset_index(drop=True).sort_values(by='2010', ascending=False, na_position='last')
top_coal_consumption_asia_2010['portion_asia'] = top_coal_consumption_asia_2010['2010']/prod_coal_asia_sum.loc['2010'][0]*100
top_coal_consumption_asia_2010['cumulative'] = top_coal_consumption_asia_2010['portion_asia'].cumsum()
top_coal_consumption_asia_2010.head()


,Country,2010,portion_asia,cumulative
8,China,81.119270,77.035835,77.035835
15,India,11.430284,10.854899,87.890733
17,Japan,4.686440,4.450531,92.341264
38,South Korea,3.025770,2.873457,95.214721
2,Australia,2.070844,1.966601,97.181322


In [578]:
top_coal_consumption_asia_2020 = con_coal_asia[['Country','2020']]
top_coal_consumption_asia_2020['2020'] = pd.to_numeric(top_coal_consumption_asia_2020['2020'], errors='coerce')
top_coal_consumption_asia_2020 = top_coal_consumption_asia_2020.reset_index(drop=True).sort_values(by='2020', ascending=False, na_position='last')
top_coal_consumption_asia_2020['portion_asia'] = top_coal_consumption_asia_2020['2020']/prod_coal_asia_sum.loc['2020'][0]*100
top_coal_consumption_asia_2020['cumulative'] = top_coal_consumption_asia_2020['portion_asia'].cumsum()
top_coal_consumption_asia_2020.head()

,Country,2020,portion_asia,cumulative
8,China,91.908036,71.848766,71.848766
15,India,15.485358,12.105621,83.954387
17,Japan,4.633663,3.622349,87.576735
38,South Korea,3.200547,2.502016,90.078751
16,Indonesia,2.639182,2.063171,92.141922


In [579]:
top_production_coal_asia_1980 = prod_coal_asia[['Country','1980']]
top_production_coal_asia_1980['1980'] = pd.to_numeric(top_production_coal_asia_1980['1980'], errors='coerce')
top_production_coal_asia_1980 = top_production_coal_asia_1980.reset_index(drop=True).sort_values(by='1980', ascending=False, na_position='last')
top_production_coal_asia_1980['portion_asia'] = top_production_coal_asia_1980['1980']/prod_coal_asia_sum.loc['1980'][0]*100
top_production_coal_asia_1980['cumulative'] = top_production_coal_asia_1980['portion_asia'].cumsum()
top_production_coal_asia_1980.head()

,Country,1980,portion_asia,cumulative
8,China,13.811305,68.200833,68.200833
2,Australia,2.374453,11.725153,79.925985
15,India,1.873129,9.249593,89.175578
30,North Korea,0.795598,3.928699,93.104277
38,South Korea,0.483504,2.387565,95.491842


In [580]:
top_production_coal_asia_1990 = prod_coal_asia[['Country','1990']]
top_production_coal_asia_1990['1990'] = pd.to_numeric(top_production_coal_asia_1990['1990'], errors='coerce')
top_production_coal_asia_1990 = top_production_coal_asia_1990.reset_index(drop=True).sort_values(by='1990', ascending=False, na_position='last')
top_production_coal_asia_1990['portion_asia'] = top_production_coal_asia_1990['1990']/prod_coal_asia_sum.loc['1990'][0]*100
top_production_coal_asia_1990['cumulative'] = top_production_coal_asia_1990['portion_asia'].cumsum()
top_production_coal_asia_1990.head()

,Country,1990,portion_asia,cumulative
8,China,24.037381,69.730185,69.730185
2,Australia,4.388971,12.731993,82.462178
15,India,3.699749,10.732624,93.194802
30,North Korea,0.871294,2.527543,95.722345
38,South Korea,0.446953,1.296568,97.018913


In [581]:
top_production_coal_asia_2000 = prod_coal_asia[['Country','2000']]
top_production_coal_asia_2000['2000'] = pd.to_numeric(top_production_coal_asia_2000['2000'], errors='coerce')
top_production_coal_asia_2000 = top_production_coal_asia_2000.reset_index(drop=True).sort_values(by='2000', ascending=False, na_position='last')
top_production_coal_asia_2000['portion_asia'] = top_production_coal_asia_2000['2000']/prod_coal_asia_sum.loc['2000'][0]*100
top_production_coal_asia_2000['cumulative'] = top_production_coal_asia_2000['portion_asia'].cumsum()
top_production_coal_asia_2000.head()

,Country,2000,portion_asia,cumulative
8,China,30.827508,66.706087,66.706087
2,Australia,6.627498,14.340909,81.046996
15,India,5.396460,11.677128,92.724123
16,Indonesia,1.774650,3.840075,96.564198
30,North Korea,0.590436,1.277614,97.841813


In [582]:
top_production_coal_asia_2010 = prod_coal_asia[['Country','2010']]
top_production_coal_asia_2010['2010'] = pd.to_numeric(top_production_coal_asia_2010['2010'], errors='coerce')
top_production_coal_asia_2010 = top_production_coal_asia_2010.reset_index(drop=True).sort_values(by='2010', ascending=False, na_position='last')
top_production_coal_asia_2010['portion_asia'] = top_production_coal_asia_2010['2010']/prod_coal_asia_sum.loc['2010'][0]*100
top_production_coal_asia_2010['cumulative'] = top_production_coal_asia_2010['portion_asia'].cumsum()
top_production_coal_asia_2010.head()

,Country,2010,portion_asia,cumulative
8,China,76.355779,72.512131,72.512131
2,Australia,9.954672,9.453567,81.965698
15,India,8.618543,8.184697,90.150395
16,Indonesia,7.345464,6.975703,97.126098
47,Vietnam,1.179894,1.120500,98.246598


In [583]:
top_production_coal_asia_2020 = prod_coal_asia[['Country','2020']]
top_production_coal_asia_2020['2020'] = pd.to_numeric(top_production_coal_asia_2020['2020'], errors='coerce')
top_production_coal_asia_2020 = top_production_coal_asia_2020.reset_index(drop=True).sort_values(by='2020', ascending=False, na_position='last')
top_production_coal_asia_2020['portion_asia'] = top_production_coal_asia_2020['2020']/prod_coal_asia_sum.loc['2020'][0]*100
top_production_coal_asia_2020['cumulative'] = top_production_coal_asia_2020['portion_asia'].cumsum()
top_production_coal_asia_2020.head()

,Country,2020,portion_asia,cumulative
8,China,88.852984,69.460491,69.460491
15,India,12.161194,9.506968,78.967459
2,Australia,11.888074,9.293458,88.260917
16,Indonesia,11.105554,8.681725,96.942642
47,Vietnam,1.155147,0.903032,97.845674


Middle East

In [584]:
con_coal_me = con_coal[con_coal['Continent']=='Middle East']
con_coal_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
106,Middle East,Bahrain,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
107,Middle East,Iran,0.049958695,0.041720134,0.056283351,0.053104368,0.041069692,0.040411672,0.03978632,0.03912402,...,0.031418175,0.0316039,0.035316046,0.038807489,0.04353957,0.043157399,0.042745331,0.04269698,0.040368645,1.767115
108,Middle East,Iraq,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
109,Middle East,Israel,0,0,0.022275342,0.041788044,0.066651806,0.072849079,0.080066788,0.085293405,...,0.291769648,0.273641401,0.267863291,0.228919784,0.20693284,0.195036992,0.146335553,0.126305367,0.145939376,8.102916
110,Middle East,Jordan,0,0,0,0,0,0,0,0,...,0.004934903,0.00518687,0.004684798,0.004176032,0.003203518,0.003032923,0.002089984,0.00400465,0.003542801,0.043342


In [585]:
con_coal_me_sum = con_coal_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_coal_me_sum = con_coal_me_sum.sum(axis=0)
con_coal_me_sum = con_coal_me_sum.transpose()
con_coal_me_sum = pd.DataFrame(con_coal_me_sum)
con_coal_me_sum = con_coal_me_sum.rename(columns={0:'Total_Coal_Consumption_Middle_East'})
con_coal_me_sum.head()

,Total_Coal_Consumption_Middle_East
1980,0.049959
1981,0.041720
1982,0.078559
1983,0.094892
1984,0.107721


In [586]:
prod_coal_me = prod_coal[prod_coal['Continent']=='Middle East']
prod_coal_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
106,Middle East,Bahrain,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
107,Middle East,Iran,0.025125988,0.02520971,0.028358455,0.028665823,0.02900106,0.030900937,0.035283941,0.030878049,...,0.030834363,0.029920444,0.033670016,0.037270719,0.044049648,0.047822611,0.042293513,0.043038724,0.045829006,1.390378
108,Middle East,Iraq,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
109,Middle East,Israel,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
110,Middle East,Jordan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [587]:
prod_coal_me_sum = prod_coal_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_coal_me_sum = prod_coal_me_sum.sum(axis=0)
prod_coal_me_sum = prod_coal_me_sum.transpose()
prod_coal_me_sum = pd.DataFrame(prod_coal_me_sum)
prod_coal_me_sum = prod_coal_me_sum.rename(columns={0:'Total_Production_Middle_East'})
prod_coal_me_sum.head()

,Total_Production_Middle_East
1980,0.025126
1981,0.025210
1982,0.028358
1983,0.028666
1984,0.029001


In [588]:
fig_coal_me = go.Figure()
fig_coal_me = fig_coal_me.add_trace(go.Scatter(x=con_coal_me_sum.index, y=con_coal_me_sum['Total_Coal_Consumption_Middle_East'], name='Total Coal Consumption Middle East'))
fig_coal_me = fig_coal_me.add_trace(go.Scatter(x=prod_coal_me_sum.index, y=prod_coal_me_sum['Total_Production_Middle_East'], name='Total Coal Production Middle East')) 
fig_coal_me.update_layout(title='Total Coal Consumption Middle East vs Total Coal Production Middle East 1980 - 2021')
fig_coal_me.show()

Central & South America

In [589]:
con_coal_csa = con_coal[con_coal['Continent']=='Central & South America']
con_coal_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
120,Central & South America,Antarctica,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
121,Central & South America,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
122,Central & South America,Argentina,0.034095908,0.030519097,0.031685348,0.02887245,0.023295615,0.027976784,0.036422449,0.038287703,...,0.04538628,0.0433368,0.040984481,0.039407759,0.039504321,0.031761984,0.021913297,0.025146053,0.033276501,1.419933
123,Central & South America,Aruba,--,--,--,--,--,--,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
124,Central & South America,Barbados,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [590]:
con_coal_csa_sum = con_coal_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_coal_csa_sum = con_coal_csa_sum.sum(axis=0)
con_coal_csa_sum = con_coal_csa_sum.transpose()
con_coal_csa_sum = pd.DataFrame(con_coal_csa_sum)
con_coal_csa_sum = con_coal_csa_sum.rename(columns={0:'Total_Coal_Consumption_Central_South_America'})
con_coal_csa_sum.head()

,Total_Coal_Consumption_Central_South_America
1980,0.379655
1981,0.386823
1982,0.383407
1983,0.424763
1984,0.499408


In [591]:
prod_coal_csa = prod_coal[prod_coal['Continent']=='Central & South America']
prod_coal_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
120,Central & South America,Antarctica,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
121,Central & South America,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
122,Central & South America,Argentina,0.008734745,0.011153597,0.011534342,0.010884836,0.011399962,0.008958713,0.008152428,0.008353999,...,0.001769346,0.001276617,0.000761491,0.000515126,0.000441217,0.000531085,0.000492729,0.000443972,0.000457588,0.203232
123,Central & South America,Aruba,--,--,--,--,--,--,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
124,Central & South America,Barbados,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [592]:
prod_coal_csa_sum = prod_coal_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_coal_csa_sum = prod_coal_csa_sum.sum(axis=0)
prod_coal_csa_sum = prod_coal_csa_sum.transpose()
prod_coal_csa_sum = pd.DataFrame(prod_coal_csa_sum)
prod_coal_csa_sum = prod_coal_csa_sum.rename(columns={0:'Total_Production_Central_South_America'})
prod_coal_csa_sum.head()

,Total_Production_Central_South_America
1980,0.220723
1981,0.234615
1982,0.252328
1983,0.268962
1984,0.325556


In [593]:
fig_coal_csa = go.Figure()
fig_coal_csa = fig_coal_csa.add_trace(go.Scatter(x=con_coal_csa_sum.index, y=con_coal_csa_sum['Total_Coal_Consumption_Central_South_America'], name='Total Coal Consumption Central South America'))
fig_coal_csa = fig_coal_csa.add_trace(go.Scatter(x=prod_coal_csa_sum.index, y=prod_coal_csa_sum['Total_Production_Central_South_America'], name='Total Coal Production Central South America'))
fig_coal_csa.update_layout(title='Total Coal Consumption Central South America vs Total Coal Production Central South America 1980 - 2021')
fig_coal_csa.show()

North America

In [594]:
con_coal_na = con_coal[con_coal['Continent']=='North America']
con_coal_na.tail()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
167,North America,Greenland,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
168,North America,Mexico,0.088556325,0.078430665,0.093431144,0.107056502,0.106333979,0.114127198,0.127087743,0.122640476,...,0.537472337,0.516109708,0.56117679,0.499536498,0.518771153,0.43680597,0.416172748,0.296681606,0.193315285,13.007324
169,North America,Saint Pierre and Miquelon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
170,North America,U.S. Territories,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
171,North America,United States,15.42280949,15.90752642,15.3215813,15.8944418,17.07062199,17.47842765,17.26040504,18.00845072,...,18.03863314,17.99763202,15.54887019,14.22590456,13.83746526,13.25153239,11.31559469,9.181109147,10.54754993,776.197063


In [595]:
con_coal_na_sum = con_coal_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_coal_na_sum = con_coal_na_sum.sum(axis=0)
con_coal_na_sum = con_coal_na_sum.transpose()
con_coal_na_sum = pd.DataFrame(con_coal_na_sum)
con_coal_na_sum = con_coal_na_sum.rename(columns={0:'Total_Coal_Consumption_North_America'})
con_coal_na_sum.head()

,Total_Coal_Consumption_North_America
1980,16.335291
1981,16.840399
1982,16.327384
1983,16.913976
1984,18.220474


In [596]:
prod_coal_na = prod_coal[prod_coal['Continent']=='North America']
prod_coal_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
165,North America,Bermuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
166,North America,Canada,0.779975237,0.849124614,0.904434344,0.936186495,1.22511752,1.300619853,1.207520596,1.293682458,...,1.480125647,1.416396127,1.285989862,1.22024172,1.291085727,1.217667588,1.160431594,1.003463954,1.045295139,55.915781
167,North America,Greenland,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
168,North America,Mexico,0.065222808,0.064230425,0.077236981,0.098098144,0.106881792,0.109647797,0.118769277,0.131247969,...,0.320666491,0.323819965,0.241205327,0.290725037,0.276650911,0.252953928,0.207907905,0.180383683,0.173279657,8.938013
169,North America,Saint Pierre and Miquelon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [597]:
prod_coal_na_sum = prod_coal_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_coal_na_sum = prod_coal_na_sum.sum(axis=0)
prod_coal_na_sum = prod_coal_na_sum.transpose()
prod_coal_na_sum = pd.DataFrame(prod_coal_na_sum)
prod_coal_na_sum = prod_coal_na_sum.rename(columns={0:'Total_Production_North_America'})
prod_coal_na_sum.head()

,Total_Production_North_America
1980,19.442924
1981,19.290128
1982,19.620444
1983,18.280955
1984,21.051215


In [598]:
fig_coal_na = go.Figure()
fig_coal_na = fig_coal_na.add_trace(go.Scatter(x=con_coal_na_sum.index, y=con_coal_na_sum['Total_Coal_Consumption_North_America'], name='Total Coal Consumption North America'))
fig_coal_na = fig_coal_na.add_trace(go.Scatter(x=prod_coal_na_sum.index, y=prod_coal_na_sum['Total_Production_North_America'], name='Total Coal Production North America'))
fig_coal_na = fig_coal_na.update_layout(title='Total Coal Consumption North America vs Total Coal Production North America 1980 - 2021')      
fig_coal_na.show()

Europe

In [599]:
con_coal_europe = con_coal[con_coal['Continent']=='Europe']
con_coal_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
172,Europe,Albania,0.024317315,0.025770486,0.029070075,0.031462709,0.033997866,0.036247979,0.035668553,0.037287487,...,0.00264252,0.003339684,0.002822255,0.002255383,0.00260389,0.003414521,0.00409317,0.004619879,0.007010067,0.456742
173,Europe,Austria,0.100247518,0.104601612,0.094376028,0.102594385,0.123716691,0.117533867,0.094748649,0.100352987,...,0.081812187,0.077223866,0.091476382,0.085730542,0.084691891,0.080961855,0.07779845,0.069798919,0.068126869,4.009763
174,Europe,Belgium,0.463628179,0.457563799,0.448400885,0.409855165,0.455290636,0.430400001,0.385421605,0.396197842,...,0.143667105,0.127233439,0.116802435,0.104282352,0.100496045,0.100796905,0.101275435,0.081919799,0.088284168,11.414849
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0.130475952,0.135798872,0.134451995,0.136395113,0.145681483,0.141078266,0.126301872,0.12840964,0.145326005,3.058326
176,Europe,Bulgaria,0.37598987,0.3749737,0.396440831,0.404166633,0.404966715,0.409436424,0.414353109,0.42763979,...,0.243146679,0.261017403,0.268744703,0.232931074,0.254385034,0.229086069,0.208471842,0.168248265,0.210162607,12.580327


In [600]:
con_coal_europe_sum = con_coal_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_coal_europe_sum = con_coal_europe_sum.sum(axis=0)
con_coal_europe_sum = con_coal_europe_sum.transpose()
con_coal_europe_sum = pd.DataFrame(con_coal_europe_sum)
con_coal_europe_sum = con_coal_europe_sum.rename(columns={0:'Total_Coal_Consumption_Europe'})
con_coal_europe_sum.head()

,Total_Coal_Consumption_Europe
1980,22.720847
1981,22.652893
1982,23.006503
1983,23.193848
1984,23.167302


In [601]:
prod_coal_europe = prod_coal[prod_coal['Continent']=='Europe']
prod_coal_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
172,Europe,Albania,0.013229039,0.014367221,0.015859919,0.017259324,0.018752022,0.020058133,0.020216733,0.019908863,...,8.40E-05,0.000155334,0.000226703,0.000130611,0.00042915,0.000914277,0.001135556,0.001184828,0.002246032,0.230346
173,Europe,Austria,0.039374646,0.042068339,0.045311765,0.041793472,0.040240476,0.042343206,0.040803952,0.038288923,...,0,0,0,0,0,0,0,0,0,0.668599
174,Europe,Belgium,0.168837875,0.164852877,0.174364969,0.162370358,0.16793091,0.165432846,0.148844265,0.115002495,...,0,0,0,0,0,0,0,0,0,1.427172
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0.090430188,0.090572066,0.08892256,0.094093162,0.104283154,0.104076723,0.096884826,0.092667414,0.098401056,2.517485
176,Europe,Bulgaria,0.210907655,0.204122422,0.224883001,0.226104622,0.234942178,0.215563777,0.245880923,0.257022108,...,0.199794393,0.213287847,0.243602608,0.212256479,0.23682489,0.211202881,0.195414221,0.155619282,0.197428505,8.791081


In [602]:
prod_coal_europe_sum = prod_coal_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_coal_europe_sum = prod_coal_europe_sum.sum(axis=0)
prod_coal_europe_sum = prod_coal_europe_sum.transpose()
prod_coal_europe_sum = pd.DataFrame(prod_coal_europe_sum)
prod_coal_europe_sum = prod_coal_europe_sum.rename(columns={0:'Total_Production_Europe'})
prod_coal_europe_sum.head()

,Total_Production_Europe
1980,20.381340
1981,20.297834
1982,21.050901
1983,21.053260
1984,19.829260


In [603]:
fig_coal_europe = go.Figure()
fig_coal_europe = fig_coal_europe.add_trace(go.Scatter(x=con_coal_europe_sum.index, y=con_coal_europe_sum['Total_Coal_Consumption_Europe'], name='Total Coal Consumption Europe'))
fig_coal_europe = fig_coal_europe.add_trace(go.Scatter(x=prod_coal_europe_sum.index, y=prod_coal_europe_sum['Total_Production_Europe'], name='Total Coal Production Europe'))
fig_coal_europe.update_layout(title='Total Coal Consumption Europe vs Total Coal Production Europe 1980 - 2021')    
fig_coal_europe.show()

Eurasia

In [604]:
con_coal_eurasia = con_coal[con_coal['Continent']=='Eurasia']
con_coal_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,3.09E-05,2.46E-05,2.75E-05,2.83E-05,3.72E-05,4.40E-05,0.000112207,0.000311904,0.00046824,0.009697
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,0.000137434,5.17E-05,3.63E-06,7.76E-06,6.55E-05,6.13E-05,0.000316159,0.000480431,0.000642527,0.005360
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0.013853422,0.01900722,0.017122224,0.014760258,0.015354533,0.009944537,0.007648056,0.006050076,0.005227797,0.300291
220,Eurasia,Former U.S.S.R.,15.90984917,14.98822833,15.01614382,14.95639295,14.81675603,14.61632587,14.81763389,15.54037446,...,--,--,--,--,--,--,--,--,--,181.694862
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.007358489,0.008268422,0.008122823,0.007297869,0.008091186,0.007980056,0.006197122,0.005964831,0.005342816,0.109185


In [605]:
con_coal_eurasia_sum = con_coal_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_coal_eurasia_sum = con_coal_eurasia_sum.sum(axis=0)
con_coal_eurasia_sum = con_coal_eurasia_sum.transpose()
con_coal_eurasia_sum = pd.DataFrame(con_coal_eurasia_sum)
con_coal_eurasia_sum = con_coal_eurasia_sum.rename(columns={0:'Total_Coal_Consumption_Eurasia'})
con_coal_eurasia_sum.head()

,Total_Coal_Consumption_Eurasia
1980,15.909849
1981,14.988228
1982,15.016144
1983,14.956393
1984,14.816756


In [606]:
prod_coal_eurasia = prod_coal[prod_coal['Continent']=='Eurasia']
prod_coal_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0.000000
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0.000000
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0.000000
220,Eurasia,Former U.S.S.R.,17.15256454,16.86509139,17.20047673,17.15256454,17.05674016,17.3921255,17.99102789,18.20663276,...,--,--,--,--,--,--,--,--,--,207.520441
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.007016911,0.005165986,0.005218864,0.005028912,0.004545526,0.002357568,0.000259502,0.001684219,0.00274102,0.056257


In [607]:
prod_coal_eurasia_sum = prod_coal_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_coal_eurasia_sum = prod_coal_eurasia_sum.sum(axis=0)
prod_coal_eurasia_sum = prod_coal_eurasia_sum.transpose()
prod_coal_eurasia_sum = pd.DataFrame(prod_coal_eurasia_sum)
prod_coal_eurasia_sum = prod_coal_eurasia_sum.rename(columns={0:'Total_Production_Eurasia'})
prod_coal_eurasia_sum.head()

,Total_Production_Eurasia
1980,17.152565
1981,16.865091
1982,17.200477
1983,17.152565
1984,17.056740


In [608]:
fig_coal_eurasia = go.Figure()
fig_coal_eurasia = fig_coal_eurasia.add_trace(go.Scatter(x=con_coal_eurasia_sum.index, y=con_coal_eurasia_sum['Total_Coal_Consumption_Eurasia'], name='Total Coal Consumption Eurasia'))
fig_coal_eurasia = fig_coal_eurasia.add_trace(go.Scatter(x=prod_coal_eurasia_sum.index, y=prod_coal_eurasia_sum['Total_Production_Eurasia'], name='Total Coal Production Eurasia'))
fig_coal_eurasia.update_layout(title='Total Coal Consumption Eurasia vs Total Coal Production Eurasia 1980 - 2021')
fig_coal_eurasia.show()

Overall

In [609]:

fig_coal_con_region = go.Figure()
fig_coal_con_region.add_trace(go.Scatter(x=con_coal_asia_sum.index, y=con_coal_africa_sum['Total_Coal_Consumption_Africa'], mode='lines', name='Africa'))
fig_coal_con_region.add_trace(go.Scatter(x=con_coal_asia_sum.index, y=con_coal_asia_sum['Total_Coal_Consumption_Asia'], mode='lines', name='Asia'))
fig_coal_con_region.add_trace(go.Scatter(x=con_coal_me_sum.index, y=con_coal_me_sum['Total_Coal_Consumption_Middle_East'], mode='lines', name='Middle East'))
fig_coal_con_region.add_trace(go.Scatter(x=con_coal_csa_sum.index, y=con_coal_csa_sum['Total_Coal_Consumption_Central_South_America'], mode='lines', name='Central South America'))
fig_coal_con_region.add_trace(go.Scatter(x=con_coal_na_sum.index, y=con_coal_na_sum['Total_Coal_Consumption_North_America'], mode='lines', name='North America'))
fig_coal_con_region.add_trace(go.Scatter(x=con_coal_europe_sum.index, y=con_coal_europe_sum['Total_Coal_Consumption_Europe'], mode='lines', name='Europe'))
fig_coal_con_region.add_trace(go.Scatter(x=con_coal_eurasia_sum.index, y=con_coal_eurasia_sum['Total_Coal_Consumption_Eurasia'], mode='lines', name='Eurasia'))
fig_coal_con_region.update_layout(xaxis_title='Year', yaxis_title='Coal Volume Consumption ', title="Total Coal Consumption by Region 1980 - 2021")



fig_coal_con_region.show()

In [610]:
prod_coal_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Coal
0,Africa,Algeria,7.59E-05,0.000240414,0.000404907,0.0005694,0.000575727,0.000582054,0.000278374,0.000240414,...,0,0,0,0,0,0,0,0,0,0.000000
1,Africa,Angola,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Africa,Botswana,0.008262057,0.008484744,0.009241913,0.008796519,0.00875198,0.009731845,0.010912138,0.010845329,...,0.033315425,0.038212317,0.046534212,0.04192044,0.049608123,0.048393175,0.047178203,0.041939121,0.044340069,0.952191
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [611]:
fig_coal_prod_region = go.Figure()
fig_coal_prod_region.add_trace(go.Scatter(x=prod_coal_africa_sum.index, y=prod_coal_africa_sum['Total_Production_Africa'], mode='lines', name='Africa'))
fig_coal_prod_region.add_trace(go.Scatter(x=prod_coal_asia_sum.index, y=prod_coal_asia_sum['Total_Production_Asia'], mode='lines', name='Asia'))
fig_coal_prod_region.add_trace(go.Scatter(x=prod_coal_me_sum.index, y=prod_coal_me_sum['Total_Production_Middle_East'], mode='lines', name='Middle East'))
fig_coal_prod_region.add_trace(go.Scatter(x=prod_coal_csa_sum.index, y=prod_coal_csa_sum['Total_Production_Central_South_America'], mode='lines', name='Central South America'))
fig_coal_prod_region.add_trace(go.Scatter(x=prod_coal_na_sum.index, y=prod_coal_na_sum['Total_Production_North_America'], mode='lines', name='North America'))
fig_coal_prod_region.add_trace(go.Scatter(x=prod_coal_europe_sum.index, y=prod_coal_europe_sum['Total_Production_Europe'], mode='lines', name='Europe'))
fig_coal_prod_region.add_trace(go.Scatter(x=prod_coal_eurasia_sum.index, y=prod_coal_eurasia_sum['Total_Production_Eurasia'], mode='lines', name='Eurasia'))
fig_coal_prod_region.update_layout(xaxis_title='Year', yaxis_title='Coal Volume Production ', title="Total Coal Production by Region 1980 - 2021")
fig_coal_prod_region.show()

#### Natural Gas

In [612]:
con_ng = pd.read_csv('Consumption_Data/Consumption_NaturalGas.csv')
con_ng.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_NaturalGas
0,Africa,Algeria,0.5428,0.40828,0.65254,0.77408,0.8024,0.68912,0.71036,0.77408,...,1.235699254,1.466614646,1.485966983,1.491480102,1.547998949,1.680477876,1.734907225,1.676235642,1.767236857,42.209019
1,Africa,Angola,0.0096324,0.0092136,0.01047,0.012564,0.012564,0.013611,0.014658,0.016752,...,0.01508572,0.00854118,0.028581524,0.029690759,0.030060504,0.042686734,0.047838582,0.048942549,0.033854996,0.954702
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0,0,0,0,0.001468293,0.000700207,0.002047254,0.00602556,0.006322011,0.016563
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [613]:
con_ng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 45 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Continent               230 non-null    object 
 1   Country                 230 non-null    object 
 2   1980                    230 non-null    object 
 3   1981                    230 non-null    object 
 4   1982                    230 non-null    object 
 5   1983                    230 non-null    object 
 6   1984                    230 non-null    object 
 7   1985                    230 non-null    object 
 8   1986                    230 non-null    object 
 9   1987                    230 non-null    object 
 10  1988                    230 non-null    object 
 11  1989                    230 non-null    object 
 12  1990                    230 non-null    object 
 13  1991                    230 non-null    object 
 14  1992                    230 non-null    ob

In [614]:
prod_ng = pd.read_csv('Production_Data/Production_NaturalGas.csv')
prod_ng.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
0,Africa,Algeria,0.48498,0.91096,1.11156,1.55052,1.6048,1.6048,1.5694,1.7995,...,2.987160805,3.123939826,3.114376171,3.517358923,3.573222006,3.608138943,3.339705133,3.162318268,3.830585865,111.846113
1,Africa,Angola,0.0096324,0.0092136,0.01047,0.012564,0.012564,0.013611,0.014658,0.016752,...,0.034201695,0.027028582,0.028581524,0.063337546,0.213433672,0.238824919,0.250232581,0.277831763,0.216598378,2.019490
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [615]:
prod_ng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 45 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Continent              230 non-null    object 
 1   Country                230 non-null    object 
 2   1980                   230 non-null    object 
 3   1981                   230 non-null    object 
 4   1982                   230 non-null    object 
 5   1983                   230 non-null    object 
 6   1984                   230 non-null    object 
 7   1985                   230 non-null    object 
 8   1986                   230 non-null    object 
 9   1987                   230 non-null    object 
 10  1988                   230 non-null    object 
 11  1989                   230 non-null    object 
 12  1990                   230 non-null    object 
 13  1991                   230 non-null    object 
 14  1992                   230 non-null    object 
 15  1993  

#### Natural Gas Production & Consumption by Regions

Africa

In [616]:
con_ng_africa = con_ng[con_ng['Continent']=='Africa']
con_ng_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_NaturalGas
0,Africa,Algeria,0.5428,0.40828,0.65254,0.77408,0.8024,0.68912,0.71036,0.77408,...,1.235699254,1.466614646,1.485966983,1.491480102,1.547998949,1.680477876,1.734907225,1.676235642,1.767236857,42.209019
1,Africa,Angola,0.0096324,0.0092136,0.01047,0.012564,0.012564,0.013611,0.014658,0.016752,...,0.01508572,0.00854118,0.028581524,0.029690759,0.030060504,0.042686734,0.047838582,0.048942549,0.033854996,0.954702
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0,0,0,0,0.001468293,0.000700207,0.002047254,0.00602556,0.006322011,0.016563
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [617]:
con_ng_africa_sum = con_ng_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ng_africa_sum = con_ng_africa_sum.sum(axis=0)
con_ng_africa_sum = con_ng_africa_sum.transpose()
con_ng_africa_sum = pd.DataFrame(con_ng_africa_sum)
con_ng_africa_sum = con_ng_africa_sum.rename(columns={0:'Total_NG_Consumption_Africa'})
con_ng_africa_sum.head()

,Total_NG_Consumption_Africa
1980,0.833810
1981,0.688067
1982,0.934025
1983,1.141378
1984,1.245614


In [618]:
prod_ng_africa = prod_ng[prod_ng['Continent']=='Africa']
prod_ng_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
0,Africa,Algeria,0.48498,0.91096,1.11156,1.55052,1.6048,1.6048,1.5694,1.7995,...,2.987160805,3.123939826,3.114376171,3.517358923,3.573222006,3.608138943,3.339705133,3.162318268,3.830585865,111.846113
1,Africa,Angola,0.0096324,0.0092136,0.01047,0.012564,0.012564,0.013611,0.014658,0.016752,...,0.034201695,0.027028582,0.028581524,0.063337546,0.213433672,0.238824919,0.250232581,0.277831763,0.216598378,2.019490
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [619]:
prod_ng_africa_sum = prod_ng_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ng_africa_sum = prod_ng_africa_sum.sum(axis=0)
prod_ng_africa_sum = prod_ng_africa_sum.transpose()
prod_ng_africa_sum = pd.DataFrame(prod_ng_africa_sum)
prod_ng_africa_sum = prod_ng_africa_sum.rename(columns={0:'Total_NG_Production_Africa'})
prod_ng_africa_sum.head()

,Total_NG_Production_Africa
1980,0.776095
1981,1.190747
1982,1.426549
1983,1.942190
1984,2.071354


In [620]:
fig_ng_africa = go.Figure()
fig_ng_africa = fig_ng_africa.add_trace(go.Scatter(x=con_ng_africa_sum.index, y=con_ng_africa_sum['Total_NG_Consumption_Africa'], name='Total NG Consumption Africa'))
fig_ng_africa = fig_ng_africa.add_trace(go.Scatter(x=prod_ng_africa_sum.index, y=prod_ng_africa_sum['Total_NG_Production_Africa'], name='Total NG Production Africa'))
fig_ng_africa.update_layout(title='Total Natural Gas Production & Consumption Africa 1980 - 2021')
fig_ng_africa.show()

In [621]:
prod_ng_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
0,Africa,Algeria,0.48498,0.91096,1.11156,1.55052,1.6048,1.6048,1.5694,1.7995,...,2.987160805,3.123939826,3.114376171,3.517358923,3.573222006,3.608138943,3.339705133,3.162318268,3.830585865,111.846113
1,Africa,Angola,0.0096324,0.0092136,0.01047,0.012564,0.012564,0.013611,0.014658,0.016752,...,0.034201695,0.027028582,0.028581524,0.063337546,0.213433672,0.238824919,0.250232581,0.277831763,0.216598378,2.019490
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [622]:
top_ng_production_africa_1980 = prod_ng_africa[['Country','1980']]
top_ng_production_africa_1980['1980'] = pd.to_numeric(top_ng_production_africa_1980['1980'], errors='coerce')
top_ng_production_africa_1980 = top_ng_production_africa_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_ng_production_africa_1980['portion_africa'] = top_ng_production_africa_1980['1980']/prod_ng_africa_sum.loc['1980'][0]*100
top_ng_production_africa_1980['cumulative'] =   top_ng_production_africa_1980['portion_africa'].cumsum()
top_ng_production_africa_1980.head()

,Country,1980,portion_africa,cumulative
0,Algeria,0.484980,62.489781,62.489781
28,Libya,0.188460,24.283113,86.772893
38,Nigeria,0.039786,5.126435,91.899328
15,Egypt,0.031410,4.047185,95.946514
52,Tunisia,0.016068,2.070365,98.016879


In [623]:
top_ng_production_africa_1990 = prod_ng_africa[['Country','1990']]
top_ng_production_africa_1990['1990']  = pd.to_numeric(top_ng_production_africa_1990['1990'], errors='coerce')
top_ng_production_africa_1990 = top_ng_production_africa_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_ng_production_africa_1990['portion_africa'] = top_ng_production_africa_1990['1990']/prod_ng_africa_sum.loc['1990'][0]*100
top_ng_production_africa_1990['cumulative'] =   top_ng_production_africa_1990['portion_africa'].cumsum()
top_ng_production_africa_1990.head()

,Country,1990,portion_africa,cumulative
0,Algeria,2.108660,74.921317,74.921317
15,Egypt,0.299436,10.639050,85.560366
28,Libya,0.229288,8.146679,93.707045
38,Nigeria,0.137156,4.873197,98.580242
1,Angola,0.019892,0.706771,99.287014


In [624]:
top_ng_production_africa_2000 = prod_ng_africa[['Country','2000']]
top_ng_production_africa_2000['2000']  = pd.to_numeric(top_ng_production_africa_2000['2000'], errors='coerce')
top_ng_production_africa_2000 = top_ng_production_africa_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_ng_production_africa_2000['portion_africa'] = top_ng_production_africa_2000['2000']/prod_ng_africa_sum.loc['2000'][0]*100
top_ng_production_africa_2000['cumulative'] =   top_ng_production_africa_2000['portion_africa'].cumsum()
top_ng_production_africa_2000.head()

,Country,2000,portion_africa,cumulative
0,Algeria,3.312952,67.724434,67.724434
15,Egypt,0.676639,13.832070,81.556504
38,Nigeria,0.460706,9.417901,90.974405
28,Libya,0.221849,4.535105,95.509510
52,Tunisia,0.077944,1.593365,97.102875


In [625]:
top_ng_production_africa_2010 = prod_ng_africa[['Country','2010']]
top_ng_production_africa_2010['2010']  = pd.to_numeric(top_ng_production_africa_2010['2010'], errors='coerce')
top_ng_production_africa_2010 = top_ng_production_africa_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_ng_production_africa_2010['portion_africa'] = top_ng_production_africa_2010['2010']/prod_ng_africa_sum.loc['2010'][0]*100
top_ng_production_africa_2010['cumulative'] =   top_ng_production_africa_2010['portion_africa'].cumsum()
top_ng_production_africa_2010.head(6)

,Country,2010,portion_africa,cumulative
0,Algeria,3.173258,40.778885,40.778885
15,Egypt,2.267665,29.141290,69.920175
38,Nigeria,1.071900,13.774760,83.694935
28,Libya,0.621694,7.989266,91.684200
16,Equatorial Guinea,0.249210,3.202548,94.886748
35,Mozambique,0.122083,1.568857,96.455606


In [626]:
top_ng_production_africa_2020 = prod_ng_africa[['Country','2020']]
top_ng_production_africa_2020['2020']  = pd.to_numeric(top_ng_production_africa_2020['2020'], errors='coerce')
top_ng_production_africa_2020 = top_ng_production_africa_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_ng_production_africa_2020['portion_africa'] = top_ng_production_africa_2020['2020']/prod_ng_africa_sum.loc['2020'][0]*100
top_ng_production_africa_2020['cumulative'] = top_ng_production_africa_2020['portion_africa'].cumsum()
top_ng_production_africa_2020.head(6)

,Country,2020,portion_africa,cumulative
0,Algeria,3.162318,36.444841,36.444841
15,Egypt,2.204099,25.401625,61.846466
38,Nigeria,1.721739,19.842567,81.689033
28,Libya,0.475986,5.485606,87.174640
1,Angola,0.277832,3.201934,90.376574
35,Mozambique,0.207440,2.390691,92.767265


In [627]:
top_ng_consumption_africa_1980 = con_ng_africa[['Country','1980']]
top_ng_consumption_africa_1980['1980']  = pd.to_numeric(top_ng_consumption_africa_1980['1980'], errors='coerce')
top_ng_consumption_africa_1980 = top_ng_consumption_africa_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_ng_consumption_africa_1980['portion_africa'] = top_ng_consumption_africa_1980['1980']/con_ng_africa_sum.loc['1980'][0]*100
top_ng_consumption_africa_1980['cumulative'] = top_ng_consumption_africa_1980['portion_africa'].cumsum()
top_ng_consumption_africa_1980.head()

,Country,1980,portion_africa,cumulative
0,Algeria,0.542800,65.098748,65.098748
28,Libya,0.188460,22.602266,87.701014
38,Nigeria,0.039786,4.771590,92.472603
15,Egypt,0.031410,3.767044,96.239648
52,Tunisia,0.016068,1.927057,98.166705


In [628]:
top_ng_consumption_africa_1990 = con_ng_africa[['Country','1990']]
top_ng_consumption_africa_1990['1990'] = pd.to_numeric(top_ng_consumption_africa_1990['1990'], errors='coerce')
top_ng_consumption_africa_1990 = top_ng_consumption_africa_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_ng_consumption_africa_1990['portion_africa'] = top_ng_consumption_africa_1990['1990']/con_ng_africa_sum.loc['1990'][0]*100
top_ng_consumption_africa_1990['cumulative'] = top_ng_consumption_africa_1990['portion_africa'].cumsum()
top_ng_consumption_africa_1990.head()

,Country,1990,portion_africa,cumulative
0,Algeria,0.803580,53.032331,53.032331
15,Egypt,0.299436,19.761304,72.793634
28,Libya,0.183221,12.091698,84.885332
38,Nigeria,0.137156,9.051629,93.936961
52,Tunisia,0.066744,4.404776,98.341737


In [629]:
top_ng_consumption_africa_2000 = con_ng_africa[['Country','2000']]
top_ng_consumption_africa_2000['2000'] = pd.to_numeric(top_ng_consumption_africa_2000['2000'], errors='coerce')
top_ng_consumption_africa_2000 = top_ng_consumption_africa_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_ng_consumption_africa_2000['portion_africa'] = top_ng_consumption_africa_2000['2000']/con_ng_africa_sum.loc['2000'][0]*100
top_ng_consumption_africa_2000['cumulative'] = top_ng_consumption_africa_2000['portion_africa'].cumsum()
top_ng_consumption_africa_2000.head()

,Country,2000,portion_africa,cumulative
0,Algeria,0.818288,37.159735,37.159735
15,Egypt,0.676639,30.727226,67.886961
38,Nigeria,0.248840,11.300231,79.187192
28,Libya,0.192269,8.731234,87.918426
52,Tunisia,0.124324,5.645762,93.564187


In [630]:
top_ng_consumption_africa_2010 = con_ng_africa[['Country','2010']]
top_ng_consumption_africa_2010['2010'] = pd.to_numeric(top_ng_consumption_africa_2010['2010'], errors='coerce')
top_ng_consumption_africa_2010 = top_ng_consumption_africa_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_ng_consumption_africa_2010['portion_africa'] = top_ng_consumption_africa_2010['2010']/con_ng_africa_sum.loc['2010'][0]*100
top_ng_consumption_africa_2010['cumulative'] = top_ng_consumption_africa_2010['portion_africa'].cumsum()
top_ng_consumption_africa_2010.head()

,Country,2010,portion_africa,cumulative
15,Egypt,1.707127,45.533852,45.533852
0,Algeria,1.080881,28.830112,74.363964
28,Libya,0.253056,6.749701,81.113665
38,Nigeria,0.185983,4.960695,86.074360
47,South Africa,0.148269,3.954749,90.029108


In [631]:
top_ng_consumption_africa_2020 = con_ng_africa[['Country','2020']]
top_ng_consumption_africa_2020['2020'] = pd.to_numeric(top_ng_consumption_africa_2020['2020'], errors='coerce')
top_ng_consumption_africa_2020 = top_ng_consumption_africa_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_ng_consumption_africa_2020['portion_africa'] = top_ng_consumption_africa_2020['2020']/con_ng_africa_sum.loc['2020'][0]*100
top_ng_consumption_africa_2020['cumulative'] = top_ng_consumption_africa_2020['portion_africa'].cumsum()
top_ng_consumption_africa_2020.head()

,Country,2020,portion_africa,cumulative
15,Egypt,2.082837,37.229489,37.229489
0,Algeria,1.676236,29.961726,67.191214
38,Nigeria,0.699845,12.509318,79.700532
28,Libya,0.243738,4.356674,84.057206
52,Tunisia,0.192714,3.444650,87.501856


Asia

In [632]:
con_ng['Continent'].unique()

array(['Africa', 'Asia & Oceania', 'Middle East',
       'Central & South America', 'North America', 'Europe', 'Eurasia'],
      dtype=object)

In [633]:
con_ng_asia = con_ng[con_ng['Continent']== 'Asia & Oceania']
con_ng_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_NaturalGas
57,Asia & Oceania,Afghanistan,0.002094,0.003141,0.005235,0.005235,0.005235,0,0,0.023034,...,0.005712559,0.00524668,0.005405671,0.006111883,0.006111883,0.005998338,0.005039074,0.003140122,0.003140122,0.427453
58,Asia & Oceania,American Samoa,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
59,Asia & Oceania,Australia,0.339066,0.387504,0.428571,0.434889,0.46376,0.495873,0.541284,0.554876,...,1.387400626,1.417984243,1.437351379,1.561518923,1.632483937,1.619996534,1.654034793,1.62290333,1.540937369,40.231410
60,Asia & Oceania,Bangladesh,0.04705,0.05646,0.061165,0.064929,0.089395,0.0941,0.106333,0.123271,...,0.839804826,0.880321726,0.989399256,1.083876497,1.087707153,1.051660195,1.169220507,1.1254972,1.170206456,19.968178
61,Asia & Oceania,Bhutan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [634]:
con_ng_asia_sum = con_ng_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ng_asia_sum = con_ng_asia_sum.sum(axis=0)
con_ng_asia_sum = con_ng_asia_sum.transpose()
con_ng_asia_sum = pd.DataFrame(con_ng_asia_sum)
con_ng_asia_sum = con_ng_asia_sum.rename(columns={0:'Total_NG_Consumption_Asia'})
con_ng_asia_sum.head()

,Total_NG_Consumption_Asia
1980,2.770283
1981,2.902779
1982,2.957398
1983,3.224079
1984,3.829029


In [635]:
prod_ng_asia = prod_ng[prod_ng['Continent']== 'Asia & Oceania']
prod_ng_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
57,Asia & Oceania,Afghanistan,0.06282,0.082713,0.084807,0.088995,0.088995,0.109935,0.109935,0.103653,...,0.005712559,0.00524668,0.005405671,0.006111883,0.006111883,0.005998338,0.005039074,0.003140122,0.003140122,1.229456
58,Asia & Oceania,American Samoa,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
59,Asia & Oceania,Australia,0.329589,0.385398,0.424359,0.444366,0.464814,0.495873,0.541284,0.554876,...,2.299111459,2.333267558,2.503116861,3.236309442,4.062974972,4.70322399,5.399933383,5.350861029,5.479908661,70.271894
60,Asia & Oceania,Bangladesh,0.04705,0.05646,0.061165,0.064929,0.089395,0.0941,0.106333,0.123271,...,0.839804826,0.880321726,0.989399256,1.083876497,1.017077236,1.014599887,1.008007376,0.927326335,0.880847291,19.209324
61,Asia & Oceania,Bhutan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [636]:
prod_ng_asia_sum = prod_ng_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ng_asia_sum = prod_ng_asia_sum.sum(axis=0)
prod_ng_asia_sum = prod_ng_asia_sum.transpose()
prod_ng_asia_sum = pd.DataFrame(prod_ng_asia_sum)
prod_ng_asia_sum = prod_ng_asia_sum.rename(columns={0:'Total_NG_Production_Asia'})
prod_ng_asia_sum.head()

,Total_NG_Production_Asia
1980,2.698593
1981,2.868678
1982,2.919132
1983,3.241797
1984,3.789489


In [637]:
fig_ng_asia = go.Figure()
fig_ng_asia = fig_ng_asia.add_trace(go.Scatter(x=con_ng_asia_sum.index, y=con_ng_asia_sum['Total_NG_Consumption_Asia'], name='Total NG Consumption Asia'))
fig_ng_asia = fig_ng_asia.add_trace(go.Scatter(x=prod_ng_asia_sum.index, y=prod_ng_asia_sum['Total_NG_Production_Asia'], name='Total NG Production Asia'))
fig_ng_asia = fig_ng_asia.update_layout(title='Total Natural Gas Production & Consumption Asia 1980 - 2021')    
fig_ng_asia.show()

In [638]:
top_ng_consumption_asia_1980 = con_ng_asia[['Country','1980']]
top_ng_consumption_asia_1980['1980']  = pd.to_numeric(top_ng_consumption_asia_1980['1980'], errors='coerce')
top_ng_consumption_asia_1980 = top_ng_consumption_asia_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_ng_consumption_asia_1980['portion_asia'] = top_ng_consumption_asia_1980['1980']/con_ng_asia_sum.loc['1980'][0]*100
top_ng_consumption_asia_1980['cumulative'] = top_ng_consumption_asia_1980['portion_asia'].cumsum()
top_ng_consumption_asia_1980.head()

,Country,1980,portion_asia,cumulative
17,Japan,0.967113,34.910260,34.910260
8,China,0.581255,20.981791,55.892051
2,Australia,0.339066,12.239399,68.131451
32,Pakistan,0.278850,10.065759,78.197209
16,Indonesia,0.272800,9.847369,88.044579


In [639]:
top_ng_consumption_asia_1990 = con_ng_asia[['Country','1990']]
top_ng_consumption_asia_1990['1990']  = pd.to_numeric(top_ng_consumption_asia_1990['1990'], errors='coerce')
top_ng_consumption_asia_1990 = top_ng_consumption_asia_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_ng_consumption_asia_1990['portion_asia'] = top_ng_consumption_asia_1990['1990']/con_ng_asia_sum.loc['1990'][0]*100
top_ng_consumption_asia_1990['cumulative'] = top_ng_consumption_asia_1990['portion_asia'].cumsum()
top_ng_consumption_asia_1990.head()

,Country,1990,portion_asia,cumulative
17,Japan,2.192509,35.706731,35.706731
16,Indonesia,0.692637,11.280138,46.986868
2,Australia,0.652500,10.626475,57.613344
8,China,0.569020,9.266945,66.880289
15,India,0.459249,7.479231,74.359520


In [640]:
top_ng_consumption_asia_2000 = con_ng_asia[['Country','2000']]
top_ng_consumption_asia_2000['2000']  = pd.to_numeric(top_ng_consumption_asia_2000['2000'], errors='coerce')
top_ng_consumption_asia_2000 = top_ng_consumption_asia_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_ng_consumption_asia_2000['portion_asia'] = top_ng_consumption_asia_2000['2000']/con_ng_asia_sum.loc['2000'][0]*100
top_ng_consumption_asia_2000['cumulative'] = top_ng_consumption_asia_2000['portion_asia'].cumsum()
top_ng_consumption_asia_2000.head()

,Country,2000,portion_asia,cumulative
17,Japan,3.085481,28.711805,28.711805
16,Indonesia,1.043940,9.714333,38.426138
8,China,0.942975,8.774815,47.200953
2,Australia,0.849553,7.905473,55.106426
32,Pakistan,0.799207,7.436988,62.543414


In [641]:
top_ng_consumption_asia_2010 = con_ng_asia[['Country','2010']]
top_ng_consumption_asia_2010['2010']  = pd.to_numeric(top_ng_consumption_asia_2010['2010'], errors='coerce')
top_ng_consumption_asia_2010 = top_ng_consumption_asia_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_ng_consumption_asia_2010['portion_asia'] = top_ng_consumption_asia_2010['2010']/con_ng_asia_sum.loc['2010'][0]*100
top_ng_consumption_asia_2010['cumulative'] = top_ng_consumption_asia_2010['portion_asia'].cumsum()
top_ng_consumption_asia_2010.head()

,Country,2010,portion_asia,cumulative
17,Japan,4.080289,18.877077,18.877077
8,China,3.938045,18.218995,37.096072
15,India,2.427777,11.231884,48.327956
38,South Korea,1.712363,7.922088,56.250043
16,Indonesia,1.591700,7.363852,63.613895


In [642]:
top_ng_consumption_asia_2020 = con_ng_asia[['Country','2020']]
top_ng_consumption_asia_2020['2020']  = pd.to_numeric(top_ng_consumption_asia_2020['2020'], errors='coerce')
top_ng_consumption_asia_2020 = top_ng_consumption_asia_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_ng_consumption_asia_2020['portion_asia'] = top_ng_consumption_asia_2020['2020']/con_ng_asia_sum.loc['2020'][0]*100
top_ng_consumption_asia_2020['cumulative'] = top_ng_consumption_asia_2020['portion_asia'].cumsum()
top_ng_consumption_asia_2020.head()

,Country,2020,portion_asia,cumulative
8,China,11.970265,37.881015,37.881015
17,Japan,4.231232,13.390126,51.271141
15,India,2.320725,7.344148,58.615289
38,South Korea,2.171209,6.870993,65.486282
41,Thailand,1.718030,5.436867,70.923149


In [643]:
top_ng_production_asia_1980 = prod_ng_asia[['Country','1980']]
top_ng_production_asia_1980['1980']  = pd.to_numeric(top_ng_production_asia_1980['1980'], errors='coerce')
top_ng_production_asia_1980 = top_ng_production_asia_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_ng_production_asia_1980['portion_asia'] = top_ng_production_asia_1980['1980']/prod_ng_asia_sum.loc['1980'][0]*100
top_ng_production_asia_1980['cumulative'] = top_ng_production_asia_1980['portion_asia'].cumsum()
top_ng_production_asia_1980.head()

,Country,1980,portion_asia,cumulative
16,Indonesia,0.719400,26.658336,26.658336
8,China,0.581255,21.539187,48.197524
5,Brunei,0.383624,14.215704,62.413228
2,Australia,0.329589,12.213365,74.626592
32,Pakistan,0.278850,10.333163,84.959755


In [644]:
top_ng_production_asia_1990 = prod_ng_asia[['Country','1990']]
top_ng_production_asia_1990['1990']  = pd.to_numeric(top_ng_production_asia_1990['1990'], errors='coerce')
top_ng_production_asia_1990 = top_ng_production_asia_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_ng_production_asia_1990['portion_asia'] = top_ng_production_asia_1990['1990']/prod_ng_asia_sum.loc['1990'][0]*100
top_ng_production_asia_1990['cumulative'] = top_ng_production_asia_1990['portion_asia'].cumsum()
top_ng_production_asia_1990.head()

,Country,1990,portion_asia,cumulative
16,Indonesia,1.762200,30.370437,30.370437
2,Australia,0.780117,13.444839,43.815276
8,China,0.584708,10.077084,53.892360
21,Malaysia,0.523036,9.014211,62.906571
15,India,0.459249,7.914875,70.821446


In [645]:
top_ng_production_asia_2000 = prod_ng_asia[['Country','2000']]
top_ng_production_asia_2000['2000']  = pd.to_numeric(top_ng_production_asia_2000['2000'], errors='coerce')
top_ng_production_asia_2000 = top_ng_production_asia_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_ng_production_asia_2000['portion_asia'] = top_ng_production_asia_2000['2000']/prod_ng_asia_sum.loc['2000'][0]*100
top_ng_production_asia_2000['cumulative'] = top_ng_production_asia_2000['portion_asia'].cumsum()
top_ng_production_asia_2000.head()

,Country,2000,portion_asia,cumulative
16,Indonesia,2.438169,25.301097,25.301097
21,Malaysia,1.542132,16.002845,41.303942
2,Australia,1.250564,12.977218,54.281160
8,China,1.005639,10.435604,64.716764
32,Pakistan,0.799207,8.293447,73.010211


In [646]:
top_ng_production_asia_2010 = prod_ng_asia[['Country','2010']]
top_ng_production_asia_2010['2010']  = pd.to_numeric(top_ng_production_asia_2010['2010'], errors='coerce')
top_ng_production_asia_2010 = top_ng_production_asia_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_ng_production_asia_2010['portion_asia'] = top_ng_production_asia_2010['2010']/prod_ng_asia_sum.loc['2010'][0]*100
top_ng_production_asia_2010['cumulative'] = top_ng_production_asia_2010['portion_asia'].cumsum()
top_ng_production_asia_2010.head()

,Country,2010,portion_asia,cumulative
8,China,3.484123,19.453372,19.453372
16,Indonesia,3.179551,17.752812,37.206184
21,Malaysia,2.285866,12.762983,49.969168
2,Australia,1.961126,10.949819,60.918987
15,India,1.935261,10.805404,71.724391


In [647]:
top_ng_production_asia_2020 = prod_ng_asia[['Country','2020']]
top_ng_production_asia_2020['2020']  = pd.to_numeric(top_ng_production_asia_2020['2020'], errors='coerce')
top_ng_production_asia_2020 = top_ng_production_asia_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_ng_production_asia_2020['portion_asia'] = top_ng_production_asia_2020['2020']/prod_ng_asia_sum.loc['2020'][0]*100
top_ng_production_asia_2020['cumulative'] = top_ng_production_asia_2020['portion_asia'].cumsum()
top_ng_production_asia_2020.head()

,Country,2020,portion_asia,cumulative
8,China,6.940324,29.268650,29.268650
2,Australia,5.350861,22.565586,51.834236
21,Malaysia,2.468160,10.408693,62.242929
16,Indonesia,2.278514,9.608921,71.851850
41,Thailand,1.244398,5.247861,77.099711


Middle East

In [648]:
con_ng_me = con_ng[con_ng['Continent']== 'Middle East']
con_ng_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_NaturalGas
106,Middle East,Bahrain,0.1047,0.12564,0.13611,0.14658,0.139251,0.171708,0.18846,0.227199,...,0.580504439,0.573109478,0.573109478,0.566823814,0.587492554,0.593766429,0.654039156,0.65273731,0.685962006,14.955641
107,Middle East,Iran,0.244992,0.16368,0.2112,0.32736,0.502656,0.6336,0.566016,0.59664,...,5.879930549,6.471242649,6.935424139,7.407321196,7.715467662,8.090628434,8.217422332,8.315724993,8.667751797,138.961773
108,Middle East,Iraq,0.064914,0.024081,0.025128,0.018846,0.02094,0.024081,0.028269,0.036645,...,0.043593295,0.033462199,0.037048755,0.04075523,0.413523097,0.534251366,0.666064164,0.690837073,0.680936856,5.384423
109,Middle East,Israel,0.005195,0.005195,0.003117,0.002078,0.002078,0.002078,0.0014546,0.0021819,...,0.185462895,0.290194882,0.311549882,0.359864886,0.384963906,0.388921203,0.390604019,0.404896533,0.419732882,3.757181
110,Middle East,Jordan,0,0,0,0,0,0,0,0,...,0.039981453,0.013258099,0.085733731,0.149449007,0.154746283,0.157436007,0.154860061,0.148654981,0.124447525,1.769991


In [649]:
con_ng_me_sum = con_ng_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ng_me_sum = con_ng_me_sum.sum(axis=0)
con_ng_me_sum = con_ng_me_sum.transpose()
con_ng_me_sum = pd.DataFrame(con_ng_me_sum)
con_ng_me_sum = con_ng_me_sum.rename(columns={0:'Total_NG_Consumption_Middle_East'})
con_ng_me_sum.head()

,Total_NG_Consumption_Middle_East
1980,1.373220
1981,1.361498
1982,1.272069
1983,1.496933
1984,1.993850


In [650]:
prod_ng_me = prod_ng[prod_ng['Continent']== 'Middle East']
prod_ng_me.head()    

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
106,Middle East,Bahrain,0.1047,0.12564,0.13611,0.14658,0.139251,0.171708,0.18846,0.227199,...,0.543529634,0.573109478,0.573109478,0.566823814,0.587492554,0.594441374,0.656803551,0.663726014,0.689062861,14.936195
107,Middle East,Iran,0.264,0.22176,0.264,0.32736,0.502656,0.6336,0.566016,0.59664,...,6.015302832,6.507416509,6.891679872,7.479444178,8.001126487,8.510536342,8.833860784,9.128470138,9.258014137,141.525437
108,Middle East,Iraq,0.064914,0.024081,0.025128,0.018846,0.02094,0.024081,0.047115,0.138204,...,0.043593295,0.033462199,0.037048755,0.04075523,0.359910085,0.376918351,0.396026101,0.309833289,0.394769334,4.683411
109,Middle East,Israel,0.005195,0.005195,0.003117,0.002078,0.002078,0.002078,0.0014546,0.0021819,...,0.165449057,0.287263198,0.309080656,0.346170019,0.362864573,0.37329401,0.374017799,0.525600086,0.644412393,3.842827
110,Middle East,Jordan,0,0,0,0,0,0,0,0,...,0.004912599,0.004291126,0.004468689,0.003788034,0.003640065,0.003430418,0.003428768,0.005502066,0.005490491,0.242795


In [651]:
prod_ng_me_sum = prod_ng_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ng_me_sum = prod_ng_me_sum.sum(axis=0)
prod_ng_me_sum = prod_ng_me_sum.transpose()
prod_ng_me_sum = pd.DataFrame(prod_ng_me_sum)
prod_ng_me_sum = prod_ng_me_sum.rename(columns={0:'Total_NG_Production_Middle_East'})
prod_ng_me_sum.head()

,Total_NG_Production_Middle_East
1980,1.491693
1981,1.520090
1982,1.438992
1983,1.589069
1984,2.101691


In [652]:
print(con_ng_me_sum.columns)

Index(['Total_NG_Consumption_Middle_East'], dtype='object')


In [653]:
fig_ng_me = go.Figure()
fig_ng_me = fig_ng_me.add_trace(go.Scatter(x=con_ng_me_sum.index, y=con_ng_me_sum['Total_NG_Consumption_Middle_East'], name='Total NG Consumption Middle East'))
fig_ng_me = fig_ng_me.add_trace(go.Scatter(x=prod_ng_me_sum.index, y=prod_ng_me_sum['Total_NG_Production_Middle_East'], name='Total NG Production Middle East'))
fig_ng_me = fig_ng_me.update_layout(title='Total Natural Gas Production & Consumption Middle East 1980 - 2021') 
fig_ng_me.show()

Central & South America

In [654]:
con_ng['Continent'].unique()

array(['Africa', 'Asia & Oceania', 'Middle East',
       'Central & South America', 'North America', 'Europe', 'Eurasia'],
      dtype=object)

In [655]:
con_ng_csa = con_ng[con_ng['Continent']== 'Central & South America']
con_ng_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_NaturalGas
120,Central & South America,Antarctica,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
121,Central & South America,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
122,Central & South America,Argentina,0.368334,0.371412,0.467856,0.460674,0.586872,0.593028,0.64125,0.62073,...,1.72242856,1.737374751,1.734127183,1.78723184,1.810186042,1.910364385,1.980163285,1.848911044,1.824563637,50.652487
123,Central & South America,Aruba,--,--,--,--,--,--,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
124,Central & South America,Barbados,0.0004188,0.0004188,0.0004188,0.0004188,0.0004188,0.0004188,0.0004188,0.0004188,...,0.000818899,0.000525081,0.000655878,0.000739234,0.000805376,0.000750586,0.000774371,0.00080395,0.000862582,0.039506


In [656]:
con_ng_csa_sum = con_ng_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ng_csa_sum = con_ng_csa_sum.sum(axis=0)
con_ng_csa_sum = con_ng_csa_sum.transpose()
con_ng_csa_sum = pd.DataFrame(con_ng_csa_sum)
con_ng_csa_sum = con_ng_csa_sum.rename(columns={0:'Total_NG_Consumption_Central_South_America'})
con_ng_csa_sum.head()

,Total_NG_Consumption_Central_South_America
1980,1.389831
1981,1.481334
1982,1.580583
1983,1.579619
1984,1.884346


In [657]:
prod_ng_csa = prod_ng[prod_ng['Continent']== 'Central & South America']
prod_ng_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
120,Central & South America,Antarctica,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
121,Central & South America,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
122,Central & South America,Argentina,0.28728,0.354996,0.409374,0.45657,0.506844,0.513,0.561222,0.540702,...,1.312312463,1.310098213,1.34331197,1.449078437,1.509896001,1.548289462,1.61358324,1.479978283,1.491854584,46.843003
123,Central & South America,Aruba,--,--,--,--,--,--,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
124,Central & South America,Barbados,0.0004188,0.0004188,0.0004188,0.0004188,0.0004188,0.0004188,0.0004188,0.0003141,...,0.000818899,0.000525081,0.000655878,0.000634977,0.000551623,0.000551623,0.000551623,0.000551623,0.000551623,0.038067


In [658]:
prod_ng_csa_sum = prod_ng_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ng_csa_sum = prod_ng_csa_sum.sum(axis=0)
prod_ng_csa_sum = prod_ng_csa_sum.transpose()
prod_ng_csa_sum = pd.DataFrame(prod_ng_csa_sum)
prod_ng_csa_sum = prod_ng_csa_sum.rename(columns={0:'Total_NG_Production_Central_South_America'})
prod_ng_csa_sum.head()

,Total_NG_Production_Central_South_America
1980,1.369422
1981,1.517969
1982,1.591101
1983,1.644515
1984,1.882318


In [659]:
fig_ng_csa = go.Figure()
fig_ng_csa = fig_ng_csa.add_trace(go.Scatter(x=con_ng_csa_sum.index, y=con_ng_csa_sum['Total_NG_Consumption_Central_South_America'], name='Total NG Consumption Central South America'))
fig_ng_csa = fig_ng_csa.add_trace(go.Scatter(x=prod_ng_csa_sum.index, y=prod_ng_csa_sum['Total_NG_Production_Central_South_America'], name='Total NG Production Central South America'))
fig_ng_csa = fig_ng_csa.update_layout(title='Total Natural Gas Production & Consumption Central South America 1980 - 2021')
fig_ng_csa.show()

North America

In [660]:
con_ng_na = con_ng[con_ng['Continent']== 'North America']
con_ng_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_NaturalGas
165,North America,Bermuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
166,North America,Canada,1.915011,1.864104,1.901757,1.90026,2.055323,2.20397,2.15982,2.14368,...,4.115150439,4.275888143,4.251207033,4.071426084,4.346019532,4.630510995,4.571284746,4.427291343,4.580516341,131.661210
167,North America,Greenland,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
168,North America,Mexico,0.843744,0.910845,0.992036,0.99414,0.99803,1.04049,0.92488,0.938543,...,2.620203332,2.550020102,2.736557814,2.844381592,2.861173018,3.022042393,3.014750733,2.961715766,3.011911022,75.395306
169,North America,Saint Pierre and Miquelon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [661]:
con_ng_na_sum = con_ng_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ng_na_sum = con_ng_na_sum.sum(axis=0)
con_ng_na_sum = con_ng_na_sum.transpose()
con_ng_na_sum = pd.DataFrame(con_ng_na_sum)
con_ng_na_sum = con_ng_na_sum.rename(columns={0:'Total_NG_Consumption_North_America'})
con_ng_na_sum.head()

,Total_NG_Consumption_North_America
1980,22.994214
1981,22.522259
1982,21.250015
1983,20.115236
1984,21.446966


In [662]:
prod_ng_na = prod_ng[prod_ng['Continent']== 'North America']
prod_ng_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
165,North America,Bermuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
166,North America,Canada,2.805903,2.70204,2.785629,2.61222,2.839953,3.095738,2.902068,3.149545,...,5.349782343,5.598649417,5.781002514,5.849182194,6.547925259,6.829722817,6.647772666,6.534892552,6.776646861,220.966536
167,North America,Greenland,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
168,North America,Mexico,0.9504,1.020357,1.09934,1.080404,1.064495,1.049949,0.931186,0.943798,...,1.758943467,1.62758005,1.471924672,1.337893831,1.15088215,1.031749759,0.985764215,0.932234646,0.871335169,54.139951
169,North America,Saint Pierre and Miquelon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [663]:
prod_ng_na_sum = prod_ng_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ng_na_sum = prod_ng_na_sum.sum(axis=0)
prod_ng_na_sum = prod_ng_na_sum.transpose()
prod_ng_na_sum = pd.DataFrame(prod_ng_na_sum)
prod_ng_na_sum = prod_ng_na_sum.rename(columns={0:'Total_NG_Production_North_America'})
prod_ng_na_sum.head()

,Total_NG_Production_North_America
1980,23.663903
1981,23.421552
1982,22.203994
1983,20.286013
1984,21.912381


In [664]:
fig_ng_na = go.Figure()
fig_ng_na = fig_ng_na.add_trace(go.Scatter(x=con_ng_na_sum.index, y=con_ng_na_sum['Total_NG_Consumption_North_America'], name='Total NG Consumption North America'))
fig_ng_na = fig_ng_na.add_trace(go.Scatter(x=prod_ng_na_sum.index, y=prod_ng_na_sum['Total_NG_Production_North_America'], name='Total NG Production North America'))
fig_ng_na = fig_ng_na.update_layout(title='Total Natural Gas Production & Consumption North America 1980 - 2021')
fig_ng_na.show()

Europe

In [665]:
con_ng_europe = con_ng[con_ng['Continent']== 'Europe']
con_ng_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_NaturalGas
172,Europe,Albania,0.01047,0.011517,0.01047,0.011517,0.011517,0.01047,0.014658,0.016752,...,0.000665617,0.001121039,0.001226137,0.001576448,0.001751609,0.001503678,0.002730014,0.001953109,0.002065834,0.201258
173,Europe,Austria,0.185256,0.172515,0.16709,0.170345,0.188825,0.202852,0.196378,0.20501,...,0.31130138,0.284155402,0.303215693,0.316651424,0.343709147,0.320216953,0.334443627,0.316412865,0.335047088,11.623614
174,Europe,Belgium,0.395115,0.363165,0.30033,0.316305,0.32589,0.32589,0.28968,0.32376,...,0.63940759,0.560064367,0.612696048,0.622886109,0.62996552,0.649099473,0.654131166,0.650444579,0.649508699,21.971845
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0.00861121,0.006063727,0.007176008,0.007498926,0.008144761,0.008130732,0.007830971,0.007004228,0.008083093,0.352067
176,Europe,Bulgaria,0.16992,0.156704,0.16992,0.173696,0.170864,0.206736,0.212175,0.210289,...,0.108003865,0.103828925,0.114407042,0.115445167,0.120273534,0.113609724,0.10634737,0.108334185,0.122604928,6.115728


In [666]:
con_ng_europe_sum = con_ng_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ng_europe_sum = con_ng_europe_sum.sum(axis=0)
con_ng_europe_sum = con_ng_europe_sum.transpose()
con_ng_europe_sum = pd.DataFrame(con_ng_europe_sum)
con_ng_europe_sum = con_ng_europe_sum.rename(columns={0:'Total_NG_Consumption_Europe'})
con_ng_europe_sum.head()

,Total_NG_Consumption_Europe
1980,10.642745
1981,10.513927
1982,10.554687
1983,10.787350
1984,11.338709


In [667]:
prod_ng_europe= prod_ng[prod_ng['Continent']== 'Europe']
prod_ng_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
172,Europe,Albania,0.01047,0.011517,0.01047,0.011517,0.011517,0.01047,0.014658,0.016752,...,0.000670314,0.00112895,0.001234789,0.001587572,0.001763969,0.001514289,0.002749278,0.001965963,0.002080412,0.201383
173,Europe,Austria,0.0747,0.055335,0.050995,0.046397,0.048555,0.04428,0.042081,0.044239,...,0.052495522,0.047839456,0.045742602,0.042953495,0.045959516,0.037358684,0.033483292,0.027438091,0.024666475,2.226702
174,Europe,Belgium,0.0014756,0.0011781,0.0011594,0.0006444,0.0013936,0.0014938,0.0009693,0.001059,...,0,0,0,0,0,0,0.000142556,0.000219317,0.000190075,0.012402
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0.032331
176,Europe,Bulgaria,0.004248,0.0039648,0.002832,0.003304,0.002832,0.003776,0.004715,0.0041492,...,0.009925563,0.006426624,0.003725802,0.003320394,0.002856253,0.001174854,0.00138468,0.002229218,0.001280455,0.261180


In [668]:
prod_ng_europe_sum = prod_ng_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ng_europe_sum = prod_ng_europe_sum.sum(axis=0)
prod_ng_europe_sum = prod_ng_europe_sum.transpose()
prod_ng_europe_sum = pd.DataFrame(prod_ng_europe_sum)
prod_ng_europe_sum = prod_ng_europe_sum.rename(columns={0:'Total_NG_Production_Europe'})
prod_ng_europe_sum.head()

,Total_NG_Production_Europe
1980,8.776540
1981,8.640931
1982,8.253435
1983,8.504295
1984,8.593558


In [669]:
fig_ng_europe = go.Figure()
fig_ng_europe = fig_ng_europe.add_trace(go.Scatter(x=con_ng_europe_sum.index, y=con_ng_europe_sum['Total_NG_Consumption_Europe'], name='Total NG Consumption Europe'))
fig_ng_europe = fig_ng_europe.add_trace(go.Scatter(x=prod_ng_europe_sum.index, y=prod_ng_europe_sum['Total_NG_Production_Europe'], name='Total NG Production Europe'))
fig_ng_europe = fig_ng_europe.update_layout(title='Total Natural Gas Production & Consumption Europe 1980 - 2021')
fig_ng_europe.show()

Eurasia

In [670]:
con_ng_eurasia = con_ng[con_ng['Continent']== 'Eurasia']
con_ng_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_NaturalGas
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0.08910928,0.092437013,0.07579835,0.081935472,0.087037952,0.091778387,0.092362389,0.094077841,0.102277487,2.066687
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,0.350151403,0.370857294,0.402285878,0.394595184,0.382652424,0.390056364,0.424011666,0.428291379,0.461260619,10.811203
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0.74371257,0.735362832,0.634653281,0.630368584,0.648569392,0.687991123,0.682607833,0.632018183,0.667594807,19.882942
220,Eurasia,Former U.S.S.R.,13.86112,14.42556,15.506478,16.805178,19.104384,20.951664,22.081572,23.046012,...,--,--,--,--,--,--,--,--,--,246.591268
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.074774003,0.08025694,0.088315684,0.082795996,0.084267913,0.083108826,0.094079191,0.093453474,0.09423454,2.137644


In [671]:
con_ng_eurasia_sum = con_ng_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ng_eurasia_sum = con_ng_eurasia_sum.sum(axis=0)
con_ng_eurasia_sum = con_ng_eurasia_sum.transpose()
con_ng_eurasia_sum = pd.DataFrame(con_ng_eurasia_sum)
con_ng_eurasia_sum = con_ng_eurasia_sum.rename(columns={0:'Total_NG_Consumption_Eurasia'})
con_ng_eurasia_sum.head()

,Total_NG_Consumption_Eurasia
1980,13.861120
1981,14.425560
1982,15.506478
1983,16.805178
1984,19.104384


In [672]:
prod_ng_eurasia = prod_ng[prod_ng['Continent']== 'Eurasia']
prod_ng_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_NaturalGas
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0.000000
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,0.617479244,0.647059088,0.66554649,0.646393704,0.6266863,0.664261666,0.853190222,0.908651959,1.160078497,13.398365
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0.001391623,0.00109865,0.00109865,0.000805679,0.002160669,0.002306458,0.002525143,0.002452248,0.00270738,0.159833
220,Eurasia,Former U.S.S.R.,15.9848,16.41357,17.66232,18.908073,21.407808,23.433624,24.82407,26.017308,...,--,--,--,--,--,--,--,--,--,279.889259
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.000199177,3.32E-05,0.000331961,0.000211248,0.000244444,0.000299651,0.000299651,0.000202514,0.000373315,0.018066


In [673]:
prod_ng_eurasia_sum = prod_ng_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ng_eurasia_sum = prod_ng_eurasia_sum.sum(axis=0)
prod_ng_eurasia_sum = prod_ng_eurasia_sum.transpose()
prod_ng_eurasia_sum = pd.DataFrame(prod_ng_eurasia_sum)
prod_ng_eurasia_sum = prod_ng_eurasia_sum.rename(columns={0:'Total_NG_Production_Eurasia'})
prod_ng_eurasia_sum.head()

,Total_NG_Production_Eurasia
1980,15.984800
1981,16.413570
1982,17.662320
1983,18.908073
1984,21.407808


In [674]:
fig_ng_eurasia = go.Figure()
fig_ng_eurasia = fig_ng_eurasia.add_trace(go.Scatter(x=con_ng_eurasia_sum.index, y=con_ng_eurasia_sum['Total_NG_Consumption_Eurasia'], name='Total NG Consumption Eurasia'))
fig_ng_eurasia = fig_ng_eurasia.add_trace(go.Scatter(x=prod_ng_eurasia_sum.index, y=prod_ng_eurasia_sum['Total_NG_Production_Eurasia'], name='Total NG Production Eurasia'))
fig_ng_eurasia = fig_ng_eurasia.update_layout(title='Total Natural Gas Production & Consumption Eurasia 1980 - 2021')
fig_ng_eurasia.show()

Total Natural Gas Consumption by Region

In [675]:
fig_ng_region = go.Figure()
fig_ng_region = fig_ng_region.add_trace(go.Scatter(x=con_ng_africa_sum.index, y=con_ng_africa_sum['Total_NG_Consumption_Africa'], name='Africa'))
fig_ng_region = fig_ng_region.add_trace(go.Scatter(x=con_ng_asia_sum.index, y=con_ng_asia_sum['Total_NG_Consumption_Asia'], name='Asia'))
fig_ng_region = fig_ng_region.add_trace(go.Scatter(x=con_ng_me_sum.index, y=con_ng_me_sum['Total_NG_Consumption_Middle_East'], name='Middle East'))
fig_ng_region = fig_ng_region.add_trace(go.Scatter(x=con_ng_csa_sum.index, y=con_ng_csa_sum['Total_NG_Consumption_Central_South_America'], name='Central & South America'))
fig_ng_region = fig_ng_region.add_trace(go.Scatter(x=con_ng_na_sum.index, y=con_ng_na_sum['Total_NG_Consumption_North_America'], name='North America'))
fig_ng_region = fig_ng_region.add_trace(go.Scatter(x=con_ng_europe_sum.index, y=con_ng_europe_sum['Total_NG_Consumption_Europe'], name='Europe'))
fig_ng_region = fig_ng_region.add_trace(go.Scatter(x=con_ng_eurasia_sum.index, y=con_ng_eurasia_sum['Total_NG_Consumption_Eurasia'], name='Eurasia'))
fig_ng_region = fig_ng_region.update_layout(title='Total Natural Gas Consumption 1980 - 2021')
fig_ng_region.show()



In [676]:
prod_ng_all_sum = prod_ng_africa_sum.join(prod_ng_asia_sum).join(prod_ng_me_sum).join(prod_ng_csa_sum).join(prod_ng_na_sum).join(prod_ng_europe_sum).join(prod_ng_eurasia_sum)
prod_ng_all_sum['Total_NG_Production'] = prod_ng_all_sum.sum(axis=1)
prod_ng_all_sum['portion_africa'] = prod_ng_all_sum['Total_NG_Production_Africa']/prod_ng_all_sum['Total_NG_Production']*100
prod_ng_all_sum['portion_asia'] = prod_ng_all_sum['Total_NG_Production_Asia']/prod_ng_all_sum['Total_NG_Production']*100
prod_ng_all_sum['portion_me'] = prod_ng_all_sum['Total_NG_Production_Middle_East']/prod_ng_all_sum['Total_NG_Production']*100
prod_ng_all_sum['portion_csa'] = prod_ng_all_sum['Total_NG_Production_Central_South_America']/prod_ng_all_sum['Total_NG_Production']*100
prod_ng_all_sum['portion_na'] = prod_ng_all_sum['Total_NG_Production_North_America']/prod_ng_all_sum['Total_NG_Production']*100
prod_ng_all_sum['portion_europe'] = prod_ng_all_sum['Total_NG_Production_Europe']/prod_ng_all_sum['Total_NG_Production']*100
prod_ng_all_sum['portion_eurasia'] = prod_ng_all_sum['Total_NG_Production_Eurasia']/prod_ng_all_sum['Total_NG_Production']*100

prod_ng_all_sum.tail()

,Total_NG_Production_Africa,Total_NG_Production_Asia,Total_NG_Production_Middle_East,Total_NG_Production_Central_South_America,Total_NG_Production_North_America,Total_NG_Production_Europe,Total_NG_Production_Eurasia,Total_NG_Production,portion_africa,portion_asia,portion_me,portion_csa,portion_na,portion_europe,portion_eurasia
2017,8.638687,21.906465,24.040810,6.348136,35.988142,9.164586,31.053866,137.140693,6.299142,15.973716,17.530034,4.628923,26.241768,6.682616,22.643801
2018,9.101071,23.011802,24.589621,6.218765,39.743621,8.703050,32.635930,144.003859,6.320019,15.979990,17.075668,4.318471,27.598997,6.043623,22.663233
2019,9.120405,24.096994,25.464422,6.134377,42.820720,8.108205,33.271812,149.016935,6.120381,16.170641,17.088274,4.116564,28.735473,5.441130,22.327538
2020,8.676998,23.712484,25.864575,5.476538,42.199108,7.503836,31.186116,144.619656,5.999875,16.396446,17.884550,3.786856,29.179372,5.188670,21.564230
2021,9.689106,24.534090,26.569944,5.392434,43.442938,7.204944,34.222671,151.056127,6.414242,16.241705,17.589451,3.569822,28.759468,4.769713,22.655599


Total Natural Gas Production by Region

In [677]:
fig_ng_prod_region = go.Figure()
fig_ng_prod_region = fig_ng_prod_region.add_trace(go.Scatter(x=prod_ng_africa_sum.index, y=prod_ng_africa_sum['Total_NG_Production_Africa'], name='Africa'))
fig_ng_prod_region = fig_ng_prod_region.add_trace(go.Scatter(x=prod_ng_asia_sum.index, y=prod_ng_asia_sum['Total_NG_Production_Asia'], name='Asia'))
fig_ng_prod_region = fig_ng_prod_region.add_trace(go.Scatter(x=prod_ng_me_sum.index, y=prod_ng_me_sum['Total_NG_Production_Middle_East'], name='Middle East'))
fig_ng_prod_region = fig_ng_prod_region.add_trace(go.Scatter(x=prod_ng_csa_sum.index, y=prod_ng_csa_sum['Total_NG_Production_Central_South_America'], name='Central & South America'))
fig_ng_prod_region = fig_ng_prod_region.add_trace(go.Scatter(x=prod_ng_na_sum.index, y=prod_ng_na_sum['Total_NG_Production_North_America'], name='North America'))
fig_ng_prod_region = fig_ng_prod_region.add_trace(go.Scatter(x=prod_ng_europe_sum.index, y=prod_ng_europe_sum['Total_NG_Production_Europe'], name='Europe'))
fig_ng_prod_region = fig_ng_prod_region.add_trace(go.Scatter(x=prod_ng_eurasia_sum.index, y=prod_ng_eurasia_sum['Total_NG_Production_Eurasia'], name='Eurasia'))
fig_ng_prod_region = fig_ng_prod_region.update_layout(title='Total Natural Gas Production 1980 - 2021')
fig_ng_prod_region.show()

In [678]:
fig_ng_prod_mix = go.Figure()
fig_ng_prod_mix = fig_ng_prod_mix.add_trace(go.Bar(x=prod_ng_all_sum.index, y=prod_ng_all_sum['portion_africa'], name='Africa'))
fig_ng_prod_mix = fig_ng_prod_mix.add_trace(go.Bar(x=prod_ng_all_sum.index, y=prod_ng_all_sum['portion_asia'], name='Asia'))
fig_ng_prod_mix = fig_ng_prod_mix.add_trace(go.Bar(x=prod_ng_all_sum.index, y=prod_ng_all_sum['portion_me'], name='Middle East'))
fig_ng_prod_mix = fig_ng_prod_mix.add_trace(go.Bar(x=prod_ng_all_sum.index, y=prod_ng_all_sum['portion_csa'], name='Central & South America'))
fig_ng_prod_mix = fig_ng_prod_mix.add_trace(go.Bar(x=prod_ng_all_sum.index, y=prod_ng_all_sum['portion_na'], name='North America'))
fig_ng_prod_mix = fig_ng_prod_mix.add_trace(go.Bar(x=prod_ng_all_sum.index, y=prod_ng_all_sum['portion_europe'], name='Europe'))
fig_ng_prod_mix = fig_ng_prod_mix.add_trace(go.Bar(x=prod_ng_all_sum.index, y=prod_ng_all_sum['portion_eurasia'], name='Eurasia'))
fig_ng_prod_mix = fig_ng_prod_mix.update_layout(title='Total Natural Gas Production 1980 - 2021', barmode='stack')
fig_ng_prod_mix.show()

#### Nuclear & Renewable Energy

In [679]:
con_re = pd.read_csv('Consumption_Data/Consumption_Neuclear+renewables.csv')
con_re.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Neuclear
0,Africa,Algeria,0.002606933,0.003841991,0.004866486,0.002389748,0.005629292,0.006549389,0.002522158,0.004945129,...,0.010393744,0.008358618,0.006348635,0.005049059,0.012308752,0.014856347,0.016825538,0.012355114,0.017043145,0.246540
1,Africa,Angola,0.005505642,0.005540091,0.006272403,0.006312011,0.006264,0.0062682,0.00689436,0.00687654,...,0.045167094,0.04796844,0.047498943,0.054748262,0.071806122,0.091140144,0.09877426,0.106784956,0.102802014,1.123556
2,Africa,Benin,0.000399204,0.000464032,0.000566392,0.000620984,0.00054592,0.000569804,0.00056298,0.000576628,...,0.003693806,0.003222888,0.00363169,0.003807988,0.003847105,0.004458737,0.003794266,0.002259265,0.002268038,0.074941
3,Africa,Botswana,2.20E-41,0.00003412,0.000279784,0.00054592,0.000634632,0.000760876,0.00081888,0.000245664,...,0.00963751,0.005774338,0.005036773,0.00567596,0.004440721,0.004113034,0.006011594,0.006322836,0.006322836,0.170053
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0.002920575,0.002626869,0.00249933,0.003579496,0.003657759,0.004923817,0.005615422,0.005271876,0.005676647,0.064742


In [680]:
prod_re = pd.read_csv('Production_Data/Production_Nuclear+renewables.csv')
prod_re.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Neuclear
0,Africa,Algeria,0.002576225,0.003783987,0.004955198,0.002451164,0.00538704,0.006675633,0.002580162,0.005115729,...,0.003119907,0.00238701,0.002059499,0.003063362,0.005854309,0.00711778,0.007453485,0.006325333,0.007955883,0.138442
1,Africa,Angola,0.005505642,0.005540091,0.006272403,0.006312011,0.006264,0.0062682,0.00689436,0.00687654,...,0.045167094,0.04796844,0.047498943,0.054748262,0.071806122,0.091140144,0.09877426,0.106784956,0.102802014,1.123556
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0.000019082,0.00001902,0.000055914,0.00009232,0.000055278,4.64E-05,7.18E-05,6.19E-05,7.07E-05,0.000732
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0.000019082,0.00002853,0.000027957,0.00004616,0.000046065,0.000054624,0.00005343,0.000052638,0.000052638,0.000391
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0.001106756,0.00096051,0.000987814,0.00143096,0.001455654,0.002066608,0.00190567,0.001859876,0.002054882,0.033589


#### Nuclear & Renewable Energy Production & Consumption by Regions

Africa

In [681]:
con_re_africa = con_re[con_re['Continent']== 'Africa']
con_re_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Neuclear
0,Africa,Algeria,0.002606933,0.003841991,0.004866486,0.002389748,0.005629292,0.006549389,0.002522158,0.004945129,...,0.010393744,0.008358618,0.006348635,0.005049059,0.012308752,0.014856347,0.016825538,0.012355114,0.017043145,0.246540
1,Africa,Angola,0.005505642,0.005540091,0.006272403,0.006312011,0.006264,0.0062682,0.00689436,0.00687654,...,0.045167094,0.04796844,0.047498943,0.054748262,0.071806122,0.091140144,0.09877426,0.106784956,0.102802014,1.123556
2,Africa,Benin,0.000399204,0.000464032,0.000566392,0.000620984,0.00054592,0.000569804,0.00056298,0.000576628,...,0.003693806,0.003222888,0.00363169,0.003807988,0.003847105,0.004458737,0.003794266,0.002259265,0.002268038,0.074941
3,Africa,Botswana,2.20E-41,0.00003412,0.000279784,0.00054592,0.000634632,0.000760876,0.00081888,0.000245664,...,0.00963751,0.005774338,0.005036773,0.00567596,0.004440721,0.004113034,0.006011594,0.006322836,0.006322836,0.170053
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0.002920575,0.002626869,0.00249933,0.003579496,0.003657759,0.004923817,0.005615422,0.005271876,0.005676647,0.064742


In [682]:
con_re_africa_sum = con_re_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_re_africa_sum = con_re_africa_sum.sum(axis=0)
con_re_africa_sum = con_re_africa_sum.transpose()
con_re_africa_sum = pd.DataFrame(con_re_africa_sum)
con_re_africa_sum = con_re_africa_sum.rename(columns={0:'Total_RE_Consumption_Africa'})
con_re_africa_sum.head()

,Total_RE_Consumption_Africa
1980,0.628175
1981,0.554975
1982,0.546272
1983,0.531879
1984,0.517153


In [683]:
prod_re_africa = prod_re[prod_re['Continent']== 'Africa']
prod_re_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Neuclear
0,Africa,Algeria,0.002576225,0.003783987,0.004955198,0.002451164,0.00538704,0.006675633,0.002580162,0.005115729,...,0.003119907,0.00238701,0.002059499,0.003063362,0.005854309,0.00711778,0.007453485,0.006325333,0.007955883,0.138442
1,Africa,Angola,0.005505642,0.005540091,0.006272403,0.006312011,0.006264,0.0062682,0.00689436,0.00687654,...,0.045167094,0.04796844,0.047498943,0.054748262,0.071806122,0.091140144,0.09877426,0.106784956,0.102802014,1.123556
2,Africa,Benin,0,0,0,0,0,0,0,0,...,0.000019082,0.00001902,0.000055914,0.00009232,0.000055278,4.64E-05,7.18E-05,6.19E-05,7.07E-05,0.000732
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0.000019082,0.00002853,0.000027957,0.00004616,0.000046065,0.000054624,0.00005343,0.000052638,0.000052638,0.000391
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0.001106756,0.00096051,0.000987814,0.00143096,0.001455654,0.002066608,0.00190567,0.001859876,0.002054882,0.033589


In [684]:
prod_re_africa_sum = prod_re_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_re_africa_sum = prod_re_africa_sum.sum(axis=0)
prod_re_africa_sum = prod_re_africa_sum.transpose()
prod_re_africa_sum = pd.DataFrame(prod_re_africa_sum)
prod_re_africa_sum = prod_re_africa_sum.rename(columns={0:'Total_RE_Production_Africa'})
prod_re_africa_sum.head()

,Total_RE_Production_Africa
1980,0.626587
1981,0.552298
1982,0.544617
1983,0.530010
1984,0.514130


In [685]:
fig_re_africa = go.Figure()
fig_re_africa = fig_re_africa.add_trace(go.Scatter(x=con_re_africa_sum.index, y=con_re_africa_sum['Total_RE_Consumption_Africa'], name='Total Consumption'))
fig_re_africa = fig_re_africa.add_trace(go.Scatter(x=prod_re_africa_sum.index, y=prod_re_africa_sum['Total_RE_Production_Africa'], name='Total Production'))
fig_re_africa = fig_re_africa.update_layout(title='Total Nuclear & Renewable Energy Consumption & Production Africa 1980 - 2021')
fig_re_africa.show()

Asia

In [686]:
con_re_asia = con_re[con_re['Continent']== 'Asia & Oceania']
con_re_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Neuclear
57,Asia & Oceania,Afghanistan,0.00738587,0.007536615,0.007390981,0.007847934,0.00790308,0.00804419,0.007447998,0.007876764,...,0.020778165,0.02216852,0.022529794,0.024570116,0.0257575,0.025797592,0.024685194,0.023570174,0.024430387,0.455932
58,Asia & Oceania,American Samoa,0,0,0,0,0,0,0,0,...,0.000019082,0.00001902,0.000018638,0.000027696,0.000046065,0.00004552,0.000044525,0.000043865,0.000043865,0.000317
59,Asia & Oceania,Australia,0.133988864,0.14927951,0.145706523,0.130429593,0.128791941,0.145220687,0.148477395,0.139897468,...,0.301941981,0.324378047,0.289463048,0.34163906,0.368926528,0.381902286,0.472231631,0.552277272,0.618524061,9.109658
60,Asia & Oceania,Bangladesh,0.00677315,0.005663374,0.007117005,0.007216836,0.009533069,0.007720333,0.0047007,0.005334528,...,0.010241825,0.015210182,0.019006398,0.024488344,0.027807894,0.028702311,0.033007215,0.033966047,0.03433713,0.504679
61,Asia & Oceania,Bhutan,7.94E-05,9.36E-05,0.000107516,9.78E-05,0.000114228,0.000155228,0.00229786,0.008603852,...,0.053274931,0.050977444,0.052960956,0.053381648,0.051350755,0.047553396,0.05906527,0.058396323,0.061202537,1.080067


In [687]:
con_re_asia_sum = con_re_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_re_asia_sum = con_re_asia_sum.sum(axis=0)
con_re_asia_sum = con_re_asia_sum.transpose()
con_re_asia_sum = pd.DataFrame(con_re_asia_sum)
con_re_asia_sum = con_re_asia_sum.rename(columns={0:'Total_RE_Consumption_Asia'})
con_re_asia_sum.head()

,Total_RE_Consumption_Asia
1980,3.714677
1981,3.990137
1982,4.223904
1983,4.681917
1984,4.986841


In [688]:
top_re_con_asia_1980 = con_re_asia[['Country','1980']]
top_re_con_asia_1980['1980'] = pd.to_numeric(top_re_con_asia_1980['1980'],errors='coerce')
top_re_con_asia_1980 = top_re_con_asia_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_re_con_asia_1980['portion_asia'] =  top_re_con_asia_1980['1980']/con_re_asia_sum.loc['1980'][0]*100
top_re_con_asia_1980['cumulative'] = top_re_con_asia_1980['portion_asia'].cumsum()
top_re_con_asia_1980 .head()

,Country,1980,portion_asia,cumulative
17,Japan,1.709851,46.029590,46.029590
8,China,0.592360,15.946473,61.976064
15,India,0.517489,13.930941,75.907005
28,New Zealand,0.219124,5.898867,81.805872
2,Australia,0.133989,3.607013,85.412885


In [689]:
top_re_con_asia_1990 = con_re_asia[['Country','1990']]
top_re_con_asia_1990['1990'] = pd.to_numeric(top_re_con_asia_1990['1990'],errors='coerce')
top_re_con_asia_1990 = top_re_con_asia_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_re_con_asia_1990['portion_asia'] =  top_re_con_asia_1990['1990']/con_re_asia_sum.loc['1990'][0]*100
top_re_con_asia_1990['cumulative'] = top_re_con_asia_1990['portion_asia'].cumsum()
top_re_con_asia_1990 .head()

,Country,1990,portion_asia,cumulative
17,Japan,2.974232,41.146952,41.146952
8,China,1.273583,17.619358,58.766310
15,India,0.809678,11.201473,69.967784
38,South Korea,0.559043,7.734073,77.701856
40,Taiwan,0.409233,5.661525,83.363381


In [690]:
top_re_con_asia_2000 = con_re_asia[['Country','2000']]
top_re_con_asia_2000['2000'] = pd.to_numeric(top_re_con_asia_2000['2000'],errors='coerce')
top_re_con_asia_2000 = top_re_con_asia_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_re_con_asia_2000['portion_asia'] =  top_re_con_asia_2000['2000']/con_re_asia_sum.loc['2000'][0]*100
top_re_con_asia_2000['cumulative'] = top_re_con_asia_2000['portion_asia'].cumsum()
top_re_con_asia_2000 .head()

,Country,2000,portion_asia,cumulative
17,Japan,4.120302,40.129586,40.129586
8,China,2.002911,19.507304,59.636889
38,South Korea,1.087514,10.591814,70.228703
15,India,1.028320,10.015300,80.244003
40,Taiwan,0.453408,4.415957,84.659960


In [691]:
top_re_con_asia_2010 = con_re_asia[['Country','2010']]
top_re_con_asia_2010['2010'] = pd.to_numeric(top_re_con_asia_2010['2010'],errors='coerce')
top_re_con_asia_2010 = top_re_con_asia_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_re_con_asia_2010['portion_asia'] =  top_re_con_asia_2010['2010']/con_re_asia_sum.loc['2010'][0]*100
top_re_con_asia_2010['cumulative'] = top_re_con_asia_2010['portion_asia'].cumsum()
top_re_con_asia_2010 .head()

,Country,2010,portion_asia,cumulative
8,China,8.257256,45.597059,45.597059
17,Japan,3.866047,21.348543,66.945602
15,India,1.747442,9.649479,76.595082
38,South Korea,1.482946,8.188913,84.783995
40,Taiwan,0.500996,2.766528,87.550523


In [692]:
top_re_con_asia_2020 = con_re_asia[['Country','2020']]
top_re_con_asia_2020['2020'] = pd.to_numeric(top_re_con_asia_2020['2020'],errors='coerce')
top_re_con_asia_2020 = top_re_con_asia_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_re_con_asia_2020['portion_asia'] =  top_re_con_asia_2020['2020']/con_re_asia_sum.loc['2020'][0]*100
top_re_con_asia_2020['cumulative'] = top_re_con_asia_2020['portion_asia'].cumsum()
top_re_con_asia_2020 .head()

,Country,2020,portion_asia,cumulative
8,China,22.759751,65.757221,65.757221
15,India,3.335490,9.636860,75.394081
17,Japan,2.085216,6.024584,81.418665
38,South Korea,1.770696,5.115875,86.534541
47,Vietnam,0.771040,2.227682,88.762222


In [693]:
prod_re_asia = prod_re[prod_re['Continent']== 'Asia & Oceania']
prod_re_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Neuclear
57,Asia & Oceania,Afghanistan,0.00738587,0.007536615,0.007390981,0.007847934,0.00790308,0.00804419,0.007447998,0.007876764,...,0.008443785,0.00951,0.009635846,0.00978592,0.010023744,0.00878536,0.00792545,0.005991959,0.006113553,0.278773
58,Asia & Oceania,American Samoa,0,0,0,0,0,0,0,0,...,0.000019082,0.00001902,0.000018638,0.000027696,0.000046065,0.00004552,0.000044525,0.000043865,0.000043865,0.000317
59,Asia & Oceania,Australia,0.137267075,0.149843793,0.145948359,0.130563946,0.129195,0.145516263,0.154308312,0.144438597,...,0.337836588,0.355808043,0.321310703,0.363992193,0.381061895,0.396371066,0.483889582,0.562944894,0.628734882,9.672690
60,Asia & Oceania,Bangladesh,0.006492502,0.00546692,0.006920551,0.006964252,0.00936468,0.007720333,0.0047007,0.005334528,...,0.010241825,0.007482002,0.007473838,0.01144768,0.011921622,0.012382715,0.009853383,0.011204595,0.01197146,0.370796
61,Asia & Oceania,Bhutan,6.23E-05,7.32E-05,8.36E-05,8.42E-05,0.00008352,0.000083576,0.0031338,0.012169392,...,0.071853271,0.0679965,0.072054508,0.07274816,0.070488663,0.06272656,0.079708655,0.078527123,0.078965773,1.458875


In [694]:
prod_re_asia_sum = prod_re_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_re_asia_sum = prod_re_asia_sum.sum(axis=0)
prod_re_asia_sum = prod_re_asia_sum.transpose()
prod_re_asia_sum = pd.DataFrame(prod_re_asia_sum)
prod_re_asia_sum = prod_re_asia_sum.rename(columns={0:'Total_RE_Production_Asia'})
prod_re_asia_sum.head()


,Total_RE_Production_Asia
1980,3.773345
1981,4.042434
1982,4.301334
1983,4.771872
1984,5.094001


In [695]:
fig_re_asia = go.Figure()
fig_re_asia = fig_re_asia.add_trace(go.Scatter(x=con_re_asia_sum.index, y=con_re_asia_sum['Total_RE_Consumption_Asia'], name='Total Consumption'))
fig_re_asia = fig_re_asia.add_trace(go.Scatter(x=prod_re_asia_sum.index, y=prod_re_asia_sum['Total_RE_Production_Asia'], name='Total Production'))
fig_re_asia = fig_re_asia.update_layout(title='Total Nuclear & Renewable Energy Consumption & Production Asia 1980 - 2021')
fig_re_asia.show()

In [696]:
top_re_prod_asia_1980 = prod_re_asia[['Country','1980']]
top_re_prod_asia_1980['1980'] = pd.to_numeric(top_re_prod_asia_1980['1980'],errors='coerce')
top_re_prod_asia_1980 = top_re_prod_asia_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_re_prod_asia_1980['portion_asia'] =  top_re_prod_asia_1980['1980']/prod_re_asia_sum.loc['1980'][0]*100
top_re_prod_asia_1980['cumulative'] = top_re_prod_asia_1980['portion_asia'].cumsum()
top_re_prod_asia_1980 .head()

,Country,1980,portion_asia,cumulative
17,Japan,1.769908,46.905534,46.905534
8,China,0.598536,15.862212,62.767746
15,India,0.517616,13.717688,76.485434
28,New Zealand,0.219124,5.807152,82.292586
2,Australia,0.137267,3.637809,85.930394


In [697]:
top_re_prod_asia_1990 = prod_re_asia[['Country','1990']]
top_re_prod_asia_1990['1990'] = pd.to_numeric(top_re_prod_asia_1990['1990'],errors='coerce')
top_re_prod_asia_1990 = top_re_prod_asia_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_re_prod_asia_1990['portion_asia'] =  top_re_prod_asia_1990['1990']/prod_re_asia_sum.loc['1990'][0]*100
top_re_prod_asia_1990['cumulative'] = top_re_prod_asia_1990['portion_asia'].cumsum()
top_re_prod_asia_1990 .head()

,Country,1990,portion_asia,cumulative
17,Japan,3.021250,41.337168,41.337168
8,China,1.301706,17.810128,59.147296
15,India,0.803726,10.996696,70.143992
38,South Korea,0.555083,7.594727,77.738719
40,Taiwan,0.404701,5.537170,83.275889


In [698]:
top_re_prod_asia_2000 = prod_re_asia[['Country','2000']]
top_re_prod_asia_2000['2000'] = pd.to_numeric(top_re_prod_asia_2000['2000'],errors='coerce')    
top_re_prod_asia_2000 = top_re_prod_asia_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_re_prod_asia_2000['portion_asia'] =  top_re_prod_asia_2000['2000']/prod_re_asia_sum.loc['2000'][0]*100
top_re_prod_asia_2000['cumulative'] = top_re_prod_asia_2000['portion_asia'].cumsum()
top_re_prod_asia_2000 .head()

,Country,2000,portion_asia,cumulative
17,Japan,4.123961,38.977984,38.977984
8,China,2.438038,23.043334,62.021318
38,South Korea,1.078743,10.195836,72.217154
15,India,0.956975,9.044933,81.262087
40,Taiwan,0.445073,4.206645,85.468731


In [699]:
top_re_prod_asia_2010 = prod_re_asia[['Country','2010']]
top_re_prod_asia_2010['2010'] = pd.to_numeric(top_re_prod_asia_2010['2010'],errors='coerce')
top_re_prod_asia_2010 = top_re_prod_asia_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_re_prod_asia_2010['portion_asia'] =  top_re_prod_asia_2010['2010']/prod_re_asia_sum.loc['2010'][0]*100
top_re_prod_asia_2010['cumulative'] = top_re_prod_asia_2010['portion_asia'].cumsum()
top_re_prod_asia_2010 .head()


,Country,2010,portion_asia,cumulative
8,China,8.464404,46.290824,46.290824
17,Japan,3.859475,21.107011,67.397835
15,India,1.700188,9.298124,76.695959
38,South Korea,1.479681,8.092198,84.788157
40,Taiwan,0.495414,2.709363,87.497520


In [700]:
top_re_prod_asia_2020 = prod_re_asia[['Country','2020']]
top_re_prod_asia_2020['2020'] = pd.to_numeric(top_re_prod_asia_2020['2020'],errors='coerce')
top_re_prod_asia_2020 = top_re_prod_asia_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_re_prod_asia_2020['portion_asia'] =  top_re_prod_asia_2020['2020']/prod_re_asia_sum.loc['2020'][0]*100
top_re_prod_asia_2020['cumulative'] = top_re_prod_asia_2020['portion_asia'].cumsum()
top_re_prod_asia_2020 .head()

,Country,2020,portion_asia,cumulative
8,China,22.833278,66.161994,66.161994
15,India,3.266171,9.464097,75.626091
17,Japan,2.154439,6.242731,81.868822
38,South Korea,1.766786,5.119461,86.988283
47,Vietnam,0.746661,2.163535,89.151818


Middle East

In [701]:
con_re_me = con_re[con_re['Continent']== 'Middle East']
con_re_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Neuclear
106,Middle East,Bahrain,0,0,0,0,0,0,0,0,...,6.56E-05,8.87E-05,4.96E-05,0.00001816,0.000222723,0.000774568,0.00078851,0.00078719,0.000806491,0.004060
107,Middle East,Iran,0.057767686,0.064432308,0.06669655,0.064571872,0.06543792,0.06550269,0.064096656,0.066014784,...,0.180941051,0.169355201,0.16384309,0.208210327,0.195388743,0.210329752,0.204072124,0.197428327,0.164044693,4.610871
108,Middle East,Iraq,0.007095006,0.007139401,0.006314219,0.006248891,0.00630576,0.006309988,0.006204924,0.006293076,...,0.087000679,0.069940502,0.06873523,0.072157988,0.060589398,0.094175485,0.164914203,0.109933266,0.109789029,1.480847
109,Middle East,Israel,-0.00056298,-0.000617572,-0.000685812,-0.000661928,-0.000678988,-0.000794996,-0.001013364,-0.001113894,...,-0.01051273,-0.007350578,-0.006344346,-0.003137959,-0.002079245,-0.002623199,0.004462344,0.012957449,0.021652389,-0.122411
110,Middle East,Jordan,0,0,0,0,0,-0.000075064,-0.00069883,-0.000143239,...,0.00171802,0.001907925,0.004064516,0.009007841,0.012919788,0.019992802,0.0268292,0.038091796,0.0450834,0.205044


In [702]:
con_re_me_sum = con_re_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_re_me_sum = con_re_me_sum.sum(axis=0)
con_re_me_sum = con_re_me_sum.transpose()
con_re_me_sum = pd.DataFrame(con_re_me_sum)
con_re_me_sum = con_re_me_sum.rename(columns={0:'Total_RE_Consumption_Middle_East'})
con_re_me_sum.head()

,Total_RE_Consumption_Middle_East
1980,0.100147
1981,0.108079
1982,0.110549
1983,0.105972
1984,0.112241


In [703]:
prod_re_me = prod_re[prod_re['Continent']== 'Middle East']
prod_re_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Neuclear
106,Middle East,Bahrain,0,0,0,0,0,0,0,0,...,7.63E-06,7.85E-05,7.69E-05,0.000076164,7.60E-05,0.000075108,0.00008905,0.00008773,0.000107031,0.000682
107,Middle East,Iran,0.057767686,0.064432308,0.06669655,0.064571872,0.06543792,0.06550269,0.064096656,0.066014784,...,0.182405047,0.173531035,0.165988847,0.21455405,0.206933595,0.215254742,0.210833883,0.208893691,0.175968645,4.532986
108,Middle East,Iraq,0.007095006,0.007139401,0.006314219,0.006248891,0.00630576,0.006309988,0.006204924,0.006293076,...,0.045367455,0.02814009,0.024024382,0.031333408,0.020372154,0.019817769,0.047555051,0.046439358,0.046295121,0.820414
109,Middle East,Israel,0,0,0,0,0,0,0,0.00010419,...,0.00543837,0.00917715,0.011387818,0.015808877,0.017195143,0.017381357,0.025995476,0.034258565,0.043908865,0.196253
110,Middle East,Jordan,0,0,0,0,0,0,0.000031338,0.000197961,...,0.000582001,0.00059913,0.00214337,0.00798568,0.012888987,0.019652805,0.026326742,0.03742825,0.044612132,0.162660


In [704]:
prod_re_me_sum = prod_re_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_re_me_sum = prod_re_me_sum.sum(axis=0)
prod_re_me_sum = prod_re_me_sum.transpose()
prod_re_me_sum = pd.DataFrame(prod_re_me_sum)
prod_re_me_sum = prod_re_me_sum.rename(columns={0:'Total_RE_Production_Middle_East'})
prod_re_me_sum.head()

,Total_RE_Production_Middle_East
1980,0.099922
1981,0.107854
1982,0.110352
1983,0.105831
1984,0.112157


In [705]:
fig_re_me = go.Figure()
fig_re_me = fig_re_me.add_trace(go.Scatter(x=con_re_me_sum.index, y=con_re_me_sum['Total_RE_Consumption_Middle_East'], name='Total Consumption'))
fig_re_me = fig_re_me.add_trace(go.Scatter(x=prod_re_me_sum.index, y=prod_re_me_sum['Total_RE_Production_Middle_East'], name='Total Production'))
fig_re_me = fig_re_me.update_layout(title='Total Nuclear & Renewable Energy Consumption & Production Middle East 1980 - 2021')
fig_re_me.show()

Central & South America

In [706]:
con_re_csa = con_re[con_re['Continent']== 'Central & South America']
con_re_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Neuclear
120,Central & South America,Antarctica,0,0,0,0,0,0,0,0,...,4.77E-06,0.000004755,0.000009319,0.000009232,0,0,0,0,0,0.000229
121,Central & South America,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,5.72E-06,0.000005706,4.99E-05,0.00004616,5.66E-05,0.00009104,0.000115765,0.000140368,0.000171249,0.000694
122,Central & South America,Argentina,0.208606362,0.204995813,0.219665828,0.233705475,0.265210873,0.278679805,0.281814149,0.2993988,...,0.427256414,0.42042759,0.423240959,0.415048695,0.415517088,0.430437023,0.444009572,0.462794574,0.46158558,15.272389
123,Central & South America,Aruba,--,--,--,--,--,--,0,0,...,0.001459773,0.00151209,0.001630825,0.001380295,0.001324019,0.001335666,0.00130647,0.001287104,0.001287104,0.016248
124,Central & South America,Barbados,0,0,0,0,0,0,0,0,...,0.000038164,0.0000951,0.000130466,0.000212336,0.000294816,0.000300432,0.00039182,0.000579018,0.000706402,0.002826


In [707]:
con_re_csa_sum = con_re_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_re_csa_sum = con_re_csa_sum.sum(axis=0)
con_re_csa_sum = con_re_csa_sum.transpose()
con_re_csa_sum = pd.DataFrame(con_re_csa_sum)
con_re_csa_sum = con_re_csa_sum.rename(columns={0:'Total_RE_Consumption_Central_South_America'})
con_re_csa_sum.head()

,Total_RE_Consumption_Central_South_America
1980,2.195940
1981,2.233626
1982,2.417209
1983,2.652514
1984,2.913554


In [708]:
prod_re_csa = prod_re[prod_re['Continent']== 'Central & South America']
prod_re_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Neuclear
120,Central & South America,Antarctica,0,0,0,0,0,0,0,0,...,4.77E-06,0.000004755,0.000009319,0.000009232,0,0,0,0,0,0.000229
121,Central & South America,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,5.72E-06,0.000005706,4.99E-05,0.00004616,5.66E-05,0.00009104,0.000115765,0.000140368,0.000171249,0.000694
122,Central & South America,Argentina,0.206775359,0.203436764,0.218124529,0.232833817,0.265284576,0.27875692,0.281884184,0.298851006,...,0.486657403,0.500061029,0.480059351,0.503851028,0.514692221,0.515031111,0.515976815,0.446798437,0.450323603,15.765753
123,Central & South America,Aruba,--,--,--,--,--,--,0,0,...,0.001459773,0.00151209,0.001630825,0.001380295,0.001324019,0.001335666,0.00130647,0.001287104,0.001287104,0.016248
124,Central & South America,Barbados,0,0,0,0,0,0,0,0,...,0.000038164,0.0000951,0.000130466,0.000212336,0.000294816,0.000300432,0.00039182,0.000579018,0.000706402,0.002826


In [709]:
prod_re_csa_sum = prod_re_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_re_csa_sum = prod_re_csa_sum.sum(axis=0)
prod_re_csa_sum = prod_re_csa_sum.transpose()
prod_re_csa_sum = pd.DataFrame(prod_re_csa_sum)
prod_re_csa_sum = prod_re_csa_sum.rename(columns={0:'Total_RE_Production_Central_South_America'})
prod_re_csa_sum.head()

,Total_RE_Production_Central_South_America
1980,2.249169
1981,2.306281
1982,2.535795
1983,2.815858
1984,3.157275


In [710]:
fig_re_csa = go.Figure()
fig_re_csa = fig_re_csa.add_trace(go.Scatter(x=con_re_csa_sum.index, y=con_re_csa_sum['Total_RE_Consumption_Central_South_America'], name='Total Consumption'))
fig_re_csa = fig_re_csa.add_trace(go.Scatter(x=prod_re_csa_sum.index, y=prod_re_csa_sum['Total_RE_Production_Central_South_America'], name='Total Production'))
fig_re_csa = fig_re_csa.update_layout(title='Total Nuclear & Renewable Energy Consumption & Production Central & South America 1980 - 2021')
fig_re_csa.show()

North America

In [711]:
con_re_na = con_re[con_re['Continent']== 'North America']
con_re_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Neuclear
165,North America,Bermuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
166,North America,Canada,2.952241781,3.101167835,3.017216196,3.21842274,3.427330243,3.687764059,3.906951539,3.988046391,...,4.927038237,4.923546881,4.792754607,4.829424982,4.926553522,4.846407889,4.723992783,4.661202349,4.635691421,179.503929
167,North America,Greenland,0,0,0,0,0,0,0,0,...,0.003568334,0.00372792,0.003904661,0.003665104,0.003537792,0.003767417,0.003562,0.0043865,0.0043865,0.065164
168,North America,Mexico,0.188516826,0.270033477,0.255339572,0.231362703,0.260591213,0.291399866,0.245119039,0.233812401,...,0.497594706,0.602592607,0.564509147,0.573463891,0.60605716,0.660360867,0.63223479,0.719714025,0.83252511,18.642062
169,North America,Saint Pierre and Miquelon,0,0,0,0,0,0,0,0,...,0.000019082,0,0,0,0,0,0,0,0,0.000077


In [712]:
con_re_na_sum = con_re_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_re_na_sum = con_re_na_sum.sum(axis=0)
con_re_na_sum = con_re_na_sum.transpose()
con_re_na_sum = pd.DataFrame(con_re_na_sum)
con_re_na_sum = con_re_na_sum.rename(columns={0:'Total_RE_Consumption_North_America'})
con_re_na_sum.head()

,Total_RE_Consumption_North_America
1980,11.308270
1981,11.792478
1982,12.383341
1983,13.147947
1984,13.678316


In [713]:
prod_re_na = prod_re[prod_re['Continent']== 'North America']
prod_re_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Neuclear
165,North America,Bermuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
166,North America,Canada,3.036118194,3.207584522,3.108207165,3.327067289,3.551323505,3.818588709,4.00884693,4.127925648,...,5.127012941,5.113315545,5.022716096,5.067332197,5.157981919,5.034795216,4.906290343,4.826160097,4.734499506,184.435834
167,North America,Greenland,0,0,0,0,0,0,0,0,...,0.003568334,0.00372792,0.003904661,0.003665104,0.003537792,0.003767417,0.003562,0.0043865,0.0043865,0.065164
168,North America,Mexico,0.18333787,0.264732744,0.250707939,0.230104358,0.25872408,0.288431223,0.242462106,0.236104959,...,0.488574421,0.591757233,0.548612733,0.541993127,0.572146617,0.6228323,0.596911452,0.677688744,0.792221749,18.150989
169,North America,Saint Pierre and Miquelon,0,0,0,0,0,0,0,0,...,0.000019082,0,0,0,0,0,0,0,0,0.000077


In [714]:
prod_re_na_sum = prod_re_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_re_na_sum = prod_re_na_sum.sum(axis=0)
prod_re_na_sum = prod_re_na_sum.transpose()
prod_re_na_sum = pd.DataFrame(prod_re_na_sum)
prod_re_na_sum = prod_re_na_sum.rename(columns={0:'Total_RE_Production_North_America'})
prod_re_na_sum.head()

,Total_RE_Production_North_America
1980,11.386967
1981,11.893594
1982,12.469701
1983,13.255333
1984,13.800442


In [715]:
fig_re_na = go.Figure()
fig_re_na = fig_re_na.add_trace(go.Scatter(x=con_re_na_sum.index, y=con_re_na_sum['Total_RE_Consumption_North_America'], name='Total Consumption'))
fig_re_na = fig_re_na.add_trace(go.Scatter(x=prod_re_na_sum.index, y=prod_re_na_sum['Total_RE_Production_North_America'], name='Total Production'))
fig_re_na = fig_re_na.update_layout(title='Total Nuclear & Renewable Energy Consumption & Production North America 1980 - 2021')
fig_re_na.show()

Europe

In [716]:
con_re_europe = con_re[con_re['Continent']== 'Europe']
con_re_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Neuclear
172,Europe,Albania,0.02889723,0.03003651,0.030738285,0.031635646,0.032069488,0.032923257,0.033298938,0.039634183,...,0.073667478,0.054950102,0.059168857,0.077233874,0.05122586,0.073971008,0.054301137,0.054376661,0.076762123,1.998845
173,Europe,Austria,0.314114744,0.329799793,0.328128524,0.330362568,0.330926189,0.356392367,0.348731273,0.27861929,...,0.529178416,0.526577494,0.492039548,0.507455566,0.512029603,0.495443817,0.525334875,0.522123342,0.509280809,18.065506
174,Europe,Belgium,0.13495419,0.143981036,0.162707627,0.238263034,0.280975991,0.340192238,0.392887413,0.407466967,...,0.567089888,0.514176594,0.470866435,0.594506569,0.594967729,0.509507389,0.611331433,0.553109539,0.672615332,19.523982
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0.048905968,0.037061868,0.036351706,0.029379007,0.019334394,0.02930313,0.032510435,0.020090726,0.035204883,1.109597
176,Europe,Bulgaria,0.127835242,0.155158839,0.165739031,0.184957224,0.18936046,0.193499141,0.17611139,0.176287513,...,0.192745014,0.205328,0.207092564,0.209915316,0.201277379,0.21921548,0.210877643,0.220075546,0.226154631,8.076440


In [717]:
con_re_europe_sum = con_re_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_re_europe_sum = con_re_europe_sum.sum(axis=0)
con_re_europe_sum = con_re_europe_sum.transpose()
con_re_europe_sum = pd.DataFrame(con_re_europe_sum)
con_re_europe_sum = con_re_europe_sum.rename(columns={0:'Total_RE_Consumption_Europe'})
con_re_europe_sum.head()

,Total_RE_Consumption_Europe
1980,7.651363
1981,8.368064
1982,8.674342
1983,9.489478
1984,10.680681


In [718]:
prod_re_europe = prod_re[prod_re['Continent']== 'Europe']
prod_re_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Neuclear
172,Europe,Albania,0.030322582,0.031547162,0.032334236,0.033316898,0.03383604,0.034631805,0.03515079,0.041457201,...,0.065741402,0.044485498,0.054395469,0.071143454,0.04128115,0.077087938,0.046091746,0.046610949,0.078631899,1.901920
173,Europe,Austria,0.299267987,0.316966398,0.317780833,0.316694628,0.3042738,0.32563299,0.32821332,0.273571683,...,0.47753144,0.476530099,0.446893048,0.47428113,0.472148241,0.456079143,0.503072321,0.491420018,0.457379451,16.841135
174,Europe,Belgium,0.129565222,0.133805659,0.159937719,0.244219026,0.281318312,0.345336836,0.395393388,0.422422976,...,0.560835952,0.483215695,0.41890767,0.588415272,0.591801415,0.465422264,0.629038475,0.552326144,0.695321419,19.076575
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0.068380347,0.05596635,0.051301095,0.051797521,0.034826061,0.059045084,0.056210052,0.0429877,0.063086643,1.502151
176,Europe,Bulgaria,0.102110384,0.132007313,0.143007948,0.162663068,0.165762446,0.159328908,0.149250948,0.155151842,...,0.213691442,0.237726784,0.243382247,0.231943591,0.22206614,0.249101104,0.235489231,0.229710599,0.254519143,8.285138


In [719]:
prod_re_europe_sum = prod_re_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_re_europe_sum = prod_re_europe_sum.sum(axis=0)
prod_re_europe_sum = prod_re_europe_sum.transpose()
prod_re_europe_sum = pd.DataFrame(prod_re_europe_sum)
prod_re_europe_sum = prod_re_europe_sum.rename(columns={0:'Total_RE_Production_Europe'})
prod_re_europe_sum.head()

,Total_RE_Production_Europe
1980,7.472297
1981,8.249006
1982,8.555073
1983,9.362117
1984,10.530915


In [720]:
fig_re_europe = go.Figure()
fig_re_europe = fig_re_europe.add_trace(go.Scatter(x=con_re_europe_sum.index, y=con_re_europe_sum['Total_RE_Consumption_Europe'], name='Total Consumption'))
fig_re_europe = fig_re_europe.add_trace(go.Scatter(x=prod_re_europe_sum.index, y=prod_re_europe_sum['Total_RE_Production_Europe'], name='Total Production'))
fig_re_europe = fig_re_europe.update_layout(title='Total Nuclear & Renewable Energy Consumption & Production Europe 1980 - 2021')
fig_re_europe.show()

Eurasia

In [721]:
con_re_eurasia = con_re[con_re['Continent']== 'Eurasia']
con_re_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Neuclear
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0.042005675,0.041580114,0.046260628,0.043991887,0.045182586,0.038481999,0.041993866,0.043283904,0.039668574,1.202431
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,0.014105394,0.01269628,0.016359663,0.01670132,0.014029954,0.01403485,0.012249339,0.008918935,0.008998921,0.586030
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0.026605268,0.015611877,0.013060177,0.015487673,0.017386172,0.005243862,0.002327971,0.014613803,0.071008016,0.620964
220,Eurasia,Former U.S.S.R.,2.681172229,2.598288915,2.885607616,3.069049694,3.664546377,3.993325699,3.876426507,4.181357059,...,--,--,--,--,--,--,--,--,--,45.424582
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.082632772,0.083144501,0.081700196,0.089363819,0.091518139,0.098303637,0.088445865,0.080510242,0.099856789,2.218518


In [722]:
con_re_eurasia_sum = con_re_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_re_eurasia_sum = con_re_eurasia_sum.sum(axis=0)
con_re_eurasia_sum = con_re_eurasia_sum.transpose()
con_re_eurasia_sum = pd.DataFrame(con_re_eurasia_sum)
con_re_eurasia_sum = con_re_eurasia_sum.rename(columns={0:'Total_RE_Consumption_Eurasia'})
con_re_eurasia_sum.head()

,Total_RE_Consumption_Eurasia
1980,2.681172
1981,2.598289
1982,2.885608
1983,3.069050
1984,3.664546


In [723]:
prod_re_eurasia = prod_re[prod_re['Continent']== 'Eurasia']
prod_re_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Neuclear
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0.045980655,0.04536061,0.050525628,0.047246935,0.049007438,0.043337275,0.045262562,0.04674026,0.041807898,1.260966
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,0.01536101,0.01394166,0.016895347,0.020051904,0.018039054,0.018517536,0.016869187,0.012378703,0.014188573,0.590332
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0.003427365,0.00335754,0.00335132,0.004303712,0.00759175,0.007494999,0.009517966,0.015268081,0.071575474,0.141775
220,Eurasia,Former U.S.S.R.,2.744290817,2.668262211,2.957665644,3.147201554,3.745755389,4.091587887,3.974808115,4.296911263,...,--,--,--,--,--,--,--,--,--,46.500640
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.078121708,0.07847652,0.077990711,0.08534984,0.084814878,0.090439136,0.07950099,0.073159802,0.090054845,2.112227


In [724]:
prod_re_eurasia_sum = prod_re_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_re_eurasia_sum = prod_re_eurasia_sum.sum(axis=0)
prod_re_eurasia_sum = prod_re_eurasia_sum.transpose()
prod_re_eurasia_sum = pd.DataFrame(prod_re_eurasia_sum)
prod_re_eurasia_sum = prod_re_eurasia_sum.rename(columns={0:'Total_RE_Production_Eurasia'})
prod_re_eurasia_sum.head()

,Total_RE_Production_Eurasia
1980,2.744291
1981,2.668262
1982,2.957666
1983,3.147202
1984,3.745755


In [725]:
fig_re_eurasia = go.Figure()
fig_re_eurasia = fig_re_eurasia.add_trace(go.Scatter(x=con_re_eurasia_sum.index, y=con_re_eurasia_sum['Total_RE_Consumption_Eurasia'], name='Total Consumption'))
fig_re_eurasia = fig_re_eurasia.add_trace(go.Scatter(x=prod_re_eurasia_sum.index, y=prod_re_eurasia_sum['Total_RE_Production_Eurasia'], name='Total Production'))
fig_re_eurasia = fig_re_eurasia.update_layout(title='Total Nuclear & Renewable Energy Consumption & Production Eurasia 1980 - 2021')
fig_re_eurasia.show()

Total Nuclear & Renewable Energy by Region

In [726]:
fig_re_region = go.Figure()
fig_re_region = fig_re_region.add_trace(go.Scatter(x=con_re_africa_sum.index, y=con_re_africa_sum['Total_RE_Consumption_Africa'], name='Africa'))
fig_re_region = fig_re_region.add_trace(go.Scatter(x=con_re_asia_sum.index, y=con_re_asia_sum['Total_RE_Consumption_Asia'], name='Asia'))
fig_re_region = fig_re_region.add_trace(go.Scatter(x=con_re_me_sum.index, y=con_re_me_sum['Total_RE_Consumption_Middle_East'], name='Middle East'))
fig_re_region = fig_re_region.add_trace(go.Scatter(x=con_re_csa_sum.index, y=con_re_csa_sum['Total_RE_Consumption_Central_South_America'], name='Central & South America'))
fig_re_region = fig_re_region.add_trace(go.Scatter(x=con_re_na_sum.index, y=con_re_na_sum['Total_RE_Consumption_North_America'], name='North America'))
fig_re_region = fig_re_region.add_trace(go.Scatter(x=con_re_europe_sum.index, y=con_re_europe_sum['Total_RE_Consumption_Europe'], name='Europe'))
fig_re_region = fig_re_region.add_trace(go.Scatter(x=con_re_eurasia_sum.index, y=con_re_eurasia_sum['Total_RE_Consumption_Eurasia'], name='Eurasia'))
fig_re_region = fig_re_region.update_layout(title='Total Nuclear & Renewable Energy Consumption 1980 - 2021')
fig_re_region.show()

Total Nuclear & Renewable Energy Production by Region

In [727]:
fig_re_prod = go.Figure()
fig_re_prod = fig_re_prod.add_trace(go.Scatter(x=prod_re_africa_sum.index, y=prod_re_africa_sum['Total_RE_Production_Africa'], name='Africa'))
fig_re_prod = fig_re_prod.add_trace(go.Scatter(x=prod_re_asia_sum.index, y=prod_re_asia_sum['Total_RE_Production_Asia'], name='Asia'))
fig_re_prod = fig_re_prod.add_trace(go.Scatter(x=prod_re_me_sum.index, y=prod_re_me_sum['Total_RE_Production_Middle_East'], name='Middle East'))
fig_re_prod = fig_re_prod.add_trace(go.Scatter(x=prod_re_csa_sum.index, y=prod_re_csa_sum['Total_RE_Production_Central_South_America'], name='Central & South America'))
fig_re_prod = fig_re_prod.add_trace(go.Scatter(x=prod_re_na_sum.index, y=prod_re_na_sum['Total_RE_Production_North_America'], name='North America'))
fig_re_prod = fig_re_prod.add_trace(go.Scatter(x=prod_re_europe_sum.index, y=prod_re_europe_sum['Total_RE_Production_Europe'], name='Europe'))
fig_re_prod = fig_re_prod.add_trace(go.Scatter(x=prod_re_eurasia_sum.index, y=prod_re_eurasia_sum['Total_RE_Production_Eurasia'], name='Eurasia'))
fig_re_prod = fig_re_prod.update_layout(title='Total Nuclear & Renewable Energy Production 1980 - 2021')
fig_re_prod.show()

#### Petroleum

In [728]:
con_ptl = pd.read_csv('Consumption_Data/Consumption_Petroleum.csv')
con_ptl.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
0,Africa,Algeria,0.232740836,0.243614268,0.274305829,0.260878271,0.288521698,0.316506726,0.318808593,0.324214207,...,0.790232024,0.824079844,0.859255,0.838878,0.804754979,0.826374511,0.872734898,0.79156158,0.795902222,21.695803
1,Africa,Angola,0.043228106,0.043109997,0.045265496,0.047420996,0.041066701,0.053887496,0.051300896,0.051506939,...,0.287508463,0.326254106,0.323138243,0.279812496,0.245592324,0.225400068,0.258252532,0.246590031,0.191971407,5.325508
2,Africa,Benin,0.006127321,0.00611058,0.00407372,0.00611058,0.006127321,0.00712901,0.006938075,0.005365737,...,0.062745586,0.066091262,0.07305374,0.088991268,0.092208488,0.103359397,0.092545774,0.098417665,0.097672206,1.549656
3,Africa,Botswana,0.006088236,0.006071602,0.004047734,0.004047734,0.004058824,0.007285922,0.007122555,0.008547459,...,0.04150615,0.043050315,0.047425,0.044297,0.040708713,0.042608707,0.04342218,0.041461441,0.043748831,1.042886
4,Africa,Burkina Faso,0.0060363,0.006019807,0.006019807,0.006019807,0.0060363,0.005819147,0.006886218,0.007191413,...,0.042193525,0.039385885,0.050896672,0.047726755,0.055806821,0.061206856,0.062082512,0.059278999,0.062549368,0.924135


In [729]:
prod_ptl = pd.read_csv('Production_Data/Production_Pertroleum.csv')
prod_ptl.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Petrolium
0,Africa,Algeria,2.31538521,2.12255246,2.108013674,2.052859918,2.248413236,2.295407851,2.396066243,2.413230191,...,3.520136978,3.435423036,3.579756852,3.390287443,3.251294692,3.139809743,3.127441395,2.80412633,2.818340169,126.127498
1,Africa,Angola,0.31996,0.27654,0.25952,0.376531656,0.443722452,0.491390169,0.599880635,0.765800396,...,3.852835324,3.727531423,3.87386645,3.817211432,3.613781189,3.484186228,3.177631628,2.774720182,2.504617249,87.709676
2,Africa,Benin,0,0,0,0.00896,0.01573,0.01793,0.01793,0.01569,...,0,0,0,0,0,0,0,0,0,0.177631
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


#### Petroleum Production & Consumption by Regions

Africa

In [730]:
con_ptl_africa = con_ptl[con_ptl['Continent']== 'Africa']
con_ptl_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
0,Africa,Algeria,0.232740836,0.243614268,0.274305829,0.260878271,0.288521698,0.316506726,0.318808593,0.324214207,...,0.790232024,0.824079844,0.859255,0.838878,0.804754979,0.826374511,0.872734898,0.79156158,0.795902222,21.695803
1,Africa,Angola,0.043228106,0.043109997,0.045265496,0.047420996,0.041066701,0.053887496,0.051300896,0.051506939,...,0.287508463,0.326254106,0.323138243,0.279812496,0.245592324,0.225400068,0.258252532,0.246590031,0.191971407,5.325508
2,Africa,Benin,0.006127321,0.00611058,0.00407372,0.00611058,0.006127321,0.00712901,0.006938075,0.005365737,...,0.062745586,0.066091262,0.07305374,0.088991268,0.092208488,0.103359397,0.092545774,0.098417665,0.097672206,1.549656
3,Africa,Botswana,0.006088236,0.006071602,0.004047734,0.004047734,0.004058824,0.007285922,0.007122555,0.008547459,...,0.04150615,0.043050315,0.047425,0.044297,0.040708713,0.042608707,0.04342218,0.041461441,0.043748831,1.042886
4,Africa,Burkina Faso,0.0060363,0.006019807,0.006019807,0.006019807,0.0060363,0.005819147,0.006886218,0.007191413,...,0.042193525,0.039385885,0.050896672,0.047726755,0.055806821,0.061206856,0.062082512,0.059278999,0.062549368,0.924135


In [731]:
con_ptl_africa_sum = con_ptl_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ptl_africa_sum = con_ptl_africa_sum.sum(axis=0)
con_ptl_africa_sum = con_ptl_africa_sum.transpose()
con_ptl_africa_sum = pd.DataFrame(con_ptl_africa_sum)
con_ptl_africa_sum = con_ptl_africa_sum.rename(columns={0:'Total_Ptl_Consumption_Africa'})
con_ptl_africa_sum.head()

,Total_Ptl_Consumption_Africa
1980,3.071978
1981,3.281095
1982,3.450872
1983,3.535079
1984,3.672176


In [732]:
prod_ptl_africa = prod_ptl[prod_ptl['Continent']== 'Africa']
prod_ptl_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Petrolium
0,Africa,Algeria,2.31538521,2.12255246,2.108013674,2.052859918,2.248413236,2.295407851,2.396066243,2.413230191,...,3.520136978,3.435423036,3.579756852,3.390287443,3.251294692,3.139809743,3.127441395,2.80412633,2.818340169,126.127498
1,Africa,Angola,0.31996,0.27654,0.25952,0.376531656,0.443722452,0.491390169,0.599880635,0.765800396,...,3.852835324,3.727531423,3.87386645,3.817211432,3.613781189,3.484186228,3.177631628,2.774720182,2.504617249,87.709676
2,Africa,Benin,0,0,0,0.00896,0.01573,0.01793,0.01793,0.01569,...,0,0,0,0,0,0,0,0,0,0.177631
3,Africa,Botswana,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [733]:
prod_ptl_africa_sum = prod_ptl_africa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ptl_africa_sum = prod_ptl_africa_sum.sum(axis=0)
prod_ptl_africa_sum = prod_ptl_africa_sum.transpose()
prod_ptl_africa_sum = pd.DataFrame(prod_ptl_africa_sum)
prod_ptl_africa_sum = prod_ptl_africa_sum.rename(columns={0:'Total_Ptl_Production_Africa'})
prod_ptl_africa_sum.head()

,Total_Ptl_Production_Africa
1980,13.139405
1981,10.254687
1982,10.202050
1983,10.222938
1984,11.149524


In [734]:
fig_ptl_africa = go.Figure()
fig_ptl_africa = fig_ptl_africa.add_trace(go.Scatter(x=con_ptl_africa_sum.index, y=con_ptl_africa_sum['Total_Ptl_Consumption_Africa'], name='Total Consumption'))
fig_ptl_africa = fig_ptl_africa.add_trace(go.Scatter(x=prod_ptl_africa_sum.index, y=prod_ptl_africa_sum['Total_Ptl_Production_Africa'], name='Total Production'))
fig_ptl_africa = fig_ptl_africa.update_layout(title='Total Petroleum Consumption & Production Africa  1980 - 2021')
fig_ptl_africa.show()

In [735]:
con_ptl_africa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
0,Africa,Algeria,0.232740836,0.243614268,0.274305829,0.260878271,0.288521698,0.316506726,0.318808593,0.324214207,...,0.790232024,0.824079844,0.859255,0.838878,0.804754979,0.826374511,0.872734898,0.79156158,0.795902222,21.695803
1,Africa,Angola,0.043228106,0.043109997,0.045265496,0.047420996,0.041066701,0.053887496,0.051300896,0.051506939,...,0.287508463,0.326254106,0.323138243,0.279812496,0.245592324,0.225400068,0.258252532,0.246590031,0.191971407,5.325508
2,Africa,Benin,0.006127321,0.00611058,0.00407372,0.00611058,0.006127321,0.00712901,0.006938075,0.005365737,...,0.062745586,0.066091262,0.07305374,0.088991268,0.092208488,0.103359397,0.092545774,0.098417665,0.097672206,1.549656
3,Africa,Botswana,0.006088236,0.006071602,0.004047734,0.004047734,0.004058824,0.007285922,0.007122555,0.008547459,...,0.04150615,0.043050315,0.047425,0.044297,0.040708713,0.042608707,0.04342218,0.041461441,0.043748831,1.042886
4,Africa,Burkina Faso,0.0060363,0.006019807,0.006019807,0.006019807,0.0060363,0.005819147,0.006886218,0.007191413,...,0.042193525,0.039385885,0.050896672,0.047726755,0.055806821,0.061206856,0.062082512,0.059278999,0.062549368,0.924135


In [736]:
top_ptl_con_africa_1980 = con_ptl_africa[['Country','1980']]
top_ptl_con_africa_1980['1980'] = pd.to_numeric(top_ptl_con_africa_1980['1980'], errors='coerce')
top_ptl_con_africa_1980 = top_ptl_con_africa_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_ptl_con_africa_1980['portion_africa'] = top_ptl_con_africa_1980['1980']/con_ptl_africa_sum.loc['1980'][0]*100
top_ptl_con_africa_1980['cumulative'] = top_ptl_con_africa_1980['portion_africa'].cumsum()
top_ptl_con_africa_1980.head()


,Country,1980,portion_africa,cumulative
47,South Africa,0.647961,21.092643,21.092643
15,Egypt,0.559835,18.223925,39.316568
38,Nigeria,0.347140,11.300204,50.616772
0,Algeria,0.232741,7.576254,58.193025
28,Libya,0.213289,6.943056,65.136081


In [737]:
top_ptl_con_africa_1990 = con_ptl_africa[['Country','1990']]
top_ptl_con_africa_1990['1990'] = pd.to_numeric(top_ptl_con_africa_1990['1990'], errors='coerce')
top_ptl_con_africa_1990 = top_ptl_con_africa_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_ptl_con_africa_1990['portion_africa'] = top_ptl_con_africa_1990['1990']/con_ptl_africa_sum.loc['1990'][0]*100
top_ptl_con_africa_1990['cumulative'] = top_ptl_con_africa_1990['portion_africa'].cumsum()
top_ptl_con_africa_1990.head()


,Country,1990,portion_africa,cumulative
15,Egypt,0.990391,23.025844,23.025844
47,South Africa,0.782140,18.184169,41.210013
38,Nigeria,0.512627,11.918191,53.128204
0,Algeria,0.379428,8.821420,61.949624
28,Libya,0.324814,7.551680,69.501304


In [738]:
top_ptl_con_africa_2000 = con_ptl_africa[['Country','2000']]
top_ptl_con_africa_2000['2000'] = pd.to_numeric(top_ptl_con_africa_2000['2000'], errors='coerce')
top_ptl_con_africa_2000 = top_ptl_con_africa_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_ptl_con_africa_2000['portion_africa'] = top_ptl_con_africa_2000['2000']/con_ptl_africa_sum.loc['2000'][0]*100
top_ptl_con_africa_2000['cumulative'] = top_ptl_con_africa_2000['portion_africa'].cumsum()
top_ptl_con_africa_2000.head()

,Country,2000,portion_africa,cumulative
15,Egypt,1.155857,22.233857,22.233857
47,South Africa,0.938368,18.050271,40.284127
28,Libya,0.534334,10.278345,50.562473
38,Nigeria,0.458982,8.828885,59.391357
0,Algeria,0.438188,8.428904,67.820261


In [739]:
top_ptl_con_africa_2010 = con_ptl_africa[['Country','2010']]
top_ptl_con_africa_2010['2010'] = pd.to_numeric(top_ptl_con_africa_2010['2010'], errors='coerce')
top_ptl_con_africa_2010 = top_ptl_con_africa_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_ptl_con_africa_2010['portion_africa'] = top_ptl_con_africa_2010['2010']/con_ptl_africa_sum.loc['2010'][0]*100
top_ptl_con_africa_2010['cumulative'] = top_ptl_con_africa_2010['portion_africa'].cumsum()
top_ptl_con_africa_2010.head()

,Country,2010,portion_africa,cumulative
15,Egypt,1.623577,21.764254,21.764254
47,South Africa,1.217447,16.320037,38.084290
0,Algeria,0.685783,9.193009,47.277299
28,Libya,0.646682,8.668855,55.946155
38,Nigeria,0.582799,7.812500,63.758654


In [740]:
top_ptl_con_africa_2020 = con_ptl_africa[['Country','2020']]
top_ptl_con_africa_2020['2020'] = pd.to_numeric(top_ptl_con_africa_2020['2020'], errors='coerce')
top_ptl_con_africa_2020 = top_ptl_con_africa_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_ptl_con_africa_2020['portion_africa'] = top_ptl_con_africa_2020['2020']/con_ptl_africa_sum.loc['2020'][0]*100
top_ptl_con_africa_2020['cumulative'] = top_ptl_con_africa_2020['portion_africa'].cumsum()
top_ptl_con_africa_2020.head()

,Country,2020,portion_africa,cumulative
15,Egypt,1.305122,16.398544,16.398544
47,South Africa,1.104178,13.873735,30.272279
38,Nigeria,0.967691,12.158811,42.431090
0,Algeria,0.791562,9.945784,52.376874
34,Morocco,0.524340,6.588203,58.965078


In [741]:
top_ptl_prod_africa_1980 = prod_ptl_africa[['Country','1980']]
top_ptl_prod_africa_1980['1980'] = pd.to_numeric(top_ptl_prod_africa_1980['1980'], errors='coerce')
top_ptl_prod_africa_1980 = top_ptl_prod_africa_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_ptl_prod_africa_1980['portion_africa'] = top_ptl_prod_africa_1980['1980']/prod_ptl_africa_sum.loc['1980'][0]*100
top_ptl_prod_africa_1980['cumulative'] = top_ptl_prod_africa_1980['portion_africa'].cumsum()
top_ptl_prod_africa_1980.head()

,Country,1980,portion_africa,cumulative
38,Nigeria,4.422161,33.655720,33.655720
28,Libya,3.840462,29.228585,62.884306
0,Algeria,2.315385,17.621690,80.505996
15,Egypt,1.320366,10.048904,90.554899
20,Gabon,0.377130,2.870221,93.425121


In [742]:
top_ptl_prod_africa_1990 = prod_ptl_africa[['Country','1990']]
top_ptl_prod_africa_1990['1990'] = pd.to_numeric(top_ptl_prod_africa_1990['1990'], errors='coerce')
top_ptl_prod_africa_1990 = top_ptl_prod_africa_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_ptl_prod_africa_1990 ['portion_africa'] = top_ptl_prod_africa_1990['1990']/prod_ptl_africa_sum.loc['1990'][0]*100
top_ptl_prod_africa_1990['cumulative'] = top_ptl_prod_africa_1990['portion_africa'].cumsum()
top_ptl_prod_africa_1990.head()

,Country,1990,portion_africa,cumulative
38,Nigeria,3.884628,27.736443,27.736443
28,Libya,2.953899,21.090990,48.827432
0,Algeria,2.643000,18.871154,67.698586
15,Egypt,1.959142,13.988374,81.686961
1,Angola,1.010433,7.214543,88.901504


In [743]:
top_ptl_prod_africa_2000 = prod_ptl_africa[['Country','2000']]
top_ptl_prod_africa_2000['2000'] = pd.to_numeric(top_ptl_prod_africa_2000['2000'], errors='coerce')
top_ptl_prod_africa_2000 = top_ptl_prod_africa_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_ptl_prod_africa_2000['portion_africa'] = top_ptl_prod_africa_2000['2000']/prod_ptl_africa_sum.loc['2000'][0]*100
top_ptl_prod_africa_2000['cumulative'] = top_ptl_prod_africa_2000['portion_africa'].cumsum()
top_ptl_prod_africa_2000.head()

,Country,2000,portion_africa,cumulative
38,Nigeria,4.659259,28.373638,28.373638
28,Libya,3.075829,18.730974,47.104612
0,Algeria,2.877078,17.520634,64.625246
15,Egypt,1.728799,10.527924,75.153169
1,Angola,1.592120,9.695584,84.848754


In [744]:
top_ptl_prod_africa_2010 = prod_ptl_africa[['Country','2010']]
top_ptl_prod_africa_2010['2010'] = pd.to_numeric(top_ptl_prod_africa_2010['2010'], errors='coerce')
top_ptl_prod_africa_2010 = top_ptl_prod_africa_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_ptl_prod_africa_2010['portion_africa'] = top_ptl_prod_africa_2010['2010']/prod_ptl_africa_sum.loc['2010'][0]*100
top_ptl_prod_africa_2010['cumulative'] = top_ptl_prod_africa_2010['portion_africa'].cumsum()
top_ptl_prod_africa_2010.head()

,Country,2010,portion_africa,cumulative
38,Nigeria,5.247503,23.911718,23.911718
1,Angola,4.075000,18.568883,42.480601
28,Libya,3.826750,17.437661,59.918262
0,Algeria,3.752901,17.101145,77.019407
15,Egypt,1.484397,6.764071,83.783478


In [745]:
top_ptl_prod_africa_2020 = prod_ptl_africa[['Country','2020']]
top_ptl_prod_africa_2020['2020'] = pd.to_numeric(top_ptl_prod_africa_2020['2020'], errors='coerce')
top_ptl_prod_africa_2020 = top_ptl_prod_africa_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_ptl_prod_africa_2020['portion_africa'] = top_ptl_prod_africa_2020['2020']/prod_ptl_africa_sum.loc['2020'][0]*100
top_ptl_prod_africa_2020['cumulative'] = top_ptl_prod_africa_2020['portion_africa'].cumsum()
top_ptl_prod_africa_2020.head()

,Country,2020,portion_africa,cumulative
38,Nigeria,3.930915,26.891355,26.891355
0,Algeria,2.804126,19.183004,46.074358
1,Angola,2.774720,18.981837,65.056195
15,Egypt,1.444825,9.884032,74.940227
28,Libya,0.878446,6.009440,80.949668


Asia

In [746]:
con_ptl['Continent'].unique()

array(['Africa', 'Asia & Oceania', 'Middle East',
       'Central & South America', 'North America', 'Europe', 'Eurasia'],
      dtype=object)

In [747]:
con_ptl_asia = con_ptl[con_ptl['Continent']== 'Asia & Oceania']
con_ptl_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
57,Asia & Oceania,Afghanistan,0.014624098,0.01666759,0.016250901,0.022917937,0.022980726,0.027084834,0.028329758,0.028675112,...,0.069950161,0.05528505,0.069916271,0.051517535,0.047425619,0.060538076,0.048550172,0.046011906,0.047513034,1.384603
58,Asia & Oceania,American Samoa,0.005893112,0.006926477,0.00965509,0.006086904,0.006945454,0.006506691,0.006716584,0.006973009,...,0.004911509,0.004911509,0.004911507,0.004924965,0.004911509,0.004911509,0.004911509,0.004678618,0.005402545,0.296398
59,Asia & Oceania,Australia,1.225279062,1.195341055,1.26261968,1.21632527,1.289833149,1.285845086,1.295406942,1.326613993,...,2.308754931,2.283645693,2.274030723,2.264430337,2.363417525,2.415695145,2.382939067,2.054869706,2.092467467,74.817896
60,Asia & Oceania,Bangladesh,0.068065241,0.070000498,0.072121725,0.063636816,0.065938203,0.067879271,0.072970216,0.070795729,...,0.222972384,0.238612049,0.240133227,0.2401794,0.261981113,0.295615386,0.260047464,0.246451698,0.340353325,6.320251
61,Asia & Oceania,Bhutan,3.84E-05,0.000383126,0.000383126,0.000383126,0.000384176,0.000383126,0.000401478,0.000477224,...,0.00638686,0.005985097,0.006696,0.007617004,0.008325307,0.009268214,0.008923738,0.008457161,0.008733073,0.133256


In [748]:
con_ptl_asia_sum = con_ptl_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ptl_asia_sum = con_ptl_asia_sum.sum(axis=0)
con_ptl_asia_sum = con_ptl_asia_sum.transpose()
con_ptl_asia_sum = pd.DataFrame(con_ptl_asia_sum)
con_ptl_asia_sum = con_ptl_asia_sum.rename(columns={0:'Total_Ptl_Consumption_Asia'})
con_ptl_asia_sum.head()

,Total_Ptl_Consumption_Asia
1980,23.343826
1981,23.031164
1982,22.482536
1983,22.298415
1984,22.884483


In [749]:
prod_ptl_asia = prod_ptl[prod_ptl['Continent']== 'Asia & Oceania']
prod_ptl_asia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Petrolium
57,Asia & Oceania,Afghanistan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
58,Asia & Oceania,American Samoa,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
59,Asia & Oceania,Australia,0.8809884,0.907481,0.8454042,0.9283842,1.08922756,1.27246025,1.152651,1.21546025,...,0.771798621,0.810895374,0.732742806,0.678135634,0.616626017,0.669224661,0.878312784,0.877532315,0.841209898,44.453500
60,Asia & Oceania,Bangladesh,0,0,0,0.000422222,0.000846758,0.001499031,0.001507305,0.001503475,...,0.008752481,0.008752481,0.008752481,0.00877646,0.025489164,0.024081623,0.022603373,0.022374583,0.021841799,0.334548
61,Asia & Oceania,Bhutan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [750]:
prod_ptl_asia_sum = prod_ptl_asia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ptl_asia_sum = prod_ptl_asia_sum.sum(axis=0)
prod_ptl_asia_sum = prod_ptl_asia_sum.transpose()
prod_ptl_asia_sum = pd.DataFrame(prod_ptl_asia_sum)
prod_ptl_asia_sum = prod_ptl_asia_sum.rename(columns={0:'Total_Ptl_Production_Asia'})
prod_ptl_asia_sum.head()

,Total_Ptl_Production_Asia
1980,10.604357
1981,10.541090
1982,10.175703
1983,10.803810
1984,11.758629


In [751]:
fig_ptl_asia = go.Figure()
fig_ptl_asia = fig_ptl_asia.add_trace(go.Scatter(x=con_ptl_asia_sum.index, y=con_ptl_asia_sum['Total_Ptl_Consumption_Asia'], name='Total Consumption'))
fig_ptl_asia = fig_ptl_asia.add_trace(go.Scatter(x=prod_ptl_asia_sum.index, y=prod_ptl_asia_sum['Total_Ptl_Production_Asia'], name='Total Production'))
fig_ptl_asia = fig_ptl_asia.update_layout(title='Total Petroleum Consumption & Production Asia 1980 - 2021')
fig_ptl_asia.show()

In [752]:
top_ptl_con_1980 = con_ptl_asia[['Country','1980']]
top_ptl_con_1980['1980'] = pd.to_numeric(top_ptl_con_1980['1980'], errors='coerce')
top_ptl_con_1980 = top_ptl_con_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_ptl_con_1980['portion_asia'] = top_ptl_con_1980['1980']/con_ptl_asia_sum.loc['1980'][0]*100
top_ptl_con_1980['cumulative'] = top_ptl_con_1980['portion_asia'].cumsum()
top_ptl_con_1980.head()

,Country,1980,portion_asia,cumulative
17,Japan,10.735107,45.986922,45.986922
8,China,4.124019,17.666423,63.653345
15,India,1.352522,5.793918,69.447263
2,Australia,1.225279,5.248836,74.696098
38,South Korea,1.186886,5.084366,79.780465


In [753]:
top_ptl_con_asia_1990 = con_ptl_asia[['Country','1990']]
top_ptl_con_asia_1990['1990'] = pd.to_numeric(top_ptl_con_asia_1990['1990'], errors='coerce')
top_ptl_con_asia_1990 = top_ptl_con_asia_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_ptl_con_asia_1990['portion_asia'] = top_ptl_con_asia_1990['1990']/con_ptl_asia_sum.loc['1990'][0]*100
top_ptl_con_asia_1990['cumulative'] = top_ptl_con_asia_1990['portion_asia'].cumsum()
top_ptl_con_asia_1990.head()

,Country,1990,portion_asia,cumulative
17,Japan,10.839079,37.759009,37.759009
8,China,4.911002,17.107965,54.866974
15,India,2.435769,8.485242,63.352216
38,South Korea,2.203745,7.676965,71.029181
2,Australia,1.475460,5.139911,76.169092


In [754]:
top_ptl_con_asia_2000 = con_ptl_asia[['Country','2000']]
top_ptl_con_asia_2000['2000'] = pd.to_numeric(top_ptl_con_asia_2000['2000'], errors='coerce')
top_ptl_con_asia_2000 = top_ptl_con_asia_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_ptl_con_asia_2000['portion_asia'] = top_ptl_con_asia_2000['2000']/con_ptl_asia_sum.loc['2000'][0]*100
top_ptl_con_asia_2000['cumulative'] = top_ptl_con_asia_2000['portion_asia'].cumsum()
top_ptl_con_asia_2000.head()

,Country,2000,portion_asia,cumulative
17,Japan,11.124251,25.545960,25.545960
8,China,9.652491,22.166179,47.712139
15,India,4.709073,10.814013,58.526152
38,South Korea,4.438430,10.192503,68.718655
16,Indonesia,2.470055,5.672285,74.390940


In [755]:
top_ptl_con_asia_2010 = con_ptl_asia[['Country','2010']]
top_ptl_con_asia_2010['2010'] = pd.to_numeric(top_ptl_con_asia_2010['2010'], errors='coerce')
top_ptl_con_asia_2010 = top_ptl_con_asia_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_ptl_con_asia_2010['portion_asia'] = top_ptl_con_asia_2010['2010']/con_ptl_asia_sum.loc['2010'][0]*100
top_ptl_con_asia_2010['cumulative'] = top_ptl_con_asia_2010['portion_asia'].cumsum()
top_ptl_con_asia_2010.head()

,Country,2010,portion_asia,cumulative
8,China,18.686604,33.105488,33.105488
17,Japan,8.871108,15.716198,48.821686
15,India,6.595881,11.685368,60.507055
38,South Korea,4.648181,8.234792,68.741847
16,Indonesia,3.048761,5.401235,74.143082


In [756]:
top_ptl_con_asia_2020 = con_ptl_asia[['Country','2020']]
top_ptl_con_asia_2020['2020'] = pd.to_numeric(top_ptl_con_asia_2020['2020'], errors='coerce')
top_ptl_con_asia_2020 = top_ptl_con_asia_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_ptl_con_asia_2020['portion_asia'] = top_ptl_con_asia_2020['2020']/con_ptl_asia_sum.loc['2020'][0]*100
top_ptl_con_asia_2020['cumulative'] = top_ptl_con_asia_2020['portion_asia'].cumsum()
top_ptl_con_asia_2020.head()

,Country,2020,portion_asia,cumulative
8,China,29.389711,43.717944,43.717944
15,India,8.816590,13.114902,56.832846
17,Japan,6.765238,10.063464,66.896309
38,South Korea,4.997970,7.434608,74.330917
16,Indonesia,2.952345,4.391688,78.722605


In [757]:
top_ptl__prod_asia_1980 = prod_ptl_asia[['Country','1980']]
top_ptl__prod_asia_1980['1980'] = pd.to_numeric(top_ptl__prod_asia_1980['1980'], errors='coerce')
top_ptl__prod_asia_1980 = top_ptl__prod_asia_1980.reset_index(drop=True).sort_values(by='1980', ascending=False,na_position='last')
top_ptl__prod_asia_1980['portion_asia'] = top_ptl__prod_asia_1980['1980']/prod_ptl_asia_sum.loc['1980'][0]*100
top_ptl__prod_asia_1980['cumulative'] = top_ptl__prod_asia_1980['portion_asia'].cumsum()
top_ptl__prod_asia_1980.head()

,Country,1980,portion_asia,cumulative
8,China,4.548403,42.891828,42.891828
16,Indonesia,3.469567,32.718317,75.610145
2,Australia,0.880988,8.307796,83.917942
21,Malaysia,0.588010,5.544985,89.462926
5,Brunei,0.562159,5.301208,94.764135


In [758]:
top_ptl__prod_asia_1990 = prod_ptl_asia[['Country','1990']]
top_ptl__prod_asia_1990['1990'] = pd.to_numeric(top_ptl__prod_asia_1990['1990'], errors='coerce')
top_ptl__prod_asia_1990 = top_ptl__prod_asia_1990.reset_index(drop=True).sort_values(by='1990', ascending=False,na_position='last')
top_ptl__prod_asia_1990['portion_asia'] = top_ptl__prod_asia_1990['1990']/prod_ptl_asia_sum.loc['1990'][0]*100
top_ptl__prod_asia_1990['cumulative'] = top_ptl__prod_asia_1990['portion_asia'].cumsum()
top_ptl__prod_asia_1990.head()

,Country,1990,portion_asia,cumulative
8,China,5.952549,42.629997,42.629997
16,Indonesia,3.177147,22.753579,65.383576
15,India,1.396909,10.004158,75.387734
21,Malaysia,1.306473,9.356487,84.744221
2,Australia,1.269327,9.090458,93.834679


In [759]:
top_ptl__prod_asia_2000 = prod_ptl_asia[['Country','2000']]
top_ptl__prod_asia_2000['2000'] = pd.to_numeric(top_ptl__prod_asia_2000['2000'], errors='coerce')
top_ptl__prod_asia_2000 = top_ptl__prod_asia_2000.reset_index(drop=True).sort_values(by='2000', ascending=False,na_position='last')
top_ptl__prod_asia_2000['portion_asia'] = top_ptl__prod_asia_2000['2000']/prod_ptl_asia_sum.loc['2000'][0]*100
top_ptl__prod_asia_2000['cumulative'] = top_ptl__prod_asia_2000['portion_asia'].cumsum()
top_ptl__prod_asia_2000.head()

,Country,2000,portion_asia,cumulative
8,China,6.990412,41.939232,41.939232
16,Indonesia,3.134539,18.805781,60.745013
2,Australia,1.583015,9.497354,70.242367
21,Malaysia,1.543698,9.261469,79.503837
15,India,1.506785,9.040012,88.543849


In [760]:
top_ptl__prod_asia_2010 = prod_ptl_asia[['Country','2010']]
top_ptl__prod_asia_2010['2010'] = pd.to_numeric(top_ptl__prod_asia_2010['2010'], errors='coerce')
top_ptl__prod_asia_2010 = top_ptl__prod_asia_2010.reset_index(drop=True).sort_values(by='2010', ascending=False,na_position='last')
top_ptl__prod_asia_2010['portion_asia'] = top_ptl__prod_asia_2010['2010']/prod_ptl_asia_sum.loc['2010'][0]*100
top_ptl__prod_asia_2010['cumulative'] = top_ptl__prod_asia_2010['portion_asia'].cumsum()
top_ptl__prod_asia_2010.head()

,Country,2010,portion_asia,cumulative
8,China,8.757405,49.990973,49.990973
16,Indonesia,2.047802,11.689722,61.680695
15,India,1.765142,10.076178,71.756874
21,Malaysia,1.430225,8.164330,79.921204
2,Australia,1.085176,6.194645,86.115849


In [761]:
top_ptl__prod_asia_2020 = prod_ptl_asia[['Country','2020']]
top_ptl__prod_asia_2020['2020'] = pd.to_numeric(top_ptl__prod_asia_2020['2020'], errors='coerce')
top_ptl__prod_asia_2020 = top_ptl__prod_asia_2020.reset_index(drop=True).sort_values(by='2020', ascending=False,na_position='last')
top_ptl__prod_asia_2020['portion_asia'] = top_ptl__prod_asia_2020['2020']/prod_ptl_asia_sum.loc['2020'][0]*100
top_ptl__prod_asia_2020['cumulative'] = top_ptl__prod_asia_2020['portion_asia'].cumsum()
top_ptl__prod_asia_2020.head()

,Country,2020,portion_asia,cumulative
8,China,8.375406,54.287792,54.287792
16,Indonesia,1.525825,9.890105,64.177898
15,India,1.516645,9.830604,74.008501
21,Malaysia,1.237100,8.018648,82.027150
2,Australia,0.877532,5.687998,87.715148


Middle East

In [762]:
con_ptl_me = con_ptl[con_ptl['Continent']== 'Middle East']
con_ptl_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
106,Middle East,Bahrain,0.032379245,0.032290777,0.032290777,0.030272604,0.032379245,0.032290777,0.032527772,0.038620139,...,0.114115104,0.115452356,0.12157146,0.1263442,0.125449438,0.127593349,0.137456625,0.133411255,0.117638874,3.241501
107,Middle East,Iran,1.249544875,1.225009957,1.393976847,1.657987614,1.620172592,1.668548045,1.75303149,1.82992823,...,3.724924154,3.744518839,3.520810708,3.22291186,3.429196966,3.540354937,3.453822232,3.146774921,3.178182441,113.084062
108,Middle East,Iraq,0.457364255,0.449808893,0.430891697,0.525477679,0.50584065,0.542292965,0.58643309,0.59445068,...,1.597897,1.560104,1.329023,1.655434,1.890502284,2.002352864,1.700833593,1.362792813,1.297570679,42.177651
109,Middle East,Israel,0.350857482,0.35632614,0.35160815,0.32613626,0.269124773,0.282349904,0.304413894,0.336488409,...,0.453867955,0.415667719,0.440061074,0.44749589,0.481137604,0.469180189,0.474113726,0.426006798,0.443433991,18.464180
110,Middle East,Jordan,0.077824544,0.092295242,0.104880957,0.111173815,0.117788499,0.12166191,0.130262149,0.139650533,...,0.289351626,0.324388658,0.277975053,0.234357,0.239853917,0.220465346,0.227207574,0.176112663,0.17624902,7.954004


In [763]:
con_ptl_me_sum = con_ptl_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ptl_me_sum = con_ptl_me_sum.sum(axis=0)
con_ptl_me_sum = con_ptl_me_sum.transpose()
con_ptl_me_sum = pd.DataFrame(con_ptl_me_sum)
con_ptl_me_sum = con_ptl_me_sum.rename(columns={0:'Total_Ptl_Consumption_Middle_East'})
con_ptl_me_sum.head()

,Total_Ptl_Consumption_Middle_East
1980,4.353613
1981,4.639664
1982,4.961577
1983,5.498737
1984,5.596714


In [764]:
prod_ptl_me = prod_ptl[prod_ptl['Continent']== 'Middle East']
prod_ptl_me.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Petrolium
106,Middle East,Bahrain,0.116052774,0.111407333,0.106735188,0.10420951,0.102511868,0.100455532,0.106776342,0.105126684,...,0.440929151,0.451370841,0.451058702,0.450975985,0.444753011,0.43994903,0.440162985,0.384412476,0.39157903,8.734657
107,Middle East,Iran,3.595868362,2.979975687,4.772082503,5.256025048,4.701583028,4.852409277,4.397241335,4.970544048,...,7.07173293,7.346027447,7.482209286,9.391374771,10.07692964,9.591895563,6.846164782,6.238555285,7.223937568,305.128101
108,Middle East,Iraq,5.378475363,2.132031055,2.157945915,2.143006075,2.58331542,3.0603103,3.6060566,4.4318537,...,6.554034956,7.23833245,8.69292569,9.577750352,9.589614974,9.931088601,10.14651261,8.827752592,8.799139,202.223079
109,Middle East,Israel,0.07592,0.07788,0.00216,0.00216,0.00217,0.00216,0.00216,0.000651724,...,0.000842691,0.000842691,0.000842691,0.000845,0,0,0,0,0,0.177989
110,Middle East,Jordan,0,0,0,0,0,0,0.00218,0.00109,...,4.78E-05,4.78E-05,4.78E-05,4.80E-05,4.78E-05,4.78E-05,4.78E-05,4.36E-05,4.35E-05,0.007241


In [765]:
prod_ptl_me_sum = prod_ptl_me.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ptl_me_sum = prod_ptl_me_sum.sum(axis=0)
prod_ptl_me_sum = prod_ptl_me_sum.transpose()
prod_ptl_me_sum = pd.DataFrame(prod_ptl_me_sum)
prod_ptl_me_sum = prod_ptl_me_sum.rename(columns={0:'Total_Ptl_Production_Middle_East'})
prod_ptl_me_sum.head()

,Total_Ptl_Production_Middle_East
1980,40.647386
1981,34.831897
1982,28.116600
1983,25.837469
1984,25.433135


In [766]:
fig_ptl_me = go.Figure()
fig_ptl_me = fig_ptl_me.add_trace(go.Scatter(x=con_ptl_me_sum.index, y=con_ptl_me_sum['Total_Ptl_Consumption_Middle_East'], name='Total Consumption'))
fig_ptl_me = fig_ptl_me.add_trace(go.Scatter(x=prod_ptl_me_sum.index, y=prod_ptl_me_sum['Total_Ptl_Production_Middle_East'], name='Total Production'))
fig_ptl_me = fig_ptl_me.update_layout(title='Total Petroleum Consumption & Production Middle East 1980 - 2021')
fig_ptl_me.show()

Central & South America

In [767]:
con_ptl_csa = con_ptl[con_ptl['Continent']== 'Central & South America']
con_ptl_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
120,Central & South America,Antarctica,0.00213195,0.002126125,0.002126125,0.002126125,0.002771535,0.002976575,0.002763963,0.002763963,...,0.000190919,0.000190919,0.00018941,0.000189246,0.000186509,0.000186509,0.000186509,0.000157013,0.000167087,0.085141
121,Central & South America,Antigua and Barbuda,0.005804855,0.005168746,0.007029494,0.004134997,0.004560958,0.004548496,0.004624394,0.005755917,...,0.009916654,0.010139449,0.01033443,0.010489332,0.009966558,0.010236337,0.010266446,0.008645598,0.009201665,0.321976
122,Central & South America,Argentina,1.102051856,1.062836481,1.030312086,1.050200405,1.024447508,0.921991082,0.992453443,0.999429765,...,1.56905934,1.576160671,1.634033684,1.595071,1.402704816,1.270231936,1.258250833,0.910387122,1.225471882,48.193798
123,Central & South America,Aruba,--,--,--,--,--,--,0.001453868,0.001453868,...,0.016187071,0.016386298,0.016791,0.016521,0.017210446,0.016569926,0.017258441,0.01453378,0.015468564,0.496912
124,Central & South America,Barbados,0.014188604,0.014783413,0.014994604,0.015839371,0.015247455,0.016895329,0.017740095,0.016871076,...,0.024345011,0.021297151,0.022528,0.022835,0.021450166,0.022899732,0.022899732,0.019284313,0.020524641,0.848446


In [768]:
con_ptl_csa_sum = con_ptl_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ptl_csa_sum = con_ptl_csa_sum.sum(axis=0)
con_ptl_csa_sum = con_ptl_csa_sum.transpose()
con_ptl_csa_sum = pd.DataFrame(con_ptl_csa_sum)
con_ptl_csa_sum = con_ptl_csa_sum.rename(columns={0:'Total_Ptl_Consumption_Central_South_America'})
con_ptl_csa_sum.head()

,Total_Ptl_Consumption_Central_South_America
1980,7.070993
1981,6.916840
1982,6.826470
1983,6.495828
1984,6.443785


In [769]:
prod_ptl_csa = prod_ptl[prod_ptl['Continent']== 'Central & South America']
prod_ptl_csa.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Petrolium
120,Central & South America,Antarctica,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
121,Central & South America,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
122,Central & South America,Argentina,1.09515556,1.1017016,1.0907716,1.08518856,1.0694318,1.0341133,0.9925733,0.9864248,...,1.326100838,1.30908065,1.315550653,1.276828441,1.208819093,1.254309097,1.304008846,1.231858832,1.291034915,58.768786
123,Central & South America,Aruba,--,--,--,--,--,--,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
124,Central & South America,Barbados,0,0,0.00214,0.00214,0.0043,0.00428,0.00428,0.002995588,...,0.00214142,0.00214142,0.00214142,0.002147287,0.00214142,0.00214142,0.00214142,0.002147287,0.00214142,0.102214


In [770]:
prod_ptl_csa_sum = prod_ptl_csa.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ptl_csa_sum = prod_ptl_csa_sum.sum(axis=0)
prod_ptl_csa_sum = prod_ptl_csa_sum.transpose()
prod_ptl_csa_sum = pd.DataFrame(prod_ptl_csa_sum)
prod_ptl_csa_sum = prod_ptl_csa_sum.rename(columns={0:'Total_Ptl_Production_Central_South_America'})
prod_ptl_csa_sum.head()

,Total_Ptl_Production_Central_South_America
1980,8.294547
1981,8.275871
1982,7.835343
1983,7.761489
1984,8.158450


In [771]:
fig_ptl_csa = go.Figure()
fig_ptl_csa = fig_ptl_csa.add_trace(go.Scatter(x=con_ptl_csa_sum.index, y=con_ptl_csa_sum['Total_Ptl_Consumption_Central_South_America'], name='Total Consumption'))
fig_ptl_csa = fig_ptl_csa.add_trace(go.Scatter(x=prod_ptl_csa_sum.index, y=prod_ptl_csa_sum['Total_Ptl_Production_Central_South_America'], name='Total Production'))
fig_ptl_csa = fig_ptl_csa.update_layout(title='Total Petroleum Consumption & Production Central & South America 1980 - 2021')
fig_ptl_csa.show()

North America

In [772]:
con_ptl_na = con_ptl[con_ptl['Continent']== 'North America']
con_ptl_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
165,North America,Bermuda,0.007977284,0.006908714,0.007327424,0.007746134,0.007557427,0.007327424,0.006898958,0.009947155,...,0.007802241,0.010104616,0.008497225,0.01081503,0.010668011,0.011036412,0.011029603,0.010088665,0.010186187,0.375344
166,North America,Canada,3.896123436,3.658928995,3.253608905,2.97229282,3.042406885,3.037175182,3.048982856,3.184137221,...,4.903361803,4.853165921,4.958156188,5.015402839,4.992827406,5.084573775,4.979847774,4.388220912,4.551970394,171.155630
167,North America,Greenland,0.007817076,0.007374328,0.007163633,0.005056682,0.008028349,0.007163633,0.004761882,0.003294473,...,0.008790343,0.007822882,0.008401949,0.008482534,0.008546504,0.008554698,0.008455163,0.007733841,0.007808601,0.331906
168,North America,Mexico,2.680407222,2.943691055,3.101897385,2.8500952,2.848456623,3.007957802,3.104555664,3.218602161,...,4.128185883,4.010070968,3.936013626,4.049421792,3.966391048,3.687231416,3.850891943,3.01867853,3.010815162,154.284542
169,North America,Saint Pierre and Miquelon,0.002129031,0.002123214,0.001910892,0.002123214,0.002129031,0.001910892,0.00235233,0.003024068,...,0.001287322,0.001361465,0.001361465,0.001386152,0.001361465,0.001481302,0.001273758,0.001165176,0.001176441,0.069014


In [773]:
con_ptl_na_sum = con_ptl_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ptl_na_sum = con_ptl_na_sum.sum(axis=0)
con_ptl_na_sum = con_ptl_na_sum.transpose()
con_ptl_na_sum = pd.DataFrame(con_ptl_na_sum)
con_ptl_na_sum = con_ptl_na_sum.rename(columns={0:'Total_Ptl_Consumption_North_America'})
con_ptl_na_sum.head()

,Total_Ptl_Consumption_North_America
1980,40.753419
1981,38.509692
1982,36.570272
1983,35.869606
1984,36.944114


In [774]:
prod_ptl_na = prod_ptl[prod_ptl['Continent']== 'North America']
prod_ptl_na.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Petrolium
165,North America,Bermuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
166,North America,Canada,3.53406102,3.203855809,3.15762815,3.32497234,3.5464927,3.60915029,3.6027904,3.78818984,...,7.979768084,8.629165327,8.824457343,8.970339939,9.702707817,10.52665566,10.74436607,10.27309824,10.84909825,248.656195
167,North America,Greenland,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
168,North America,Mexico,4.51468829,5.39274597,6.36541205,6.249886199,6.45618549,6.37931175,5.80625897,6.03683075,...,6.04273667,5.837588198,5.43210089,5.169780732,4.672601858,4.341851969,3.994545991,4.035322832,4.030191561,262.876990
169,North America,Saint Pierre and Miquelon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [775]:
prod_ptl_na_sum = prod_ptl_na.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ptl_na_sum = prod_ptl_na_sum.sum(axis=0)
prod_ptl_na_sum = prod_ptl_na_sum.transpose()
prod_ptl_na_sum = pd.DataFrame(prod_ptl_na_sum)
prod_ptl_na_sum = prod_ptl_na_sum.rename(columns={0:'Total_Ptl_Production_North_America'})
prod_ptl_na_sum.head()

,Total_Ptl_Production_North_America
1980,28.522507
1981,29.008905
1982,29.981190
1983,30.102970
1984,31.084721


In [776]:
fig_ptl_na = go.Figure()
fig_ptl_na = fig_ptl_na.add_trace(go.Scatter(x=con_ptl_na_sum.index, y=con_ptl_na_sum['Total_Ptl_Consumption_North_America'], name='Total Consumption'))
fig_ptl_na = fig_ptl_na.add_trace(go.Scatter(x=prod_ptl_na_sum.index, y=prod_ptl_na_sum['Total_Ptl_Production_North_America'], name='Total Production'))
fig_ptl_na = fig_ptl_na.update_layout(title='Total Petroleum Consumption & Production North America 1980 - 2021')
fig_ptl_na.show()

Europe

In [777]:
con_ptl_europe = con_ptl[con_ptl['Continent']== 'Europe']
con_ptl_europe.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
172,Europe,Albania,0.099297277,0.094720496,0.092567758,0.094720496,0.092821368,0.090415019,0.082449887,0.072966858,...,0.054955503,0.056805568,0.053080053,0.05483,0.070199116,0.064531253,0.052484736,0.048199032,0.051509691,2.383221
173,Europe,Austria,0.497764429,0.479105205,0.443829415,0.437411255,0.425243473,0.415063222,0.431790733,0.437767514,...,0.550363122,0.536045252,0.539556685,0.549273798,0.549871027,0.560842403,0.578893457,0.501184129,0.528564132,21.903795
174,Europe,Belgium,1.12837983,1.031169895,0.955138205,0.870128975,0.813053147,0.852212849,0.959582872,0.964330176,...,1.286751519,1.261412977,1.294651458,1.324669915,1.335104211,1.423103145,1.329297537,1.162101967,1.304013015,50.498193
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0.061869314,0.063016853,0.064332658,0.074407,0.072629599,0.071416463,0.070663781,0.064893495,0.068238224,1.717136
176,Europe,Bulgaria,0.665221009,0.637387643,0.639555628,0.639555628,0.63913391,0.628715702,0.62026056,0.598940545,...,0.162174283,0.178371523,0.195558794,0.193727,0.182606693,0.186944775,0.191793197,0.176131908,0.185210066,13.062424


In [778]:
con_ptl_europe_sum = con_ptl_europe.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ptl_europe_sum = con_ptl_europe_sum.sum(axis=0)
con_ptl_europe_sum = con_ptl_europe_sum.transpose()
con_ptl_europe_sum = pd.DataFrame(con_ptl_europe_sum)
con_ptl_europe_sum = con_ptl_europe_sum.rename(columns={0:'Total_Ptl_Consumption_Europe'})
con_ptl_europe_sum.head()

,Total_Ptl_Consumption_Europe
1980,34.352347
1981,31.494244
1982,30.397382
1983,29.547442
1984,28.938635


In [779]:
prod_ptl_erope = prod_ptl[prod_ptl['Continent']== 'Europe']
prod_ptl_erope.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Petrolium
172,Europe,Albania,0.10154,0.10126,0.14728,0.1726,0.12692,0.12657,0.12657,0.12427,...,0.039140806,0.04719216,0.046397743,0.052494579,0.039121891,0.036820603,0.041423179,0.039832346,0.037035108,2.145501
173,Europe,Austria,0.063714284,0.05650001,0.05870001,0.05650001,0.052239892,0.04991001,0.04991001,0.047080093,...,0.03706414,0.03928568,0.037804654,0.033407379,0.031167532,0.029156978,0.027553737,0.023838119,0.023823477,1.870779
174,Europe,Belgium,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
175,Europe,Bosnia and Herzegovina,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0.000000
176,Europe,Bulgaria,0.00646,0.00644,0.00644,0.00644,0.00861,0.01288,0.01288,0.01288,...,0.002145788,0.002145788,0.002145788,0.002151667,0.002145788,0.002145788,0.002145788,0.002151667,0.002145788,0.164284


In [780]:
prod_ptl_europe_sum = prod_ptl_erope.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ptl_europe_sum = prod_ptl_europe_sum.sum(axis=0)
prod_ptl_europe_sum = prod_ptl_europe_sum.transpose()
prod_ptl_europe_sum = pd.DataFrame(prod_ptl_europe_sum)
prod_ptl_europe_sum = prod_ptl_europe_sum.rename(columns={0:'Total_Ptl_Production_Europe'})
prod_ptl_europe_sum.head()

,Total_Ptl_Production_Europe
1980,6.277921
1981,6.639482
1982,7.409406
1983,8.346887
1984,8.927659


In [781]:
fig_ptl_europe = go.Figure()
fig_ptl_europe = fig_ptl_europe.add_trace(go.Scatter(x=con_ptl_europe_sum.index, y=con_ptl_europe_sum['Total_Ptl_Consumption_Europe'], name='Total Consumption'))
fig_ptl_europe = fig_ptl_europe.add_trace(go.Scatter(x=prod_ptl_europe_sum.index, y=prod_ptl_europe_sum['Total_Ptl_Production_Europe'], name='Total Production'))
fig_ptl_europe = fig_ptl_europe.update_layout(title='Total Petroleum Consumption & Production Europe 1980 - 2021')
fig_ptl_europe.show()

Eurasia

In [782]:
con_ptl_eurasia = con_ptl[con_ptl['Continent']== 'Eurasia']
con_ptl_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Petrolium
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0.015759131,0.015696952,0.016651356,0.013772,0.016653142,0.016173671,0.021244013,0.025893453,0.025998959,0.820739
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,0.202940083,0.201464708,0.199667388,0.197653,0.209892639,0.204277527,0.2106614,0.190824389,0.192448622,7.238268
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0.334217966,0.350313412,0.288218658,0.284028,0.259404026,0.288686629,0.269226389,0.270742983,0.272633757,11.031491
220,Eurasia,Former U.S.S.R.,19.11784282,18.93843357,19.23517455,18.97022724,18.93718505,18.97022724,19.02533629,19.0686231,...,--,--,--,--,--,--,--,--,--,224.171550
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.04539067,0.047830848,0.054615065,0.066561,0.060732308,0.060104828,0.064225813,0.060322707,0.067725349,1.243850


In [783]:
con_ptl_eurasia_sum = con_ptl_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
con_ptl_eurasia_sum = con_ptl_eurasia_sum.sum(axis=0)
con_ptl_eurasia_sum = con_ptl_eurasia_sum.transpose()
con_ptl_eurasia_sum = pd.DataFrame(con_ptl_eurasia_sum)
con_ptl_eurasia_sum = con_ptl_eurasia_sum.rename(columns={0:'Total_Ptl_Consumption_Eurasia'})
con_ptl_eurasia_sum.head()

,Total_Ptl_Consumption_Eurasia
1980,19.117843
1981,18.938434
1982,19.235175
1983,18.970227
1984,18.937185


In [784]:
prod_ptl_eurasia = prod_ptl[prod_ptl['Continent']== 'Eurasia']
prod_ptl_eurasia.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Production_Petrolium
217,Eurasia,Armenia,--,--,--,--,--,--,--,--,...,0,0,0,0,0,0,0,0,0,0.000000
218,Eurasia,Azerbaijan,--,--,--,--,--,--,--,--,...,1.869983051,1.818827096,1.822507973,1.795358307,1.695173828,1.695458624,1.648523028,1.496800037,1.526918862,36.745905
219,Eurasia,Belarus,--,--,--,--,--,--,--,--,...,0.064385943,0.064385943,0.05598343,0.070301218,0.06859469,0.072242621,0.073064474,0.073972904,0.072912029,2.227244
220,Eurasia,Former U.S.S.R.,25.62498691,25.88701752,26.04251444,26.19363363,26.04170789,25.39333236,26.19515443,26.51339306,...,--,--,--,--,--,--,--,--,--,306.581247
221,Eurasia,Georgia,--,--,--,--,--,--,--,--,...,0.002146198,0.002146198,0.001715782,0.000860831,0.000895123,0.000809469,0.000734388,0.00068305,0.000635357,0.086351


In [785]:
prod_ptl_eurasia_sum = prod_ptl_eurasia.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ptl_eurasia_sum = prod_ptl_eurasia_sum.sum(axis=0)
prod_ptl_eurasia_sum = prod_ptl_eurasia_sum.transpose()
prod_ptl_eurasia_sum = pd.DataFrame(prod_ptl_eurasia_sum)
prod_ptl_eurasia_sum = prod_ptl_eurasia_sum.rename(columns={0:'Total_Ptl_Production_Eurasia'})
prod_ptl_eurasia_sum.head()

,Total_Ptl_Production_Eurasia
1980,25.624987
1981,25.887018
1982,26.042514
1983,26.193634
1984,26.041708


In [786]:
fig_ptl_eurasia = go.Figure()
fig_ptl_eurasia = fig_ptl_eurasia.add_trace(go.Scatter(x=con_ptl_eurasia_sum.index, y=con_ptl_eurasia_sum['Total_Ptl_Consumption_Eurasia'], name='Total Consumption'))
fig_ptl_eurasia = fig_ptl_eurasia.add_trace(go.Scatter(x=prod_ptl_eurasia_sum.index, y=prod_ptl_eurasia_sum['Total_Ptl_Production_Eurasia'], name='Total Production'))  
fig_ptl_eurasia = fig_ptl_eurasia.update_layout(title='Total Petroleum Consumption & Production Eurasia 1980 - 2021')
fig_ptl_eurasia.show()

Total Consumption & Production by Region

In [787]:
fig_ptl_region = go.Figure()
fig_ptl_region = fig_ptl_region.add_trace(go.Scatter(x=con_ptl_africa_sum.index, y=con_ptl_africa_sum['Total_Ptl_Consumption_Africa'], name='Africa'))
fig_ptl_region = fig_ptl_region.add_trace(go.Scatter(x=con_ptl_asia_sum.index, y=con_ptl_asia_sum['Total_Ptl_Consumption_Asia'], name='Asia'))
fig_ptl_region = fig_ptl_region.add_trace(go.Scatter(x=con_ptl_me_sum.index, y=con_ptl_me_sum['Total_Ptl_Consumption_Middle_East'], name='Middle East'))
fig_ptl_region = fig_ptl_region.add_trace(go.Scatter(x=con_ptl_csa_sum.index, y=con_ptl_csa_sum['Total_Ptl_Consumption_Central_South_America'], name='Central South America'))
fig_ptl_region = fig_ptl_region.add_trace(go.Scatter(x=con_ptl_na_sum.index, y=con_ptl_na_sum['Total_Ptl_Consumption_North_America'], name='North America'))
fig_ptl_region = fig_ptl_region.add_trace(go.Scatter(x=con_ptl_europe_sum.index, y=con_ptl_europe_sum['Total_Ptl_Consumption_Europe'], name='Europe'))
fig_ptl_region = fig_ptl_region.add_trace(go.Scatter(x=con_ptl_eurasia_sum.index, y=con_ptl_eurasia_sum['Total_Ptl_Consumption_Eurasia'], name='Eurasia'))
fig_ptl_region = fig_ptl_region.update_layout(title='Total Petroleum Consumption  1980 - 2021')
fig_ptl_region.show()

In [788]:
fig_prod_ptl_region = go.Figure()
fig_prod_ptl_region = fig_prod_ptl_region.add_trace(go.Scatter(x=prod_ptl_africa_sum.index, y=prod_ptl_africa_sum['Total_Ptl_Production_Africa'], name='Africa'))
fig_prod_ptl_region = fig_prod_ptl_region.add_trace(go.Scatter(x=prod_ptl_asia_sum.index, y=prod_ptl_asia_sum['Total_Ptl_Production_Asia'], name='Asia'))
fig_prod_ptl_region = fig_prod_ptl_region.add_trace(go.Scatter(x=prod_ptl_me_sum.index, y=prod_ptl_me_sum['Total_Ptl_Production_Middle_East'], name='Middle East'))
fig_prod_ptl_region = fig_prod_ptl_region.add_trace(go.Scatter(x=prod_ptl_csa_sum.index, y=prod_ptl_csa_sum['Total_Ptl_Production_Central_South_America'], name='Central South America'))
fig_prod_ptl_region = fig_prod_ptl_region.add_trace(go.Scatter(x=prod_ptl_na_sum.index, y=prod_ptl_na_sum['Total_Ptl_Production_North_America'], name='North America'))
fig_prod_ptl_region = fig_prod_ptl_region.add_trace(go.Scatter(x=prod_ptl_europe_sum.index, y=prod_ptl_europe_sum['Total_Ptl_Production_Europe'], name='Europe'))
fig_prod_ptl_region = fig_prod_ptl_region.add_trace(go.Scatter(x=prod_ptl_eurasia_sum.index, y=prod_ptl_eurasia_sum['Total_Ptl_Production_Eurasia'], name='Eurasia'))
fig = fig_prod_ptl_region.update_layout(title='Total Petroleum Production  1980 - 2021')
fig.show()

In [789]:
prod_ptl_all_sum = prod_ptl_me_sum.join(prod_ptl_asia_sum).join(prod_ptl_europe_sum).join(prod_ptl_eurasia_sum).join(prod_ptl_na_sum).join(prod_ptl_africa_sum).join(prod_ptl_csa_sum)
prod_ptl_all_sum['Total_Production'] = prod_ptl_all_sum.sum(axis=1)
prod_ptl_all_sum['portion_me'] = prod_ptl_all_sum['Total_Ptl_Production_Middle_East']/prod_ptl_all_sum['Total_Production']*100
prod_ptl_all_sum['portion_asia'] = prod_ptl_all_sum['Total_Ptl_Production_Asia']/prod_ptl_all_sum['Total_Production']*100
prod_ptl_all_sum['portion_europe'] = prod_ptl_all_sum['Total_Ptl_Production_Europe']/prod_ptl_all_sum['Total_Production']*100
prod_ptl_all_sum['portion_eurasia'] = prod_ptl_all_sum['Total_Ptl_Production_Eurasia']/prod_ptl_all_sum['Total_Production']*100
prod_ptl_all_sum['portion_na'] = prod_ptl_all_sum['Total_Ptl_Production_North_America']/prod_ptl_all_sum['Total_Production']*100
prod_ptl_all_sum['portion_africa'] = prod_ptl_all_sum['Total_Ptl_Production_Africa']/prod_ptl_all_sum['Total_Production']*100
prod_ptl_all_sum['portion_csa'] = prod_ptl_all_sum['Total_Ptl_Production_Central_South_America']/prod_ptl_all_sum['Total_Production']*100
prod_ptl_all_sum.head()

,Total_Ptl_Production_Middle_East,Total_Ptl_Production_Asia,Total_Ptl_Production_Europe,Total_Ptl_Production_Eurasia,Total_Ptl_Production_North_America,Total_Ptl_Production_Africa,Total_Ptl_Production_Central_South_America,Total_Production,portion_me,portion_asia,portion_europe,portion_eurasia,portion_na,portion_africa,portion_csa
1980,40.647386,10.604357,6.277921,25.624987,28.522507,13.139405,8.294547,133.111109,30.536434,7.966545,4.716301,19.250825,21.427593,9.871005,6.231296
1981,34.831897,10.541090,6.639482,25.887018,29.008905,10.254687,8.275871,125.438949,27.768008,8.403362,5.292999,20.637145,23.125915,8.175042,6.597529
1982,28.116600,10.175703,7.409406,26.042514,29.981190,10.202050,7.835343,119.762805,23.476905,8.496547,6.186734,21.745077,25.033807,8.518546,6.542384
1983,25.837469,10.803810,8.346887,26.193634,30.102970,10.222938,7.761489,119.269196,21.663153,9.058340,6.998360,21.961776,25.239518,8.571314,6.507539
1984,25.433135,11.758629,8.927659,26.041708,31.084721,11.149524,8.158450,122.553827,20.752624,9.594665,7.284684,21.249200,25.364138,9.097655,6.657034


In [790]:
fig_prod_ptl_all_sum = go.Figure()
fig_prod_ptl_all_sum = fig_prod_ptl_all_sum.add_trace(go.Bar(x=prod_ptl_all_sum.index, y=prod_ptl_all_sum['portion_me'], name='Middle East'))
fig_prod_ptl_all_sum = fig_prod_ptl_all_sum.add_trace(go.Bar(x=prod_ptl_all_sum.index, y=prod_ptl_all_sum['portion_asia'], name='Asia'))
fig_prod_ptl_all_sum = fig_prod_ptl_all_sum.add_trace(go.Bar(x=prod_ptl_all_sum.index, y=prod_ptl_all_sum['portion_europe'], name='Europe'))
fig_prod_ptl_all_sum = fig_prod_ptl_all_sum.add_trace(go.Bar(x=prod_ptl_all_sum.index, y=prod_ptl_all_sum['portion_eurasia'], name='Eurasia'))
fig_prod_ptl_all_sum = fig_prod_ptl_all_sum.add_trace(go.Bar(x=prod_ptl_all_sum.index, y=prod_ptl_all_sum['portion_na'], name='North America'))
fig_prod_ptl_all_sum = fig_prod_ptl_all_sum.add_trace(go.Bar(x=prod_ptl_all_sum.index, y=prod_ptl_all_sum['portion_africa'], name='Africa'))
fig_prod_ptl_all_sum = fig_prod_ptl_all_sum.add_trace(go.Bar(x=prod_ptl_all_sum.index, y=prod_ptl_all_sum['portion_csa'], name='Central South America'))
fig_prod_ptl_all_sum = fig_prod_ptl_all_sum.update_layout(title='Petroleum Production Mix  by Region 1980 - 2021',barmode='stack')
fig_prod_ptl_all_sum.show()

#### Energy Mix

Africa

In [791]:
energy_mix_africa = pd.DataFrame()
energy_mix_africa = energy_mix_africa.append(con_coal_africa_sum['Total_Coal_Consumption_Africa'])
energy_mix_africa = energy_mix_africa.append(con_ng_africa_sum['Total_NG_Consumption_Africa'])
energy_mix_africa = energy_mix_africa.append(con_re_africa_sum['Total_RE_Consumption_Africa']) 
energy_mix_africa = energy_mix_africa.append(con_ptl_africa_sum['Total_Ptl_Consumption_Africa']) 
energy_mix_africa = energy_mix_africa.transpose()  
energy_mix_africa.head()

,Total_Coal_Consumption_Africa,Total_NG_Consumption_Africa,Total_RE_Consumption_Africa,Total_Ptl_Consumption_Africa
1980,2.106965,0.833810,0.628175,3.071978
1981,2.296101,0.688067,0.554975,3.281095
1982,2.464263,0.934025,0.546272,3.450872
1983,2.598829,1.141378,0.531879,3.535079
1984,2.763172,1.245614,0.517153,3.672176


In [792]:
energy_mix_africa['Total_Energy_Consumption'] = energy_mix_africa.sum(axis=1)
energy_mix_africa.head()

,Total_Coal_Consumption_Africa,Total_NG_Consumption_Africa,Total_RE_Consumption_Africa,Total_Ptl_Consumption_Africa,Total_Energy_Consumption
1980,2.106965,0.833810,0.628175,3.071978,6.640928
1981,2.296101,0.688067,0.554975,3.281095,6.820238
1982,2.464263,0.934025,0.546272,3.450872,7.395432
1983,2.598829,1.141378,0.531879,3.535079,7.807166
1984,2.763172,1.245614,0.517153,3.672176,8.198115


In [793]:
energy_mix_africa['Coal_Portion'] = energy_mix_africa['Total_Coal_Consumption_Africa']/energy_mix_africa['Total_Energy_Consumption']*100
energy_mix_africa['NG_Portion'] = energy_mix_africa['Total_NG_Consumption_Africa']/energy_mix_africa['Total_Energy_Consumption']*100
energy_mix_africa['RE_Portion'] = energy_mix_africa['Total_RE_Consumption_Africa']/energy_mix_africa['Total_Energy_Consumption']*100
energy_mix_africa['PTL_Portion'] = energy_mix_africa['Total_Ptl_Consumption_Africa']/energy_mix_africa['Total_Energy_Consumption']*100
energy_mix_africa.head()

,Total_Coal_Consumption_Africa,Total_NG_Consumption_Africa,Total_RE_Consumption_Africa,Total_Ptl_Consumption_Africa,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,2.106965,0.833810,0.628175,3.071978,6.640928,31.726969,12.555627,9.459143,46.258260
1981,2.296101,0.688067,0.554975,3.281095,6.820238,33.665995,10.088605,8.137186,48.108213
1982,2.464263,0.934025,0.546272,3.450872,7.395432,33.321420,12.629759,7.386609,46.662212
1983,2.598829,1.141378,0.531879,3.535079,7.807166,33.287735,14.619626,6.812706,45.279934
1984,2.763172,1.245614,0.517153,3.672176,8.198115,33.704971,15.193907,6.308196,44.792926


In [794]:
fig_energy_mix_africa = go.Figure()
fig_energy_mix_africa = fig_energy_mix_africa.add_trace(go.Scatter(x=energy_mix_africa.index, y=energy_mix_africa['Total_Coal_Consumption_Africa'], name='Coal', mode='lines +markers'))
fig_energy_mix_africa = fig_energy_mix_africa.add_trace(go.Scatter(x=energy_mix_africa.index, y=energy_mix_africa['Total_NG_Consumption_Africa'], name='Natural Gas'))
fig_energy_mix_africa = fig_energy_mix_africa.add_trace(go.Scatter(x=energy_mix_africa.index, y=energy_mix_africa['Total_RE_Consumption_Africa'], name='Nuclear & Renewable'))
fig_energy_mix_africa = fig_energy_mix_africa.add_trace(go.Scatter(x=energy_mix_africa.index, y=energy_mix_africa['Total_Ptl_Consumption_Africa'], name='Petroleum', mode = 'lines+markers'))
fig_energy_mix_africa = fig_energy_mix_africa.update_layout(title='Energy Mix Consumption for Africa  1980 - 2021')
fig_energy_mix_africa.show()

In [795]:
fig_energy_portion_africa = go.Figure()
fig_energy_portion_africa = fig_energy_portion_africa.add_trace(go.Bar(x=energy_mix_africa.index, y=energy_mix_africa['Coal_Portion'], name='Coal'))  
fig_energy_portion_africa = fig_energy_portion_africa.add_trace(go.Bar(x=energy_mix_africa.index, y=energy_mix_africa['NG_Portion'], name='Natural Gas'))
fig_energy_portion_africa = fig_energy_portion_africa.add_trace(go.Bar(x=energy_mix_africa.index, y=energy_mix_africa['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_portion_africa = fig_energy_portion_africa.add_trace(go.Bar(x=energy_mix_africa.index, y=energy_mix_africa['PTL_Portion'], name='Petroleum'))
fig_energy_portion_africa = fig_energy_portion_africa.update_layout(title='Energy Mix Consumption for Africa  1980 - 2021',barmode= 'stack')
fig_energy_portion_africa.show()

In [799]:
energy_prod_mix_africa = pd.DataFrame()
energy_prod_mix_africa = energy_prod_mix_africa.append(prod_coal_africa_sum['Total_Production_Africa'])
energy_prod_mix_africa = energy_prod_mix_africa.append(prod_ng_africa_sum['Total_NG_Production_Africa'])
energy_prod_mix_africa = energy_prod_mix_africa.append(prod_re_africa_sum['Total_RE_Production_Africa'])
energy_prod_mix_africa = energy_prod_mix_africa.append(prod_ptl_africa_sum['Total_Ptl_Production_Africa'])
energy_prod_mix_africa = energy_prod_mix_africa.transpose()
energy_prod_mix_africa.head()

,Total_Production_Africa,Total_NG_Production_Africa,Total_RE_Production_Africa,Total_Ptl_Production_Africa
1980,2.644093,0.776095,0.626587,13.139405
1981,2.914258,1.190747,0.552298,10.254687
1982,3.154886,1.426549,0.544617,10.202050
1983,3.308514,1.942190,0.530010,10.222938
1984,3.695633,2.071354,0.514130,11.149524


In [801]:
energy_prod_mix_africa['Total_Energy_Production'] = energy_prod_mix_africa.sum(axis=1)
energy_prod_mix_africa['Coal_Portion'] = energy_prod_mix_africa['Total_Production_Africa']/energy_prod_mix_africa['Total_Energy_Production']*100
energy_prod_mix_africa['NG_Portion'] = energy_prod_mix_africa['Total_NG_Production_Africa']/energy_prod_mix_africa['Total_Energy_Production']*100
energy_prod_mix_africa['RE_Portion'] = energy_prod_mix_africa['Total_RE_Production_Africa']/energy_prod_mix_africa['Total_Energy_Production']*100
energy_prod_mix_africa['PTL_Portion'] = energy_prod_mix_africa['Total_Ptl_Production_Africa']/energy_prod_mix_africa['Total_Energy_Production']*100

energy_prod_mix_africa.head()


,Total_Production_Africa,Total_NG_Production_Africa,Total_RE_Production_Africa,Total_Ptl_Production_Africa,Total_Energy_Production,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,2.644093,0.776095,0.626587,13.139405,34.372359,7.692498,2.257904,1.822938,38.226660
1981,2.914258,1.190747,0.552298,10.254687,29.823981,9.771526,3.992582,1.851860,34.384032
1982,3.154886,1.426549,0.544617,10.202050,30.656203,10.291183,4.653379,1.776530,33.278908
1983,3.308514,1.942190,0.530010,10.222938,32.007305,10.336747,6.067960,1.655903,31.939390
1984,3.695633,2.071354,0.514130,11.149524,34.861284,10.600967,5.941703,1.474789,31.982541


In [810]:
fig_energy_prod_vol_mix_africa = go.Figure()
fig_energy_prod_vol_mix_africa = fig_energy_prod_vol_mix_africa.add_trace(go.Scatter(x=energy_prod_mix_africa.index, y=energy_prod_mix_africa['Total_Production_Africa'], name='Coal', mode='lines +markers'))
fig_energy_prod_vol_mix_africa = fig_energy_prod_vol_mix_africa.add_trace(go.Scatter(x=energy_prod_mix_africa.index, y=energy_prod_mix_africa['Total_NG_Production_Africa'], name='Natural Gas', mode='lines +markers'))
fig_energy_prod_vol_mix_africa = fig_energy_prod_vol_mix_africa.add_trace(go.Scatter(x=energy_prod_mix_africa.index, y=energy_prod_mix_africa['Total_RE_Production_Africa'], name='Nuclear & Renewable', mode='lines +markers'))
fig_energy_prod_vol_mix_africa = fig_energy_prod_vol_mix_africa.add_trace(go.Scatter(x=energy_prod_mix_africa.index, y=energy_prod_mix_africa['Total_Ptl_Production_Africa'], name='Petroleum', mode='lines +markers'))
fig_energy_prod_vol_mix_africa = fig_energy_prod_vol_mix_africa.update_layout(title='Energy Mix Production Volume for Africa  1980 - 2021')
fig_energy_prod_vol_mix_africa.show()

In [804]:
fig_energy_prod_portion_mix_africa = go.Figure()
fig_energy_prod_portion_mix_africa = fig_energy_prod_portion_mix_africa.add_trace(go.Bar(x=energy_prod_mix_africa.index, y=energy_prod_mix_africa['Coal_Portion'], name='Coal'))
fig_energy_prod_portion_mix_africa = fig_energy_prod_portion_mix_africa.add_trace(go.Bar(x=energy_prod_mix_africa.index, y=energy_prod_mix_africa['NG_Portion'], name='Natural Gas'))
fig_energy_prod_portion_mix_africa = fig_energy_prod_portion_mix_africa.add_trace(go.Bar(x=energy_prod_mix_africa.index, y=energy_prod_mix_africa['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_prod_portion_mix_africa = fig_energy_prod_portion_mix_africa.add_trace(go.Bar(x=energy_prod_mix_africa.index, y=energy_prod_mix_africa['PTL_Portion'], name='Petroleum'))
fig_energy_prod_portion_mix_africa = fig_energy_prod_portion_mix_africa.update_layout(title='Energy Mix Production for Africa  1980 - 2021',barmode= 'stack')
fig_energy_prod_portion_mix_africa.show()

Asia

In [ ]:
energy_mix_asia = pd.DataFrame()
energy_mix_asia = energy_mix_asia.append(con_coal_asia_sum['Total_Coal_Consumption_Asia'])
energy_mix_asia = energy_mix_asia.append(con_ng_asia_sum['Total_NG_Consumption_Asia'])
energy_mix_asia = energy_mix_asia.append(con_re_asia_sum['Total_RE_Consumption_Asia'])
energy_mix_asia = energy_mix_asia.append(con_ptl_asia_sum['Total_Ptl_Consumption_Asia'])
energy_mix_asia = energy_mix_asia.transpose()
energy_mix_asia.head()

,Total_Coal_Consumption_Asia,Total_NG_Consumption_Asia,Total_RE_Consumption_Asia,Total_Ptl_Consumption_Asia
1980,21.191995,2.770283,3.714677,23.343826
1981,21.812177,2.902779,3.990137,23.031164
1982,23.192326,2.957398,4.223904,22.482536
1983,24.446940,3.224079,4.681917,22.298415
1984,26.824249,3.829029,4.986841,22.884483


In [ ]:
energy_mix_asia['Total_Energy_Consumption'] = energy_mix_asia.sum(axis=1)
energy_mix_asia.head()

,Total_Coal_Consumption_Asia,Total_NG_Consumption_Asia,Total_RE_Consumption_Asia,Total_Ptl_Consumption_Asia,Total_Energy_Consumption
1980,21.191995,2.770283,3.714677,23.343826,51.020781
1981,21.812177,2.902779,3.990137,23.031164,51.736257
1982,23.192326,2.957398,4.223904,22.482536,52.856164
1983,24.446940,3.224079,4.681917,22.298415,54.651351
1984,26.824249,3.829029,4.986841,22.884483,58.524602


In [ ]:
energy_mix_asia['Coal_Portion'] = energy_mix_asia['Total_Coal_Consumption_Asia']/energy_mix_asia['Total_Energy_Consumption']*100
energy_mix_asia['NG_Portion'] = energy_mix_asia['Total_NG_Consumption_Asia']/energy_mix_asia['Total_Energy_Consumption']*100
energy_mix_asia['RE_Portion'] = energy_mix_asia['Total_RE_Consumption_Asia']/energy_mix_asia['Total_Energy_Consumption']*100
energy_mix_asia['PTL_Portion'] = energy_mix_asia['Total_Ptl_Consumption_Asia']/energy_mix_asia['Total_Energy_Consumption']*100
energy_mix_asia.head()

,Total_Coal_Consumption_Asia,Total_NG_Consumption_Asia,Total_RE_Consumption_Asia,Total_Ptl_Consumption_Asia,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,21.191995,2.770283,3.714677,23.343826,51.020781,41.536007,5.429715,7.280713,45.753565
1981,21.812177,2.902779,3.990137,23.031164,51.736257,42.160331,5.610726,7.712458,44.516486
1982,23.192326,2.957398,4.223904,22.482536,52.856164,43.878186,5.595181,7.991318,42.535315
1983,24.446940,3.224079,4.681917,22.298415,54.651351,44.732545,5.899358,8.566882,40.801215
1984,26.824249,3.829029,4.986841,22.884483,58.524602,45.834142,6.542597,8.520930,39.102331


In [ ]:
fig_energy_mix_asia = go.Figure()
fig_energy_mix_asia = fig_energy_mix_asia.add_trace(go.Scatter(x=energy_mix_asia.index, y=energy_mix_asia['Total_Coal_Consumption_Asia'], name='Coal'))
fig_energy_mix_asia = fig_energy_mix_asia.add_trace(go.Scatter(x=energy_mix_asia.index, y=energy_mix_asia['Total_NG_Consumption_Asia'], name='Natural Gas'))
fig_energy_mix_asia = fig_energy_mix_asia.add_trace(go.Scatter(x=energy_mix_asia.index, y=energy_mix_asia['Total_RE_Consumption_Asia'], name='Nuclear & Renewable'))
fig_energy_mix_asia = fig_energy_mix_asia.add_trace(go.Scatter(x=energy_mix_asia.index, y=energy_mix_asia['Total_Ptl_Consumption_Asia'], name='Petroleum'))
fig_energy_mix_asia = fig_energy_mix_asia.update_layout(title='Energy Mix Consumption for Asia  1980 - 2021')
fig_energy_mix_asia.show()

In [ ]:
fig_energy_portion_asia = go.Figure()
fig_energy_portion_asia = fig_energy_portion_asia.add_trace(go.Bar(x=energy_mix_asia.index, y=energy_mix_asia['Coal_Portion'], name='Coal'))
fig_energy_portion_asia = fig_energy_portion_asia.add_trace(go.Bar(x=energy_mix_asia.index, y=energy_mix_asia['NG_Portion'], name='Natural Gas'))
fig_energy_portion_asia = fig_energy_portion_asia.add_trace(go.Bar(x=energy_mix_asia.index, y=energy_mix_asia['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_portion_asia = fig_energy_portion_asia.add_trace(go.Bar(x=energy_mix_asia.index, y=energy_mix_asia['PTL_Portion'], name='Petroleum'))
fig_energy_portion_asia = fig_energy_portion_asia.update_layout(title='Energy Mix Consumption for Asia  1980 - 2021',barmode= 'stack')
fig_energy_portion_asia.show()

In [808]:
energy_prod_mix_asia = pd.DataFrame()
energy_prod_mix_asia = energy_prod_mix_asia.append(prod_coal_asia_sum['Total_Production_Asia'])
energy_prod_mix_asia = energy_prod_mix_asia.append(prod_ng_asia_sum['Total_NG_Production_Asia'])
energy_prod_mix_asia = energy_prod_mix_asia.append(prod_re_asia_sum['Total_RE_Production_Asia'])
energy_prod_mix_asia = energy_prod_mix_asia.append(prod_ptl_asia_sum['Total_Ptl_Production_Asia'])
energy_prod_mix_asia = energy_prod_mix_asia.transpose()
energy_prod_mix_asia.head()

,Total_Production_Asia,Total_NG_Production_Asia,Total_RE_Production_Asia,Total_Ptl_Production_Asia
1980,20.250933,2.698593,3.773345,10.604357
1981,20.834413,2.868678,4.042434,10.541090
1982,22.087943,2.919132,4.301334,10.175703
1983,23.581426,3.241797,4.771872,10.803810
1984,25.692378,3.789489,5.094001,11.758629


In [809]:
energy_prod_mix_asia['Total_Energy_Production'] = energy_prod_mix_asia.sum(axis=1)
energy_prod_mix_asia['Coal_Portion'] = energy_prod_mix_asia['Total_Production_Asia']/energy_prod_mix_asia['Total_Energy_Production']*100
energy_prod_mix_asia['NG_Portion'] = energy_prod_mix_asia['Total_NG_Production_Asia']/energy_prod_mix_asia['Total_Energy_Production']*100
energy_prod_mix_asia['RE_Portion'] = energy_prod_mix_asia['Total_RE_Production_Asia']/energy_prod_mix_asia['Total_Energy_Production']*100
energy_prod_mix_asia['PTL_Portion'] = energy_prod_mix_asia['Total_Ptl_Production_Asia']/energy_prod_mix_asia['Total_Energy_Production']*100
energy_prod_mix_asia.head()

,Total_Production_Asia,Total_NG_Production_Asia,Total_RE_Production_Asia,Total_Ptl_Production_Asia,Total_Energy_Production,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,20.250933,2.698593,3.773345,10.604357,37.327228,54.252443,7.229556,10.108827,28.409174
1981,20.834413,2.868678,4.042434,10.541090,38.286615,54.416963,7.492640,10.558348,27.532049
1982,22.087943,2.919132,4.301334,10.175703,39.484112,55.941344,7.393181,10.893835,25.771640
1983,23.581426,3.241797,4.771872,10.803810,42.398905,55.618008,7.645945,11.254706,25.481341
1984,25.692378,3.789489,5.094001,11.758629,46.334497,55.449784,8.178547,10.993970,25.377699


In [812]:
fig_energy_prod_vol_mix_asia = go.Figure()
fig_energy_prod_vol_mix_asia = fig_energy_prod_vol_mix_asia.add_trace(go.Scatter(x=energy_prod_mix_asia.index, y=energy_prod_mix_asia['Total_Production_Asia'], name='Coal', mode='lines +markers'))
fig_energy_prod_vol_mix_asia = fig_energy_prod_vol_mix_asia.add_trace(go.Scatter(x=energy_prod_mix_asia.index, y=energy_prod_mix_asia['Total_NG_Production_Asia'], name='Natural Gas', mode='lines +markers'))
fig_energy_prod_vol_mix_asia = fig_energy_prod_vol_mix_asia.add_trace(go.Scatter(x=energy_prod_mix_asia.index, y=energy_prod_mix_asia['Total_RE_Production_Asia'], name='Nuclear & Renewable', mode='lines +markers'))
fig_energy_prod_vol_mix_asia = fig_energy_prod_vol_mix_asia.add_trace(go.Scatter(x=energy_prod_mix_asia.index, y=energy_prod_mix_asia['Total_Ptl_Production_Asia'], name='Petroleum', mode='lines +markers'))
fig_energy_prod_vol_mix_asia = fig_energy_prod_vol_mix_asia.update_layout(title='Energy Mix Production Volume for Asia  1980 - 2021')
fig_energy_prod_vol_mix_asia.show()

In [813]:
fig_energy_prod_portion_mix_asia = go.Figure()
fig_energy_prod_portion_mix_asia = fig_energy_prod_portion_mix_asia.add_trace(go.Bar(x=energy_prod_mix_asia.index, y=energy_prod_mix_asia['Coal_Portion'], name='Coal'))
fig_energy_prod_portion_mix_asia = fig_energy_prod_portion_mix_asia.add_trace(go.Bar(x=energy_prod_mix_asia.index, y=energy_prod_mix_asia['NG_Portion'], name='Natural Gas'))
fig_energy_prod_portion_mix_asia = fig_energy_prod_portion_mix_asia.add_trace(go.Bar(x=energy_prod_mix_asia.index, y=energy_prod_mix_asia['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_prod_portion_mix_asia = fig_energy_prod_portion_mix_asia.add_trace(go.Bar(x=energy_prod_mix_asia.index, y=energy_prod_mix_asia['PTL_Portion'], name='Petroleum'))
fig_energy_prod_portion_mix_asia = fig_energy_prod_portion_mix_asia.update_layout(title='Energy Mix Production for Asia  1980 - 2021',barmode= 'stack')
fig_energy_prod_portion_mix_asia.show()

Middle East

In [ ]:
energy_mix_me = pd.DataFrame()
energy_mix_me = energy_mix_me.append(con_coal_me_sum['Total_Coal_Consumption_Middle_East'])
energy_mix_me = energy_mix_me.append(con_ng_me_sum['Total_NG_Consumption_Middle_East'])
energy_mix_me = energy_mix_me.append(con_re_me_sum['Total_RE_Consumption_Middle_East'])
energy_mix_me = energy_mix_me.append(con_ptl_me_sum['Total_Ptl_Consumption_Middle_East'])
energy_mix_me = energy_mix_me.transpose()
energy_mix_me.head()

,Total_Coal_Consumption_Middle_East,Total_NG_Consumption_Middle_East,Total_RE_Consumption_Middle_East,Total_Ptl_Consumption_Middle_East
1980,0.049959,1.373220,0.100147,4.353613
1981,0.041720,1.361498,0.108079,4.639664
1982,0.078559,1.272069,0.110549,4.961577
1983,0.094892,1.496933,0.105972,5.498737
1984,0.107721,1.993850,0.112241,5.596714


In [ ]:
energy_mix_me['Total_Energy_Consumption'] = energy_mix_me.sum(axis=1)
energy_mix_me.head()

,Total_Coal_Consumption_Middle_East,Total_NG_Consumption_Middle_East,Total_RE_Consumption_Middle_East,Total_Ptl_Consumption_Middle_East,Total_Energy_Consumption
1980,0.049959,1.373220,0.100147,4.353613,5.876938
1981,0.041720,1.361498,0.108079,4.639664,6.150960
1982,0.078559,1.272069,0.110549,4.961577,6.422753
1983,0.094892,1.496933,0.105972,5.498737,7.196534
1984,0.107721,1.993850,0.112241,5.596714,7.810526


In [ ]:
energy_mix_me['Coal_Portion'] = energy_mix_me['Total_Coal_Consumption_Middle_East']/energy_mix_me['Total_Energy_Consumption']*100
energy_mix_me['NG_Portion'] = energy_mix_me['Total_NG_Consumption_Middle_East']/energy_mix_me['Total_Energy_Consumption']*100
energy_mix_me['RE_Portion'] = energy_mix_me['Total_RE_Consumption_Middle_East']/energy_mix_me['Total_Energy_Consumption']*100
energy_mix_me['PTL_Portion'] = energy_mix_me['Total_Ptl_Consumption_Middle_East']/energy_mix_me['Total_Energy_Consumption']*100
energy_mix_me.head()

,Total_Coal_Consumption_Middle_East,Total_NG_Consumption_Middle_East,Total_RE_Consumption_Middle_East,Total_Ptl_Consumption_Middle_East,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,0.049959,1.373220,0.100147,4.353613,5.876938,0.850080,23.366248,1.704063,74.079609
1981,0.041720,1.361498,0.108079,4.639664,6.150960,0.678270,22.134716,1.757101,75.429913
1982,0.078559,1.272069,0.110549,4.961577,6.422753,1.223131,19.805665,1.721208,77.249996
1983,0.094892,1.496933,0.105972,5.498737,7.196534,1.318585,20.800744,1.472538,76.408133
1984,0.107721,1.993850,0.112241,5.596714,7.810526,1.379184,25.527730,1.437049,71.656037


In [ ]:
fig_energy_mix_me = go.Figure()
fig_energy_mix_me = fig_energy_mix_me.add_trace(go.Scatter(x=energy_mix_me.index, y=energy_mix_me['Total_Coal_Consumption_Middle_East'], name='Coal'))
fig_energy_mix_me = fig_energy_mix_me.add_trace(go.Scatter(x=energy_mix_me.index, y=energy_mix_me['Total_NG_Consumption_Middle_East'], name='Natural Gas'))
fig_energy_mix_me = fig_energy_mix_me.add_trace(go.Scatter(x=energy_mix_me.index, y=energy_mix_me['Total_RE_Consumption_Middle_East'], name='Nuclear & Renewable'))
fig_energy_mix_me = fig_energy_mix_me.add_trace(go.Scatter(x=energy_mix_me.index, y=energy_mix_me['Total_Ptl_Consumption_Middle_East'], name='Petroleum'))
fig_energy_mix_me = fig_energy_mix_me.update_layout(title='Energy Mix Consumption for Middle East  1980 - 2021')
fig_energy_mix_me.show()

In [ ]:
prod_ptl_me_sum.head()

,Total_Ptl_Production_Middle_East
1980,40.647386
1981,34.831897
1982,28.116600
1983,25.837469
1984,25.433135


In [ ]:
energy_prod_mix_me = pd.DataFrame()
energy_prod_mix_me = energy_prod_mix_me.append(prod_coal_me_sum['Total_Production_Middle_East'])
energy_prod_mix_me = energy_prod_mix_me.append(prod_ng_me_sum['Total_NG_Production_Middle_East'])
energy_prod_mix_me = energy_prod_mix_me.append(prod_re_me_sum['Total_RE_Production_Middle_East'])
energy_prod_mix_me = energy_prod_mix_me.append(prod_ptl_me_sum['Total_Ptl_Production_Middle_East']) 
energy_prod_mix_me = energy_prod_mix_me.transpose()
energy_prod_mix_me.head()

,Total_Production_Middle_East,Total_NG_Production_Middle_East,Total_RE_Production_Middle_East,Total_Ptl_Production_Middle_East
1980,0.025126,1.491693,0.099922,40.647386
1981,0.025210,1.520090,0.107854,34.831897
1982,0.028358,1.438992,0.110352,28.116600
1983,0.028666,1.589069,0.105831,25.837469
1984,0.029001,2.101691,0.112157,25.433135


In [ ]:
fig_energy_portion_me = go.Figure()
fig_energy_portion_me = fig_energy_portion_me.add_trace(go.Bar(x=energy_mix_me.index, y=energy_mix_me['Coal_Portion'], name='Coal'))
fig_energy_portion_me = fig_energy_portion_me.add_trace(go.Bar(x=energy_mix_me.index, y=energy_mix_me['NG_Portion'], name='Natural Gas'))
fig_energy_portion_me = fig_energy_portion_me.add_trace(go.Bar(x=energy_mix_me.index, y=energy_mix_me['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_portion_me = fig_energy_portion_me.add_trace(go.Bar(x=energy_mix_me.index, y=energy_mix_me['PTL_Portion'], name='Petroleum'))
fig_energy_portion_me = fig_energy_portion_me.update_layout(title='Energy Mix Consumption for Middle East  1980 - 2021',barmode= 'stack')
fig_energy_portion_me.show()

In [ ]:
energy_prod_mix_me.head()

,Total_Production_Middle_East,Total_NG_Production_Middle_East,Total_RE_Production_Middle_East,Total_Ptl_Production_Middle_East
1980,0.025126,1.491693,0.099922,40.647386
1981,0.025210,1.520090,0.107854,34.831897
1982,0.028358,1.438992,0.110352,28.116600
1983,0.028666,1.589069,0.105831,25.837469
1984,0.029001,2.101691,0.112157,25.433135


In [ ]:
energy_prod_mix_me['Total_Energy_Consumption'] = energy_prod_mix_me.sum(axis=1)
energy_prod_mix_me.head()

,Total_Production_Middle_East,Total_NG_Production_Middle_East,Total_RE_Production_Middle_East,Total_Ptl_Production_Middle_East,Total_Energy_Consumption
1980,0.025126,1.491693,0.099922,40.647386,42.264127
1981,0.025210,1.520090,0.107854,34.831897,36.485050
1982,0.028358,1.438992,0.110352,28.116600,29.694303
1983,0.028666,1.589069,0.105831,25.837469,27.561035
1984,0.029001,2.101691,0.112157,25.433135,27.675984


In [ ]:
energy_prod_mix_me['Coal_Portion'] = energy_prod_mix_me['Total_Production_Middle_East']/energy_prod_mix_me['Total_Energy_Consumption']*100
energy_prod_mix_me['NG_Portion'] = energy_prod_mix_me['Total_NG_Production_Middle_East']/energy_prod_mix_me['Total_Energy_Consumption']*100
energy_prod_mix_me['RE_Portion'] = energy_prod_mix_me['Total_RE_Production_Middle_East']/energy_prod_mix_me['Total_Energy_Consumption']*100
energy_prod_mix_me['PTL_Portion'] = energy_prod_mix_me['Total_Ptl_Production_Middle_East']/energy_prod_mix_me['Total_Energy_Consumption']*100
energy_prod_mix_me.head()

,Total_Production_Middle_East,Total_NG_Production_Middle_East,Total_RE_Production_Middle_East,Total_Ptl_Production_Middle_East,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,0.025126,1.491693,0.099922,40.647386,42.264127,0.059450,3.529454,0.236423,96.174673
1981,0.025210,1.520090,0.107854,34.831897,36.485050,0.069096,4.166335,0.295612,95.468957
1982,0.028358,1.438992,0.110352,28.116600,29.694303,0.095501,4.846020,0.371628,94.686850
1983,0.028666,1.589069,0.105831,25.837469,27.561035,0.104009,5.765635,0.383989,93.746368
1984,0.029001,2.101691,0.112157,25.433135,27.675984,0.104788,7.593916,0.405250,91.896046


In [816]:
fig_energy_prod_vol_mix_me = go.Figure()
fig_energy_prod_vol_mix_me = fig_energy_prod_vol_mix_me.add_trace(go.Scatter(x=energy_prod_mix_me.index, y=energy_prod_mix_me['Total_Production_Middle_East'], name='Coal', mode='lines +markers'))
fig_energy_prod_vol_mix_me = fig_energy_prod_vol_mix_me.add_trace(go.Scatter(x=energy_prod_mix_me.index, y=energy_prod_mix_me['Total_NG_Production_Middle_East'], name='Natural Gas', mode='lines +markers'))
fig_energy_prod_vol_mix_me = fig_energy_prod_vol_mix_me.add_trace(go.Scatter(x=energy_prod_mix_me.index, y=energy_prod_mix_me['Total_RE_Production_Middle_East'], name='Nuclear & Renewable', mode='lines +markers'))
fig_energy_prod_vol_mix_me = fig_energy_prod_vol_mix_me.add_trace(go.Scatter(x=energy_prod_mix_me.index, y=energy_prod_mix_me['Total_Ptl_Production_Middle_East'], name='Petroleum', mode='lines +markers'))
fig_energy_prod_vol_mix_me = fig_energy_prod_vol_mix_me.update_layout(title='Energy Mix Production Volume for Middle East  1980 - 2021')
fig_energy_prod_vol_mix_me.show()

In [ ]:
fig_energy_prod_mix_me = go.Figure()
fig_energy_prod_mix_me = fig_energy_prod_mix_me.add_trace(go.Bar(x=energy_prod_mix_me.index, y=energy_prod_mix_me['Coal_Portion'], name='Coal'))
fig_energy_prod_mix_me = fig_energy_prod_mix_me.add_trace(go.Bar(x=energy_prod_mix_me.index, y=energy_prod_mix_me['NG_Portion'], name='Natural Gas'))
fig_energy_prod_mix_me = fig_energy_prod_mix_me.add_trace(go.Bar(x=energy_prod_mix_me.index, y=energy_prod_mix_me['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_prod_mix_me = fig_energy_prod_mix_me.add_trace(go.Bar(x=energy_prod_mix_me.index, y=energy_prod_mix_me['PTL_Portion'], name='Petroleum'))
fig_energy_prod_mix_me = fig_energy_prod_mix_me.update_layout(title='Energy Production Mix for Middle East  1980 - 2021', barmode= 'stack')
fig_energy_prod_mix_me.show()

Central & South America

In [ ]:
energy_mix_csa = pd.DataFrame()
energy_mix_csa = energy_mix_csa.append(con_coal_csa_sum['Total_Coal_Consumption_Central_South_America'])
energy_mix_csa = energy_mix_csa.append(con_ng_csa_sum['Total_NG_Consumption_Central_South_America'])
energy_mix_csa = energy_mix_csa.append(con_re_csa_sum['Total_RE_Consumption_Central_South_America'])
energy_mix_csa = energy_mix_csa.append(con_ptl_csa_sum['Total_Ptl_Consumption_Central_South_America'])
energy_mix_csa = energy_mix_csa.transpose()
energy_mix_csa.head()

,Total_Coal_Consumption_Central_South_America,Total_NG_Consumption_Central_South_America,Total_RE_Consumption_Central_South_America,Total_Ptl_Consumption_Central_South_America
1980,0.379655,1.389831,2.195940,7.070993
1981,0.386823,1.481334,2.233626,6.916840
1982,0.383407,1.580583,2.417209,6.826470
1983,0.424763,1.579619,2.652514,6.495828
1984,0.499408,1.884346,2.913554,6.443785


In [ ]:
energy_mix_csa['Total_Energy_Consumption'] = energy_mix_csa.sum(axis=1)
energy_mix_csa.head()

,Total_Coal_Consumption_Central_South_America,Total_NG_Consumption_Central_South_America,Total_RE_Consumption_Central_South_America,Total_Ptl_Consumption_Central_South_America,Total_Energy_Consumption
1980,0.379655,1.389831,2.195940,7.070993,11.036419
1981,0.386823,1.481334,2.233626,6.916840,11.018623
1982,0.383407,1.580583,2.417209,6.826470,11.207669
1983,0.424763,1.579619,2.652514,6.495828,11.152724
1984,0.499408,1.884346,2.913554,6.443785,11.741093


In [ ]:
energy_mix_csa['Coal_Portion'] = energy_mix_csa['Total_Coal_Consumption_Central_South_America']/energy_mix_csa['Total_Energy_Consumption']*100
energy_mix_csa['NG_Portion'] = energy_mix_csa['Total_NG_Consumption_Central_South_America']/energy_mix_csa['Total_Energy_Consumption']*100
energy_mix_csa['RE_Portion'] = energy_mix_csa['Total_RE_Consumption_Central_South_America']/energy_mix_csa['Total_Energy_Consumption']*100
energy_mix_csa['PTL_Portion'] = energy_mix_csa['Total_Ptl_Consumption_Central_South_America']/energy_mix_csa['Total_Energy_Consumption']*100
energy_mix_csa.head()

,Total_Coal_Consumption_Central_South_America,Total_NG_Consumption_Central_South_America,Total_RE_Consumption_Central_South_America,Total_Ptl_Consumption_Central_South_America,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,0.379655,1.389831,2.195940,7.070993,11.036419,3.440022,12.593133,19.897215,64.069629
1981,0.386823,1.481334,2.233626,6.916840,11.018623,3.510625,13.443912,20.271374,62.774090
1982,0.383407,1.580583,2.417209,6.826470,11.207669,3.420932,14.102691,21.567457,60.908920
1983,0.424763,1.579619,2.652514,6.495828,11.152724,3.808602,14.163525,23.783555,58.244318
1984,0.499408,1.884346,2.913554,6.443785,11.741093,4.253502,16.049149,24.815018,54.882331


In [ ]:
fig_energy_mix_csa = go.Figure()
fig_energy_mix_csa = fig_energy_mix_csa.add_trace(go.Scatter(x=energy_mix_csa.index, y=energy_mix_csa['Total_Coal_Consumption_Central_South_America'], name='Coal'))
fig_energy_mix_csa = fig_energy_mix_csa.add_trace(go.Scatter(x=energy_mix_csa.index, y=energy_mix_csa['Total_NG_Consumption_Central_South_America'], name='Natural Gas'))
fig_energy_mix_csa = fig_energy_mix_csa.add_trace(go.Scatter(x=energy_mix_csa.index, y=energy_mix_csa['Total_RE_Consumption_Central_South_America'], name='Nuclear & Renewable'))
fig_energy_mix_csa = fig_energy_mix_csa.add_trace(go.Scatter(x=energy_mix_csa.index, y=energy_mix_csa['Total_Ptl_Consumption_Central_South_America'], name='Petroleum'))
fig_energy_mix_csa = fig_energy_mix_csa.update_layout(title='Energy Mix Consumption for Central South America  1980 - 2021')
fig_energy_mix_csa.show()

In [ ]:
fig_energy_portion_csa = go.Figure()
fig_energy_portion_csa = fig_energy_portion_csa.add_trace(go.Bar(x=energy_mix_csa.index, y=energy_mix_csa['Coal_Portion'], name='Coal'))
fig_energy_portion_csa = fig_energy_portion_csa.add_trace(go.Bar(x=energy_mix_csa.index, y=energy_mix_csa['NG_Portion'], name='Natural Gas'))
fig_energy_portion_csa = fig_energy_portion_csa.add_trace(go.Bar(x=energy_mix_csa.index, y=energy_mix_csa['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_portion_csa = fig_energy_portion_csa.add_trace(go.Bar(x=energy_mix_csa.index, y=energy_mix_csa['PTL_Portion'], name='Petroleum'))
fig_energy_portion_csa = fig_energy_portion_csa.update_layout(title='Energy Mix Consumption for Central South America  1980 - 2021',barmode= 'stack')
fig_energy_portion_csa.show()

In [819]:
energy_prod_mix_csa = pd.DataFrame()
energy_prod_mix_csa = energy_prod_mix_csa.append(prod_coal_csa_sum['Total_Production_Central_South_America'])
energy_prod_mix_csa = energy_prod_mix_csa.append(prod_ng_csa_sum['Total_NG_Production_Central_South_America'])
energy_prod_mix_csa = energy_prod_mix_csa.append(prod_re_csa_sum['Total_RE_Production_Central_South_America'])
energy_prod_mix_csa = energy_prod_mix_csa.append(prod_ptl_csa_sum['Total_Ptl_Production_Central_South_America'])
energy_prod_mix_csa = energy_prod_mix_csa.transpose()
energy_prod_mix_csa.head()

,Total_Production_Central_South_America,Total_NG_Production_Central_South_America,Total_RE_Production_Central_South_America,Total_Ptl_Production_Central_South_America
1980,0.220723,1.369422,2.249169,8.294547
1981,0.234615,1.517969,2.306281,8.275871
1982,0.252328,1.591101,2.535795,7.835343
1983,0.268962,1.644515,2.815858,7.761489
1984,0.325556,1.882318,3.157275,8.158450


In [821]:
energy_prod_mix_csa['Total_Energy_Consumption'] = energy_prod_mix_csa.sum(axis=1)
energy_prod_mix_csa['Coal_Portion'] = energy_prod_mix_csa['Total_Production_Central_South_America']/energy_prod_mix_csa['Total_Energy_Consumption']*100
energy_prod_mix_csa['NG_Portion'] = energy_prod_mix_csa['Total_NG_Production_Central_South_America']/energy_prod_mix_csa['Total_Energy_Consumption']*100
energy_prod_mix_csa['RE_Portion'] = energy_prod_mix_csa['Total_RE_Production_Central_South_America']/energy_prod_mix_csa['Total_Energy_Consumption']*100
energy_prod_mix_csa['PTL_Portion'] = energy_prod_mix_csa['Total_Ptl_Production_Central_South_America']/energy_prod_mix_csa['Total_Energy_Consumption']*100

energy_prod_mix_csa.head()

,Total_Production_Central_South_America,Total_NG_Production_Central_South_America,Total_RE_Production_Central_South_America,Total_Ptl_Production_Central_South_America,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,0.220723,1.369422,2.249169,8.294547,24.267721,0.909534,5.642977,9.268149,34.179339
1981,0.234615,1.517969,2.306281,8.275871,24.669473,0.951034,6.153228,9.348726,33.547012
1982,0.252328,1.591101,2.535795,7.835343,24.429133,1.032898,6.513129,10.380208,32.073765
1983,0.268962,1.644515,2.815858,7.761489,24.981647,1.076639,6.582892,11.271705,31.068764
1984,0.325556,1.882318,3.157275,8.158450,27.047196,1.203659,6.959381,11.673206,30.163754


In [823]:
fig_energy_prod_vol_mix_csa = go.Figure()
fig_energy_prod_vol_mix_csa = fig_energy_prod_vol_mix_csa.add_trace(go.Scatter(x=energy_prod_mix_csa.index, y=energy_prod_mix_csa['Total_Production_Central_South_America'], name='Coal', mode='lines +markers'))
fig_energy_prod_vol_mix_csa = fig_energy_prod_vol_mix_csa.add_trace(go.Scatter(x=energy_prod_mix_csa.index, y=energy_prod_mix_csa['Total_NG_Production_Central_South_America'], name='Natural Gas'))
fig_energy_prod_vol_mix_csa = fig_energy_prod_vol_mix_csa.add_trace(go.Scatter(x=energy_prod_mix_csa.index, y=energy_prod_mix_csa['Total_RE_Production_Central_South_America'], name='Nuclear & Renewable'))
fig_energy_prod_vol_mix_csa = fig_energy_prod_vol_mix_csa.add_trace(go.Scatter(x=energy_prod_mix_csa.index, y=energy_prod_mix_csa['Total_Ptl_Production_Central_South_America'], name='Petroleum'))
fig_energy_prod_vol_mix_csa = fig_energy_prod_vol_mix_csa.update_layout(title='Energy Production Volume Mix for Central South America  1980 - 2021')
fig_energy_prod_vol_mix_csa.show()

In [824]:
fig_energy_prod_portion_mix_csa = go.Figure()
fig_energy_prod_portion_mix_csa = fig_energy_prod_portion_mix_csa.add_trace(go.Bar(x=energy_prod_mix_csa.index, y=energy_prod_mix_csa['Coal_Portion'], name='Coal'))
fig_energy_prod_portion_mix_csa = fig_energy_prod_portion_mix_csa.add_trace(go.Bar(x=energy_prod_mix_csa.index, y=energy_prod_mix_csa['NG_Portion'], name='Natural Gas'))
fig_energy_prod_portion_mix_csa = fig_energy_prod_portion_mix_csa.add_trace(go.Bar(x=energy_prod_mix_csa.index, y=energy_prod_mix_csa['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_prod_portion_mix_csa = fig_energy_prod_portion_mix_csa.add_trace(go.Bar(x=energy_prod_mix_csa.index, y=energy_prod_mix_csa['PTL_Portion'], name='Petroleum'))
fig_energy_prod_portion_mix_csa = fig_energy_prod_portion_mix_csa.update_layout(title='Energy Production Portion Mix for Central South America  1980 - 2021',barmode= 'stack')
fig_energy_prod_portion_mix_csa.show()

North America

In [ ]:
energy_mix_na = pd.DataFrame()
energy_mix_na = energy_mix_na.append(con_coal_na_sum['Total_Coal_Consumption_North_America'])
energy_mix_na = energy_mix_na.append(con_ng_na_sum['Total_NG_Consumption_North_America'])
energy_mix_na = energy_mix_na.append(con_re_na_sum['Total_RE_Consumption_North_America'])
energy_mix_na = energy_mix_na.append(con_ptl_na_sum['Total_Ptl_Consumption_North_America'])
energy_mix_na = energy_mix_na.transpose()
energy_mix_na.head()

,Total_Coal_Consumption_North_America,Total_NG_Consumption_North_America,Total_RE_Consumption_North_America,Total_Ptl_Consumption_North_America
1980,16.335291,22.994214,11.308270,40.753419
1981,16.840399,22.522259,11.792478,38.509692
1982,16.327384,21.250015,12.383341,36.570272
1983,16.913976,20.115236,13.147947,35.869606
1984,18.220474,21.446966,13.678316,36.944114


In [ ]:
energy_mix_na['Total_Energy_Consumption'] = energy_mix_na.sum(axis=1)
energy_mix_na.head()

,Total_Coal_Consumption_North_America,Total_NG_Consumption_North_America,Total_RE_Consumption_North_America,Total_Ptl_Consumption_North_America,Total_Energy_Consumption
1980,16.335291,22.994214,11.308270,40.753419,91.391194
1981,16.840399,22.522259,11.792478,38.509692,89.664829
1982,16.327384,21.250015,12.383341,36.570272,86.531012
1983,16.913976,20.115236,13.147947,35.869606,86.046765
1984,18.220474,21.446966,13.678316,36.944114,90.289870


In [ ]:
energy_mix_na['Coal_Portion'] = energy_mix_na['Total_Coal_Consumption_North_America']/energy_mix_na['Total_Energy_Consumption']*100
energy_mix_na['NG_Portion'] = energy_mix_na['Total_NG_Consumption_North_America']/energy_mix_na['Total_Energy_Consumption']*100
energy_mix_na['RE_Portion'] = energy_mix_na['Total_RE_Consumption_North_America']/energy_mix_na['Total_Energy_Consumption']*100
energy_mix_na['PTL_Portion'] = energy_mix_na['Total_Ptl_Consumption_North_America']/energy_mix_na['Total_Energy_Consumption']*100
energy_mix_na.head()

,Total_Coal_Consumption_North_America,Total_NG_Consumption_North_America,Total_RE_Consumption_North_America,Total_Ptl_Consumption_North_America,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,16.335291,22.994214,11.308270,40.753419,91.391194,17.874031,25.160208,12.373478,44.592282
1981,16.840399,22.522259,11.792478,38.509692,89.664829,18.781499,25.118276,13.151733,42.948493
1982,16.327384,21.250015,12.383341,36.570272,86.531012,18.868823,24.557687,14.310871,42.262619
1983,16.913976,20.115236,13.147947,35.869606,86.046765,19.656725,23.377097,15.280001,41.686176
1984,18.220474,21.446966,13.678316,36.944114,90.289870,20.179976,23.753457,15.149336,40.917231


In [ ]:
fig_energy_mix_na = go.Figure()
fig_energy_mix_na = fig_energy_mix_na.add_trace(go.Scatter(x=energy_mix_na.index, y=energy_mix_na['Total_Coal_Consumption_North_America'], name='Coal'))
fig_energy_mix_na = fig_energy_mix_na.add_trace(go.Scatter(x=energy_mix_na.index, y=energy_mix_na['Total_NG_Consumption_North_America'], name='Natural Gas'))
fig_energy_mix_na = fig_energy_mix_na.add_trace(go.Scatter(x=energy_mix_na.index, y=energy_mix_na['Total_RE_Consumption_North_America'], name='Nuclear & Renewable'))
fig_energy_mix_na = fig_energy_mix_na.add_trace(go.Scatter(x=energy_mix_na.index, y=energy_mix_na['Total_Ptl_Consumption_North_America'], name='Petroleum'))
fig_energy_mix_na = fig_energy_mix_na.update_layout(title='Energy Mix Consumption for North America  1980 - 2021')
fig_energy_mix_na.show()

In [ ]:
energy_portion_na = go.Figure()
energy_portion_na = energy_portion_na.add_trace(go.Bar(x=energy_mix_na.index, y=energy_mix_na['Coal_Portion'], name='Coal'))
energy_portion_na = energy_portion_na.add_trace(go.Bar(x=energy_mix_na.index, y=energy_mix_na['NG_Portion'], name='Natural Gas'))
energy_portion_na = energy_portion_na.add_trace(go.Bar(x=energy_mix_na.index, y=energy_mix_na['RE_Portion'], name='Nuclear & Renewable'))
energy_portion_na = energy_portion_na.add_trace(go.Bar(x=energy_mix_na.index, y=energy_mix_na['PTL_Portion'], name='Petroleum'))
energy_portion_na = energy_portion_na.update_layout(title='Energy Mix Consumption for North America  1980 - 2021',barmode= 'stack')
energy_portion_na.show()

In [838]:
energy_prod_mix_na = pd.DataFrame()
energy_prod_mix_na = energy_prod_mix_na.append(prod_coal_na_sum['Total_Production_North_America'])
energy_prod_mix_na = energy_prod_mix_na.append(prod_ng_na_sum['Total_NG_Production_North_America'])
energy_prod_mix_na = energy_prod_mix_na.append(prod_re_na_sum['Total_RE_Production_North_America'])
energy_prod_mix_na = energy_prod_mix_na.append(prod_ptl_na_sum['Total_Ptl_Production_North_America'])
energy_prod_mix_na = energy_prod_mix_na.transpose()
energy_prod_mix_na.head()

,Total_Production_North_America,Total_NG_Production_North_America,Total_RE_Production_North_America,Total_Ptl_Production_North_America
1980,19.442924,23.663903,11.386967,28.522507
1981,19.290128,23.421552,11.893594,29.008905
1982,19.620444,22.203994,12.469701,29.981190
1983,18.280955,20.286013,13.255333,30.102970
1984,21.051215,21.912381,13.800442,31.084721


In [839]:
energy_prod_mix_na['Total_Energy_Production'] = energy_prod_mix_na.sum(axis=1)
energy_prod_mix_na['portion_coal'] = energy_prod_mix_na['Total_Production_North_America']/energy_prod_mix_na['Total_Energy_Production']*100
energy_prod_mix_na['portion_ng'] = energy_prod_mix_na['Total_NG_Production_North_America']/energy_prod_mix_na['Total_Energy_Production']*100
energy_prod_mix_na['portion_re'] = energy_prod_mix_na['Total_RE_Production_North_America']/energy_prod_mix_na['Total_Energy_Production']*100
energy_prod_mix_na['portion_ptl'] = energy_prod_mix_na['Total_Ptl_Production_North_America']/energy_prod_mix_na['Total_Energy_Production']*100

energy_prod_mix_na.head()

,Total_Production_North_America,Total_NG_Production_North_America,Total_RE_Production_North_America,Total_Ptl_Production_North_America,Total_Energy_Production,portion_coal,portion_ng,portion_re,portion_ptl
1980,19.442924,23.663903,11.386967,28.522507,83.016301,23.420609,28.505128,13.716544,34.357719
1981,19.290128,23.421552,11.893594,29.008905,83.614179,23.070403,28.011460,14.224375,34.693763
1982,19.620444,22.203994,12.469701,29.981190,84.275328,23.281362,26.346968,14.796383,35.575287
1983,18.280955,20.286013,13.255333,30.102970,81.925271,22.314183,24.761606,16.179785,36.744425
1984,21.051215,21.912381,13.800442,31.084721,87.848759,23.963020,24.943301,15.709319,35.384360


In [844]:
fig_energy_prod_vol_mix_na = go.Figure()
fig_energy_prod_vol_mix_na = fig_energy_prod_vol_mix_na.add_trace(go.Scatter(x=energy_prod_mix_na.index, y=energy_prod_mix_na['Total_Production_North_America'], name='Coal'))
fig_energy_prod_vol_mix_na = fig_energy_prod_vol_mix_na.add_trace(go.Scatter(x=energy_prod_mix_na.index, y=energy_prod_mix_na['Total_NG_Production_North_America'], name='Natural Gas'))
fig_energy_prod_vol_mix_na = fig_energy_prod_vol_mix_na.add_trace(go.Scatter(x=energy_prod_mix_na.index, y=energy_prod_mix_na['Total_RE_Production_North_America'], name='Nuclear & Renewable'))
fig_energy_prod_vol_mix_na = fig_energy_prod_vol_mix_na.add_trace(go.Scatter(x=energy_prod_mix_na.index, y=energy_prod_mix_na['Total_Ptl_Production_North_America'], name='Petroleum'))
fig_energy_prod_vol_mix_na.update_xaxes(rangeslider_visible=True)
fig_energy_prod_vol_mix_na = fig_energy_prod_vol_mix_na.update_layout(title='Energy Production Volume Mix for North America  1980 - 2021')

In [845]:
fig_energy_prod_portion_mix_na = go.Figure()
fig_energy_prod_portion_mix_na = fig_energy_prod_portion_mix_na.add_trace(go.Bar(x=energy_prod_mix_na.index, y=energy_prod_mix_na['portion_coal'], name='Coal'))
fig_energy_prod_portion_mix_na = fig_energy_prod_portion_mix_na.add_trace(go.Bar(x=energy_prod_mix_na.index, y=energy_prod_mix_na['portion_ng'], name='Natural Gas'))
fig_energy_prod_portion_mix_na = fig_energy_prod_portion_mix_na.add_trace(go.Bar(x=energy_prod_mix_na.index, y=energy_prod_mix_na['portion_re'], name='Nuclear & Renewable'))
fig_energy_prod_portion_mix_na = fig_energy_prod_portion_mix_na.add_trace(go.Bar(x=energy_prod_mix_na.index, y=energy_prod_mix_na['portion_ptl'], name='Petroleum'))
fig_energy_prod_portion_mix_na = fig_energy_prod_portion_mix_na.update_layout(title='Energy Production Portion Mix for North America  1980 - 2021')
fig_energy_prod_portion_mix_na.show()

Europe

In [ ]:
energy_mix_europe = pd.DataFrame()
energy_mix_europe = energy_mix_europe.append(con_coal_europe_sum['Total_Coal_Consumption_Europe'])
energy_mix_europe = energy_mix_europe.append(con_ng_europe_sum['Total_NG_Consumption_Europe'])
energy_mix_europe = energy_mix_europe.append(con_re_europe_sum['Total_RE_Consumption_Europe'])
energy_mix_europe = energy_mix_europe.append(con_ptl_europe_sum['Total_Ptl_Consumption_Europe'])
energy_mix_europe = energy_mix_europe.transpose()
energy_mix_europe.head()

,Total_Coal_Consumption_Europe,Total_NG_Consumption_Europe,Total_RE_Consumption_Europe,Total_Ptl_Consumption_Europe
1980,22.720847,10.642745,7.651363,34.352347
1981,22.652893,10.513927,8.368064,31.494244
1982,23.006503,10.554687,8.674342,30.397382
1983,23.193848,10.787350,9.489478,29.547442
1984,23.167302,11.338709,10.680681,28.938635


In [ ]:
energy_mix_europe['Total_Energy_Consumption'] = energy_mix_europe.sum(axis=1)
energy_mix_europe.head()

,Total_Coal_Consumption_Europe,Total_NG_Consumption_Europe,Total_RE_Consumption_Europe,Total_Ptl_Consumption_Europe,Total_Energy_Consumption
1980,22.720847,10.642745,7.651363,34.352347,75.367302
1981,22.652893,10.513927,8.368064,31.494244,73.029128
1982,23.006503,10.554687,8.674342,30.397382,72.632915
1983,23.193848,10.787350,9.489478,29.547442,73.018118
1984,23.167302,11.338709,10.680681,28.938635,74.125327


In [ ]:
energy_mix_europe['Coal_Portion'] = energy_mix_europe['Total_Coal_Consumption_Europe']/energy_mix_europe['Total_Energy_Consumption']*100
energy_mix_europe['NG_Portion'] = energy_mix_europe['Total_NG_Consumption_Europe']/energy_mix_europe['Total_Energy_Consumption']*100
energy_mix_europe['RE_Portion'] = energy_mix_europe['Total_RE_Consumption_Europe']/energy_mix_europe['Total_Energy_Consumption']*100
energy_mix_europe['PTL_Portion'] = energy_mix_europe['Total_Ptl_Consumption_Europe']/energy_mix_europe['Total_Energy_Consumption']*100
energy_mix_europe.head()

,Total_Coal_Consumption_Europe,Total_NG_Consumption_Europe,Total_RE_Consumption_Europe,Total_Ptl_Consumption_Europe,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,22.720847,10.642745,7.651363,34.352347,75.367302,30.146823,14.121170,10.152099,45.579908
1981,22.652893,10.513927,8.368064,31.494244,73.029128,31.018983,14.396895,11.458530,43.125592
1982,23.006503,10.554687,8.674342,30.397382,72.632915,31.675038,14.531549,11.942715,41.850698
1983,23.193848,10.787350,9.489478,29.547442,73.018118,31.764511,14.773525,12.996060,40.465905
1984,23.167302,11.338709,10.680681,28.938635,74.125327,31.254233,15.296673,14.408949,39.040145


In [ ]:
fig_energy_mix_europe = go.Figure()
fig_energy_mix_europe = fig_energy_mix_europe.add_trace(go.Scatter(x=energy_mix_europe.index, y=energy_mix_europe['Total_Coal_Consumption_Europe'], name='Coal'))
fig_energy_mix_europe = fig_energy_mix_europe.add_trace(go.Scatter(x=energy_mix_europe.index, y=energy_mix_europe['Total_NG_Consumption_Europe'], name='Natural Gas'))
fig_energy_mix_europe = fig_energy_mix_europe.add_trace(go.Scatter(x=energy_mix_europe.index, y=energy_mix_europe['Total_RE_Consumption_Europe'], name='Nuclear & Renewable'))
fig_energy_mix_europe = fig_energy_mix_europe.add_trace(go.Scatter(x=energy_mix_europe.index, y=energy_mix_europe['Total_Ptl_Consumption_Europe'], name='Petroleum'))
fig_energy_mix_europe = fig_energy_mix_europe.update_layout(title='Energy Mix Consumption for Europe  1980 - 2021')
fig_energy_mix_europe.show()

In [ ]:
energy_portion_europe = go.Figure()
energy_portion_europe = energy_portion_europe.add_trace(go.Bar(x=energy_mix_europe.index, y=energy_mix_europe['Coal_Portion'], name='Coal'))
energy_portion_europe = energy_portion_europe.add_trace(go.Bar(x=energy_mix_europe.index, y=energy_mix_europe['NG_Portion'], name='Natural Gas'))
energy_portion_europe = energy_portion_europe.add_trace(go.Bar(x=energy_mix_europe.index, y=energy_mix_europe['RE_Portion'], name='Nuclear & Renewable'))
energy_portion_europe = energy_portion_europe.add_trace(go.Bar(x=energy_mix_europe.index, y=energy_mix_europe['PTL_Portion'], name='Petroleum'))
energy_portion_europe = energy_portion_europe.update_layout(title='Energy Mix Consumption for Europe  1980 - 2021',barmode= 'stack')
energy_portion_europe.show()

Eurasia

In [ ]:
energy_mix_eurasia = pd.DataFrame()
energy_mix_eurasia = energy_mix_eurasia.append(con_coal_eurasia_sum['Total_Coal_Consumption_Eurasia'])
energy_mix_eurasia = energy_mix_eurasia.append(con_ng_eurasia_sum['Total_NG_Consumption_Eurasia'])
energy_mix_eurasia = energy_mix_eurasia.append(con_re_eurasia_sum['Total_RE_Consumption_Eurasia'])
energy_mix_eurasia = energy_mix_eurasia.append(con_ptl_eurasia_sum['Total_Ptl_Consumption_Eurasia'])
energy_mix_eurasia = energy_mix_eurasia.transpose()
energy_mix_eurasia.head()

,Total_Coal_Consumption_Eurasia,Total_NG_Consumption_Eurasia,Total_RE_Consumption_Eurasia,Total_Ptl_Consumption_Eurasia
1980,15.909849,13.861120,2.681172,19.117843
1981,14.988228,14.425560,2.598289,18.938434
1982,15.016144,15.506478,2.885608,19.235175
1983,14.956393,16.805178,3.069050,18.970227
1984,14.816756,19.104384,3.664546,18.937185


In [ ]:
energy_mix_eurasia['Total_Energy_Consumption'] = energy_mix_eurasia.sum(axis=1)
energy_mix_eurasia.head()

,Total_Coal_Consumption_Eurasia,Total_NG_Consumption_Eurasia,Total_RE_Consumption_Eurasia,Total_Ptl_Consumption_Eurasia,Total_Energy_Consumption
1980,15.909849,13.861120,2.681172,19.117843,51.569984
1981,14.988228,14.425560,2.598289,18.938434,50.950511
1982,15.016144,15.506478,2.885608,19.235175,52.643404
1983,14.956393,16.805178,3.069050,18.970227,53.800848
1984,14.816756,19.104384,3.664546,18.937185,56.522871


In [ ]:
energy_mix_eurasia['Coal_Portion'] = energy_mix_eurasia['Total_Coal_Consumption_Eurasia']/energy_mix_eurasia['Total_Energy_Consumption']*100
energy_mix_eurasia['NG_Portion'] = energy_mix_eurasia['Total_NG_Consumption_Eurasia']/energy_mix_eurasia['Total_Energy_Consumption']*100
energy_mix_eurasia['RE_Portion'] = energy_mix_eurasia['Total_RE_Consumption_Eurasia']/energy_mix_eurasia['Total_Energy_Consumption']*100
energy_mix_eurasia['PTL_Portion'] = energy_mix_eurasia['Total_Ptl_Consumption_Eurasia']/energy_mix_eurasia['Total_Energy_Consumption']*100
energy_mix_eurasia.head()

,Total_Coal_Consumption_Eurasia,Total_NG_Consumption_Eurasia,Total_RE_Consumption_Eurasia,Total_Ptl_Consumption_Eurasia,Total_Energy_Consumption,Coal_Portion,NG_Portion,RE_Portion,PTL_Portion
1980,15.909849,13.861120,2.681172,19.117843,51.569984,30.850987,26.878271,5.199095,37.071648
1981,14.988228,14.425560,2.598289,18.938434,50.950511,29.417229,28.312886,5.099633,37.170253
1982,15.016144,15.506478,2.885608,19.235175,52.643404,28.524265,29.455690,5.481423,36.538622
1983,14.956393,16.805178,3.069050,18.970227,53.800848,27.799549,31.235898,5.704463,35.260090
1984,14.816756,19.104384,3.664546,18.937185,56.522871,26.213736,33.799387,6.483298,33.503579


In [ ]:
fig_energy_mix_eurasia = go.Figure()
fig_energy_mix_eurasia = fig_energy_mix_eurasia.add_trace(go.Scatter(x=energy_mix_eurasia.index, y=energy_mix_eurasia['Total_Coal_Consumption_Eurasia'], name='Coal'))
fig_energy_mix_eurasia = fig_energy_mix_eurasia.add_trace(go.Scatter(x=energy_mix_eurasia.index, y=energy_mix_eurasia['Total_NG_Consumption_Eurasia'], name='Natural Gas'))
fig_energy_mix_eurasia = fig_energy_mix_eurasia.add_trace(go.Scatter(x=energy_mix_eurasia.index, y=energy_mix_eurasia['Total_RE_Consumption_Eurasia'], name='Nuclear & Renewable'))
fig_energy_mix_eurasia = fig_energy_mix_eurasia.add_trace(go.Scatter(x=energy_mix_eurasia.index, y=energy_mix_eurasia['Total_Ptl_Consumption_Eurasia'], name='Petroleum'))
fig_energy_mix_eurasia = fig_energy_mix_eurasia.update_layout(title='Energy Mix Consumption for Eurosia  1980 - 2021')
fig_energy_mix_eurasia.show()

In [ ]:
fig_energy_portion_eurasia = go.Figure()
fig_energy_portion_eurasia = fig_energy_portion_eurasia.add_trace(go.Bar(x=energy_mix_eurasia.index, y=energy_mix_eurasia['Coal_Portion'], name='Coal'))
fig_energy_portion_eurasia = fig_energy_portion_eurasia.add_trace(go.Bar(x=energy_mix_eurasia.index, y=energy_mix_eurasia['NG_Portion'], name='Natural Gas'))
fig_energy_portion_eurasia = fig_energy_portion_eurasia.add_trace(go.Bar(x=energy_mix_eurasia.index, y=energy_mix_eurasia['RE_Portion'], name='Nuclear & Renewable'))
fig_energy_portion_eurasia = fig_energy_portion_eurasia.add_trace(go.Bar(x=energy_mix_eurasia.index, y=energy_mix_eurasia['PTL_Portion'], name='Petroleum'))
fig_energy_portion_eurasia = fig_energy_portion_eurasia.update_layout(title='Energy Mix Consumption for Eurosia  1980 - 2021',barmode= 'stack')
fig_energy_portion_eurasia.show()

### Energy Production Mix

In [ ]:
prod_coal_africa_sum.head()

,Total_Production_Africa
1980,2.644093
1981,2.914258
1982,3.154886
1983,3.308514
1984,3.695633


In [ ]:
prod_coal_all_sum  = prod_coal.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_coal_all_sum = prod_coal_all_sum.sum(axis=0)
prod_coal_all_sum = pd.DataFrame(prod_coal_all_sum)
prod_coal_all_sum = prod_coal_all_sum.rename(columns={0: 'Coal_Production'})
prod_coal_all_sum.head()

,Coal_Production
1980,80.117704
1981,80.461549
1982,83.395337
1983,83.674348
1984,87.679784


In [ ]:
prod_ng_all_sum = prod_ng.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ng_all_sum = prod_ng_all_sum.sum(axis=0)
prod_ng_all_sum = pd.DataFrame(prod_ng_all_sum)
prod_ng_all_sum = prod_ng_all_sum.rename(columns={0: 'NG_Production'})
prod_ng_all_sum.head()

,NG_Production
1980,54.761046
1981,55.573537
1982,55.495523
1983,56.115952
1984,61.758598


In [ ]:
prod_re_all_sum = prod_re.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_re_all_sum = prod_re_all_sum.sum(axis=0)
prod_re_all_sum = pd.DataFrame(prod_re_all_sum)
prod_re_all_sum = prod_re_all_sum.rename(columns={0: 'RE_Production'})
prod_re_all_sum.head()

,RE_Production
1980,28.352578
1981,29.819730
1982,31.474538
1983,33.988222
1984,36.954675


In [ ]:
prod_ptl_all_sum = prod_ptl.iloc[:,2:44].apply(pd.to_numeric, errors='coerce')
prod_ptl_all_sum = prod_ptl_all_sum.sum(axis=0)
prod_ptl_all_sum = pd.DataFrame(prod_ptl_all_sum)
prod_ptl_all_sum = prod_ptl_all_sum.rename(columns={0: 'PTL_Production'})
prod_ptl_all_sum.head()

,PTL_Production
1980,133.111109
1981,125.438949
1982,119.762805
1983,119.269196
1984,122.553827


In [ ]:
prod_energy_all_sum = prod_coal_all_sum.join(prod_ng_all_sum).join(prod_re_all_sum).join(prod_ptl_all_sum)
prod_energy_all_sum['Energy_Production'] = prod_energy_all_sum.sum(axis=1)
prod_energy_all_sum['portion_coal'] = prod_energy_all_sum['Coal_Production']/prod_energy_all_sum['Energy_Production']*100
prod_energy_all_sum['portion_ng'] = prod_energy_all_sum['NG_Production']/prod_energy_all_sum['Energy_Production']*100
prod_energy_all_sum['portion_re'] = prod_energy_all_sum['RE_Production']/prod_energy_all_sum['Energy_Production']*100
prod_energy_all_sum['portion_ptl'] = prod_energy_all_sum['PTL_Production']/prod_energy_all_sum['Energy_Production']*100
prod_energy_all_sum.head()

,Coal_Production,NG_Production,RE_Production,PTL_Production,Energy_Production,portion_coal,portion_ng,portion_re,portion_ptl
1980,80.117704,54.761046,28.352578,133.111109,296.342436,27.035515,18.478975,9.567505,44.918005
1981,80.461549,55.573537,29.819730,125.438949,291.293766,27.622132,19.078176,10.236996,43.062696
1982,83.395337,55.495523,31.474538,119.762805,290.128203,28.744306,19.127931,10.848493,41.279270
1983,83.674348,56.115952,33.988222,119.269196,293.047718,28.553148,19.149083,11.598187,40.699582
1984,87.679784,61.758598,36.954675,122.553827,308.946883,28.380213,19.990037,11.961498,39.668251


In [ ]:
fig_prod_energy_all_sum = go.Figure()
fig_prod_energy_all_sum = fig_prod_energy_all_sum.add_trace(go.Scatter(x=prod_energy_all_sum.index, y=prod_energy_all_sum['Energy_Production'], name='Energy', mode='lines+markers'))
fig_prod_energy_all_sum = fig_prod_energy_all_sum.add_trace(go.Scatter(x=prod_energy_all_sum.index, y=prod_energy_all_sum['Coal_Production'], name='Coal',mode='lines+markers'))
fig_prod_energy_all_sum = fig_prod_energy_all_sum.add_trace(go.Scatter(x=prod_energy_all_sum.index, y=prod_energy_all_sum['NG_Production'], name='Natural Gas',mode='lines+markers'))
fig_prod_energy_all_sum = fig_prod_energy_all_sum.add_trace(go.Scatter(x=prod_energy_all_sum.index, y=prod_energy_all_sum['RE_Production'], name='Nuclear & Renewable',mode='lines+markers'))
fig_prod_energy_all_sum = fig_prod_energy_all_sum.add_trace(go.Scatter(x=prod_energy_all_sum.index, y=prod_energy_all_sum['PTL_Production'], name='Petroleum',mode='lines+markers'))
fig_prod_energy_all_sum = fig_prod_energy_all_sum.update_layout(title='Energy Production Volume  1980 - 2021')
fig_prod_energy_all_sum.show()

In [ ]:
fig_prod_energy_all = go.Figure()
fig_prod_energy_all = fig_prod_energy_all.add_trace(go.Bar(x=prod_energy_all_sum.index, y=prod_energy_all_sum['portion_coal'], name='Coal'))
fig_prod_energy_all = fig_prod_energy_all.add_trace(go.Bar(x=prod_energy_all_sum.index, y=prod_energy_all_sum['portion_ng'], name='Natural Gas'))
fig_prod_energy_all = fig_prod_energy_all.add_trace(go.Bar(x=prod_energy_all_sum.index, y=prod_energy_all_sum['portion_re'], name='Nuclear & Renewable'))
fig_prod_energy_all = fig_prod_energy_all.add_trace(go.Bar(x=prod_energy_all_sum.index, y=prod_energy_all_sum['portion_ptl'], name='Petroleum'))
fig_prod_energy_all = fig_prod_energy_all.update_layout(title='Energy Production Mix  1980 - 2021', barmode= 'stack')
fig_prod_energy_all.show()

#### Top 10 Coal Producer & Global Shares

In [ ]:
prod_coal.iloc[:,2:45] = prod_coal.iloc[:,2:45].apply(pd.to_numeric, errors='coerce')

In [ ]:
top_production_coal_1980 = prod_coal[['Country', '1980']].sort_values(by=['1980'], ascending=False).head(10)
sum_coal_1980 = prod_coal['1980'].sum()
top_production_coal_1980['portion_prod_1980'] = (top_production_coal_1980['1980']/sum_coal_1980)*100
top_production_coal_1980

,Country,1980,portion_prod_1980
171,United States,18.597725,23.213004
220,Former U.S.S.R.,17.152565,21.409206
65,China,13.811305,17.238768
189,"Germany, East",4.725552,5.898261
206,Poland,4.066769,5.075993
216,United Kingdom,3.310692,4.132285
190,"Germany, West",3.150368,3.932174
47,South Africa,2.525071,3.151702
59,Australia,2.374453,2.963706
184,Former Czechoslovakia,1.977908,2.468752


In [ ]:
top_production_coal_1990 = prod_coal[['Country', '1990']].sort_values(by=['1990'], ascending=False).head(10)
sum_coal_1990 = prod_coal['1990'].sum()
top_production_coal_1990['portion_prod_1990'] = (top_production_coal_1990['1990']/sum_coal_1990)*100
top_production_coal_1990

,Country,1990,portion_prod_1990
65,China,24.037381,24.250637
171,United States,22.487548,22.687055
220,Former U.S.S.R.,17.032784,17.183897
59,Australia,4.388971,4.427909
189,"Germany, East",4.171783,4.208795
206,Poland,3.803122,3.836862
47,South Africa,3.770739,3.804193
72,India,3.699749,3.732573
190,"Germany, West",2.781189,2.805863
216,United Kingdom,2.360596,2.381538


In [ ]:
top_production_coal_2000 = prod_coal[['Country', '2000']].sort_values(by=['2000'], ascending=False).head(10)
sum_coal_2000 = prod_coal['2000'].sum()
top_production_coal_2000['portion_prod_2000'] = (top_production_coal_2000['2000']/sum_coal_2000)*100
top_production_coal_2000

,Country,2000,portion_prod_2000
65,China,30.827508,32.007357
171,United States,22.735478,23.605624
59,Australia,6.627498,6.881150
225,Russia,5.742782,5.962573
72,India,5.396460,5.602997
47,South Africa,4.929281,5.117938
206,Poland,2.797945,2.905030
188,Germany,2.578540,2.677227
222,Kazakhstan,1.990207,2.066378
73,Indonesia,1.774650,1.842570


In [ ]:
top_production_coal_2010 = prod_coal[['Country', '2010']].sort_values(by=['2010'], ascending=False).head(10)
sum_coal_2010 = prod_coal['2010'].sum()
top_production_coal_2010['portion_prod_2010'] = (top_production_coal_2010['2010']/sum_coal_2010)*100
top_production_coal_2010

,Country,2010,portion_prod_2010
65,China,76.355779,48.890206
171,United States,22.038226,14.110961
59,Australia,9.954672,6.373924
72,India,8.618543,5.518408
73,Indonesia,7.345464,4.703262
225,Russia,7.061008,4.521126
47,South Africa,5.579088,3.572261
222,Kazakhstan,2.948104,1.887656
206,Poland,2.241278,1.435078
188,Germany,1.932962,1.237665


In [ ]:
top_production_coal_2020 = prod_coal[['Country', '2020']].sort_values(by=['2020'], ascending=False).head(10)
sum_coal_2020 = prod_coal['2020'].sum()
top_production_coal_2020['portion_prod_2020'] = (top_production_coal_2020['2020']/sum_coal_2020)*100
top_production_coal_2020

,Country,2020,portion_prod_2020
65,China,88.852984,53.870113
72,India,12.161194,7.373133
59,Australia,11.888074,7.207545
73,Indonesia,11.105554,6.733116
171,United States,10.703214,6.489184
225,Russia,9.651781,5.851717
47,South Africa,5.394765,3.270758
222,Kazakhstan,2.814983,1.706678
206,Poland,1.694473,1.027331
131,Colombia,1.295995,0.785741


#### Top 10 Natural Gas Producer & Global Shares

In [ ]:
prod_ng.iloc[:, 2:45] = prod_ng.iloc[:, 2:45].apply(pd.to_numeric, errors='coerce')

In [ ]:
top_production_ng_1980 = prod_ng[['Country', '1980']].sort_values(by=['1980'], ascending=False).head(10)
sum_ng_1980 = prod_ng['1980'].sum()
top_production_ng_1980['portion_pro_1980'] = (top_production_ng_1980['1980']/sum_ng_1980)*100
top_production_ng_1980

,Country,1980,portion_pro_1980
171,United States,19.907600,36.353579
220,Former U.S.S.R.,15.984800,29.190093
203,Netherlands,3.037812,5.547396
166,Canada,2.805903,5.123903
216,United Kingdom,1.381212,2.522253
208,Romania,1.284000,2.344732
205,Norway,1.004115,1.833630
168,Mexico,0.950400,1.735540
73,Indonesia,0.719400,1.313708
164,Venezuela,0.615747,1.124425


In [ ]:
top_production_ng_1990 = prod_ng[['Country', '1990']].sort_values(by=['1990'], ascending=False).head(10)
sum_ng_1990 = prod_ng['1990'].sum()
top_production_ng_1990['portion_pro_1990'] = (top_production_ng_1990['1990']/sum_ng_1990)*100
top_production_ng_1990

,Country,1990,portion_pro_1990
220,Former U.S.S.R.,29.530332,39.004219
171,United States,18.326155,24.205530
166,Canada,3.902866,5.154979
203,Netherlands,2.402379,3.173107
0,Algeria,2.108660,2.785158
216,United Kingdom,1.804866,2.383901
73,Indonesia,1.762200,2.327547
116,Saudi Arabia,1.127623,1.489385
208,Romania,1.071070,1.414689
205,Norway,1.064816,1.406429


In [ ]:
top_production_ng_2000 = prod_ng[['Country', '2000']].sort_values(by=['2000'], ascending=False).head(10)
sum_ng_2000 = prod_ng['2000'].sum()
top_production_ng_2000['portion_pro_2000'] = (top_production_ng_2000['2000']/sum_ng_2000)*100
top_production_ng_2000

,Country,2000,portion_pro_2000
171,United States,19.661529,22.021278
225,Russia,19.508977,21.850417
166,Canada,6.599102,7.391117
216,United Kingdom,4.051389,4.537631
0,Algeria,3.312952,3.710568
73,Indonesia,2.438169,2.730794
203,Netherlands,2.299076,2.575008
107,Iran,2.246509,2.516131
229,Uzbekistan,2.025626,2.268739
205,Norway,2.022456,2.265188


In [ ]:
top_production_ng_2010 = prod_ng[['Country', '2010']].sort_values(by=['2010'], ascending=False).head(10)
sum_ng_2010 = prod_ng['2010'].sum()
top_production_ng_2010['portion_pro_2010'] = (top_production_ng_2010['2010']/sum_ng_2010)*100
top_production_ng_2010

,Country,2010,portion_pro_2010
225,Russia,22.015649,18.825534
171,United States,21.805763,18.646062
166,Canada,5.571874,4.764497
107,Iran,5.450319,4.660556
115,Qatar,4.578720,3.915254
205,Norway,4.001002,3.421248
65,China,3.484123,2.979266
116,Saudi Arabia,3.241211,2.771553
73,Indonesia,3.179551,2.718827
0,Algeria,3.173258,2.713446


In [ ]:
top_production_ng_2020 = prod_ng[['Country', '2020']].sort_values(by=['2020'], ascending=False).head(10)
sum_ng_2020 = prod_ng['2020'].sum()
top_production_ng_2020['portion_pro_2020'] = (top_production_ng_2020['2020']/sum_ng_2020)*100
top_production_ng_2020

,Country,2020,portion_pro_2020
171,United States,34.731981,24.016086
225,Russia,24.170206,16.712947
107,Iran,9.128470,6.312054
65,China,6.940324,4.799019
115,Qatar,6.536303,4.519651
166,Canada,6.534893,4.518675
59,Australia,5.350861,3.699954
116,Saudi Arabia,4.206837,2.908897
205,Norway,4.175657,2.887337
0,Algeria,3.162318,2.186645


#### Top 10 Nuclear & Renewable Energy & Global Shares

In [ ]:
prod_re.iloc[:, 2:45] = prod_re.iloc[:, 2:45].apply(pd.to_numeric, errors='coerce')

In [ ]:
top_production_re_1980 = prod_re[['Country', '1980']].sort_values(by=['1980'], ascending=False).head(10)
sum_re_1980 = prod_re['1980'].sum()
top_production_re_1980['portion_pro_1980'] = (top_production_re_1980['1980']/sum_re_1980)*100
top_production_re_1980

,Country,1980,portion_pro_1980
171,United States,8.167511,28.806944
166,Canada,3.036118,10.708438
220,Former U.S.S.R.,2.744291,9.679158
74,Japan,1.769908,6.242493
187,France,1.440150,5.079431
127,Brazil,1.437510,5.070122
213,Sweden,0.871722,3.074579
205,Norway,0.859264,3.030640
190,"Germany, West",0.697464,2.459966
65,China,0.598536,2.111046


In [ ]:
top_production_re_1990 = prod_re[['Country', '1990']].sort_values(by=['1990'], ascending=False).head(10)
sum_re_1990 = prod_re['1990'].sum()
top_production_re_1990['portion_pro_1990'] = (top_production_re_1990['1990']/sum_re_1990)*100
top_production_re_1990

,Country,1990,portion_pro_1990
171,United States,12.144393,26.046288
220,Former U.S.S.R.,4.707286,10.095796
166,Canada,3.893037,8.349463
187,France,3.678392,7.889110
74,Japan,3.021250,6.479727
127,Brazil,2.452306,5.259504
190,"Germany, West",1.673756,3.589733
213,Sweden,1.415378,3.035585
65,China,1.301706,2.791792
205,Norway,1.250060,2.681026


In [ ]:
top_production_re_2000 = prod_re[['Country', '2000']].sort_values(by=['2000'], ascending=False).head(10)
sum_re_2000 = prod_re['2000'].sum()
top_production_re_2000['portion_pro_2000'] = (top_production_re_2000['2000']/sum_re_2000)*100
top_production_re_2000

,Country,2000,portion_pro_2000
171,United States,13.964078,24.246221
187,France,4.811295,8.353986
166,Canada,4.492069,7.799706
74,Japan,4.123961,7.160550
127,Brazil,3.444276,5.980393
225,Russia,3.050930,5.297416
65,China,2.438038,4.233234
188,Germany,2.049545,3.558682
205,Norway,1.435556,2.492596
213,Sweden,1.391409,2.415942


In [ ]:
top_production_re_2010 = prod_re[['Country', '2010']].sort_values(by=['2010'], ascending=False).head(10)
sum_re_2010 = prod_re['2010'].sum()
top_production_re_2010['portion_pro_2010'] = (top_production_re_2010['2010']/sum_re_2010)*100
top_production_re_2010

,Country,2010,portion_pro_2010
171,United States,16.749439,22.719651
65,China,8.464404,11.481478
187,France,5.109924,6.931318
127,Brazil,5.034744,6.829340
166,Canada,4.571422,6.200871
74,Japan,3.859475,5.235156
225,Russia,3.409521,4.624820
188,Germany,2.542137,3.448263
72,India,1.700188,2.306207
212,Spain,1.582579,2.146678


#### Top 10 Petroleum Producer & Global Shares

In [ ]:
prod_ptl.iloc[:, 2:45] = prod_ptl.iloc[:, 2:45].apply(pd.to_numeric, errors='coerce')

In [ ]:
top_production_ptl_1980 = prod_ptl[['Country', '1980']].sort_values(by=['1980'], ascending=False).head(10)
sum_ptl_1980 = prod_ptl['1980'].sum()
top_production_ptl_1980['portion_pro_1980'] = (top_production_ptl_1980['1980']/sum_ptl_1980)*100
top_production_ptl_1980

,Country,1980,portion_pro_1980
220,Former U.S.S.R.,25.624987,19.250825
116,Saudi Arabia,22.084850,16.591290
171,United States,20.473758,15.380954
108,Iraq,5.378475,4.040591
164,Venezuela,4.999819,3.756125
65,China,4.548403,3.416997
168,Mexico,4.514688,3.391669
38,Nigeria,4.422161,3.322158
28,Libya,3.840462,2.885155
111,Kuwait,3.734649,2.805663


In [ ]:
top_production_ptl_1990 = prod_ptl[['Country', '1990']].sort_values(by=['1990'], ascending=False).head(10)
sum_ptl_1990 = prod_ptl['1990'].sum()
top_production_ptl_1990['portion_pro_1990'] = (top_production_ptl_1990['1990']/sum_ptl_1990)*100
top_production_ptl_1990

,Country,1990,portion_pro_1990
220,Former U.S.S.R.,24.198385,17.766739
171,United States,17.709484,13.002511
116,Saudi Arabia,14.792507,10.860833
107,Iran,6.692022,4.913361
168,Mexico,6.165967,4.527126
65,China,5.952549,4.370431
164,Venezuela,4.959888,3.641608
118,United Arab Emirates,4.695729,3.447659
108,Iraq,4.381184,3.216717
216,United Kingdom,4.031928,2.960289


In [ ]:
top_production_ptl_2000 = prod_ptl[['Country', '2000']].sort_values(by=['2000'], ascending=False).head(10)
sum_ptl_2000 = prod_ptl['2000'].sum()
top_production_ptl_2000['portion_pro_2000'] = (top_production_ptl_2000['2000']/sum_ptl_2000)*100
top_production_ptl_2000

,Country,2000,portion_pro_2000
116,Saudi Arabia,19.751398,12.632614
171,United States,14.909567,9.535872
225,Russia,14.296582,9.143819
107,Iran,8.084904,5.170949
164,Venezuela,7.352966,4.702816
168,Mexico,7.286146,4.660079
65,China,6.990412,4.470933
205,Norway,6.823984,4.364489
108,Iraq,5.499736,3.517526
118,United Arab Emirates,5.347763,3.420326


In [ ]:
top_production_ptl_2010 = prod_ptl[['Country', '2010']].sort_values(by=['2010'], ascending=False).head(10)
sum_ptl_2010 = prod_ptl['2010'].sum()
top_production_ptl_2010['portion_pro_2010'] = (top_production_ptl_2010['2010']/sum_ptl_2010)*100
top_production_ptl_2010

,Country,2010,portion_pro_2010
225,Russia,21.674508,12.674385
116,Saudi Arabia,20.093515,11.749883
171,United States,14.315301,8.371015
107,Iran,9.079599,5.309386
65,China,8.757405,5.120980
166,Canada,6.679306,3.905791
168,Mexico,6.188541,3.618811
118,United Arab Emirates,5.816506,3.401260
164,Venezuela,5.664614,3.312440
38,Nigeria,5.247503,3.068529


In [ ]:
top_production_ptl_2020 = prod_ptl[['Country', '2020']].sort_values(by=['2020'], ascending=False).head(10)
sum_ptl_2020 = prod_ptl['2020'].sum()
top_production_ptl_2020['portion_pro_2020'] = (top_production_ptl_2020['2020']/sum_ptl_2020)*100
top_production_ptl_2020

,Country,2020,portion_pro_2020
171,United States,30.378457,16.923985
116,Saudi Arabia,22.440388,12.501648
225,Russia,22.151130,12.340501
166,Canada,10.273098,5.723193
108,Iraq,8.827753,4.917984
65,China,8.375406,4.665979
118,United Arab Emirates,7.687781,4.282900
127,Brazil,6.514799,3.629426
107,Iran,6.238555,3.475529
111,Kuwait,5.810954,3.237310


### Top Consumer & Producer

In [ ]:
total_all_con.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
0,Africa,Algeria,0.780695167,0.663391323,0.952188116,1.070561843,1.130786713,1.046418247,1.066300962,1.138318654,...,2.037584643,2.299888436,2.35214754,2.33598238,2.365986905,2.522119152,2.624873096,2.480810774,2.581123314,64.746181
1,Africa,Angola,0.058366148,0.057863688,0.062007899,0.066297007,0.059894701,0.073766696,0.072853256,0.075135479,...,0.347761278,0.382763726,0.39921871,0.364251517,0.347458949,0.359226946,0.404865374,0.402317536,0.328628418,7.403765
2,Africa,Benin,0.006526525,0.006574612,0.004640112,0.006731564,0.006673241,0.007698814,0.007501055,0.005942365,...,0.066471109,0.070275151,0.077723341,0.094785008,0.10051639,0.111506331,0.101325936,0.108653617,0.110194887,1.660207
3,Africa,Botswana,0.014748762,0.015462338,0.014427219,0.014460824,0.014444631,0.018865991,0.020060806,0.021144246,...,0.065375734,0.065661654,0.080008274,0.088008537,0.080969008,0.080276228,0.080755726,0.076835026,0.082892488,2.021542
4,Africa,Burkina Faso,0.0060363,0.006019807,0.006019807,0.006019807,0.0060363,0.005819147,0.006886218,0.007191413,...,0.0451141,0.042012755,0.053396002,0.051306252,0.05946458,0.066130673,0.067697934,0.064550875,0.068226015,0.988877


In [ ]:
total_all_con_sum = total_all_con.iloc[:, 2:45].apply(pd.to_numeric, errors='coerce')
total_all_con_sum = total_all_con_sum.sum(axis=0)
total_all_con_sum = total_all_con_sum.transpose()
total_all_con_sum = pd.DataFrame(total_all_con_sum)
total_all_con_sum = total_all_con_sum.rename(columns={0:'Total_All_Consumption'})
total_all_con_sum.head()

,Total_All_Consumption
1980,292.939915
1981,289.460953
1982,289.748584
1983,293.743169
1984,307.292935


In [ ]:
top_total_all_con_1980 = total_all_con[['Country', '1980']]
top_total_all_con_1980['1980'] = pd.to_numeric(top_total_all_con_1980['1980'],errors='coerce')
top_total_all_con_1980 = top_total_all_con_1980.reset_index(drop=True).sort_values(by=['1980'], ascending=False,na_position='last')
top_total_all_con_1980['portion'] = top_total_all_con_1980['1980']/total_all_con_sum.loc['1980'][0]*100
top_total_all_con_1980['cumulative'] = top_total_all_con_1980['portion'].cumsum()
top_total_all_con_1980.head()

,Country,1980,portion,cumulative
171,United States,78.021113,26.633828,26.633828
220,Former U.S.S.R.,51.569984,17.604287,44.238115
65,China,19.107288,6.522596,50.760712
74,Japan,15.808905,5.396637,56.157349
190,"Germany, West",11.431371,3.902292,60.059641


In [ ]:
top_total_all_con_1980 = total_all_con[['Country', '1980']]
top_total_all_con_1980['1980'] = pd.to_numeric(top_total_all_con_1980['1980'],errors='coerce')
top_total_all_con_1980 = top_total_all_con_1980.reset_index(drop=True).sort_values(by=['1980'], ascending=False,na_position='last')
top_total_all_con_1980['portion'] = top_total_all_con_1980['1980']/total_all_con_sum.loc['1980'][0]*100
top_total_all_con_1980['cumulative'] = top_total_all_con_1980['portion'].cumsum()
top_total_all_con_1980.head()

,Country,1980,portion,cumulative
171,United States,78.021113,26.633828,26.633828
220,Former U.S.S.R.,51.569984,17.604287,44.238115
65,China,19.107288,6.522596,50.760712
74,Japan,15.808905,5.396637,56.157349
190,"Germany, West",11.431371,3.902292,60.059641


In [ ]:
top_total_all_con_1990 = total_all_con[['Country', '1990']]
top_total_all_con_1990['1990'] = pd.to_numeric(top_total_all_con_1990['1990'],errors='coerce')
top_total_all_con_1990 = top_total_all_con_1990.reset_index(drop=True).sort_values(by=['1990'], ascending=False,na_position='last')
top_total_all_con_1990['portion'] = top_total_all_con_1990['1990']/total_all_con_sum.loc['1990'][0]*100
top_total_all_con_1990['cumulative'] = top_total_all_con_1990['portion'].cumsum()
top_total_all_con_1990.head()

,Country,1990,portion,cumulative
171,United States,84.432879,23.890133,23.890133
220,Former U.S.S.R.,62.940846,17.809001,41.699134
65,China,28.374378,8.028480,49.727614
74,Japan,19.258586,5.449183,55.176796
190,"Germany, West",11.585823,3.278188,58.454984


In [ ]:
top_total_all_con_2000 = total_all_con[['Country', '2000']]
top_total_all_con_2000['2000'] = pd.to_numeric(top_total_all_con_2000['2000'],errors='coerce')
top_total_all_con_2000 = top_total_all_con_2000.reset_index(drop=True).sort_values(by=['2000'], ascending=False,na_position='last')
top_total_all_con_2000['portion'] = top_total_all_con_2000['2000']/total_all_con_sum.loc['2000'][0]*100
top_total_all_con_2000['cumulative'] = top_total_all_con_2000['portion'].cumsum()
top_total_all_con_2000.head()

,Country,2000,portion,cumulative
171,United States,98.702286,24.544147,24.544147
65,China,42.466837,10.560164,35.104311
225,Russia,26.780038,6.659351,41.763662
74,Japan,22.271832,5.538303,47.301964
188,Germany,14.394891,3.579556,50.881520


In [ ]:
top_total_all_con_2010 = total_all_con[['Country', '2010']]
top_total_all_con_2010['2010'] = pd.to_numeric(top_total_all_con_2010['2010'],errors='coerce')
top_total_all_con_2010 = top_total_all_con_2010.reset_index(drop=True).sort_values(by=['2010'], ascending=False,na_position='last')
top_total_all_con_2010['portion'] = top_total_all_con_2010['2010']/total_all_con_sum.loc['2010'][0]*100
top_total_all_con_2010['cumulative'] = top_total_all_con_2010['portion'].cumsum()
top_total_all_con_2010.head()

,Country,2010,portion,cumulative
65,China,112.001176,21.241244,21.241244
171,United States,97.514231,18.493766,39.735010
225,Russia,30.303897,5.747194,45.482204
72,India,22.201384,4.210536,49.692740
74,Japan,21.503885,4.078254,53.770994


In [ ]:
top_total_all_con_2020 = total_all_con[['Country', '2020']]
top_total_all_con_2020['2020'] = pd.to_numeric(top_total_all_con_2020['2020'],errors='coerce')
top_total_all_con_2020 = top_total_all_con_2020.reset_index(drop=True).sort_values(by=['2020'], ascending=False,na_position='last')
top_total_all_con_2020 ['portion'] = top_total_all_con_2020['2020']/total_all_con_sum.loc['2020'][0]*100
top_total_all_con_2020['cumulative'] = top_total_all_con_2020['portion'].cumsum()
top_total_all_con_2020.head()

,Country,2020,portion,cumulative
65,China,156.027762,27.059731,27.059731
171,United States,93.012185,16.131006,43.190736
225,Russia,32.814969,5.691066,48.881802
72,India,29.958162,5.195613,54.077415
74,Japan,17.715349,3.072354,57.149769


In [ ]:
total_all_prod.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
0,Africa,Algeria,2.803017355,3.03753686,3.224933779,3.606400483,3.859176003,3.907465538,3.968324779,4.218086334,...,6.510417691,6.561749872,6.696192523,6.910709728,6.830371007,6.755066467,6.474600013,5.972769931,6.656881917,238.118858
1,Africa,Angola,0.335098042,0.291293691,0.276262403,0.395407667,0.462550452,0.511269369,0.621432995,0.789428936,...,3.932204112,3.802528446,3.949946917,3.93529724,3.899020984,3.814151292,3.526638469,3.159336901,2.824017641,90.852722
2,Africa,Benin,in,0,0,0.00896,0.01573,0.01793,0.01793,0.01569,...,0.000019082,0.00001902,0.000055914,0.00009232,0.000055278,4.64E-05,7.18E-05,6.19E-05,7.07E-05,0.178363
3,Africa,Botswana,0.008262057,0.008484744,0.009241913,0.008796519,0.00875198,0.009731845,0.010912138,0.010845329,...,0.033334507,0.038240847,0.046562169,0.0419666,0.049654188,0.048447799,0.047231633,0.041991759,0.044392707,0.952582
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0.001106756,0.00096051,0.000987814,0.00143096,0.001455654,0.002066608,0.00190567,0.001859876,0.002054882,0.033589


In [ ]:
total_all_prod_sum = total_all_prod.iloc[:, 2:45].apply(pd.to_numeric, errors='coerce')
total_all_prod_sum = total_all_prod_sum.sum(axis=0)
total_all_prod_sum = total_all_prod_sum.transpose()
total_all_prod_sum = pd.DataFrame(total_all_prod_sum)
total_all_prod_sum = total_all_prod_sum.rename(columns={0:'Total_All_Prod'})
total_all_prod_sum.head()

,Total_All_Prod
1980,296.342436
1981,291.293766
1982,290.128203
1983,293.047718
1984,308.946883


In [ ]:
top_total_all_prod_1980 = total_all_prod[['Country', '1980']]
top_total_all_prod_1980['1980'] = pd.to_numeric(top_total_all_prod_1980['1980'],errors='coerce')
top_total_all_prod_1980 = top_total_all_prod_1980.reset_index(drop=True).sort_values(by=['1980'], ascending=False,na_position='last')
top_total_all_prod_1980['portion'] = top_total_all_prod_1980['1980']/total_all_prod_sum.loc['1980'][0]*100
top_total_all_prod_1980['cumulative'] = top_total_all_prod_1980['portion'].cumsum()
top_total_all_prod_1980.head()

,Country,1980,portion,cumulative
183,United States,67.146595,22.658447,22.658447
60,Former U.S.S.R.,61.506642,20.755260,43.413707
174,Saudi Arabia,22.434548,7.570481,50.984188
123,China,19.539499,6.593554,57.577742
179,Canada,10.156057,3.427136,61.004878


In [ ]:
top_total_all_prod_1990 = total_all_prod[['Country', '1990']]
top_total_all_prod_1990['1990'] = pd.to_numeric(top_total_all_prod_1990['1990'],errors='coerce')
top_total_all_prod_1990 = top_total_all_prod_1990.reset_index(drop=True).sort_values(by=['1990'], ascending=False,na_position='last')
top_total_all_prod_1990['portion'] = top_total_all_prod_1990['1990']/total_all_prod_sum.loc['1990'][0]*100
top_total_all_prod_1990['cumulative'] = top_total_all_prod_1990['portion'].cumsum()
top_total_all_prod_1990.head()

,Country,1990,portion,cumulative
60,Former U.S.S.R.,75.468787,21.100830,21.100830
183,United States,70.667580,19.758428,40.859258
123,China,31.876344,8.912523,49.771781
174,Saudi Arabia,15.920130,4.451217,54.222998
179,Canada,13.164539,3.680763,57.903761


In [ ]:
top_total_all_prod_2000 = total_all_prod[['Country', '2000']]
top_total_all_prod_2000['2000'] = pd.to_numeric(top_total_all_prod_2000['2000'],errors='coerce')
top_total_all_prod_2000 = top_total_all_prod_2000.reset_index(drop=True).sort_values(by=['2000'], ascending=False,na_position='last')
top_total_all_prod_2000['portion'] = top_total_all_prod_2000['2000']/total_all_prod_sum.loc['2000'][0]*100
top_total_all_prod_2000['cumulative'] = top_total_all_prod_2000['portion'].cumsum()
top_total_all_prod_2000.head()

,Country,2000,portion,cumulative
183,United States,71.270652,17.838031,17.838031
65,Russia,42.599271,10.661992,28.500022
123,China,41.261597,10.327191,38.827213
174,Saudi Arabia,21.593113,5.404449,44.231663
179,Canada,17.739849,4.440032,48.671694


In [ ]:
top_total_all_prod_2010 = total_all_prod[['Country', '2010']]
top_total_all_prod_2010['2010'] = pd.to_numeric(top_total_all_prod_2010['2010'],errors='coerce')
top_total_all_prod_2010 = top_total_all_prod_2010.reset_index(drop=True).sort_values(by=['2010'], ascending=False,na_position='last')
top_total_all_prod_2010['portion'] = top_total_all_prod_2010['2010']/total_all_prod_sum.loc['2010'][0]*100
top_total_all_prod_2010['cumulative'] = top_total_all_prod_2010['portion'].cumsum()
top_total_all_prod_2010.head()

,Country,2010,portion,cumulative
123,China,97.061712,18.742980,18.742980
183,United States,74.908729,14.465156,33.208136
65,Russia,54.160686,10.458631,43.666768
174,Saudi Arabia,23.334726,4.506023,48.172791
179,Canada,18.228729,3.520036,51.692827


In [ ]:
top_total_all_prod_2020 = total_all_prod[['Country', '2020']]
top_total_all_prod_2020['2020'] = pd.to_numeric(top_total_all_prod_2020['2020'],errors='coerce')
top_total_all_prod_2020 = top_total_all_prod_2020.reset_index(drop=True).sort_values(by=['2020'], ascending=False,na_position='last')
top_total_all_prod_2020 ['portion'] = top_total_all_prod_2020['2020']/total_all_prod_sum.loc['2020'][0]*100
top_total_all_prod_2020['cumulative'] = top_total_all_prod_2020['portion'].cumsum()
top_total_all_prod_2020.head()

,Country,2020,portion,cumulative
123,China,127.001993,21.669499,21.669499
183,United States,95.752763,16.337652,38.007152
65,Russia,60.181599,10.268383,48.275535
174,Saudi Arabia,26.654111,4.547812,52.823347
179,Canada,22.637615,3.862505,56.685852


In [ ]:
total_all_con.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_Consumption
0,Africa,Algeria,0.780695167,0.663391323,0.952188116,1.070561843,1.130786713,1.046418247,1.066300962,1.138318654,...,2.037584643,2.299888436,2.35214754,2.33598238,2.365986905,2.522119152,2.624873096,2.480810774,2.581123314,64.746181
1,Africa,Angola,0.058366148,0.057863688,0.062007899,0.066297007,0.059894701,0.073766696,0.072853256,0.075135479,...,0.347761278,0.382763726,0.39921871,0.364251517,0.347458949,0.359226946,0.404865374,0.402317536,0.328628418,7.403765
2,Africa,Benin,0.006526525,0.006574612,0.004640112,0.006731564,0.006673241,0.007698814,0.007501055,0.005942365,...,0.066471109,0.070275151,0.077723341,0.094785008,0.10051639,0.111506331,0.101325936,0.108653617,0.110194887,1.660207
3,Africa,Botswana,0.014748762,0.015462338,0.014427219,0.014460824,0.014444631,0.018865991,0.020060806,0.021144246,...,0.065375734,0.065661654,0.080008274,0.088008537,0.080969008,0.080276228,0.080755726,0.076835026,0.082892488,2.021542
4,Africa,Burkina Faso,0.0060363,0.006019807,0.006019807,0.006019807,0.0060363,0.005819147,0.006886218,0.007191413,...,0.0451141,0.042012755,0.053396002,0.051306252,0.05946458,0.066130673,0.067697934,0.064550875,0.068226015,0.988877


In [ ]:
china_all_con = total_all_con.loc[total_all_con['Country'] == '        China']
china_all_con = china_all_con.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_all_con = china_all_con.transpose()
china_all_con = pd.DataFrame(china_all_con)
china_all_con = china_all_con.rename(columns={65:'China_All_Con'})
china_all_con.head()


,China_All_Con
1980,19.107288
1981,18.964347
1982,19.938801
1983,21.285521
1984,23.055197


In [ ]:
japan_all_con = total_all_con.loc[total_all_con['Country'] == '        Japan']
japan_all_con = japan_all_con.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
japan_all_con = japan_all_con.transpose()
japan_all_con = pd.DataFrame(japan_all_con)
japan_all_con = japan_all_con.rename(columns={74:'Japan_All_Con'})
japan_all_con.head()

,Japan_All_Con
1980,15.808905
1981,15.790211
1982,15.271856
1983,15.056980
1984,16.040604


In [ ]:

india_all_con = total_all_con.loc[total_all_con['Country'] == '        India']
india_all_con = india_all_con.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
india_all_con = india_all_con.transpose()
india_all_con = pd.DataFrame(india_all_con)
india_all_con = india_all_con.rename(columns={72:'India_All_Con'})
india_all_con.head()


,India_All_Con
1980,3.753147
1981,4.204147
1982,4.356720
1983,4.598950
1984,4.994465


In [ ]:
sky_all_con = total_all_con.loc[total_all_con['Country'] == '        South Korea']
sky_all_con = sky_all_con.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
sky_all_con = sky_all_con.transpose()
sky_all_con = pd.DataFrame(sky_all_con)
sky_all_con = sky_all_con.rename(columns={95:'South_Korea_All_Con'})
sky_all_con.tail()


,South_Korea_All_Con
2017,12.334271
2018,12.416353
2019,12.388879
2020,12.140423
2021,12.713747


In [ ]:
fig_top_asia_all_con = go.Figure()
fig_top_asia_all_con.add_trace(go.Scatter(x=china_all_con.index, y=china_all_con['China_All_Con'], mode='markers + lines', name='China'))
fig_top_asia_all_con.add_trace(go.Scatter(x=japan_all_con.index, y=japan_all_con['Japan_All_Con'], mode='markers + lines', name='Japan'))
fig_top_asia_all_con.add_trace(go.Scatter(x=india_all_con.index, y=india_all_con['India_All_Con'], mode='markers + lines', name='India'))
fig_top_asia_all_con.add_trace(go.Scatter(x=sky_all_con.index, y=sky_all_con['South_Korea_All_Con'], mode='markers + lines', name='South Korea'))
fig_top_asia_all_con.update_xaxes(rangeslider_visible=True)
fig_top_asia_all_con.update_layout(title='Top Energy Consumer in Asia (1980 - 2020)',xaxis_title='Year',yaxis_title='Energy Consumption (qBTU)')
fig_top_asia_all_con.show()

In [ ]:
total_all_prod.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Total_production
0,Africa,Algeria,2.803017355,3.03753686,3.224933779,3.606400483,3.859176003,3.907465538,3.968324779,4.218086334,...,6.510417691,6.561749872,6.696192523,6.910709728,6.830371007,6.755066467,6.474600013,5.972769931,6.656881917,238.118858
1,Africa,Angola,0.335098042,0.291293691,0.276262403,0.395407667,0.462550452,0.511269369,0.621432995,0.789428936,...,3.932204112,3.802528446,3.949946917,3.93529724,3.899020984,3.814151292,3.526638469,3.159336901,2.824017641,90.852722
2,Africa,Benin,in,0,0,0.00896,0.01573,0.01793,0.01793,0.01569,...,0.000019082,0.00001902,0.000055914,0.00009232,0.000055278,4.64E-05,7.18E-05,6.19E-05,7.07E-05,0.178363
3,Africa,Botswana,0.008262057,0.008484744,0.009241913,0.008796519,0.00875198,0.009731845,0.010912138,0.010845329,...,0.033334507,0.038240847,0.046562169,0.0419666,0.049654188,0.048447799,0.047231633,0.041991759,0.044392707,0.952582
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0.001106756,0.00096051,0.000987814,0.00143096,0.001455654,0.002066608,0.00190567,0.001859876,0.002054882,0.033589


In [ ]:
total_all_prod_sum = total_all_prod.iloc[:, 2:45].apply(pd.to_numeric, errors='coerce')
total_all_prod_sum = total_all_prod_sum.sum(axis=0)
total_all_prod_sum = total_all_prod_sum.transpose()
total_all_prod_sum = pd.DataFrame(total_all_prod_sum)
total_all_prod_sum = total_all_prod_sum.rename(columns={0:'Total_All_Prod'})
total_all_prod_sum.head()

,Total_All_Prod
1980,296.342436
1981,291.293766
1982,290.128203
1983,293.047718
1984,308.946883


In [ ]:
top_total_all_prod_1980 = total_all_prod[['Country', '1980']]
top_total_all_prod_1980['1980'] = pd.to_numeric(top_total_all_prod_1980['1980'],errors='coerce')
top_total_all_prod_1980 = top_total_all_prod_1980.reset_index(drop=True).sort_values(by=['1980'], ascending=False,na_position='last')
top_total_all_prod_1980['portion'] = top_total_all_prod_1980['1980']/total_all_prod_sum.loc['1980'][0]*100
top_total_all_prod_1980['cumulative'] = top_total_all_prod_1980['portion'].cumsum()
top_total_all_prod_1980.head()

,Country,1980,portion,cumulative
183,United States,67.146595,22.658447,22.658447
60,Former U.S.S.R.,61.506642,20.755260,43.413707
174,Saudi Arabia,22.434548,7.570481,50.984188
123,China,19.539499,6.593554,57.577742
179,Canada,10.156057,3.427136,61.004878


In [ ]:
top_total_all_prod_1990 = total_all_prod[['Country', '1990']]
top_total_all_prod_1990['1990'] = pd.to_numeric(top_total_all_prod_1990['1990'],errors='coerce')
top_total_all_prod_1990 = top_total_all_prod_1990.reset_index(drop=True).sort_values(by=['1990'], ascending=False,na_position='last')
top_total_all_prod_1990['portion'] = top_total_all_prod_1990['1990']/total_all_prod_sum.loc['1990'][0]*100
top_total_all_prod_1990['cumulative'] = top_total_all_prod_1990['portion'].cumsum()
top_total_all_prod_1990.head()

,Country,1990,portion,cumulative
60,Former U.S.S.R.,75.468787,21.100830,21.100830
183,United States,70.667580,19.758428,40.859258
123,China,31.876344,8.912523,49.771781
174,Saudi Arabia,15.920130,4.451217,54.222998
179,Canada,13.164539,3.680763,57.903761


In [ ]:
top_total_all_prod_2000 = total_all_prod[['Country', '2000']]
top_total_all_prod_2000['2000'] = pd.to_numeric(top_total_all_prod_2000['2000'],errors='coerce')
top_total_all_prod_2000 = top_total_all_prod_2000.reset_index(drop=True).sort_values(by=['2000'], ascending=False,na_position='last')
top_total_all_prod_2000['portion'] = top_total_all_prod_2000['2000']/total_all_prod_sum.loc['2000'][0]*100
top_total_all_prod_2000['cumulative'] = top_total_all_prod_2000['portion'].cumsum()
top_total_all_prod_2000.head()

,Country,2000,portion,cumulative
183,United States,71.270652,17.838031,17.838031
65,Russia,42.599271,10.661992,28.500022
123,China,41.261597,10.327191,38.827213
174,Saudi Arabia,21.593113,5.404449,44.231663
179,Canada,17.739849,4.440032,48.671694


In [ ]:
top_total_all_prod_2010 = total_all_prod[['Country', '2010']]
top_total_all_prod_2010['2010'] = pd.to_numeric(top_total_all_prod_2010['2010'],errors='coerce')
top_total_all_prod_2010 = top_total_all_prod_2010.reset_index(drop=True).sort_values(by=['2010'], ascending=False,na_position='last')
top_total_all_prod_2010['portion'] = top_total_all_prod_2010['2010']/total_all_prod_sum.loc['2010'][0]*100
top_total_all_prod_2010['cumulative'] = top_total_all_prod_2010['portion'].cumsum()
top_total_all_prod_2010.head()

,Country,2010,portion,cumulative
123,China,97.061712,18.742980,18.742980
183,United States,74.908729,14.465156,33.208136
65,Russia,54.160686,10.458631,43.666768
174,Saudi Arabia,23.334726,4.506023,48.172791
179,Canada,18.228729,3.520036,51.692827


In [ ]:
top_total_all_prod_2020 = total_all_prod[['Country', '2020']]
top_total_all_prod_2020['2020'] = pd.to_numeric(top_total_all_prod_2020['2020'],errors='coerce')
top_total_all_prod_2020 = top_total_all_prod_2020.reset_index(drop=True).sort_values(by=['2020'], ascending=False,na_position='last')
top_total_all_prod_2020 ['portion'] = top_total_all_prod_2020['2020']/total_all_prod_sum.loc['2020'][0]*100
top_total_all_prod_2020['cumulative'] = top_total_all_prod_2020['portion'].cumsum()
top_total_all_prod_2020.head()

,Country,2020,portion,cumulative
123,China,127.001993,21.669499,21.669499
183,United States,95.752763,16.337652,38.007152
65,Russia,60.181599,10.268383,48.275535
174,Saudi Arabia,26.654111,4.547812,52.823347
179,Canada,22.637615,3.862505,56.685852


In [ ]:
china_all_prod = total_all_prod.loc[total_all_prod['Country'] == '        China']
china_all_prod = china_all_prod.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_all_prod = china_all_prod.transpose()
china_all_prod = pd.DataFrame(china_all_prod)
china_all_prod = china_all_prod.rename(columns={123:'China_All_Prod'})
china_all_prod.head()

,China_All_Prod
1980,19.539499
1981,19.358625
1982,20.436553
1983,21.857729
1984,23.918051


In [ ]:
fig_china_con_prod = go.Figure()
fig_china_con_prod.add_trace(go.Scatter(x=china_all_prod.index, y=china_all_prod['China_All_Prod'], mode='markers + lines', name='Production'))
fig_china_con_prod.add_trace(go.Scatter(x=china_all_con.index, y=china_all_con['China_All_Con'], mode='markers + lines', name='Consumption'))
fig_china_con_prod.update_xaxes(rangeslider_visible=True)
fig_china_con_prod.update_layout(title_text='China Consumption & Production')
fig_china_con_prod.show()

In [ ]:
con_coal.head()

,Continent,Country,1980,1981,1982,1983,1984,1985,1986,1987,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Consumption_Coal
0,Africa,Algeria,0.002547398,0.007655064,0.020475801,0.033213824,0.034235723,0.034242132,0.034610211,0.035079318,...,0.001259621,0.000835328,0.000576922,0.000575219,0.000924225,0.000410418,0.000405436,0.000658437,0.00094109,0.594818
1,Africa,Angola,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
2,Africa,Benin,0,0,0,0,0,0,0,0,...,3.17E-05,0.000961001,0.001037911,0.001985751,0.002992505,0.002987991,0.002938642,0.001951128,0.003932631,0.019046
3,Africa,Botswana,0.008660526,0.009356617,0.010099701,0.00986717,0.009751175,0.010819193,0.01211937,0.012351123,...,0.014232074,0.016837001,0.027546501,0.038035577,0.035819574,0.033554487,0.031321952,0.029050749,0.032820821,0.808603
4,Africa,Burkina Faso,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000


In [ ]:
china_con_coal = con_coal.loc[con_coal['Country']=='        China']
china_con_coal = china_con_coal.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_con_coal = china_con_coal.transpose() 
china_con_coal = pd.DataFrame(china_con_coal)
china_con_coal = china_con_coal.rename(columns={65:'China_Con_Coal'})
china_con_coal.head()

,China_Con_Coal
1980,13.809654
1981,13.836130
1982,14.847019
1983,15.924503
1984,17.598646


In [ ]:
china_con_ng = con_ng.loc[con_ng['Country']=='        China']
china_con_ng = china_con_ng.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_con_ng = china_con_ng.transpose()
china_con_ng = pd.DataFrame(china_con_ng)
china_con_ng = china_con_ng.rename(columns={65:'China_Con_NG'})
china_con_ng.head()

,China_Con_NG
1980,0.581255
1981,0.517950
1982,0.438531
1983,0.496081
1984,0.504138


In [ ]:
china_con_re = con_re.loc[con_re['Country']=='        China']
china_con_re = china_con_re.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_con_re = china_con_re.transpose()
china_con_re = pd.DataFrame(china_con_re)
china_con_re = china_con_re.rename(columns={65:'China_Con_RE'})
china_con_re.head()

,China_Con_RE
1980,0.592360
1981,0.673220
1982,0.758539
1983,0.891974
1984,0.889890


In [ ]:
china_con_ptl = con_ptl.loc[con_ptl['Country']=='        China']
china_con_ptl = china_con_ptl.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_con_ptl = china_con_ptl.transpose()
china_con_ptl = pd.DataFrame(china_con_ptl)
china_con_ptl = china_con_ptl.rename(columns={65:'China_Con_PTL'})
china_con_ptl.head()

,China_Con_PTL
1980,4.124019
1981,3.937047
1982,3.894712
1983,3.972963
1984,4.062523


In [ ]:
china_all_con_mix = china_all_con['China_All_Con']
china_all_con_mix.head()

1980    19.107288
1981    18.964347
1982    19.938801
1983    21.285521
1984    23.055197
Name: China_All_Con, dtype: float64

In [ ]:
china_energy_mix = china_con_coal.join(china_con_ng).join(china_con_re).join(china_con_ptl).join(china_all_con_mix)
china_energy_mix['portion_coal'] = china_energy_mix['China_Con_Coal']/china_energy_mix['China_All_Con']*100
china_energy_mix['portion_ng'] = china_energy_mix['China_Con_NG']/china_energy_mix['China_All_Con']*100
china_energy_mix['portion_re'] = china_energy_mix['China_Con_RE']/china_energy_mix['China_All_Con']*100
china_energy_mix['portion_ptl'] = china_energy_mix['China_Con_PTL']/china_energy_mix['China_All_Con']*100
china_energy_mix.head()

,China_Con_Coal,China_Con_NG,China_Con_RE,China_Con_PTL,China_All_Con,portion_coal,portion_ng,portion_re,portion_ptl
1980,13.809654,0.581255,0.592360,4.124019,19.107288,72.274276,3.042059,3.100178,21.583487
1981,13.836130,0.517950,0.673220,3.937047,18.964347,72.958643,2.731178,3.549925,20.760255
1982,14.847019,0.438531,0.758539,3.894712,19.938801,74.462947,2.199385,3.804336,19.533331
1983,15.924503,0.496081,0.891974,3.972963,21.285521,74.813781,2.330603,4.190519,18.665097
1984,17.598646,0.504138,0.889890,4.062523,23.055197,76.332661,2.186657,3.859825,17.620857


In [ ]:
china_prod_coal = prod_coal.loc[prod_coal['Country']=='        China']
china_prod_coal = china_prod_coal.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_prod_coal = china_prod_coal.transpose()
china_prod_coal = pd.DataFrame(china_prod_coal)
china_prod_coal = china_prod_coal.rename(columns={65:'China_Prod_Coal'})
china_prod_coal.head()

,China_Prod_Coal
1980,13.811305
1981,13.844489
1982,14.839792
1983,15.913490
1984,17.577154


In [ ]:
china_prod_ng = prod_ng.loc[prod_ng['Country']=='        China']
china_prod_ng = china_prod_ng.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_prod_ng = china_prod_ng.transpose()
china_prod_ng = pd.DataFrame(china_prod_ng)
china_prod_ng = china_prod_ng.rename(columns={65:'China_Prod_NG'})
china_prod_ng.head()

,China_Prod_NG
1980,0.581255
1981,0.517950
1982,0.438531
1983,0.496081
1984,0.504138


In [ ]:
china_prod_re = prod_re.loc[prod_re['Country']=='        China']
china_prod_re = china_prod_re.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_prod_re = china_prod_re.transpose()
china_prod_re = pd.DataFrame(china_prod_re)
china_prod_re = china_prod_re.rename(columns={65:'China_Prod_RE'})
china_prod_re.head()

,China_Prod_RE
1980,0.598536
1981,0.677825
1982,0.770000
1983,0.899840
1984,0.897130


In [ ]:
china_prod_ptl = prod_ptl.loc[prod_ptl['Country']=='        China']
china_prod_ptl = china_prod_ptl.iloc[:, 2:44].apply(pd.to_numeric, errors='coerce')
china_prod_ptl = china_prod_ptl.transpose()
china_prod_ptl = pd.DataFrame(china_prod_ptl)
china_prod_ptl = china_prod_ptl.rename(columns={65:'China_Prod_PTL'})
china_prod_ptl.head()

,China_Prod_PTL
1980,4.548403
1981,4.318361
1982,4.388230
1983,4.548318
1984,4.939629


In [ ]:
china_all_prod.tail()

,China_All_Prod
2017,112.066649
2018,118.047776
2019,123.839249
2020,127.001993
2021,134.964189


In [ ]:
china_all_prod_mix = china_all_prod['China_All_Prod']
china_all_prod_mix.head()

1980    19.539499
1981    19.358625
1982    20.436553
1983    21.857729
1984    23.918051
Name: China_All_Prod, dtype: float64

In [ ]:
china_prod_energy_mix = china_prod_coal.join(china_prod_ng).join(china_prod_re).join(china_prod_ptl).join(china_all_prod_mix)
china_prod_energy_mix['portion_coal'] = china_prod_energy_mix['China_Prod_Coal']/china_prod_energy_mix['China_All_Prod']*100
china_prod_energy_mix['portion_ng'] = china_prod_energy_mix['China_Prod_NG']/china_prod_energy_mix['China_All_Prod']*100
china_prod_energy_mix['portion_re'] = china_prod_energy_mix['China_Prod_RE']/china_prod_energy_mix['China_All_Prod']*100
china_prod_energy_mix['portion_ptl'] = china_prod_energy_mix['China_Prod_PTL']/china_prod_energy_mix['China_All_Prod']*100
china_prod_energy_mix.head()

,China_Prod_Coal,China_Prod_NG,China_Prod_RE,China_Prod_PTL,China_All_Prod,portion_coal,portion_ng,portion_re,portion_ptl
1980,13.811305,0.581255,0.598536,4.548403,19.539499,70.684030,2.974769,3.063211,23.277990
1981,13.844489,0.517950,0.677825,4.318361,19.358625,71.515871,2.675552,3.501411,22.307166
1982,14.839792,0.438531,0.770000,4.388230,20.436553,72.613967,2.145817,3.767759,21.472456
1983,15.913490,0.496081,0.899840,4.548318,21.857729,72.804865,2.269591,4.116806,20.808738
1984,17.577154,0.504138,0.897130,4.939629,23.918051,73.489074,2.107772,3.750849,20.652304


In [ ]:
fig_china_energy_mix = go.Figure()
fig_china_energy_mix.add_trace(go.Bar(x=china_energy_mix.index, y=china_energy_mix['portion_coal'], name='Coal'))
fig_china_energy_mix.add_trace(go.Bar(x=china_energy_mix.index, y=china_energy_mix['portion_ng'], name='Natural Gas'))
fig_china_energy_mix.add_trace(go.Bar(x=china_energy_mix.index, y=china_energy_mix['portion_re'], name='Nuclear & Renewable'))
fig_china_energy_mix.add_trace(go.Bar(x=china_energy_mix.index, y=china_energy_mix['portion_ptl'], name='Petroleum'))
fig_china_energy_mix.update_layout(title_text='China Energy Consumption Mix',barmode='stack') 
fig_china_energy_mix.show()

In [ ]:
fig_china_energy_prod_mix = go.Figure()
fig_china_energy_prod_mix.add_trace(go.Bar(x=china_prod_energy_mix.index, y=china_prod_energy_mix['portion_coal'], name='Coal'))
fig_china_energy_prod_mix.add_trace(go.Bar(x=china_prod_energy_mix.index, y=china_prod_energy_mix['portion_ng'], name='Natural Gas'))
fig_china_energy_prod_mix.add_trace(go.Bar(x=china_prod_energy_mix.index, y=china_prod_energy_mix['portion_re'], name='Nuclear & Renewable'))
fig_china_energy_prod_mix.add_trace(go.Bar(x=china_prod_energy_mix.index, y=china_prod_energy_mix['portion_ptl'], name='Petroleum'))
fig_china_energy_prod_mix.update_layout(title_text='China Energy Production Mix',barmode='stack')
fig_china_energy_prod_mix.show()